# Colaboratory GPU notebook - Data Science Project
> * ___Rotem Shperling - 305699514___  
> * ___Eitan Shteinberg - 305809535___

---

## Phase 3 - Training the RNN LSTM model

** Installing Keras **

In [0]:
!pip install -q keras

In [0]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import re

Using TensorFlow backend.


** Installing Google Drive wrapper **

In [0]:
# Install a Drive FUSE wrapper.
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 16669 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .../0

Setting up dirmngr (2.1.15-1ubuntu8) ...
Setting up cron (3.0pl1-128ubuntu5) ...
Adding group `crontab' (GID 102) ...
Done.
update-rc.d: warning: start and stop actions are no longer supported; falling back to defaults
update-rc.d: warning: stop runlevel arguments (1) do not match cron Default-Stop values (none)
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Setting up kmod (24-1ubuntu2) ...
Setting up libdbus-glib-1-2:amd64 (0.108-2) ...
Setting up python3-gi (3.24.1-2build1) ...
Setting up module-init-tools (24-1ubuntu2) ...
Setting up python3-software-properties (0.96.24.17) ...
Setting up dbus (1.10.22-1ubuntu1) ...
Setting up python-apt (1.4.0~beta3build2) ...
Setting up python3-dbus (1.2.4-1build3) ...
Setting up python-software-properties (0.96.24.17) ...
Setting up software-properties-common (0.96.24.17) ...
Processing triggers for libc-bin (2.26-0ubuntu2.1) ...
Processing

** Connecting to Google Drive - Temporary keys are given after each notebook interruption **

In [0]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

In [0]:
# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
# Create a directory and mount Google Drive using that directory.
!mkdir -p drive
!google-drive-ocamlfuse drive

** Open the text files **
>__Google Colab is unable to open large text files, no matter what encoding was coded.  
Raw data was used from the localhost.__

In [0]:
obama = """ newspeech
mr president a few days ago the world watched as the seeds of democracy began to take root in iraq as a result of the sheer courage of the iraqi people and the untold sacrifices of american soldiers the success of the elections showed just how far people will go to achieve selfgovernment and rule of law
as americans we can take enormous pride in the fact that this kind of courage has been inspired by our own struggle for freedom by the tradition of democratic law secured by our forefathers and enshrined in our constitution it is a tradition that says all men are created equal under the law and that no one is above it
that is why even within the executive branch there is an office dedicated to enforcing the law of the land and applying it to people and to presidents alike
in this sense the attorney general is not like the other cabinet posts unlike the secretary of state who is the public face of the presidents foreign policy or the secretary of education whose job it is to carry out the presidents education policy the attorney generals job is not just to enforce the presidents laws it is to tell the president what the law is the job is not simply to facilitate the presidents power it is to speak truth to that power as well
the job is to protect and defend the laws of freedoms for which so many have sacrificed so much
the president is not the attorney generals client the people are and so the true test of an attorney general nominee is whether that person is ready to put the constitution of the people before the political agenda of the president as such i cannot approach this nomination for attorney general the same way i approached that of secretary of state rice or veterans affairs secretary nicholson or any other cabinet position the standard is simply higher
like the previous speaker senator dodd i wanted to give alberto gonzales the benefit of the doubt when we began this process his story is inspiring especially for so many of us like me who shared in achieving the american dream i have no question that as white house counsel he has served his president and his country to the best of his ability but in my judgment these positive qualities alone are not sufficient to warrant confirmation as the top law enforcement officer in the land
i had hoped that during his hearings judge gonzales would ease my concerns about some of the legal advice he gave to the president and i had hoped he would prove that he has the ability to distance himself from his role as the presidents lawyer so that he could perform his new role as the peoples lawyer
unfortunately rather than full explanations during these hearings i heard equivocation rather than independence i heard an unyielding insistence on protecting the presidents prerogative
i did not hear judge gonzales repudiate 212 years of what appears to be official us policy that has defined torture so narrowly that only organ failure and death would qualify a policy that he himself appears to have helped develop and at least has condoned
imagine that if the entire world accepted the definition contained in the department of justice memos we can only imagine what atrocities might befall our american pows how in the world without such basic constraints would we feel about sending our sons and daughters off to war how if we are willing to rationalize torture through legalisms and semantics can we claim to our children and the children of the world that america is different and represents a higher moral standard
this policy is not just a moral failure it is a violation of half a century of international law yet while judge gonzaless job was white house counsel he said nothing to that effect to the president of the united states he did not show an ability to speak with responsible moral clarity then and he has indicated that he still has no intention to speak such truths now
during his recent testimony he refused to refute a conclusion in the torture memo which stated that the president has the power to override our laws when acting as commander in chief think about this the nations top law enforcement officer telling its most powerful citizen that if the situation warrants the president can break the law from time to time
the truth is mr gonzales has raised serious doubts about whether given the choice between the constitution and the presidents political agenda he would put our constitution first and that is why i simply cannot support his nomination for attorney general
i understand that judge gonzales will most likely be confirmed and i look forward to working with him in that new role but i also hope that once in office he will take the lessons of this debate to heart
before serving in this distinguished body i had the privilege of teaching law for 10 years at the university of chicago among the brilliant minds to leave that institution for government service was a former dean of the law school named edward levi a man of impeccable integrity who was committed to the rule of law before politics
edward levi was chosen by president ford to serve as attorney general in the wake of watergate the president courageously chose to appoint him not because dean levi was a yes man not because he was a loyal political soldier but so that he could restore the publics confidence in a badly damaged justice department so that he could restore the publics trust and the ability of our leaders to follow the law
while he has raised serious doubts about his ability to follow this example judge gonzales can still choose to restore our trust he can still choose to put the constitution first i hope for our countrys sake that he will and part of the reason i am speaking in this chamber today is to suggest three steps that he can take upon assuming his role that would help restore that trust
first he can immediately repudiate the terror memos in question and ensure that the department of defense is not using any of its recommendations to craft interrogation policies
second judge gonzales can restore the credibility of his former position as legal counsel by appointing an independentminded universally respected lawyer to the post
and third he can provide this congress regular detailed reports on his efforts to live up to the presidents stated zero tolerance policy with respect to torture
today we are engaged in a deadly global struggle for those who would intimidate torture and murder people for exercising the most basic freedoms if we are to win this struggle and spread those freedoms we must keep our own moral compass pointed in a true direction the attorney general is one figure charged with doing this but to do it well he must demonstrate a higher loyalty than just to the president he must demonstrate a loyalty to the ideals that inspire a nation and hopefully the world
i thank the chair
 newspeech
mr president i rise to recognize the life and service of mr warren v hileman who passed away recently at the illinois veterans home in anna
last week the southern illinoisan reported that the state believes mr hileman was the last world war i veteran to have lived in illinois
he joined the us army in 1919 and served with the american expeditionary force in siberia from september of 1919 to march 1920 traveling thousands of miles across siberia in temperatures that often reached 30 below mr hileman and the 27th infantry served long after the armistice was signed in europe
in posolskaya their unit was involved in a hostile encounter that later won mr hileman the world war i victory medal which he was awarded in january of 2004
after the war he came home to illinois where he worked in a north chicago veterans hospital later he and his wife moved back down south to union county where they spent the rest of their lives
warren was only 17 years old when he first landed in vladivostok russia perhaps he was anxious about the war ahead of him perhaps he already missed the home that lay behind but above all he was ready and eager to serve this country at just 17 he was ready to make the ultimate sacrifice in the defense of freedom
today we honor his service and remember a man who returned from war to live over a century on this earth through more wars and depression through great advances for civil rights and great struggles for freedom mr hileman was there a patriot who had proudly written his own page in the story of 20th century america
it is said that whether a life is long or short its completeness depends on what it was lived for and so while warren hileman left us at the age of 103 the true completeness of his life comes from what he lived it for for his friends for his family and for the defense of the country he loved may his memory serve as a reminder for all of us to keep faith with our nations veterans and may warren hileman rest in eternal peace
 newspeech
mr president i speak today to recognize three gifted students from the state of illinois alison nichols brittany saltiel and sara siegal all students at stevenson high school in lincolnshire il
these three students created a national history day project on the mississippi burning legal case alison brittany and saras efforts to examine the circumstances of this case have led to not only a reopening of the case but also the overdue indictment of edgar ray killen for the murder of three young civil rights activists james chaney andrew goodman and michael schwerner
as a former civil rights attorney and constitutional law lecturer i know firsthand the importance of ensuring that justice and the principles of our constitution are always upheld i am proud to represent alison brittany and sara in the senate as they serve as a reminder of why all of us have committed our lives to public service
these students have demonstrated their tremendous potential in scholarship and leadership in public affairs they serve as shining examples for our nations young people of how a small group of committed individuals can truly change a community nation and the world alison brittany and sara deserve not only our congratulations they deserve our gratitude for making this country stronger
 newspeech
mr president i rise today to discuss the class action fairness act of 2005 as both a lawyer and a citizen i am a strong believer in class actions as a way for ordinary people who have been wronged by a corporation to band together and seek justice some of our great advances in civil rights and consumer protections have come from these actions
but there is overwhelming evidence that there are abuses in the class action system that should be addressed when multimillion dollar settlements are handed down and all the victims get are coupons for a free product justice is not being served and when cases are tried in counties only because its known that those judges will award big payoffs you get quick settlements without ever finding out whos right and whos wrong
every american deserves their day in court this bill while not perfect gives people that day while still providing the reasonable reforms necessary to safeguard against the most blatant abuses of the system i also hope that the federal judiciary takes seriously their expanded role in class action litigation and upholds their responsibility to fairly certify class actions so that they may protect our civil and consumer rights senator specter has pledged to work on these issues and address these serious concerns in the future and i look forward to joining him so we can improve this law
 newspeech
thank you mr president
i have come to the floor today to briefly address the pending legislation this issue forces us to face a fundamental question about who we are as a country how we progress as a society where our values lie as a people how do we treat our fellow americans who have fallen on hard times and what is our responsibility to cushion those falls when they occur we do so not only out of compassion for others but also knowing that hard times might at any moment fall on ourselves
the proponents of this bill claim it is designed to curb the worst abuses of our bankruptcy system i think that is a worthy goal shared by all those in this chamber and we can all agree that bankruptcy was never meant to serve as a get out of jail free card for use when you foolishly gamble away all your savings and dont feel like taking responsibility for your actions
but to accomplish that this bill would take us from a system where judges weed out the abusers from the honest to a system where all the honest are presumed to be abusers where declaring chapter 7 bankruptcy is made prohibitively expensive for people who have already suffered financial devastation
with this bill it doesnt matter if you run up your debt on a trip to vegas or a trip to the emergency room you are still treated the same under the law you still face the possibility that you will never get a chance to start over
it would be one thing if most people were abusing the system and falling into bankruptcy because they were irresponsible with their finances i think we need more responsibility with our finances in our society as well as from our government but we know that for the most part bankruptcies are caused as a result of bad luck
we know from a recent study which was mentioned by the distinguished senator from massachusetts that nearly half of all bankruptcies occur because of an illness that ends up sticking families with medical bills they cant keep up with
let me give you as a particular example the case of suzanne gibbons a constituent of mine a few years back suzanne had a good job as a nurse and a home on chicagos northwest side then she suffered a stroke that left her hospitalized for 5 days even though she had health insurance through her job it only covered usd4000 of the usd53000 in hospital bills as a consequence of that illness she was soon forced to leave her fulltime nursing job and take a temporary job that paid less and didnt offer health insurance then the collection agencies started coming after her for her hospital bills that she couldnt keep up with she lost her retirement savings she lost her house and eventually she was forced to declare bankruptcy if this bill passes as written without amendment suzanne will be treated by the law the same as any scam artist who cheats the system the decision about whether she can file for chapter 7 bankruptcy would not account for the fact that she fell into financial despair because of her illness
with all that debt she would have to hire a lawyer and pay hundreds of dollars more in increased paperwork after all that she still might be told she is ineligible for chapter 7 bankruptcy
as much as we like to believe that the face of this bankruptcy crisis is credit card addicts who spend their way into debt the truth is it is the face of people such as suzanne gibbons it is the face of middleclass americans
over the last 30 years bankruptcies have gone up 400 percent we have had 2100 more in illinois this year we also know what else has gone up the cost of childcare the cost of college the cost of home ownership and the cost of health care which is now at record highs people are working harder and longer for less and they are falling farther and farther behind
we are not talking about only the poor or even the working poor here these are middleclass families with two parents who both work at goodpaying jobs that put a roof over their heads they are saving every extra penny they have so their children can go to college and do better than they did but with just one illness one emergency one divorce these dreams are wiped away
this bill does a great job helping the credit card industry recover profits they are losing but what are we doing to help middleclass families to recover the dreams they are losing
the bankruptcy crisis this bill should address is not only the one facing credit card companies that are currently enjoying record profits we have to look after those hardworking families who are dealing with record hardships as senator dodd senator feinstein and others have pointed out this bill also fails to deal with the aggressive marketing practices and hidden fees credit card companies have used to raise their profits and our debt charging a penalty to consumers who make a late payment on a completely unrelated credit card is but one example of these tactics we need to end these practices so that we are making life easier not only for the credit card companies but for honest hardworking middleclass families
if we are going to crack down on bankruptcy abuse which we should we should also make it clear we intend to hold the wealthy and the powerful accountable as well
one example in my own state we had a mining company by the name of horizon that recently declared bankruptcy and then refused to pay its employees the health benefits it owed them a federal bankruptcy judge upheld the right of horizon to vacate the obligations it had made to its workers the mine workers involved had provided a total of 100000 years of service and dedication and sacrifice to this company they had spent their entire lives working hard they had deferred part of their salaries because there was an assurance that health care would be available for them these are men and women with black lung disease with bad backs with bad necks and the company made a decision to go back on their promise saying we will not pay the debt we owe these workers and a federal bankruptcy judge said that is ok you are permitted to do that
these same workers now are going to have a tough time as a consequence of this bill filing for bankruptcy the irony should not be lost on this chamber it is wrong that a bill would make it harder for those unemployed workers to declare bankruptcy while doing nothing to prevent the bankrupt company that puts them in financial hardship in the first place from shirking its responsibilities entirely
 newspeech
that is an accurate assessment by the distinguished senator from massachusetts i appreciate that amplification
the central point is what kind of message does it send when we tell hardworking middleclass americans you have to be more responsible with your finances than the companies you work for they are allowed to be irresponsible with their finances and we give them a pass when they have bad management decisions but you do not have a pass when confronting difficulties outside of your control
we need to reform our bankruptcy code so corporations keep their promises and meet their obligations to their workers i remain hopeful our companies want to do the right thing for workers doing so should not be a choice it should be a mandate
senator rockefeller and i have proposed two amendments to ensure this i strongly urge my colleagues support one will increase the required payments of wages and employee benefit plans to usd15000 per individual from the current level of usd4925 it requires companies that emerge from bankruptcy to immediately pay each retiree who lost health benefits an amount of cash equal to what a retiree would be expected to have to pay for cobra coverage for 18 months
the second amendment prevents bankruptcy courts from dismissing companies coal act obligations to pay their workers the benefits they were promised these companies made a deal to their mine workers they should be forced to honor that deal that will be an amendment that hopefully will be added to the pending bill
this bill gives a rare chance to ask ourselves who we are here to protect for whom we are here to stand up for whom we are here to speak we have to curb bankruptcy abuse and demand a measure of personal responsibility from all people we all want that we also want to make sure we are helping middleclass families who are loving their children and doing anything they can to give them the best possible life ahead
to wrap up in the 10 minutes i have been speaking about 30 of those middle class families have had to file for bankruptcy we live in a rapidly changing world with an economy that is moving just as fast we cannot always control this we cannot promise the changes will always leave everyone better off but we can do better than 1 bankruptcy every 19 seconds we can do better than forcing people to choose between the cost of health care and the cost of college we can do better than big corporations using bankruptcy laws to deny health care and benefits to their employees and we can give people the basic tools and protections they need to believe that in america your circumstances are no limit to the success you can achieve and the dreams you may fulfill
while unfortunately i cannot support this bill the way it is currently written i do look forward to working with my colleagues in amending this bill so we can still keep the promise alive
 newspeech
that is an accurate statement by the distinguished senator from massachusetts he characterizes it correctly
i add that all the statistics i have seen indicate one of the fastest growing segments engaged in bankruptcy is senior citizens who i dont think are any different than they were back in the day when we think people were more responsible and more thrifty i think they are still thrifty and responsible what has happened is they are experiencing extremely tough times partly because they are having difficulty paying for prescription medicines that are not covered under medicaid
 newspeech
mr president i rise to recognize an important anniversary in the history of this nation
one hundred and thirtyfive years ago on this day hiram r revels was sworn in as a us senator from mississippi on that day february 25 1870 senator revels became the first african american to ever serve in the us congress
but hiram revels story started in a place very far from washington dc he was born to free parents in 1822 and grew up as an apprentice to a barber in north carolina but hiram wanted to learn more and see more and so he left for indiana and then ohio where he furthered his education he was soon ordained a minister by the african methodist church and traveled to congregations all over the midwest and the south until he finally ended up in baltimore
at the beginning of the civil war he helped recruit africanamerican troops for the union and he ended up serving as a chaplain for a mississippi regiment of free blacks he stayed in mississippi after the war and continued serving as a pastor at various local churches in 1868 and he ran and was elected alderman respected by both whites and african americans he was soon elected as a mississippi state senator then in 1870 just 5 years after the end of the very war fought for his freedom hiram revels was elected the first africanamerican us senator in history
like so many of our own hirams story is americas story the story of the seemingly impossible occurring in a land where good people will give everything to make it possible the story of hope winning out against all odds the story of one mans improbable achievement paving the way for so many others
did hiram ever know what he was destined for in that barber shop when he was sweeping that floor in north carolina and so many of his brothers and sisters were enslaved did he ever dream that he would end up a us senator
we dont know but we do know that he did dream of bigger things
he dreamed of an education and so even though many kids like him didnt do it he went to college he dreamed of helping others and so even though it involved sacrifice he became a minister he dreamed of a free america and so even though it could have cost him his life he joined the union and he dreamed of lifting up his community and so even though it wasnt done by people of his color he ran for office
he dreamed of making this world a better place and in doing so he found a place in history and so we remember this day his day as a symbol of what is possible for those of us who are willing to make it so in this magical place we call america
 newspeech
mr president i rise to recognize and remember the life of earl langdon neal
mr neal was one of the finest lawyers and civic leaders chicago has ever known from mayors to citizens business leaders to college students he was a trusted friend and inspiring mentor to many including myself earl earned his law degree from michigan law school in 1952 following graduation he served his country in the us army until 1955 when he returned to chicago to join his fathers law firm neal neal
on their very first case earl and his father were forced to commute 170 miles from chicago to lincoln simply because there were no hotels in lincoln that would accept african americans but he went anyway because as his son has said it wasnt just a job for earl it was a way of life
it was a way of life that led him to serve the city of chicago as a special assistant corporation counsel responsible for countless land acquisition projects including the dan ryan expressway ohares expansion and the chicago city colleges a way of life that led him to start his own practice and earn a place on the university of illinois board of trustees a way of life that made almost every person who came to know him speak of him as a warm compassionate man who put the wellbeing of his clients above all else
earls passion for his work wasnt complicated he simply looked around his community and wanted to make it better and in so many ways from the places he made possible to the peoples lives he touched he did we honor his life pray for his family and will miss him dearly
 newspeech
mr president as the peace corps celebrates its 44th anniversary this month i would like to take this opportunity to commend its many wonderful volunteers past and present and the remarkable work they do i am very pleased to report that three universities in illinois the university of illinois at champaignurbana northwestern university and the university of chicago are three of the top peace corpsvolunteer producing colleges and universities in the country there are currently 295 volunteers from illinois working in countries around the world and i am tremendously proud of the service provided by each and every one of them
the work of the peace corps in promoting mutual understanding between our country and the rest of the world has never been more relevant than it is today the peace corps began in 1961 under president john f kennedy as a unique experiment in humanitarian service and cultural exchange and has grown to become one of the most widely respected american institutions in the world since the founding of the peace corps over 178000 volunteers have served in 138 countries
peace corps volunteers share their knowledge skills and enthusiasm by serving as health educators youth and agricultural workers teachers and business advisors of the 7700 volunteers currently serving over 3100 are working on hivaids education and prevention projects and i commend them on their critically important work on this global crisis
i also would like to honor past volunteers who have helped to build this wonderful program into what it is today who have empowered and given voice to individuals and communities in developing countries around the world individually and collectively peace corps volunteers represent the very best of our great country and i am proud to salute them on their 44th anniversary
 newspeech
mr president over the last century the nation has witnessed tremendous advances in medical science and technology we now have treatments and cures for diseases and conditions that were at one time surely fatal thirty years ago if children developed cancer doctors couldnt save their lives today more than threequarters of children with cancer survive heart disease is no longer the leading cause of death because of significant improvements in medical treatment and surgical procedures americans with aids are living many years longer and spending more time at home and not in hospitals because of new drug cocktails that prevent infections and other deadly complications
the unfortunate and bitter irony is that while the number of medical breakthroughs continues to increase so does the number of americans who will never benefit from them right now 45 million americans have no health care coverage and this number continues to rise over a 2year period over 85 million americans have not had continuous insurance coverage in this land of plenty and opportunity 350000 uninsured children with earaches and sore throats will never see a doctor sixteen million uninsured americans cannot afford to fill prescriptions uninsured women who develop breast cancer are 40 percent more likely to die as are 50 percent of uninsured men with prostate cancer the institute of medicine has reported that 18000 adults die every year because they are uninsured
for many americans medicaid represents their only real hope of obtaining health care nationally 53 million people rely on medicaid coverage including 25 million children 13 million lowincome adults and 15 million disabled and elderly americans nearly 16 percent of people who live in rural areas have medicaid coverage including more than 1 in 4 children in these areas one quarter of african americans and 20 percent of hispanics rely on medicaid as do 9 percent of women
in my home state of illinois medicaid provides health coverage for 2 million residents over 30 percent of children in illinois receive health care through kidcare nearly 65 percent of nursing home residents rely on medicaid coverage
despite medicaids critical role in providing access to care the republican budget proposes to cut medicaid by usd15 billion this cut translates into an estimated usd287 million loss for illinois experts report this funding could provide health care coverage for 200000 children or 135000 working parents in my state
some of my colleagues argue that we have no choice but to make large cuts to medicaid because of the deficit but these deficits were created by huge tax breaks for the rich not by medicaid and we should not balance the budget at the expense of health care for lowincome children their parents pregnant women and seniors we cannot keep tax cuts for the rich and cut basic health care for the poor we cannot retreat from our federal commitment to medicaid and leave the states holding the bag
i agree the medicaid program is not perfect the smithbingaman amendment to create a commission to study the program and make recommendations for improvement is a reasonable approach sound policy not politics or deficit concerns should guide any changes to the medicaid program and i am not convinced that we have examined or discussed the full range of medicaidrelated issues and options before us
we cannot and should not deny millions of americans access to basic health care medicaid is the nations safety net and we should strengthen it not destroy it i am going to vote yes for the smithbingaman amendment to strike proposed cuts in funding for medicaid and i urge my colleagues to join me
 newspeech
mr president i am pleased to join as a cosponsor of the fuels security act of 2005 which sets a renewable fuels standard for the years 2006 to 2012
to lessen our dependence on foreign oil and strengthen our economy here at home renewable fuels like ethanol ought to be a larger part of our domestic fuel supply this bill will contribute to that objective and i commend senators lugar and harkin for their leadership in crafting this legislation
yesterday during the markup of a similar bill in the senate environment and public works committee i expressed strong support for establishing a meaningful renewable fuels standard as an important part of a comprehensive national energy policy the bill before the committee set targets at 38 billion gallons in 2006 and 6 billion gallons in 2012 improving upon last years rfs provision in the energy bill conference report that set targets at 31 billion gallons and 5 billion gallons respectively
i voted for the chairmans mark yesterday because it gets the rfs debate rolling in the new congress however i also noted that it has been widely reported in the trade press that the 30state governors ethanol coalition has recommended to the president that refiners be required to purchase a minimum volume of ethanol of at least 4 billion gallons in 2006 rising to 8 billion gallons in 2012 this recommendation adds weight to the view expressed by me and others that the committees targets are too conservative
why are these specific targets so important they are important if we are to maximize the ethanol industrys ability to boost farm income by providing a new market for corn to promote economic growth in rural communities by increasing production in existing plants and attracting investment in new communitysized ethanol facilities and to reduce our alarming dependence on imported oil by expanding the volume of ethanol in our transportation fuel mix
these are important objectives they matter and that is why it is important to get the specific targets right
in committee yesterday i suggested that since ethanol production is expected to reach 4 billion gallons this year we ought to adjust the committee bills rfs targets on the senate floor to reflect current market reality i am pleased that chairman inhofe seemed open to that debate
i think the governors ethanol coalition recommendation of at least 4 billion gallons in 2006 and 8 billion gallons in 2012 is a good place to start this debate i think any rfs legislation enacted by congress should contain these levels
that is why i am pleased to cosponsor the fuels security act introduced by senators lugar and harkin today the ethanol volume targets in this bill 4 billion gallons in 2006 and 8 billion gallons in 2012 are in much greater alignment with expected ethanol production in future years than those in the committee bill
earlier this week i had the opportunity to tour the aventine ethanol plant in pekin il my visit reminded me of the work of a pekin native more than 50 years ago that person senator everett dirksen encouraged federal lawmakers to consider processing our surplus farm crops into an alcohol to create a market in our own land for our own people
today farmers across illinois including farmers near pekin are growing corn for fuel both strengthening our energy security and providing an economic boost to rural communities by enacting a meaningful rfs we are displacing more foreign oil with homegrown energy we are expanding the market for illinois corn and we are promoting the use of renewable fuel remember unlike other energy sources when you run out of ethanol you can simply grow more
 by mr reid
s 651 a bill to amend title 5 united states code to make creditable for civil service retirement purposes certain periods of service performed with air america incorporated air asia company limited or the pacific division of southern air transport incorporated while those entities were owned or controlled by the government of the united states and operate or managed by the central intelligence agency to the committee on homeland security and governmental affairs
mr president i ask unanimous consent that the text of the bill be printed in the record
 there being no objection the bill was ordered to be printed in the record as follows
be it enacted by the senate and house of representatives of the united states of america in congress assembled
a in generalsection 8332b of title 5 united states code is amended 1 by striking and at the end of paragraph 16 2 by striking the period at the end of paragraph 17 and inserting and 3 by adding after paragraph 17 the following 18 any period of service performed before 1977 while a citizen of the united states in the employ of air america incorporated air asia company limited a subsidiary of air america incorporated or the pacific division of southern air transport incorporated at a time when that corporation or subsidiary was owned or controlled by the government of the united states and operated or managed by the central intelligence agency and 4 by adding at the end the following for purposes of this subchapter service of the type described in paragraph 18 of this subsection shall be considered to have been service as an employee and the office of personnel management shall accept the certification of the director of the central intelligence agency or his designee concerning any such service b exemption from deposit requirementsection 8334g of title 5 united states code is amended 1 by striking or at the end of paragraph 5 2 by striking the period at the end of paragraph 6 and inserting or and 3 by adding after paragraph 6 the following 7 any service for which credit is allowed under section 8332b18 of this title
a in generalexcept as otherwise provided in this section the amendments made by this act shall apply with respect to annuities commencing on or after the effective date of this act b provisions relating to current annuitantsany individual who is entitled to an annuity for the month in which this act becomes effective may upon application submitted to the office of personnel management within 2 years after the effective date of this act have the amount of such annuity recomputed as if the amendments made by this act had been in effect throughout all periods of service on the basis of which such annuity is or may be based any such recomputation shall be effective as of the commencement date of the annuity and any additional amounts becoming payable for periods before the first month for which the recomputation is reflected in the individuals regular monthly annuity payments shall be payable to such individual in the form of a lumpsum payment c provisions relating to individuals eligible for but not currently receiving an annuity 1 in generalany individual not described in subsection b who becomes eligible for an annuity or for an increased annuity as a result of the enactment of this act may elect to have such individuals rights under subchapter iii of chapter 83 of title 5 united states code determined as if the amendments made by this act had been in effect throughout all periods of service on the basis of which such annuity is or would be based by submitting an appropriate application to the office of personnel management within 2 years after a the effective date of this act or b if later the date on which such individual separates from service 2 commencement date etc a in generalany entitlement to an annuity or to an increased annuity resulting from an application under paragraph 1 shall be effective as of the commencement date of such annuity subject to subparagraph b if applicable and any amounts becoming payable for periods before the first month for which regular monthly annuity payments begin to be made in accordance with the amendments made by this act shall be payable to such individual in the form of a lumpsum payment b retroactivityany determination of the amount or of the commencement date of any annuity all the requirements for entitlement to which including separation but disregarding any application requirement would have been satisfied before the effective date of this act if this act had then been in effect but would not then otherwise have been satisfied absent this act shall be made as if application for such annuity had been submitted as of the earliest date that would have been allowable after such individuals separation from service if such amendments had been in effect throughout the periods of service referred to in the first sentence of paragraph 1 d right to file on behalf of a decedentthe regulations under section 4a shall include provisions consistent with the order of precedence set forth in section 8342c of title 5 united states code under which a survivor of an individual who performed service described in section 8332b18 of such title as amended by section 1 shall be allowed to submit an application on behalf of and to receive any lumpsum payment that would otherwise have been payable to the decedent under subsection b or c such an application shall not be valid unless it is filed within 2 years after the effective date of this act or 1 year after the date of the decedents death whichever is later
a lumpsum paymentsany lumpsum payments under section 2 shall be payable out of the civil service retirement and disability fund b unfunded liabilityany increase in the unfunded liability of the civil service retirement system attributable to the enactment of this act shall be financed in accordance with section 8348f of title 5 united states code
a in generalexcept as provided in subsection b the director of the office of personnel management in consultation with the director of the central intelligence agency shall prescribe any regulations necessary to carry out this act such regulations shall include provisions under which rules similar to those established pursuant to section 201 of the federal employees retirement system act of 1986 public law 99335 100 stat 514 shall be applied with respect to any service described in section 8332b18 of title 5 united states code as amended by section 1 that was subject to title ii of the social security act b other regulationsthe director of the central intelligence agency in consultation with the director of the office of personnel management shall prescribe any regulations which may become necessary with respect to any retirement system administered by the director of the central intelligence agency as a result of the enactment of this act c special rulefor purposes of any application for any benefit which is computed or recomputed taking into account any service described in section 8332b18 of title 5 united states code as amended by section 1 section 8345i2 of such title shall be applied by deeming the reference to the date of the other event which gives rise to title to the benefit to refer to the effective date of this act if later than the date of the event that would otherwise apply
for purposes of this act 1 the terms unfunded liability survivor and survivor annuitant have the meanings given under section 8331 of title 5 united states code and 2 the term annuity as used in subsections b and c of section 2 includes a survivor annuity
 by mr specter for himself and mr santorum
s 652 a bill to provide financial assistance for the rehabilitation of the benjamin franklin national memorial in philadelphia pennsylvania and the development of an exhibit to commemorate the 300th anniversary of the birth of benjamin franklin to the committee on energy and natural resources
mr president i have sought recognition today to introduce a bill to authorize federal funding for the rehabilitation of the benjamin franklin national memorial this memorial an attraction for some 1 million visitors annually is truly a national treasure yet it has come under significant deterioration the franklin statue has not been thoroughly cleaned since 1998 there are structural impacts to the statue from changes in temperature and humidity the lighting and sound systems are obsolete and the marble walls and stained glass dome are discolored from days when smoking was permitted the bill that senator santorum and i are introducing today will help ensure that federal funding is made available to preserve and protect our nations memorial to benjamin franklin americas distinguished scientist statesman inventor and diplomat
in the 108th congress senator santorum and i introduced similar legislation to authorize this much needed funding and we were pleased that senator domenici senator thomas and their colleagues on the senate committee on energy and natural resources favorably reported an amended version of our legislation to the senate on september 28 2004 subsequently this legislature passed the senate on october 10 2004 however the limited time available prior to adjournment of the 108th congress precluded passage of this measure by the house of representatives
unlike other national memorials the benjamin franklin national memorial does not receive an annual allocation of federal funds to provide for preventative maintenance or other important activities
 by mr leahy for himself mr durbin mr kennedy and mr dodd
s 654 a bill to prohibit the expulsion return or extradition of persons by the united states to countries engaging in torture and for other purposes to the committee on foreign relations
mr president our nation has a proud history as the leading advocate of human rights around the world throughout this history we have committed ourselves to numerous international human rights treaties including the convention against torture and other cruel inhuman or degrading treatment or punishment the bill that i introduce today will reaffirm our obligations under this convention and reassure the world that we are a nation committed to the rule of law i want to thank my cosponsors senators durbin kennedy and dodd for working with me on this legislation and for their leadership on these issues
it has been nearly a year since the first horrific images from abu ghraib prison appeared in the media shocking the world and shattering the image of the united states as the administration circled the wagons and claimed the abuses were committed by a few bad apples new details about the widespread abuse of detainees continued to emerge i have spoken many times about the need for a comprehensive independent investigation into the abuse of detainees i have no doubt that such an investigation would be painful but it is also a necessary step to moving forward
prisoner abuse by us personnel is deeply troubling but it is only one aspect of a broader and serious problem while we must ensure that prisoners are treated humanely by our own personnel we must also prohibit the use of socalled extraordinary renditions to send people to other countries where they will be subject to torture article 3 of the convention against torture states that no state party shall expel return or extradite a person to another state where there are substantial grounds for believing that he would be in danger of being subjected to torture the bill i introduce today the convention against torture implementation act will ensure that we honor this commitment
we have addressed this issue before congress implemented article 3 of the convention against torture in the foreign affairs reform and restructuring act of 1998 but this administration has exploited loopholes in that law to transfer detainees to countries where they are subjected to torture attorney general gonzales recently said that us policy is not to send detainees to countries where we believe or we know that theyre going to be tortured but he acknowledged that we cant fully control what other nations do and added that he does not know whether countries have always complied with their promises in fact they have not
my proposed legislation does not broaden the obligations that we agreed to by ratifying the convention against torture it simply closes the loopholes in the 1998 law and ensures that we honor our commitment not to outsource torture to other countries
the case of maher arar provides a chilling example of extraordinary rendition and illustrates why this bill is necessary mr arar a canadian and syrian citizen was stopped by immigration officers at john f kennedy international airport in september 2002 as he attempted to change planes on his way home to canada from tunisia he claims that he was interrogated by an fbi agent and a new york city police officer and that he was denied access to a lawyer he further claims that he repeatedly told us officials that he feared he would be tortured if deported to syria after being detained for nearly two weeks in a federal detention center in new york mr arar was transferred by us authorities to syria and held at the bush administrations request mr arar claims that he was physically tortured during the first two weeks of his detention in syria and that he was subjected to severe psychological abuse over the following 10 months including being held in a gravelike cell and being forced to undergo interrogation while hearing the screams of other prisoners
according to administration officials the cia received diplomatic assurances from syria that it would not torture mr arar but those assurances amounted to little more than a wink and a nod unnamed intelligence officials were later quoted in the press saying that arar confessed under torture in syria that he had gone to afghanistan for terrorist training syria has a welldocumented history of statesponsored torture in fact president bush stated on november 7 2003 that syria has left a legacy of torture oppression misery and ruin to its people
rather than rely on assurances that a country will not torture an individual we must make our own unbiased determination we already have the necessary information to do so each year as required by law the state department publishes country reports on human rights practices the most recent report on syria states that its torture methods include administering electrical shocks pulling out fingernails forcing objects into the rectum beating sometimes while the victim was suspended from the ceiling hyperextending the spine bending the detainees into the frame of a wheel and whipping exposed body parts and using a backwardbending chair to asphyxiate the victim or fracture the victims spine
some will argue that the post911 world is different that we must use any and all means available to extract information from suspected terrorists their argument might be more credible if every person who turned up on a terrorist watch list were in fact a terrorist i cannot say whether mr arar had ties to terrorist groups or not but we do know that he was never charged with a crime after enduring months of torture at the hands of the syrians he was released and sent back to canada
nor was mr arars experience an isolated incident a recent article in the new yorker titled outsourcing torture provides disturbing details about how the administration embraced the use of rendition after the 911 attacks several press reports detail the cias use of its own gulfstream v and boeing 737 jets to secretly transfer detainees to countries around the world where it is likely that they will be tortured
the convention against torture implementation act addresses the extraordinary rendition problem in a straightforward manner it requires the state department to produce annually a list of countries where torture is known to occur the list would be based on information contained in the state departments country reports on human rights practices the bill prohibits the transfer of individuals to any country on this list or to any other country if there are substantial grounds for believing that the person would be tortured it also provides reasonable exceptions to this prohibition to allow for legal extraditions and removals
most importantly the bill closes the diplomatic assurances loophole we would no longer accept assurances from governments that we know engage in torture our past reliance on diplomatic assurances is blatantly hypocritical how can our state department denounce countries for engaging in torture while the cia secretly transfers detainees to the very same countries for interrogation the president says he does not condone torture but transferring detainees to other countries where they will be tortured does not absolve our government of responsibility by outsourcing torture to these countries we diminish our own values as a nation and lose our credibility as an advocate of human rights around the world
last june in the aftermath of the abu ghraib scandal the president was asked if he had authorized abusive interrogation techniques he replied the authorization i issued was that anything we did would conform to us law and would be consistent with international treaty obligations the legislation i introduce today will help us fulfill the presidents promise
the senate gave its advice and consent to the ratification of the convention against torture more than a decade ago it is time to honor our commitment and show the world that we will hold ourselves to the same standards that we demand of others
mr president i ask unanimous consent that the text of the bill and a sectionbysection analysis be printed in the record
 there being no objection the material was ordered to be printed in the record as follows
be it enacted by the senate and house of representatives of the united states of america in congress assembled
this act may be cited as the convention against torture implementation act of 2005
a prohibitionno person in the custody or control of a department agency or official of the united states government or of any contractor of any such department or agency shall be expelled returned or extradited to another country whether directly or indirectly if 1 the country is included on the most recent list submitted to congress by the secretary of state under section 3 or 2 there are otherwise substantial grounds for believing that the person would be in danger of being subjected to torture b exceptions 1 waivers a authoritythe secretary of state may waive the prohibition in subsection a1 with respect to a country if the secretary certifies to the appropriate congressional committees that i the acts of torture that were the basis for including that country on the list have ended and ii there is in place a mechanism that assures the secretary in a verifiable manner that a person expelled returned or extradited to that country will not be tortured in that country including at a minimum immediate unfettered and continuing access from the point of return to such person by an independent humanitarian organization b reports on waivers i reports requiredfor each person expelled returned or extradited under a waiver provided under subparagraph a the head of the appropriate government agency making such transfer shall submit to the appropriate congressional committees a report that includes the name and nationality of the person transferred the date of transfer the reason for such transfer and the name of the receiving country ii formeach report under this subparagraph shall be submitted to the extent practicable in unclassified form but may include a classified annex as necessary to protect the national security of the united states 2 extradition or removalthe prohibition in subsection a1 may not be construed to apply to the legal extradition of a person under a bilateral or multilateral extradition treaty or to the legal removal of a person under the immigration laws of the united states if before such extradition or removal the person has recourse to a united states court of competent jurisdiction to challenge such extradition or removal on the basis that there are substantial grounds for believing that the person would be in danger of being subjected to torture in the receiving country c assurances insufficientwritten or verbal assurances made to the united states by the government of a country that persons in its custody or control will not be tortured are not sufficient for believing that a person is not in danger of being subjected to torture for purposes of subsections a2 and b2 or for meeting the requirement of subsection b1aii
not later than 30 days after the effective date of this act and annually thereafter the secretary of state shall submit to the appropriate congressional committees a report listing each country where torture is known to be used the list shall be compiled on the basis of the information contained in the most recent annual report of the secretary of state submitted to the speaker of the house of representatives and the committee on foreign relations of the senate under section 116d of the foreign assistance act of 1961 22 usc 2151nd
a interim regulationsnot later than 60 days after the effective date of this act the heads of the appropriate government agencies shall prescribe interim regulations for the purpose of carrying out this act and implementing the obligations of the united states under article 3 of the convention against torture subject to any reservations understandings declarations and provisos contained in the senate resolution advising and consenting to the ratification of the convention against torture and consistent with the provisions of this act b final regulationsnot later than 180 days after interim regulations are prescribed under subsection a and following a period of notice and opportunity for public comment the heads of the appropriate government agencies shall prescribe final regulations for the purposes described in subsection a
nothing in this act shall be construed to eliminate limit or constrain in any way the obligations of the united states or the rights of any individual under the convention against torture or any other applicable law
section 2242 of the foreign affairs reform and restructuring act of 1998 public law 105277 8 usc 1231 note is repealed regulations promulgated under such section that are in effect on the date this act becomes effective shall remain in effect until the heads of the appropriate government agencies issue interim regulations under section 4a
a defined termsin this act 1 appropriate government agenciesthe term appropriate government agencies means a the intelligence community as defined in section 34 of the national security act of 1947 50 usc 401a4 and b elements of the department of state the department of defense the department of homeland security the department of justice the united states secret service the united states marshals service and any other federal law enforcement national security intelligence or homeland security agency that takes or assumes custody or control of persons or transports persons in its custody or control outside the united states other than those elements listed or designated as elements of the intelligence community under section 34 of the national security act of 1947 50 usc 401a4 2 appropriate congressional committeesthe term appropriate congressional committees means a the committees on armed services homeland security and government affairs judiciary foreign relations and the select committee on intelligence of the senate and b the committees on armed services homeland security judiciary international relations and the permanent select committee on intelligence of the house of representatives 3 convention against torturethe term convention against torture means the united nations convention against torture and other cruel inhuman or degrading treatment or punishment done at new york on december 10 1984 entered into force on june 26 1987 signed by the united states on april 18 1988 and ratified by the united states on october 21 1994 t doc 10020 4 expelled persona person who is expelled is a person who is involuntarily transferred from the territory of any country or a port of entry thereto to the territory of another country or a port of entry thereto 5 extradited persona person who is extradited is an accused person who in accordance with chapter 209 of title 18 united states code is surrendered or delivered to another country with jurisdiction to try and punish the person 6 returned persona person who is returned is a person who is transferred from the territory of any country or a port of entry thereto to the territory of another country of which the person is a national or where the person has previously resided or a port of entry thereto b same terms as in the convention against torture except as otherwise provided the terms used in this act have the meanings given those terms in the convention against torture subject to any reservations understandings declarations and provisos contained in the senate resolution advising and consenting to the ratification of the convention against torture
this act shall take effect on the date that is 30 days after the date of the enactment of this act
this act shall be classified to the united states code as a new chapter of title 50 united states code
this act shall be classified to the united states code as a new chapter of title 50 united states codeconvention against torture implementation act of 2005 sectionby section analysis
this act shall be classified to the united states code as a new chapter of title 50 united states codeconvention against torture implementation act of 2005 sectionby section analysissec 1 short title the convention against torture implementation act of 2005 sec 2 prohibition on certain transfers of persons this section implements article 3 of the convention against torture which prohibits expelling returning or extraditing persons to countries where they are in danger of being subjected to torture subsection a prohibits the transfer of a person in the custody or control of the united states government to a country included on a list generated by the state department as required by section 3 of this act or to countries where there are substantial grounds for believing that the person would be in danger of being subjected to torture subsection b allows exceptions to the prohibition if the secretary of state waives the prohibition or if the transfer is done under an extradition treaty or as a legal removal under united states immigration laws agencies that transfer a detainee under the waiver exception must submit a report of the transfer to appropriate congressional committees subsection c states that assurances made to the united states by another government that persons in its custody will not be tortured are not sufficient for the united states to conclude that a person will not be subjected to torture sec 3 reports on countries using torture this section requires the secretary of state on an annual basis to compile a list of countries where torture is known to be used the united states is prohibited from transferring persons to the countries on this list except in accordance with the exceptions contained in section 2 the list shall be compiled based on information contained in the most recent state department country reports on human rights practices which the department submits annually in accordance with section 116d of the foreign assistance act of 1961 22 usc 2151nd sec 4 regulations this section requires appropriate government agencies as defined in section 7 to prescribe regulations in accordance with this act interim regulations must be prescribed within 60 days of the effective date of the act final regulations must be prescribed through notice and comment rulemaking not more than 180 days thereafter sec 5 savings clause this section ensures that the act does not eliminate limit or constrain the obligations of the united states or the rights of any individual under the convention against torture or any other applicable law sec 6 repeal of superseded authority this section repeals section 2242 of the foreign affairs reform and restructuring act of 1998 public law 105277 8 usc 1231 note this law also implemented article 3 of the convention against torture but lacked specific guidance for agencies and allowed the united states to rely on diplomatic assurances that a government would not torture a person transferred to its custody this section also requires agency regulations promulgated under section 2242 to remain in effect until the appropriate government agencies issue new regulations in accordance with section 4 of this act sec 7 definitions this section defines appropriate government agencies appropriate congressional committees expelled person extradited person returned person and convention against torture it also states that terms used in the act unless otherwise provided have the meanings given to those terms in the convention against torture sec 8 effective date makes the act effective 30 days after its enactment sec 9 classification in united states code this section requires the act to be classified as a new chapter of title 50 in the united states code the superseded authority was classified as a note in title 8 in the united states code given the scope and applicability of the act it is more accurate to classify it in the war and national defense title than in the aliens and nationality title
mr president the entire world continues to wait for signs that the administration takes seriously its moral and legal responsibilities to eliminate torture and abuse it is long past time for the administration to give the american people and the world an ironclad assurance that these shameful tactics are no longer being used in any prison or detention facility under american control and that we are not outsourcing our torture to regimes well known for using them
i strongly support the legislation that senator leahy has introduced to deal with this urgent problem and to see that our nation is not farming out abusive interrogations to other countries the bill makes crystal clear that we cant torture by proxy
abhorrence to torture is a fundamental value our attitude toward torture speaks volumes about our national conscience our dedication to the rule of law and our essential ideals 911 is no excuse for abandoning our ideals
the line separating right from wrong must clearly exclude the reprehensible practice called extraordinary rendition the ridiculous code word for torture by proxy article 3 of the treaty against torture which the united states has ratified provides no state party shall expel return or extradite a person to another state where there are substantial grounds for believing he would be in danger of being subjected to torture the secretive us practice of rendition is a violation of international law because it involves detaining prisoners without a shred of due process and delivering them for interrogation into the hands of countries known to commit torture as one commentator noted in terms of bad behavior it stands side by side with contract killings
ask maher arar in the fall of 2002 arar a canadian citizen was returning to montreal from a family visit in tunisia and he made a stopover at kennedy airport in new york city acting in part on flawed intelligence from canadian officials us immigration officials seized mr arar at the airport he was not charged with a crime or given a chance to talk with a lawyer instead he was held in brooklyn and interrogated for days by us law enforcement authorities
when the interrogation failed to produce incriminating information mr arar was flown to jordan and handed over to jordanian authorities he was chained blindfolded and beaten in a van that transported him to the syrian border in syria he was placed in a small dark cell three feet by six feet like a grave and was held there for almost a year he was slapped beaten and whipped on his palms wrists and back with an electric cable he begged them to stop he heard other prisoners screaming as they were tortured he signed any confessions he was told to sign
mr arar was released in october 2003 syrian officials told reporters that their investigators found no link between mr arar and alqaida his confession turned out to be worthless and his suffering was pointless mr arar is now home in canada
how can any of us stand idly by knowing that this country condoned and facilitated such brutality
tragically mr arar is not the only victim on march 6 60 minutes aired a report on rendition on the program michael scheuer a recently retired cia official who created its rendition program admitted that he would have to assume that suspects the us sends to egypt are tortured its very convenient he said its finding someone else to do your dirty work
the defense department has attempted to justify this tactic on june 25 2003 defense department general counsel william haynes wrote to senator leahy stating that whenever the us transfers an individual to another country united states policy is to obtain specific assurances from the receiving country that it will not torture the individual being transferred to that country we can assure you that the united states would take steps to investigate credible allegations of torture and take appropriate action if there were reason to believe that those assurances were not being honored
mr haynes assurances are difficult to accept the state departments annual human rights report released last month criticized numerous countries for a range of interrogation practices it labeled as torture the state department identified syria egypt and saudi arabia among others as countries practicing torture press reports make clear that since 911 the us has flown 100150 suspects to countries such as these the state department condemns syria for torturing its prisoners but mr haynes blindly relies on syrias promise that the prisoners we send there will be treated humanely
recent press reports also suggest that the assurances of humane treatment sought by the cia are worth very little according to todays washington post one government official who visited several foreign prisons where suspects were rendered by the cia said `its widely understood that the interrogation practices that would be illegal in the us are being used the official also said they say they are not abusing them but we all know they do
according to the post an arab diplomat whose country is actively engaged in counterterrorism alongside the cia said it was unrealistic to believe the cia really wants to follow up on assurances he said it would be stupid to keep track of them because then you would know whats going on he said its like dont ask dont tell
so it seems that we are not fooling anybody but the american public
we are a nation of laws not hypocrites our country is strong and our constitutional system has endured because it permits us to do great things and still ensure that we treat people fairly and humanely we are not supposed to disappear people here
yet that is exactly what rendition and the related tactic of ghost detainees amounts to making people vanish into a shadowy world of secret abuse in his report on the abuses at abu ghraib prison mg antonio taguba wrote that prisoners had not been registered as required by army regulations and they were being moved around to avoid detection by the red cross general taguba called the practice deceptive contrary to army doctrine and in violation of international law last september army investigators told the senate armed services committee that as many as 100 detainees at abu ghraib had been hidden from the red cross at the cias direction
last month the associated press reported that one of the ghost detainees held at abu ghraib manadel aljamadi died in november 2003 under cia interrogation he had been suspended by his wrists with his hands cuffed behind his back according to an army guard who was asked by the interrogator to adjust aljamadis position blood gushed from his mouth as if a faucet had been turned on after he was released from his shackles
behavior like that forces us all to ask what has america become
the issue shows no signs of abating article 49 of the fourth geneva convention states that transfers of detainees from occupied territory to any other country are prohibited regardless of their motive violations of the article constitute grave breaches of the treaty and qualify as war crimes under federal law nevertheless a justice department memorandum in march 2004 reinterpreted the treaty to allow the cia to remove prisoners from iraq for the purpose of facilitating interrogation according to press reports the cia used this goldsmith memorandum as justification to transport as many as a dozen detainees out of iraq the legal analysis in the memorandum is an embarrassment yet it appears to have provided the legal justification for the cia to commit war crimes
the new york times recently reported that the us plans to transfer as many as half the 550 detainees held at guantanamo bay to prisons in other countries this week a federal judge blocked the government from transferring 13 citizens of yemen until a hearing can be held on the propriety of the move lawyers for the detainees expressed concern that the prisoners would be delivered into the hands of torture
even worse last week attorney general gonzales defended the practice of rendition despite admitting that he cant fully control what other nations do and that he doesnt know whether countries have always complied with their promises
congress cant allow these shameful tactics to continue senator leahys bill is designed to prevent them it states that no person in the custody or control of the united states can be sent to another country on the state department list of countries that commit torture nor may any person be sent to a country even if it is not on the state department list where there are grounds to believe the person would be in danger of being tortured the bill states that mere diplomatic assurances that detainees will be treated humanely are not sufficient to permit a detainees transfer instead in certain circumstances the act permits delivery of the detainee where there is an actual mechanism to verify that the person will not be tortured such as by allowing unfettered access to the detainee by humanitarian organizations
the bush administrations has clearly condoned the use of torture and abuse by our own government as well as handing prisoners over to other countries for the same purpose officials have approved and used interrogation techniques that include feigning suffocation feigning drowning stress positions sleep deprivation and the use of unmuzzled dogs according to one report the methods employed by the cia are so severe that senior officials of the federal bureau of investigation have directed its agents to stay out of many of the interviews of the highlevel detainees because the fbi fears that the techniques could subject their agents to criminal lawsuits
the antirendition bill offered today is a way to start addressing the problem it deserves to pass as soon as possible torture and other abuses of prisoners in iraq afghanistan and guantanamo have done immense damage to americas standing in the world and has clearly made the war on terrorism harder to win we need to repair that damage and reclaim our national commitment to fairness and decency
 by mr ensign for himself and ms landrieu
s 657 a bill to amend title xviii of the social security act to make a technical correction in the definition of outpatient speechlanguage pathology services to the committee on finance
mr president today i introduced a bill that would expand access to speechlanguage pathology care
speechlanguage pathology or speech therapy includes services for patients with speech hearing and language disorders which result in communication disabilities speech therapy also includes the diagnosis and treatment of swallowing disorders regardless of the presence of communications disability communications disabilities most frequently affect patients who suffer from a stroke tumor head injury or have been diagnosed with parkinsons disease amyotrophic lateral sclerosis als or other neuromuscular diseases
as a result of a legislative anomaly patients cannot receive medicare coverage for speechlanguage pathology care in a private practice setting under the medicare program the same patient is able to receive such care in a hospital skilled nursing facility or rehabilitation facility this bill would not create a new benefit rather it would provide a technical correction to a section of medicare statute that originated more than 30 years ago under current law physical therapy and occupational therapy care can be received by patients in the private practice setting
in 1972 speechlanguage pathology services were added to the medicare statute under the physical therapy definition section 14 years later occupational therapy was defined under a separate section unlike speechlanguage pathology services occupational therapy services were not incorporated within the physical therapy definition as a result a patient can receive both physical and occupational therapy care in an independent practice setting the legislation i am introducing today would enable patients to likewise receive speechlanguage therapy services in private practice settings
without this legislative fix beneficiaries may confront situations in which they either do not have access to a medicarecovered setting or do not meet the requirements to receive care from other settings this can be especially problematic in rural communities with fewer hospitals skilled nursing facilities and rehabilitation facilities
for example consider an elderly patient who is discharged from a hospital but requires followup physical therapy and speechlanguage pathology care the patient would be able to obtain necessary physical therapy care in an independent practice setting but would not be able to receive necessary speechlanguage pathology care in the same setting the patient would have to see the necessary speechlanguage pathology care in another medicare setting possibly having to travel farther distances to receive such care or not receive it all
essentially the legislation i am introducing today would ensure that patients have access to speechlanguage pathology services particularly in rural areas i urge my colleagues to join me in supporting this commonsense legislation
this legislation compliments the measure i introduced last month called the medicare access to rehabilitation services act s 438 both bills ensure access to needed therapy care within the medicare program i am committed to working toward their enactment and believe that they will help medicare beneficiaries obtain the quality health care that they deserve by mr brownback for himself ms landrieu mr allard mr bunning mr burr mr chambliss mr cornyn mr craig mr crapo mr demint mr dewine mrs dole mr domenici mr ensign mr graham mr grassley mr hagel mr inhofe mr kyl mr martinez ms murkowski mr santorum mr sessions mr shelby mr thomas mr thune mr vitter mr voinovich and mr talent
s 658 a bill to amend the public health service act to prohibit human cloning to the committee on health education labor and pensions
mr president i rise to speak on the brownbacklandrieu human cloning prohibition act which we introduce today
the brownbacklandrieu human cloning prohibition act remains the only effective ban on human cloning
this legislation has passed the us house of representatives twice by large margins this bill would also bring the us into conformity with the recent vote at the united nations where the general assembly called on all member states to prohibit all forms of human cloning by a strong 84 to 34 margin
president bush has also spoken eloquently on the brownbacklandrieu human cloning prohibition act when he wholeheartedly endorsed the legislation
the president said human cloning is deeply troubling to me and to most americans life is a creation not a commodity
our children are gifts to be loved and protected not products to be designed and manufactured allowing cloning would be taking a significant step toward a society in which human beings are grown for spare body parts and children are engineered to custom specifications and thats not acceptable i strongly support a comprehensive law against all human cloning and i endorse the bill wholeheartedly endorse the bill sponsored by senator brownback and senator mary landrieu
the president could hardly have been clearer
we should take a stand against those that would turn young human beings into commodities and spare parts we should not use human life for research purposes
the legislation introduced by sen landrieu and myself along with over one quarter of the senate answers that human life should not be used for research purposes
let there be no doubt science affirms that the young human at his or her earliest moments of life is a human it is wrong to treat another person as a piece of property that can be bought and sold created and destroyed all at the will of those in power
the issue of human cloning and specifically how we treat the young human will determine the kind of future we will give to our children and grandchildren
the essential question is whether or not we will allow human beings to be produced to preordained specifications for their eventual implantation or destruction depending upon the intentions of the technicians who created them
will we create life simply to destroy it
i firmly believe that human life should be cherished and that human dignity should be protected
i also firmly believe that ethicallysound research should proceed in the search for cures the legislation that we introduce today takes a very thoughtful approach and is careful not to ban or interfere with gene therapy ivf practices or dna cell or tissue cloning other than with cloned embryos
now some of our colleagues will tell you that they oppose `reproductive cloning but then turn around and call for `therapeutic cloning or `scnt whether intentional or not to argue that there are different types of human cloning creates a distinction that simply does not exist
all human cloning is `reproductive the question is simply what do you do with the young cloned human do you implant it and bring it to birth like the sheep dolly or do you research on and kill the young human being as advocates of socalled `therapeutic cloning would have us do
any other socalled human cloning bans outside of the brownbacklandrieu human cloning prohibition act are not enforceable once the young human has been cloned you cannot distinguish it from any other human embryo produced by ivf or embodied sexual intercourse
if socalled `therapeutic human cloning proceeds and there are no laws in the us against it one of these human clones will be implanted and there is nothing we can do to stop human cloning once we reach this point
even if we detected a clonal human pregnancy nothing could be done about it any remedies or punishments would be highly unpopular and unenforceable
 by mr hatch for himself mr baucus mr grassley and mrs lincoln
s 661 a bill to amend the internal revenue code of 1986 to provide for the modernization of the united states tax court and for other purposes to the committee on finance
mr president i rise today to introduce the tax court modernization act i am joined in this legislation by the chairman and ranking democrat of the finance committee senator grassley and senator baucus and my colleague senator lincoln
the united states tax court plays an important role in our tax system however it has been years since congress has taken a good hard look at the tax court this bipartisan piece of legislation will improve this court in a number of ways and i would like to take a moment to summarize some of its provisions
first the tcma would make minor changes in the tax courts jurisdiction these are small changes that will have a big impact on the courts efficiency for example the bill would allow the tax court to hire employees on its own just as other courts do currently the tax court is forced to hire through the executive branchs office of personnel management entangling the executive power with the judicial power restoring the constitutional separation of powers in the hiring process will increase the independence of the tax court
second the tcma would improve the way that tax court judges receive retirement benefits and other nonsalary benefits i believe that tax court judges should be treated the same way that bankruptcy court of federal claims and article iii judges are treated when it comes to fringe benefits
tax court judges are often not provided with the same benefits as similarly appointed article i and article iii judges for example congress allows article iii bankruptcy and court of federal claims judges to participate in the thrift savings plan in addition to the civil service retirement system while tax court judges are ineligible to participate in this program these disparities in the treatment of our tax court judges affect the courts ability to attract and retain seasoned judges as well as talented employees
this legislation is noncontroversial and is the result of many years of work the finance committee passed the bill three separate times during the 108th congress but it unfortunately was not included in a vehicle that made it to enactment hopefully we will be able to get these provisions to the presidents desk this year
i have spent many years observing the federal judiciary i have spent many years trying to improve the judicial branch of our government and to make it the very finest court system the world has ever known i look forward to working with my colleagues on the senate finance committee on this important piece of legislation i urge my colleagues both on the finance committee and in the senate as a whole to support this legislation
i ask unanimous consent to print in the record a summary of the provisions of the us tax court modernization act there being no objection the material was ordered to be printed in the record as follows
jurisdiction of tax court over collection due process cases currently if a taxpayers underlying tax liability does not relate to income taxes or a type of tax over which the tax court normally has deficiency jurisdiction there is no opportunity for tax court review and the taxpayer must file in a district court to obtain review this provision consolidates judicial review of collection due process activity in the tax court authority for special trial judges to hear and decide certain employment status cases this provision clarifies that the tax court may authorize its special trial judges to enter decisions in employment status cases that are subject to small case proceedings under section 7436c confirmation of authority of tax court to apply doctrine of equitable recoupment the commonlaw principle of equitable recoupment permits a party to assert an otherwise timebarred claim to reduce or defeat an opponents claim if both claims arise from the same transaction this provision confirms statutorily that the tax court may apply equitable recoupment principles to the same extent as district courts and the court of federal claims tax court filing fee in all cases commenced by filing petition this provision clarifies in keeping with current tax court procedure that the tax court is authorized to impose a usd60 filing fee for all cases commenced by petition the proposal would eliminate the need to amend section 7451 each time the tax court is granted new jurisdiction amendments to appoint employees currently the tax court has to go to the executive branch the office of personnel management to change a position it is inappropriate to require the tax court to seek permission from the executive since that branch is a party commissioner of internal revenue before the tax court this change would allow the tax court to be independent in fact and perception from the executive branch while ensuring that basic employee rights protections and remedies are retained or required in an appropriate way eg whistleblower protection civil rights merit system principles etc expanded use of tax court practice fee for pro se taxpayers the tax court is authorized to charge practitioners a fee of up to usd30 per year and to use these fees to pursue disciplinary matters the provision expands use of these fees to provide services to pro se taxpayers fees could be used for education programs for pro se taxpayers annuities for survivors of tax court judges who are assassinated the reality is that many people do not like to pay taxes there is as much risk of a tax court judge being assassinated as any other federal judge the proposal would conform the treatment of tax court judges to district court judges costofliving adjustments for tax court judicial survivor annuities all federal employees have this provision except the tax court survivors of tax court judges are subject to an obsolete method of indexing life insurance coverage for tax court judges this simply codifies current office of personnel management interpretation as was previously done for district court judges cost of life insurance coverage for tax court judges age 65 or over congress established the tax court in 1969 and required that tax court judges receive the same compensation as district court judges the district court judges were given this benefit to ensure that there was no diminution of their compensation as required by the constitution this provision is in keeping with the original intent of congress modification of timing of lumpsum payment of judges accrued annual leave district court judges are allowed to receive a lumpsum payment due to the lifetime tenure of article iii judges tax court judges while they have a 15 year term effectively have a lifetime term because they are always subject to recall participation of tax court judges in the thrift savings plan the proposal would allow tax court judges to participate in thrift savings plan currently only 19 federal government employees are left out of the thrift savings plan ie tax court judges exemption of teaching compensation of retired judges for limitation on outside earned income after retirement tax court judges should have the same ability to teach as district court judges general provisions relating to magistrate judges of the tax court magistrate is more recognizable to the american public because it is the term used by article iii courts the provision changes the term special trial judge to magistrate judge of the united states tax court and provides for alignment of term of office and removal applicable to district court magistrate judges annuities to surviving spouses and dependent children of magistrate judges of the tax court this section gives magistratesspecial trial judges the same advantages as tax court judges thus ensuring a greater pool of participants in the fund retirement and annuity program for magistrate judges a retirement and annuity program more aligned with district court magistrates and the tax court judges is key for attracting and retaining qualified judges incumbent magistrate judges of the tax court the provision provides transition rules similar to those given to the district court magistrate judges provisions for recall article iii judges are self recalling ie they decide for themselves whether they are recalled in contrast tax court judges are subject mandatory recall by the chief judge these provisions authorize the recall in a manner similar to those now applicable to the regular judges of the court
mr president i rise today to support the united states tax court modernization act i am pleased to be an original cosponsor of this important legislation along with senators hatch grassley and lincoln
in 1969 congress elevated the us tax court as a federal court of record under article i of the constitution of the united states congress created the tax court to provide a judicial forum in which affected persons could dispute tax deficiencies determined by the commissioner of the internal revenue service prior to payment of the disputed amounts that means that the tax courts jurisdictional requirements are in part a recognition that lower and middle income taxpayers cannot necessarily pay the tax deficiency before taking their dispute to court
congress also closely linked the legislation governing the tax court with the laws governing the article iii district courts unfortunately the congress did not include the tax court in the changes made for article iii courts
this legislation is designed to restore parity between the tax court and article iii courts and to modernize their personnel and pension systems
i thank senator hatch for sponsoring the legislation i also want to thank former senator breaux who sponsored the legislation in the last congress and who was a strong advocate for the tax court as well as this package of modernization provisions
this modernization package is noncontroversial and long overdue in the 108th congress the finance committee passed the tax court legislation three times as a stand alone bill as part of the national employee savings and trust equity guarantee act and as part of the tax administration good government act
the finance committee intends to markup the united states tax court modernization act next month i fully expect the committee to once again unanimously pass the legislation i also hope that soon after committee action majority leader frist and minority leader reid will bring the united states tax court modernization act to the floor for swift passage
 by ms collins for herself mr carper and mr voinovich
s 662 a bill to reform the postal laws of the united states to the committee on homeland security and governmental affairs
mr president i rise today with my friend and colleague senator carper to introduce the postal accountability and enhancement act of 2005 a bill designed to help the 225yearold postal service meet the challenges of the 21st century this legislation represents the culmination of a process that began in the summer of 2002 when i introduced a bill to establish a presidential commission charged with examining the problems the postal service faces and developing specific recommendations and legislative proposals that congress and the postal service could implement
i originally introduced the postal accountability and enhancement act last may in june of 2004 the bill was unanimously reported out of the the homeland security and governmental affairs committee that bill s 2468 had the strong endorsements of the national rural letter carriers association the national association of letter carriers the national association of postmasters of the united states and the coalition for a 21st century postal service which represents thousands of the major mailers employee groups small businesses and other users of the mail it also had the strong bipartisan support of twentytwo members of the united states senate unfortunately due to a variety of factors my efforts to have the bill considered before the full senate were stalled
since last fall administration representatives have become actively engaged in postal reform efforts and have given me their commitment to working with congress to ensure passage of a reform bill this year i have every expectation that this will be the year comprehensive postal reform legislation is signed into law
it has long been acknowledged that the financial and operational problems confronting the postal service are serious at present the postal service has more than usd90 billion in unfunded liabilities and obligations which include usd18 billion in debt to the us treasury usd76 billion for workers compensation claims usd35 billion for retirement costs and as much as usd47 billion to cover retiree health care costs the government accountability offices comptroller general david walker has pointed to the urgent need for fundamental reforms to minimize the risk of a significant taxpayer bailout or dramatic postal rate increases the postal service has been on gaos highrisk list since april of 2001 the postal service is at risk of a death spiral of decreasing volume and increasing rates that lead to further decreases in volume
in december of 2003 president bush announced the creation of a bipartisan commission charged with identifying the operational structural and financial challenges facing the us postal service the president charged this commission with examining all significant aspects of the postal service with the goal of recommending legislative and administrative reforms to ensure its longterm viability
the presidents commission conducted seven public hearings across the country at which they heard from numerous witnesses on july 31 2003 the commission released its final report making 35 legislative and administrative recommendations for the reform of the postal service
as i read through the commissions report i was struck by what i considered the commissions wake up call to congress its statement that an incremental approach to postal service reform will yield too little too late given the enterprises bleak fiscal outlook the depth of current debt and unfunded obligations the downward trend in firstclass mail volumes and the limited potential of its legacy postal network that was built for a bygone era that is a very strong statement and one that challenged both the postal service and congress to embrace farreaching reforms
to the relief of many including myself the commission did not recommend privatization of the postal service instead the commission sought to find a way for the postal service to do as cochair jim johnson described to me an overwhelmingly better job under the same general structure
the postal service plays a vital role in our economy the service itself employs more than 750000 career employees less well known is the fact that it is also the linchpin of a usd900billion mailing industry that employs 9 million americans in fields as diverse as direct mailing printing catalog production paper manufacturing and financial services the health of the postal service is essential to the vitality of thousands of companies and the millions that they employ
one of the greatest challenges for the postal service is the decrease in mail volume as business communications bills and payments move more and more to the internet the postal service has experienced declining volumes of firstclass mail for three straight years this is highly significant given that firstclass mail accounts for 48 percent of total mail volume and the revenue it generates pays for more than twothirds of the postal services institutional costs
the postal service also faces the difficult task of trying to cut costs from its nationwide infrastructure and transportation network these costs are difficult to cut even though volumes may be decreasing carriers must still deliver six days a week to more than 139 million addresses
as chairman of the committee on homeland security and governmental affairs i held a series of eight hearings including a joint hearing with the house during which we reviewed the recommendations of the presidents commission the bill senator carper and i introduce today reflects what the committee learned from dozens of witnesses
first and foremost the collinscarper bill preserves the basic features of universal service affordable rates frequent delivery and convenient community access to retail postal services as a senator representing a large rural state i want to ensure that my constituents living in the northern woods or on the islands or in our many rural small towns have the same access to postal services as the people of our cities if the postal service were no longer to provide universal service and deliver mail to every customer the affordable communication link upon which many americans rely would be jeopardized most commercial enterprises would find it uneconomical if not impossible to deliver mail and packages to rural americans at rates charged by the postal service
the collinscarper bill allows the postal service to maintain its current mail monopoly and retain its sole access to customer mailboxes it grants the postal service board of governors the authority to set rates for competitive products like express mail and parcel post as long as these prices do not result in cross subsidy from marketdominant products as a safeguard our bill establishes a 30 day prior review period during which the proposed rate changes shall be reviewed by the postal regulatory commission
it replaces the current lengthy and litigious ratesetting process with a rate capbased structure for marketdominant products such as firstclass mail periodicals and library mail this would allow the postal service to react more quickly to changes in the mailing industry the rate caps would be linked to the consumer price index the goal would be to make rate increases more predictable and less frequent and to provide incentives for the postal service to operate efficiently price changes for marketdominant products would be subject to a 45 day prior review period by the postal regulatory commission
our bill would introduce new safeguards against unfair competition by the postal service in competitive markets subsidization of competitive products by marketdominant products would be expressly forbidden and an equitable allocation of institutional costs to competitive products would be required
the presidents commission recommended that the regulator be granted the authority to make changes to the postal services universal service obligation and monopoly the vast majority of the postal community however shared my belief that these are important policy determinations that should be retained by congress the collinscarper bill keeps those public policy decisions in congressional hands
the existing postal rate commission would be transformed into the postal regulatory commission with greatly enhanced authority under current law the rate commission has very narrow authority we wanted to ensure that the postal service management has both greater latitude and stronger oversight among other things the postal regulatory commission will have the authority to regulate rates for noncompetitive products and services ensure financial transparency establish limits on the accumulation of retained earnings by the postal service obtain information from the postal service if need be through the use of new subpoena power and review and act on complaints filed by those who believe the postal service has exceeded its authority members of the postal regulatory board will be selected solely on the basis of their demonstrated experience and professional standing senate confirmation of all board members will be required
to meet the presidential commissions call for increased financial transparency the collinscarper bill will require the postal service to file with the postal regulatory commission certain securities and exchange commission financial disclosure forms along with detailed annual reports on the status of the postal services pension and postretirement health obligations
the governmental affairs committee dedicated two hearings to the examination of the commissions workforcerelated recommendations the postal service is a highly labor intensive organization using usd3 out of every usd4 to pay the wages and benefits of its employees their workforce is comprised of more than 700000 dedicated letter carriers clerks mail handlers postmasters and others many of whom place great value on their right to collectively bargain our bill reaffirms that right this bill only makes changes to the bargaining process that have been agreed to by both the postal service and the four major unions we replace the rarely used factfinding process with mediation and shorten statutory deadlines for certain phases of the bargaining process
additionally the collinscarper bill corrects what i believe to be an anomaly in the federal workers compensation law that results in high costs for the postal service under the federal employees compensation act feca federal employees with dependents are eligible for 75 percent of their takehome pay tax free plus cost of living allowances in addition there is no maximum dollar cap on feca payments as a result employees often opt not to retire staying on the more generous workers compensation program permanently
according to a march 2003 audit issued by the postal services office of inspector general the postal services workers compensation rolls include 81 cases that originated 40 to 50 years ago with the oldest recipient being 102 years old the igs office found 778 cases that originated 30 to 40 years ago and 1189 cases that originated 20 to 29 years ago
the collinscarper bill works to protect the financial resources of the postal service by converting workers compensation benefits for total or partial disability to a retirement annuity when the affected employee reaches 65 years of age this change would reflect the fact that disabled postal employees would likely retire at some point were they not receiving workers compensation i would like to note that the average postal employee retires far earlier than age 65 so this is still a generous program it is important to point out that the postal service has reduced their workplace injury rate by twentyeight percent over the past three years
the collins carper bill also puts into place a threeday waiting period before an employee is eligible to receive 45 days of continuation of pay this is consistent with every states workers compensation program that requires a three to sevenday waiting period before benefits are paid
to address the presidents commissions recommendation for improved executive compensation this bill will allow the postal service to raise their overall executive compensation level from executive level 1 to that of the vice president this would bring the postal service in line with authority granted to federal agencies this new authority will be contingent upon the development of a meaningful performance appraisal system
our bill has reached an important compromise on the issue of workshare discounts the workshare program was developed by the postal service and the postal rate commission to enable customers to pay lower rates when they perform mail preparation or transportation activities the language in our bill supports the principle that workshare discounts should generally not exceed the costs that the postal service avoids as a result of the worksharing activity however the bill spells out certain circumstances under which workshare discounts in excess of avoided costs are warranted
finally our bill would repeal a provision of public law 10818 which requires that money owed to the postal service due to an overpayment into the civil service retirement system fund be held in an escrow account repealing this provision would essentially free up usd78 billion over a period of 60 years these savings would be used to not only pay off debt to the us treasury and to fund health care liabilities but also to mitigate rate increases as well in fact failure to release these escrow funds could mean for mailers a doubledigit rate increase in 2006 an expense most american businesses and many consumers are illequipped to afford
the bill would also return to the department of treasury the responsibility for funding csrs pension benefits relating to the military service of postal retirees no other agency is required to make this payment ratepayers should not be held responsible for this usd27 billion obligation
the postal service has reached a critical juncture if we are to save and strengthen this vital service upon which so many americans rely for communication and their livelihoods the time to act is now
i look forward to working with all of my colleagues in the senate and house government reform and oversight committee chairman tom davis who together with congressman john mchugh also recently introduced a postal reform bill hr 22
i ask unanimous consent that the text of the bill be printed in the record
 there being no objection the bill was ordered to be printed in the record as follows
be it enacted by the senate and house of representatives of the united states of america in congress assembled
a short titlethis act may be cited as the postal accountability and enhancement act b table of contentsthe table of contents for this act is as follows
a short titlethis act may be cited as the postal accountability and enhancement act b table of contentsthe table of contents for this act is as followssec 1 short title table of contents
a short titlethis act may be cited as the postal accountability and enhancement act b table of contentsthe table of contents for this act is as followssec 1 short title table of contentstitle i definitions postal services
a short titlethis act may be cited as the postal accountability and enhancement act b table of contentsthe table of contents for this act is as followssec 1 short title table of contentstitle i definitions postal servicessec 101 definitionssec 102 postal services
a short titlethis act may be cited as the postal accountability and enhancement act b table of contentsthe table of contents for this act is as followssec 1 short title table of contentstitle i definitions postal servicessec 101 definitionssec 102 postal servicestitle ii modern rate regulation
section 102 of title 39 united states code is amended by striking and at the end of paragraph 3 by striking the period at the end of paragraph 4 and inserting a semicolon and by adding at the end the following 5 `postal service refers to the physical delivery of letters printed matter or packages weighing up to 70 pounds including physical acceptance collection sorting transportation or other functions ancillary thereto 6 `product means a postal service with a distinct cost or market characteristic for which a rate or rates are applied 7 `rates as used with respect to products includes fees for postal services 8 `marketdominant product or `product in the market dominant category of mail means a product subject to subchapter i of chapter 36 and 9 `competitive product or `product in the competitive category of mail means a product subject to subchapter ii of chapter 36 and 10 `year as used in chapter 36 other than subchapters i and vi thereof means a fiscal year
a in generalsection 404 of title 39 united states code is amended 1 in subsection a by striking paragraph 6 and by redesignating paragraphs 7 through 9 as paragraphs 6 through 8 respectively and 2 by adding at the end the following c except as provided in section 411 nothing in this title shall be considered to permit or require that the postal service provide any special nonpostal or similar services b conforming amendments1 section 1402b1bii of the victims of crime act of 1984 98 stat 2170 42 usc 10601b1bii is amended by striking 404a8 and inserting 404a7 2 section 2003b1 of title 39 united states code is amended by striking and nonpostal
a in generalchapter 36 of title 39 united states code is amended by striking sections 3621 and 3622 and inserting the following
chapter 36 of title 39 united states code is amended by inserting after section 3629 the following
 newspeech
today i wish to commend congressman bobby rush and other members of the congressional black caucus for their work on behalf of afrocolombians the consistent advocacy of the cbc on this human rights issue has been critical to increasing consciousness and activism in the us and colombia significant progress has made through this alliance and i look forward to working with the cbc and other community groups on this issue
throughout latin america afrolatino communities remain marginalized socially economically and politically in the case of colombia the violence and disruption of the countrys 40year civil conflict have disproportionately affected afrocolombians many are now refugees in their own country after being forced to leave their homes and they face widespread racial discrimination as they try to rebuild their lives although colombias 1991 constitution granted afrocolombians territorial rights to the land they historically held these rights are now being increasingly violated as this land is taken from them with little or no economic and educational opportunities available many afrocolombian youths have turned to coca cultivation or joined guerrilla forces
with the rise of afrocolombian advocacy groups and ngos in colombia i believe it is possible to foster meaningful partnerships and alliances for positive change in this region in addition to the cbc there are many members of the religious community in my home state of illinois and across our country who are working on behalf of afrocolombians i commend them on their dedication to this important cause together we can and will make a difference
 newspeech
there being no objection the bill was ordered to be printed in the record as follows
be it enacted by the senate and house of representatives of the united states of america in congress assembled
this act may be cited as the rural schools geography act
congress makes the following findings 1 there are significant differences between urban and rural school districts with regard to student transportation distances between schools and school districts and school of choice options local educational agencies leas in rural areas often have only 1 school servicing a particular grade level and the distance between these schools is often much greater than in urban areas these differences are not addressed by existing guidelines under the elementary and secondary education act of 1965 2 in 2000 rural schools those in communities with populations below 2500 taught 32 percent of the children in the united states but rural schools accounted for usd5670000000 of the nations spending on school transportation or nearly half of such spending 3 rural transportation costs perpupil are double that of urban transportation costs as a percentage of total spending rural areas spend 77 percent more than urban areas for education transportation 4 commutes in rural areas are much more likely to be on rougher unpaved roads this not only undermines the physical health of the students but makes transportation during poor weather much more difficult or impossible students with longer commutes are more likely to miss school because of inclement weather school attendance is an important factor in school performance 5 school students who have long commutes actively avoid advanced and highlevel courses because they do not have time for the extra homework this selfimposed restriction retards maximization of educational potential 6 students with long commutes are less likely to engage in inhome and outofhome activities such as family dinners afterschool jobs and athletic or musical extracurricular activities participation in these activities benefits overall educational progress 7 section 1116b10c of the elementary and secondary education act of 1965 instructs that the lowest achieving children be given priority for outofdistrict transportation thus the negative impacts of long commutes disproportionately affect the very students who need the most help
section 1116 of the elementary and secondary education act of 1965 20 usc 6316 is amended by adding at the end the following i geography limitsnotwithstanding subsections b and c a local educational agency shall not be required to provide a student the option to transfer to another school pursuant to this section if providing the option is impractical due to the distance to be traveled a geographical barrier or hazard the duration of the travel or an unusually high cost of travel
the secretary of education not later than 180 days after the date of enactment of this act shall promulgate such regulations as the secretary determines necessary to implement this act
 by mr obama for himself and mr inouye
s 697 a bill to amend the higher education act of 1965 to improve higher education and for other purposes to the committee on health education labor and pensions
mr president i rise today to introduce the higher education opportunity through pell grant expansion act of 2005 or hope act
right now in schools playgrounds and backyards across america children are dreaming about what they want to be when they grow up as tomorrows astronauts doctors and teachers dream about their futures their parents know that so many of those dreams are dependent on a college diploma
the families i have met in illinois are worried that they might not be able to give their kids a chance at that diploma everywhere i go i hear the same story we work hard we pay our bills we cut corners and we put away savings but we just dont know if it is going to be enough when the tuition bill comes in the mail
the facts and statistics are not encouraging college tuition is rising at a stunning rate of almost 10 percent a year and over the last 25 years it is gone up an astounding 519 percent because of these rising prices over 200000 students were priced out of a college education last year
in a country with so much wealth and opportunity for education it is difficult to imagine there are parents who are forced to say to their kids were sorry we cant afford to send you to college none of us in the senate should rest until those parents can start saying yes to their kids
this bill would start us down that path by increasing access to pell grants today these needbased awards are used by 53 million undergraduate students to fund their education unfortunately the awards just havent kept up with the rising price of tuition or even inflation as a result the current usd4050 pell grant maximum is usd700 less in real terms than the maximum grant 30 years ago pell grants now cover only 23 percent of the total cost of the average 4year public college
the hope act would correct this problem by raising the pell grant maximum to usd5100 and it would continue to raise this maximum in future years to keep up with inflation the bill also would make sure that no student sees a reduction in pell grant assistance due to recent changes in the eligibility formula
because working families are already burdened with too many taxes this bill would not add to the deficit or raise a dime of taxes instead it will close two loopholes that guarantee banks and private lenders an additional usd2 billion in taxpayer subsidies every year on top of the interest that college students and their families are already paying on their loans in a country where 200000 students were priced out of college last year our tax dollars shouldnt be spent subsidizing banks that are already making record profits
when our children dream about their future they need to know those dreams are within their reach a college education forms the foundation of the opportunity society that will keep this country strong and growing in the 21st century i know we can work together to get this done and i look forward to doing so
i urge my colleagues to support the hope act
i ask unanimous consent that the text of the bill be printed in the record
 there being no objection the bill was ordered to be printed in the record as follows
be it enacted by the senate and house of representatives of the united states of america in congress assembled
this act may be cited as the higher education opportunity through pell grant expansion act
a findingsthe senate makes the following findings 1 federal pell grants are needbased and are used by 5300000 undergraduate students to fund their college educations 2 over 90 percent of federal pell grant recipients come from a family with a combined income of less than usd40000 3 because of the rising cost of college tuition the maximum federal pell grant amount of usd4050 for academic year 20042005 is usd700 less in real terms than the maximum federal pell grant amount for academic year phone 4 federal pell grants for academic year 20032004 cover only 23 percent of the total cost of the average 4year public college b sense of the senateit is the sense of the senate that 1 eligible undergraduate students should receive the maximum federal pell grant amount established by the amendment made by section 3b of this act and 2 sufficient funds should be appropriated to allow the awarding of the maximum federal pell grant amount for which students are eligible pursuant to the amendment made by section 3b of this act
a appropriation of funds for federal pell grantsin addition to any amounts otherwise appropriated to carry out subpart 1 of part a of title iv of the higher education act of 1965 20 usc 1070a for the fiscal year ending september 30 2006 there are authorized to be appropriated and there are appropriated out of any money in the treasury not otherwise appropriated for the fiscal year ending september 30 2006 for carrying out such subpart 1 an additional usd2000000000 b authorization amount and maximum federal pell grant section 401b2a of the higher education act of 1965 20 usc 1070ab2a is amended to read as follows 2ai the amount of a federal pell grant for a student eligible under this part shall be usd5100 for academic year 20052006 less an amount equal to the amount determined to be the expected family contribution with respect to that student for that year ii the secretary shall cumulatively adjust the amount in clause i every 2 academic years beginning with academic year 20062007 to account for any percentage increase in the consumer price index for all urban consumers published by the bureau of labor statistics of the department of labor
notwithstanding any other provision of law the annual updates to the allowance for state and other taxes in the tables used in the federal needs analysis methodology to determine a students expected family contribution for the award year 20052006 under part f of title iv of the higher education act of 1965 20 usc 1087kk et seq published in the federal register on thursday december 23 2004 69 fed reg 76926 shall not apply to a student to the extent the updates will increase the students expected family contribution under such part f
section 438b2b of the higher education act of 1965 20 usc 10871b2b is amended by striking clause v and inserting the following v this subparagraph shall not apply to i any loan made or purchased after the date of enactment of the higher education opportunity through pell grant expansion act ii any loan that had not qualified before such date of enactment for receipt of a special allowance payment determined under this subparagraph or iii any loan made or purchased before such date of enactment with funds described in the first or second sentence of clause i if
section 438b2b of the higher education act of 1965 20 usc 10871b2b is amended by striking clause v and inserting the following v this subparagraph shall not apply to i any loan made or purchased after the date of enactment of the higher education opportunity through pell grant expansion act ii any loan that had not qualified before such date of enactment for receipt of a special allowance payment determined under this subparagraph or iii any loan made or purchased before such date of enactment with funds described in the first or second sentence of clause i ifaa the obligation described in the first such sentence has after such date of enactment matured or been retired or defeased or bb the maturity date or the date of retirement of the obligation described in the first such sentence has after such date of enactment been extended
 by mr baucus for himself mr bunning mr johnson mr talent and mr craig
s 702 a bill to amend the internal revenue code of 1986 to repeal the occupational taxes relating to distilled spirits wine and beer to the committee on finance
mr president it is with great pleasure that i join my colleagues senators bunning johnson and talent today in introducing legislation that will repeal the special occupational tax on taxpayers who manufacture distribute and sell alcoholic beverages
the special occupational tax is not a tax on alcoholic products but rather operates as a license fee on businesses the tax is imposed on those engaged in the business of selling alcohol beverages believe it or not this tax was originally established to help finance the civil war that war is over and this inequitable tax has outlived its original purpose repealing the sot will also simplify the tax code for thousands of small businesses
the sot on alcohol dramatically increased during the budget process in 1988 and has unfairly burdened business owners across the country since from thompson falls to sidney from chinook to billings small businesses are burdened with yet another tax in the form of the sot according to the alcohol and tobacco tax and trade bureau there are 426193 locations nationwide that pay the sot every year including 399657 retailers these retail establishments account for usd99 million out of usd103 million collected in sot revenues
in montana there are 2969 locations which together pay nearly usd1 million in the sot every year seasonal resorts in whitefish and yellowstone mom and pop convenience stores in butte and bowling alleys flower shops and restaurants across montana and the united states pay the federal government over usd100 million per year for the privilege of running businesses that sell beer wine or alcoholic beverages
the sot is extremely regressive retailers must annually pay usd250 per location wholesalers pay usd500 vintners and distillers pay usd1000 because the sot is levied on a per location basis a sole proprietorship must pay the same amount as one of the nations largest retailers and locallyowned chains having to pay per location would have to pay as much as if not more than the nations largest single site brewery this is not what congress had in mind 150 years ago and i dont believe its a situation we want today
repealing the sot on alcohol is supported by a broadbased group of business organizations and enjoys widespread bipartisan support on capitol hill last year we made progress in ending this burdensome tax on small businesses we repealed the tax for three years more can be done business owners across the united states deserve assurance that they wont be hit with this antiquated tax down the line
the legislation preserves the ttbs recordkeeping requirements while removing the agencys enforcement burden and will save over usd2 million per year the gao examined sot efficacy several times and found it fundamentally flawed the joint committee on taxation called for the elimination of sot in its june 2001 simplification study
more than 90 percent of all sot revenue comes from retailers a great majority of those are small businesses our small business sector is a great strength of our economy president bush has said that the best way to encourage job growth is to let small businesses keep more of their own money so they can invest in their business and make it easier for somebody to find work repealing the sot would provide an immediate and visible tax cut to small business owners
in recent months there has been much talk of tax reform inside the beltway president bush has made tax reform one of his key priorities and established a panel that will make recommendations to the department of treasury for a better tax system getting rid of a tax that has outlived its original purpose is one small step toward reform that makes sense for montana and our country we urge our colleagues to join us in this endeavor
i ask unanimous consent that the text of the bill be printed in the record
 there being no objection the bill was ordered to be printed in the record as follows
be it enacted by the senate and house of representatives of the united states of america in congress assembled
a repeal of occupational taxes 1 in generalthe following provisions of part ii of subchapter a of chapter 51 of the internal revenue code of 1986 relating to occupational taxes are hereby repealed a subpart a relating to proprietors of distilled spirits plants bonded wine cellars etc b subpart b relating to brewer c subpart d relating to wholesale dealers other than sections 5114 and 5116 d subpart e relating to retail dealers other than section 5124 e subpart g relating to general provisions other than sections 5142 5143 5145 and 5146 2 nonbeverage domestic drawbacksection 5131 of such code is amended by striking on payment of a special tax per annum 3 industrial use of distilled spiritssection 5276 of such code is hereby repealed b conforming amendments 1a the heading for part ii of subchapter a of chapter 51 of the internal revenue code of 1986 and the table of subparts for such part are amended to read as follows
a repeal of occupational taxes 1 in generalthe following provisions of part ii of subchapter a of chapter 51 of the internal revenue code of 1986 relating to occupational taxes are hereby repealed a subpart a relating to proprietors of distilled spirits plants bonded wine cellars etc b subpart b relating to brewer c subpart d relating to wholesale dealers other than sections 5114 and 5116 d subpart e relating to retail dealers other than section 5124 e subpart g relating to general provisions other than sections 5142 5143 5145 and 5146 2 nonbeverage domestic drawbacksection 5131 of such code is amended by striking on payment of a special tax per annum 3 industrial use of distilled spiritssection 5276 of such code is hereby repealed b conforming amendments 1a the heading for part ii of subchapter a of chapter 51 of the internal revenue code of 1986 and the table of subparts for such part are amended to read as followspart ii miscellaneous provisions
 newspeech
mr president i ask unanimous consent that the order for the quorum call be rescinded
 newspeech
mr president i rise today to urge my colleagues to think about the implications of what has been called the nuclear option and what effect that might have on this chamber and on this country i urge all of us to think not just about winning every debate but about protecting free and democratic debate
during my senate campaign i had the privilege and opportunity to meet americans from all walks of life and both ends of the political spectrum they told me about their lives about their hopes about the issues that matter to them and they also told me what they think about washington
because my colleagues have heard it themselves i know it will not surprise many of them to learn that a lot of people do not think much gets done around here on issues about which they care the most they think the atmosphere has become too partisan the arguments have become too nasty and the political agendas have become too petty
while i have not been here too long i have noticed that partisan debate is sharp and dissent is not always well received honest differences of opinion and principled compromise often seem to be the victim of a determination to score points against ones opponents
but the american people sent us here to be their voice they understand that those voices can at times become loud and argumentative but they also hope we can disagree without being disagreeable at the end of the day they expect both parties to work together to get the peoples business done
what they do not expect is for one party be it republican or democrat to change the rules in the middle of the game so they can make all the decisions while the other party is told to sit down and keep quiet
the american people want less partisanship in this town but everyone in this chamber knows that if the majority chooses to end the filibuster if they choose to change the rules and put an end to democratic debate then the fighting the bitterness and the gridlock will only get worse
i understand that republicans are getting a lot of pressure to do this from factions outside the chamber but we need to rise above the ends justify the means mentality because we are here to answer to the people all of the people not just the ones who are wearing our particular party label
the fact is that both parties have worked together to confirm 95 percent of this presidents judicial nominees the senate has accepted 205 of his 214 selections in fact we just confirmed another one of the presidents judges this week by a vote of 95 to 0 overall this is a better record than any president has had in the last 25 years for a president who received 51 percent of the vote and a senate chamber made up of 55 percent of the presidents party i would say that confirming 95 percent of their judicial nominations is a record to be proud of
again i urge my republican colleagues not to go through with changing these rules in the long run it is not a good result for either party one day democrats will be in the majority again and this rule change will be no fairer to a republican minority than it is to a democratic minority
i sense that talk of the nuclear option is more about power than about fairness i believe some of my colleagues propose this rule change because they can get away with it rather than because they know it is good for our democracy
right now we are faced with rising gas prices skyrocketing tuition costs a record number of uninsured americans and some of the most serious national security threats we have ever had while our bravest young men and women are risking their lives halfway around the world to keep us safe these are challenges we all want to meet and problems we all want to solve even if we do not always agree on how to do it but if the right of free and open debate is taken away from the minority party and the millions of americans who ask us to be their voice i fear the partisan atmosphere in washington will be poisoned to the point where no one will be able to agree on anything that does not serve anybodys best interest and it certainly is not what the patriots who founded this democracy had in mind we owe the people who sent us here more than that we owe them much more
mr president i suggest the absence of a quorum
 newspeech
mr president i ask that the quorum call be dispensed with
 newspeech
mr president today i am offering an amendment to the fiscal year 2005 emergency supplemental which i am pleased to announce is being cosponsored by senators corzine bingaman and graham this amendment would meet certain needs of our injured service members in recognition of the tremendous sacrifice they have made in defense of our country
the other day i had the opportunity to visit some of our wounded heroes at walter reed army medical center i know many of you have made the same trip i heard about their visits but there is nothing that can fully prepare you for what you see when you take that first step into the physical therapy room
these are kids in there our kids the ones we watched grow up the ones we hoped would live lives that were happy healthy and safe these kids left their homes and families for a dangerous place halfway around the world after years of being protected by their parents these kids risk their lives to protect us now some of them have come home from that war with scars that may change their lives forever scars that may never heal yet they sit there in the hospital so full of hope and still so proud of their country they are the best that america has to offer and they deserve our highest respect and they deserve our help
recently i learned that some of our most severely wounded soldiers are being forced to pay for their own meals and their own phone calls while being treated in medical hospitals up until last year there was a law on the books that prohibited soldiers from receiving both their basic subsistence allowance and free meals from the military basically this law allowed the government to charge our wounded heroes for food while they were recovering from their war injuries thankfully this body acted to change this law in 2003 so that wounded soldiers would not have to pay for their meals but we are dealing with a bureaucracy here and as we know nothing is ever simple in a bureaucracy so now because the department of defense does not consider getting physical rehabilitation or therapy services in a medical hospital as being hospitalized there are wounded veterans who still do not qualify for the free meals other veterans receive after 90 days even those classified as hospitalized on an outpatient status lose their free meals as well
also while our soldiers in the field qualify for free phone service injured service men and women who may be hospitalized hundreds or thousands of miles from home do not receive this same benefit for soldiers whose family members are not able to take off work and travel to a military hospital hearing the familiar voice of mom or dad or husband or wife on the other side of the phone can make all the difference in the world yet right now our government will not help pay for these calls and it will not help pay for these meals
now think about the sacrifices these young people have made for their country many of them literally sacrificing life and in some cases limb now at usd830 a meal they could end up with a usd250 bill from the government that sent them to war and they could get that bill every single month this is wrong and we have a moral obligation to fix it the amendment i am offering today will do this
the amendment will expand the group of hospitalized soldiers who cannot be charged for their meals to include those service members undergoing medical recuperation therapy or otherwise on medical hold the number of people affected by this amendment will be small only about 4000 service members are estimated to fall under the category of nonhospitalized the amendment is retroactive to january 1 2005 in an effort to provide those injured service members who may have already received bills for their meals with some relief from these costs
the amendment will also extend free phone service to those injured service members who are hospitalized or otherwise undergoing medical recuperation or therapy i am very proud this amendment is supported by the american legion and i hope my colleagues will join them in that support i ask all of my colleagues to join me in supporting this amendment it should be something that is very simple for us to do these are our children and they risked their lives for us when they come home with injuries we should be expected to provide them the best possible service and the best possible support this is a small price to pay for those who have sacrificed so much for their country
i want to mention and extend my thanks to the senior senator from alaska and my colleague from mississippi for working with me on this issue i am hoping that we can reach an agreement on this bill
 newspeech
yes i do dod currently charges soldiers usd830 per day for meals at the nondiscounted rate so if all the eligible soldiers ate all of their meals at military facilities through the end of this fiscal year the amendment would cost about usd102 million now that is probably a high estimate because my expectation would be these wounded soldiers would not be eating all of their meals at the hospital so it would probably end up being lower but the upper threshold would be usd102 million
 newspeech
i thank my colleague the senator from mississippi for that offer and i believe all of us feel the same way these are the soldiers that are most severely wounded we want to take the very best care of them and i very much appreciate the consideration of the senator from mississippi
 newspeech
mr president i thank the senior senator from alaska for joining me to discuss an issue of growing importance for our service members wounded in iraq and afghanistan
 newspeech
recently usa today reported that many of our injured soldiers are returning from iraq with a condition known as traumatic brain injury or tbi even though new technology and better body armor are helping soldiers survive bomb and rocket attacks the blasts are still causing brain damage to them as of january 437 cases have been diagnosed in army hospitals alone and some doctors are saying that it could become the signature wound of the iraq war
tbi is the greatest risk factor for developing epilepsy in fact a study of vietnam vets showed that 51 percent of those who suffered tbi went on to develop this disorder that is why i filed an amendment to provide usd1 million to the department of defense peer reviewed medical research program for epilepsy research including research on the relationship between tbi and epilepsy the epilepsy foundation of america supports the amendment
however i understand that this important issue is more appropriately addressed in the fiscal year 2006 appropriations process with that understanding i will not offer the amendment at this time
 newspeech
i look forward to working with the senator from alaska on this issue because epilepsy is a disorder that remains latent for many years it is important that we work now to better understand the relationship between tbi and epilepsy and prevent the onset of epilepsy in these service members
 newspeech
mr president i speak in favor of the amendment offered by the distinguished chairman and ranking member of the veterans affairs committee
a few weeks ago i met with sergeants ryan kelly jeremy feldbusch and heath calhoun all of whom had recently returned from iraq they served their country bravely in battle and in doing so each of these men sustained a disabling injury that will change their lives forever
when they came home it would have been easy for them to go about their own business or feel sorry for themselves
but they did not instead they decided that their service to our country would not end on the battlefields of iraq they would speak out for their fellow soldiers the ones who also may come home without a leg or an arm or their sight but may not have the resources to carry on and support their families
this amendment is their tribute to their brothers and sistersinarms
for only about usd3 per month it allows service members to purchase group disability insurance that would award them a maximum of usd100000 if they are deemed seriously injured for disabled veterans who may not be able to work when they come home this insurance could help them obtain longterm care send their kids to school or simply make sure that they can pay the bills and still put food on the table it wont cost the government a dime it simply needs our approval to allow it to happen
the blessings of modern technology have saved the lives of many service members who would otherwise have died from their wounds yet it also means there will be more wounded who need care every single one of us has a fundamental moral duty to take care of those men and women whove sacrificed to safeguard our freedom this amendment offers us one way to do that and i thank senators craig and akaka for their cooperation in moving this issue forward mr akaka mr president i am pleased to support this important and timely amendment
this amendment will go far to ease the financial burden that is placed on a service member and his or her loved ones as a result of traumatic injury between usd25000 and usd100000 will be paid to service members who suffer such injuries based on severity of injury
service members and their families face heavy financial burdens while hospitalized and prior to being medically discharged from the military this effort will help lessen the burden that exists on service members and their families before va benefits kick in
importantly to qualify for this necessary benefit our soldiers sailors airmen and marines do not have to do any additional paperwork they are automatically enrolled in this program by virtue of being a participant in the servicemembers group life insurance program
the insurance premium will cost the service member approximately usd1 a month and will be determined by the secretary of veterans affairs
this insurance policy is meant to supplement and not take the place of existing dod and va benefits this amendment is intended to fill a gap assistance to service members and their families during recovery from a traumatic injury in no way should anyone view this as a precedence for shifting costs to a service member
 newspeech
i see that the distinguished ranking member of the state and foreign operations subcommittee senator leahy is here on the senate floor i am wondering it he would take just a moment to discuss with me the critical issue of the avian flu
mr president an outbreak of the avian flu would be an international calamity in this age when you can get on a plane in bangkok and arrive in chicago or burlington in hours we must face the reality that this threat is not a problem isolated half a world away but is one that could affect people in illinois vermont and all across america the director of the centers for disease control recognized the grave consequences this virus could pose to international health when she recently stated that this is a very ominous situation for the globe this is the most important threat we are facing right now it is something that is clearly an emergency and is appropriately addressed in the iraq supplemental
at this point humans contract the virus overwhelmingly by coming into contact with infected animals and once contracted the virus is extremely deadly a 65 to 75 percent mortality rate for humans especially because there is no proven vaccine for the h5n1 strain further effective treatments for this strain of the virus are not widely available and must be delivered within 24 hours
the recent trends with respect to the spread of the avian flu are very alarming over the last few months there is growing evidence which suggests that the virus may be mutating and could eventually result in a form that is transmittable from human to human if this were to occur it could cause the deaths of millions of people seriously damage economic activity in southeast asia and cause panic and instability throughout the region moreover because of the dynamic nature of southeast asia with all sorts of commerce and transport in and out of the region the virus would likely spread around the world including to the united states in a matter of hours or days
i would ask my good friend the senior senator from vermont who has a long history of leadership on international health issues for his assessment of what needs to be done
 newspeech
just for the record the usd100 million figure is important for our purposes here today before the appropriations committee put together the supplemental we discussed the importance of immediately addressing the avian flu before the situation spirals out of control and that usd25 million is an appropriate amount to deal with this critical emergency i am correct
 newspeech
mr president i rise to speak about the budget resolution i think that the budget process is one of those issues that doesnt translate too well outside of washington most americans know it involves a lot of fighting over a lot of numbers but other than that what goes on here is largely obscured from public view
sometimes i think that is why washington gets away with passing a budget like this one
see a budget is fundamentally about choices not just choosing where to allocate funding but where to place our most important values and priorities and there are no free lunches here either
we must choose do we want to run up our debt with tax cuts and give the bill to our children or do we want to get our fiscal house back in order do we want to hand more corporate tax breaks to companies with record profits while handing our veterans higher health care bills or do we want to keep our promise to those willing to sacrifice in defense of our freedom these are the very real choices a budget asks us to make and they have equally real consequences on peoples lives
when we cut usd10 billion from medicaid what does that say to the 53 million americans 25 million of whom are children who rely on this program as their only source of health care the thousands of seniors and kids in illinois who will be turned away from a doctors office when they get sick because we chose to end their coverage what does that say to those americans
when we cut out a proposal to increase pell grants that will send more kids to college what does that say to the 220000 who didnt attend last year for the simple reason that they couldnt afford it what does it say to our kids who will have to compete with kids in india and china for jobs when we cut out proposals to provide new math and science teachers
when we cut usd351 million in funding for veterans nursing homes and eliminate usd100 million in state grants for va facilities what does that say to the veterans who have sacrificed for this nation but who cannot seem to get this nation to sacrifice anything for them what does it say to these veterans when we provide only around 100 new employees to deal with a backlog of 480000 compensation and pension claims that havent even been looked at yet and what does it say to the men and women who are willing to fight and die for this country when we are not doing much about the nearly 300000 veterans who go to sleep without a roof over their heads every single night what does that say to them what do we say to them
maybe we tell them that the budget process is complicated that we are in some tough times and have tough decisions to make that we are not happy about the choices but we have a huge deficit and no money left to spend
or maybe we tell them that we couldnt afford to do anything about these important problems because we chose to give out over usd100 billion in tax breaks usd100 billion on top of the trillions in tax cuts we have already given out most of which have gone to those few who already have so much
these tax cuts have driven us into the deepest debt in americas history and squandered our opportunity to deal with social security medicare medicaid and the true costs of the war in iraq and yet when we try to do something fiscally responsible like pass an amendment that forces congress to pay as it goes we get rejected by those who want to keep borrowing and spending
right now there are millions of middleclass families who are deeply in debt and struggling to pay the bills this body couldnt wait to pass a bankruptcy bill to make sure they paid every penny of that debt and yet it has now maxed out the countrys credit card many times over what does this say to americans about taking responsibility for themselves
a budget is about choices and i believe the choices we have made here are just plain wrong
in this budget we should be meeting our responsibilities to our fellow americans while still paying down the debt so we can meet our responsibilities to our children too it doesnt have to be eitheror we can do both as long as we get our priorities in order many of us democrats and republicans have been trying to do this during the budget process unfortunately the final product does not reflect those efforts in the future i hope that both parties can find a way to come together and make sure that americas budget reflects americans priorities
 newspeech
mr president i ask unanimous consent that the order for the quorum call be dispensed with
 newspeech
mr president i rise today to speak about the fiscal year 2005 emergency supplemental appropriations bill every day in iraq and afghanistan the men and women of the us armed forces risk their lives to defend ours they are completing a mission they did not ask for and in iraq a mission that is longer and more dangerous than they were ever told yet amid roadside explosions insurgent attacks and the loss of some of their closest friends they wake up each day and do their jobs they wake up each day and do whatever it takes to leave a democratic iraq for a free iraqi people
this bill is a way for us to support these efforts with its passage i sincerely hope our troops will receive all the support and all the equipment they need to do their job with its passage i hope we do not hear any more stories about troops driving convoys with unarmored humvees or about troops going into battle with armor their parents had to send them from home for their birthday and i sincerely hope this money will be used to train more iraqis to secure their own country so we can bring home our young people safe and secure
i particularly thank the chairman and ranking member of the appropriations committee for working with me on several other emergency spending needs
i say to senator cochran i appreciate that this bill provides usd25 million for the prevention of the avian flu as some of you may have read the number of cases in southeast asia is increasing and there is serious concern that this virus could mutate and jump from continent to continent potentially causing a pandemic that could kill millions of people we have to work proactively to prevent such a pandemic and i appreciate the support from the committee chairman as well as the administration on this issue
also included in the bill is an amendment i sponsored with my friend from south carolina senator graham this amendment will ensure that our injured service members who remain under medical care but are no longer hospitalized will not have to pay for their meals while receiving therapy i thank the graciousness of senator cochran for adopting that amendment on the floor without debate
i also joined with senator durbin to address the security needs of our judiciary as some of my colleagues know a federal judge in illinois recently suffered a tragic loss the murder of her mother and her husband this bill provides necessary funding for the us marshal service to step up its security for our federal judges
i commend all those who have been involved including the chairman for crafting a number of important measures in this bill i wish that i could without any further statement simply say how proud i am of our troops and move on with the supplemental unfortunately this bill also includes some immigration provisions known as real id that cause me enormous concern although i will certainly vote for the conference report because of the good measures i have already discussed it is important to state for the record my serious reservations about real despite the fact that almost all of these immigration provisions are controversial the senate did not conduct a full hearing or debate on any one of them while they may do very little to increase homeland security they come at a heavy price for struggling state budgets and our values as a compassionate country the drivers license provisions in real id for example will cost an estimated usd100 million over 5 years states will have to bear the majority of these costs at a time when budgets are tight i dont think we should be outsourcing our homeland security to states that cant afford it
the cost to our nations legacy as a refuge for asylum seekers is also heavy conferees were able to improve some aspects of real id including increasing the limit on the number of foreigners who can apply for asylum in the united states but other provisions intended to eliminate fraudulent asylum applications may end up denying asylum to people who deserve to receive it
these are costs that call for greater examination as a sovereign country we have the right to control and identify those who enter and exit i have worked with my colleagues to support hundreds of millions of dollars for more border patrol agents to help exercise that right but controlling immigration is a federal responsibility it always has been and it should not come at the expense of state budgets or basic civil liberties we should have more time to examine and debate the real id provisions as part of comprehensive immigration reform
these provisions currently in the bill are opposed by religious organizations civil liberties groups civil rights organizations church groups and hundreds of other groups the legitimate concerns of these groups have not been properly aired in the senate i am aware of the fact that the real id act despite what i say despite my reservations will become law it will become law not because it is the right thing to do but because the house majority has abused its privilege to attach this unexamined bill to mustpass legislation this is highly inappropriate and i hope that all of the senate will agree to highlight and correct the deficiencies of these immigration provisions in the year to come
i yield the floor
 newspeech
mr president recently more 1200 students from across the united states visited washington dc to take part in the national finals of we the people the citizen and the constitution the most extensive educational program in the country developed specifically to educate young people about the us constitution and bill of rights
i applaud the class from maine township high school in park ridge that skillfully represented the great state of illinois in this prestigious national event through their knowledge of the us constitution these outstanding students won the statewide competition and made illinois proud in the national competition here in our nations capitol
congratulations to nicole calabrese carly calkins emily cottrell keith dent katie eichstaedt alyssa engle katie funkhouser kathryn futris jacqueline heffernan kevin kane erin keating maddie kiem dan leung mike mangialardi kelly mckenna ryan morrisroe allison mueller jessica newton james pikul elizabeth poli ashley rezaeizadeh alex schallmo jimmy skuros ryan stegink dan widing meredith wisniewski and their teacher dan states i commend each and everyone of you for your hard work
while in washington these students participated in a 3day academic competition that simulated a congressional hearing in which they testified before a panel of judges students demonstrated their knowledge and understanding of constitutional principles and had opportunities to evaluate take and defend positions on relevant historical and contemporary issues
i wish these students the best of luck in their future endeavors and applaud their outstanding achievement
 newspeech
mr president we have all heard from folks back home about the high price of gasoline when you pull into a gas station to fill up your tank youre now paying some of the highest prices of all time
this amendment is designed to do something about that by promoting a choice at the pump that will allow consumers to choose a fuel that today is 50 cents per gallon cheaper than regular gasoline
thats why i would like to thank the chairman of the finance committee senator grassley and the ranking member of the committee senator baucus for their advocacy of this amendment i also want to thank the manager of the transportation bill chairman inhofe for working with us on this proposal these senate leaders are all committed to addressing high gas prices and their work on this amendment is an example of that commitment
i would like to thank my fellow authors of this amendment senator talent as well as my distinguished colleague from illinois senator durbin for their hard work in getting this provision passed and i thank the cosponsors of this amendment also longtime supporters of ethanol senators lugar harkin bayh coleman salazar dayton and nelson of nebraska
and of course i would like to thank the excellent staff work of elizabeth paris matt jones and russ sullivan on behalf of this provision
i am sure many of us in this chamber and many watching these proceedings would jump at the chance to fill our cars and trucks with fuel that is 50 cents cheaper than current prices what many consumers may not know is that that option is available today it is known as e85 a fuel comprised of 85 percent ethanol and i suspect most americans would agree that a fuel made of 85 percent midwestern corn is a lot more desirable than one made from 100 percent middle eastern oil
right now there are millions of cars and trucks that can run on e85 they are known as flexible fuel vehicles and the auto industry is turning out hundreds of thousands of them every year these cars and trucks arent more expensive to operate than regular cars in fact for just a onehundreddollar adjustment even regular cars could run on e85 and if e85 is good enough for the indianapolis 500 which just announced their cars will run on this fuel then you can be sure that e85 will work great in a flexible fuel vehicle
the only problem now is our short supply of e85 fuel stations while there are more than 180000 gas stations all over america only about 400 offer e85
the amendment adopted by the senate today addresses this problem by providing a tax credit to encourage the installation of more e85 fuel pumps at your local gas station its enactment will not only give motorists another option at the pump it will also send a clear message that the us senate is serious about reducing our countrys dangerous dependence on imported oil
again i thank my colleagues who have worked to adopt this amendment to help make america energy independent
 newspeech
mr president like my distinguished colleagues from arkansas and from mississippi i rise in opposition to the amendment offered by my distinguished colleague from virginia i strongly urge my colleagues to preserve the seatbelt program as it is written in the transportation reauthorization bill
this provision in the underlying bill gives states that pass primarily seatbelt laws a onetime incentive grant from that states annual traffic safety grant apportionment the purpose of this incentive is to encourage states to take specific action passage of a primary seatbelt law that will save more lives
as it so happens my state of illinois passed a primary seatbelt law in response to this incentive i know we did it in response to these incentives because i was the chief sponsor of passage of the primary seatbelt law
the same thing happened in delaware the same thing happened in tennessee you know what it works and it works faster and cheaper than any other method in terms of ensuring that people wear safety belts and save lives
it is amazing we have to keep saying this but seatbelts save lives and primary seatbelt laws save more lives the national highway traffic safety administration predicts if every state enacted primary seatbelt laws more than 1000 lives could be saved each year and 17000 injuries could be prevented seatbelt use is 11 percentage points higher in states with primary enforcement laws than in those states where laws provide for secondary enforcement and states changing from secondary to primary enforcement have seen 10 to 15 percentage point increases in usage
beyond the facts and statistics this is an issue that makes sense we should not have to just hope people wear seatbelts just as we should not have to hope they obey speed limits or hope they stop at red lights we should do what we can to make sure people will wear seatbelts that will keep them alive we teach our children to wear seatbelts when they get into a car and we all hope they listen to mom and dad and do it when we are not there but wouldnt we feel better if we knew our laws in our communities were helping to make that happen doesnt it makes sense for the federal government to maintain a consistent message on seatbelt use not through a mandate but through a simple incentive
the national safe kids campaign thinks so mothers against drunk drivers thinks so they endorse and prefer the federal incentive as written in the underlying bill
finally a federal incentive is also a federal commitment when the federal government makes a commitment and states respond accordingly then the federal government needs to keep its word one of the points that was raised by senator allen in sponsoring this amendment was that in virginia at least there seems to be some concern that primary seatbelt enforcement would result potentially in an increase in racial profiling in virginia
as somebody whose community on the south side of chicago is fairly familiar with racial profiling and who hears anecdotes each day from africanamerican drivers who believe they may have been profiled i am certainly sensitive to senator allens point as it turns out though part of the way we were able to solve this in illinois was to couple a primary seatbelt enforcement law with a racial profiling law that would ensure we were keeping track as to how traffic enforcement was taking place and to make certain it was being done in a nondiscriminatory fashion
this was the bargain that was struck at the local level the notion that we would have a primary seatbelt law enforced we would also have a data collection bill that would allow us to track and make sure our traffic laws are being applied in a nondiscriminatory fashion
that deal that was struck in illinois was premised on the notion that we would be getting these federal incentives it is not appropriate for the federal government to now pull the rug out from under states such as illinois that have done the right thing it is appropriate instead for us to keep our word maintain our commitments and make sure we continue to incentivize a law that everybody knows in fact saves the lives of our citizens
i encourage my colleagues to oppose this amendment
mr president i yield the floor
 newspeech
mr president i ask unanimous consent that the order for the quorum call be rescinded
 newspeech
mr president we have been spending a considerable amount of time in this body debating the socalled nuclear option today i want to spend a little bit of time talking about an issue that poses a more significant threat to our republic
throughout the last half of the 20th century one nation more than any other on the face of the earth defined and shaped the threats posed to the united states this nation of course was the soviet union and its successor state russia
while many have turned their attention to china or other parts of the world i believe the most important threat to the security of the united states continues to lie within the borders of the former soviet union in the form of stockpiles of nuclear biological and chemical weapons and materials we are in a race against time to prevent these weapons from getting in the hands of international terrorist organizations or rogue states the path to this potential disaster is easier than anyone could imagine there are a number of potential sources of fissile material in the former soviet union in sites that are poorly secured the material is compact easy to hide and hard to track weapons designs can be easily found on the internet
today some weapons experts believe that terrorist organizations will have enough fissile material to build a nuclear bomb in the next 10 years that is right 10 years
i rise today to instill a sense of urgency in the senate i rise today to ask how are we going to deal with this threat tomorrow a year from now a decade from now
the president has just completed an international trip that included a visit to russia i commend him for taking this trip and making our relationship with russia a priority
during the cold war the united states and the soviet union produced nearly 2000 tons of plutonium and highly enriched uranium for use in weapons that could destroy the world several times over to give an idea of just how much this is it takes only 5 to 10 kilograms of plutonium to build a nuclear weapon that could kill the entire population of st louis for decades strategic deterrence our alliances and the balance of power with the soviet union ensured the relative safety of these weapons and materials
with the end of the cold war and the collapse of the soviet union all this has changed key institutions within the soviet national security apparatus have crumbled exposing dangerous gaps in the security of nuclear weapons delivery systems and fissile material
regional powers felt fewer constraints to develop nuclear weapons rogue states accelerated weapons programs
and while this was happening international terrorist organizations who are aggressively seeking nuclear weapons gained strength and momentum
now thanks to the leadership of former senator nunn and senator lugar in creating the cooperative threat reduction program at the department of defense there is no question that we have made some great progress in securing these weapons
these same two leaders continue to work tirelessly on this issue to this day senator nunn through the nuclear threat initiative and senator lugar through his chairmanship of the foreign relations committee
the situation in russia and the rest of the former soviet union is drastically different than it was in 1991 or even 1996 or 2001 but the threat is still extremely dangerous and extremely real
in march of this year a senior russian commander concluded that 39 of 46 key russian weapons facilities had serious security shortcomings many russian nuclear research sites frequently have doors propped open security sensors turned off and guards patrolling without ammunition in their weapons
meanwhile the security situation outside of russia continues to be of grave concern fanatical terrorist organizations who want these weapons continue to search every corner of the earth resorting to virtually any means necessary the nuclear programs of nations such as iran and north korea threaten to destabilize key regions of the world we are still learning about the tremendous damage caused by aq khan the rogue pakistani weapons scientist
looking back over the past decade and a half it is clear that we could and should have done more
so as the president returns from his trip to russia we should be thinking on a bipartisan basis about the critical issues that can guide us in the future to ensure that there are no more missed opportunities
the first question we should be thinking about is what is the future of the cooperative threat reduction program what is our plan i believe the administration must spend more time working with congress to chart out a roadmap and a strategic vision of the program
there are two things the president can do to move on this issue first in the national security strategy to combat weapons of mass destruction of 2002 the administration said the national security council would prepare a 5year governmentwide strategy by march of 2003 to my knowledge this has not been completed in addition congress required the administration to submit an interagency coordination plan on how to more effectively deal with nonproliferation issues this plan is due at the end of this month
completing these plans will help the united states better address critical daytoday issues such as liability resource allocation and timetables having a better strategic vision will also help us work more efficiently and effectively with other international donors who have become increasingly involved and are making significant contributions to these efforts this is very important as the contribution of other donors can help us make up valuable lost time
mr president my second question concerns the usrussian relationship where is this relationship heading will russia be an adversary a partner or something in between
we do not ask these questions simply because we are interested in being nice and want only to get along with the russians we have to ask these questions because they directly impact our progress towards securing and destroying stockpiles of nuclear weapons and materials
in the last few years we have seen some disturbing trends in russia the rapid deterioration of democracy and the rule of law bizarre and troubling statements from president putin about the fall of the soviet union the abuses that have taken place in chechnya and russian meddling in the former soviet union from the baltics to the ukraine to georgia
the russians must understand that their actions on some of these issues are entirely unacceptable
at the same time i believe we have to do a better job of working with the russians to make sure they are moving in the right direction this starts by being thoughtful and consistent about what we say and what we do tone matters
some of the statements by our own officials have been confusing contradictory and problematic at times i have been left scratching my head about what exactly our policy is and how administration statements square with this policy
another issue is the level of sustained engagement with russia i am glad the president and secretary of state have made several trips to russia but as these trips are only a few days every year or so this is only one aspect of the relationship
an additional component which has suffered in recent years is our foreign assistance programs to russia and the rest of the former soviet union these programs are absolutely essential in maintaining our engagement with russia these programs are not giveaways they are programs that advance us interests by strengthening russian democracy and civil society enhancing economic development and dealing with international health issues in addition to curbing the nonproliferation threat
at a time when these programs are desperately needed their budgets have been cut dramatically at a time when we should be doing more to engage and shape the future of russia we seem to be doing the exact opposite
the nonproliferation threat does not exist in a vacuum the issue i just mentioned along with other important issues such as our own strategic nuclear arsenal must be considered as we move forward
finally mr president i would like my colleagues to consider how our relationship with russia and our efforts to secure and destroy weapons and materials inside the former soviet union fits in with our broader nonproliferation goals
russia is a major player in the two biggest proliferation challenges we currently face iran and north korea russias dangerous involvement with irans nuclear program has been well documented and there is no question their actions will be pivotal if the president is to successfully resolve this deteriorating situation
the russians are also an important voice in trying to make progress on the deteriorating situation in north korea the russian city of vladivostok is home to 590000 people and is very close to the north korean border putting the russians smack in the middle of the crisis that we need to resolve
in addition to all this russia holds a seat on the security council of the united nations which could consider iranian and north korean issues in the very near future
developing bilateral and multilateral strategies that deal with russias role in these growing crises will be extremely important both in terms of resolving these crises advancing our nonproliferation goals within the former soviet union and our longterm relationship with russia
i realize that at this time none of us have all the answers to these extraordinarily difficult questions but if we hope to successfully fight terror and avoid disaster before it arrives at our shores we have to start finding these answers we have a lot of work to do
i believe it is worth putting in place a process one that involves senior administration officials a bipartisan group of members of congress as well as retired senior military officers and diplomats in an effort to dramatically improve progress on these issues
i am interested in hearing from the president about his trip i am also interested in hearing if he believes that an idea similar to the one i put forward is worth considering
delay is not an option we need to start making more progress on this issue today i urge my colleagues to act
despite all the distractions we have had with the socalled nuclear option and judicial nominations this is literally a matter of life and death i hope we start paying more attention to it in this senate chamber and in the debates that are going to be coming in the coming months
mr president i yield the floor
 newspeech
mr president i rise today to bring attention to the situation of afrolatinos throughout latin america in the hopes that we can encourage more action on this issue from colombia to brazil to the dominican republic to ecuador persons of african descent continue to experience racial discrimination and remain among the poorest and most marginalized groups in the entire region while recent positive steps have been taken in some areas for example giving land titles to afrocolombians and passing explicit antidiscrimination legislation in brazil much work still needs to be done to ensure that this is the beginning of an ongoing process of reform not the end
in places where civil conflict has taken hold afrolatinos are much more likely to become victims of violence or refugees in their own countries in many areas afrolatinos are also subject to aggression by local police forces at far greater rates than those perceived to be white access to health services is another serious concern and recent studies have shown that afrolatino communities are at greater risk of contracting hivaids
in the last congress there was not one mention in the senate of the millions of afrolatinos who continue to experience this widespread discrimination and socioeconomic marginalization now is the time for more action on this issue not less emerging civil society groups are growing stronger throughout many countries in latin america and this growth should be encouraged as it presents important opportunities for partnerships and collaboration i look forward to working with my colleagues in the senate and house on this critical concern in the coming months and i believe that together we can and will make a difference
 newspeech
mr president let me begin my statement today by outlining what i think this debate is not about
i do not believe this debate is about mr bolton being rude on occasion this debate is not about mr bolton being blunt the debate is not about mr bolton occasionally losing his temper
as the distinguished senator from arizona just noted if this is the criteria many of us in the us senate would not be qualified to serve in a position that requires confirmation almost all of us lose our cool from time to time and say things we come to regret later let me add i dont think this debate is about whether mr bolton is an intelligent man
these are not the issues at the heart of the strong bipartisan objections that have been voiced on this nomination
the crux of the objections is very specific very credible allegations that mr bolton sought to shade intelligence and sideline career intelligence analysts who did not agree with his policy views this is the core of the bipartisan objections to this nomination
over and over again we heard from a range of career officials and bush administration appointees that mr bolton sought to massage intelligence to fit an ideological bias let me emphasize these are objections coming forward from bush appointees
in addition we have 102 former ambassadors and senior diplomats who oppose bolton from the nixon administration the ford administration and that bastion of fuzzyheaded liberalism the reagan administration
in an environment where reliable intelligence is one of the best tools we have to keep us safe we must heed the lessons from the iraq war intelligence must never be shaped to fit policy views dissent within the intelligence community should not be muzzled or suppressed it should be respected and encouraged
the united states senate should be sending a clear unequivocal statement to our intelligence officers we want you to play it straight and call it like you see it even if it is something we do not want to hear
i am afraid that by voting to confirm mr bolton we will fail to send that critical message
now i believe the president is entitled to the benefit of the doubt when appointing senior members of his team to that end i have supported a number of the presidents choices for top foreign policy positions including secretary rice robert zoellick to be her deputy and nick burns to fill the thirdranking position at the state department
i think we should provide some deference to the president the executive branch is primarily responsible for the daytoday operations of our foreign policy
at the same time the constitution gives the senate the power to advise and to consent this is a responsibility i take very seriously
and so because of mr boltons consistent breach of the line between practicing politics and analyzing intelligence that is pivotal to our national security i intend to vote no on the nomination of john bolton to be our representative to the united nations
i agree with much of what my colleagues have said about the problems with mr boltons qualifications to serve in this position but i would like to focus on one issue that i believe has not been covered in great detail mr boltons performance in his current job
it has been suggested we should overlook the troubling aspects of mr boltons record the fact that he appears to have attempted to manipulate intelligence data the fact he does not appear to have been entirely forthcoming before the foreign relations committee and the fact we still cannot get basic information from the state department on his nomination for one reason because mr bolton is so competent for the job i have heard this argument repeatedly from the other side of the aisle
i am baffled by this reasoning i am stupefied by the suggestion that mr bolton is such an excellent choice for the job so uniquely qualified for this job that we should just ignore all of these other problems
when i look at the record of mr bolton during the last 4 years as the top arms control and nonproliferation official at the state department i am not impressed lets look at his track record
on north korea the approach that has been advocated by both mr bolton and this administration has simply not worked under mr boltons watch there are no longer international inspectors and cameras at any site in north korea the north koreans have withdrawn from the nonproliferation treaty we now believe north korea has developed material for six to eight nuclear weapons
when north korea has one or two nuclear weapons the situation is critical they can test one weapon and hold one weapon when it has six to eight the situation is terminal north korea can now test a weapon hold a couple and sell the rest and we know that north korea will do virtually anything for the money
another area mr bolton was responsible for is the nonproliferation treaty a critical tool for helping to prevent the spread of nuclear weapons to rogue states which could ultimately fall into the hands of terrorist organizations
president bush recognized the importance of the npt and pledged to strengthen this treaty in a 2004 speech at the national defense university a week later mr bolton promised to do the same
what has happened since virtually nothing the administration has made very little progress on this issue and the nonproliferation treaty review conference currently underway is not going well
an article from msnbc reports
the united states has been losing control of the conferences agenda this week to iran and other countries a potentially serious setback to us efforts to isolate tehran
where has mr bolton been throughout this process
according to the same article
since last fall bolton mr bushs embattled nominee to be americas ambassador to the united nations has aggressively lobbied for a senior job in the second bush administration during that time mr bolton did almost no diplomatic groundwork for the npt conference officials say everyone knew the conference was coming and that it would be contentious says a former senior bush official but bolton stopped all diplomacy on this six months ago
in other words mr bolton was more interested in lobbying for the un job than doing the tough groundwork necessary for a successful review conference
lets turn to iran another issue on which mr bolton should have been working to formulate a coherent workable administration strategy instead the administrations policy has been all over the map in a hearing before the foreign relations committee last week a senior state department official described the latest iteration of the administrations policy as a patient policy
i would say the policy has been less about patience and more about paralysis a dangerous situation for a nation such as iran that is developing nuclear weapons is a state sponsor of terrorism and is meddling in iraq
perhaps this paralysis and incoherence is best illustrated by the fact that since 2001 the administration has tried to my knowledge without success to formulate a presidential directive on iran as the top nonproliferation official at the state department mr bolton should have been doing more to shape a workable policy instead of letting it drift dangerously along for the last 4 years
mr president i know my time is running short so let me conclude with a couple of simple points
two examples are frequently cited by mr bolton and his supporters as evidence of his success and competence in his current position libya and the proliferation security initiative during his confirmation hearings mr bolton touted these successes over and over again
now i agree with mr bolton that we have made important progress on these issues but reports suggest that the libya deal was struck in spite of mr bolton not because of him in fact mr bolton was sidelined from the negotiations by the white house and the british government specifically asked that mr bolton not play a role in this process
i quote from an msnbc article that specifically addresses this issue
bolton for instance often takes and is given credit for the administrations proliferation security initiative an agreement to interdict suspected wmd shipments on the high seas and the deal to dismantle libyas nuclear program a deal that bolton by the way had sought to block but a former senior bush official says that in fact boltons successor robert joseph deserves most of the credit for these achievements this official adds that it was joseph who was in charge of counterproliferation at the nsc and who had to pitch in when bolton fumbled preparations for the npt conference as well
now here is my point if there was clear evidence that mr bolton is a terrific diplomat maybe i could understand how some in the senate could overlook what i consider to be a mountain of evidence concerning his misuse of intelligence and say you know what this guy is such a capable administrator and diplomat we need him to reform the united nations
i would still believe that the misuse of intelligence in and of itself disqualifies mr bolton from the job but at least i could understand why some people would draw such a conclusion
but the record indicates that in his current job he has not had much success which leads me to ask why is it we are so confident this is the person who is going to lead reform in the united nations
the distinguished senator from arizona is exactly right we need reform in the united nations it is inexcusable some of the things that go on up there
but as a consequence of mr boltons diminished credibility and stature i think he is exactly the opposite of what we need at the united nations countries such as zimbabwe and burma and others that do not want to see reform take place at the un are going to be able to dismiss our efforts at reform by saying mr bolton is a un basher someone who is ideologically opposed to the existence of the unthereby using mr boltons own words and lack of credibility as a shield to prevent the very reforms that need to take place
moreover i have yet to hear a comprehensive plan from mr bolton or the administration for un reform
so let me close by saying this when the foreign relations committee considered mr boltons nomination i invoked the memory of adlai stevenson a great citizen of the state of illinois stevenson had the credibility the temperament and the diplomatic skill to guide the united states through some of the worst most difficult times at the united nations especially the cuban missile crisis
during this crisis we were able to isolate the soviets because of the stature and integrity of our permanent representative to the united nations
given the issues that have surfaced surrounding mr boltons nomination i simply ask my colleagues this if a crisis were to occur with north korea or iran are we sure the integrity and credibility of mr bolton would command the respect of the rest of the world would mr bolton like adlai stevenson be able to convince the world that our intelligence and our policies are right and true would mr bolton be able to isolate our enemies and build a coalition that would ultimately make our troops safer and our mission easier
i believe the answer is no there are some wonderful capable tough conservative reformminded republican diplomats who are well qualified for this task and would easily be confirmed by the senate mr bolton is not one of them
i would urge that the other side of the aisle seriously consider their position on this nomination i hope we can muster the votes to send this nomination back to the president lets start afresh i know we can do better
thank you mr president
 newspeech
mr president i am pleased to join my colleagues senators specter and harkin to introduce the gynecological cancer education and awareness act of 2005 also known as johannas law this important legislation authorizes a national gynecologic cancer early detection and awareness campaign for women and their providers this bill is named in honor of johanna silver gordon who died from ovarian cancer and whose sister sheryl silver founded johannas law alliance for womens cancer awareness we thank ms silver for her courage and her persistent efforts to turn her sisters tragedy into a crusade to raise awareness and prevent needless suffering and death from gynecologic cancers for other women
nearly 80000 american women are diagnosed with gynecologic cancers each year tragically 29000 of them die from this disease we know that early detection is the key to successful treatment of all gynecologic cancers and we have made great strides at reducing rates of cervical cancer with widespread use of pap screening tests yet we have not been able to replicate this success with uterine cancer and ovarian cancer for which effective and general screening methods do not exist for ovarian cancer which is the deadliest of the gynecologic cancers in addition to lack of screening tests doctors and researchers have not identified effective diagnostic and treatments seventy percent of all new diagnoses of ovarian cancer take place after this cancer has progressed beyond its earliest and most survivable stage
given these challenges knowing the symptoms of gynecologic cancers which can mimic gi illnesses menopause or perimenopause is key to early diagnosis the 5year survival rates for the most common gynecologic cancers are 90 percent when diagnosed early but drop to 50 percent for cancers diagnosed later
johannas law will promote early detection and awareness through a national public awareness campaign conducted by the department of health and human services women will be given written materials that provide information about gynecologic cancers and public service announcements will be developed to encourage women to talk to their doctors about gynecologic cancer the department will also give grants for demonstration projects to local and national nonprofit organizations to identify the best ways to reach and educate women about these cancers particularly those women who are high risk
johannas law will make sure that women and doctors get the information they need to help them recognize early symptoms of gynecologic cancers so that women can be diagnosed and treated earlier when their cancers are treatable i urge my colleagues to work to move this legislation forward promptly
 newspeech
mr president the department of veterans affairs estimates that on any given day as many as 200000 veterans are homeless that is 200000 men and women who have fought for this country who will go without the comfort of knowing that they will have a roof over their head and a place to call home
if 200000 of our nations veterans will go homeless tonight the va estimates that about twice as many veterans will experience homelessness this year again that is 400000 men and women who defended this great nation who will be left out on the streets at some point this year
i hope my colleagues are as distressed as i am by these numbers and i hope my colleagues will join me in supporting the bill i introduce today the shelter all veterans everywhere or save reauthorization act of 2005
this bill reauthorizes many of the soontoexpire homeless veterans programs currently serving this needy population including the homeless providers grant and per diem program and the homeless veterans reintegration program these programs work to provide muchneeded services to homeless veterans so that they can find jobs and ultimately find a stable home these programs deserve to be continued the save reauthorization act actually expands the reach of the homeless veterans reintegration program which provides job placement and training assistance to include those veterans at risk of homelessness as well as those actually homeless so that we can work to prevent homelessness before it happens
 mr cornyn for himself mr leahy mr feingold and mr alexander
s 1181 a bill to ensure an open and deliberate process in congress by providing that any future legislation to establish a new exemption to section 552 of title 5 united states code commonly referred to as the freedom of information act be stated explicitly within the text of the bill to the committee on the judiciary
 newspeech
mr president today marks the 40th anniversary of the us supreme court decision in griswold v connecticut which struck down connecticut laws that prohibited reproductive counseling and the use of contraception in recognizing a constitutional right to privacy this landmark decision secured the right of married women to use contraception and laid the groundwork for widespread access to birth control for all american women
the availability and use of contraceptives has had a profound impact on the health and lives of women across the nation widespread use of birth control has led to dramatic reductions in national rates of sexually transmitted infections unintended pregnancies and abortion contraceptive use has also significantly improved maternal and infant health outcomes and reduced maternal and infant mortality rates since 1965 maternal and infant mortality rates have declined by more than twothirds
the impact of contraception on the professional lives of women has been equally profound the ability of women to control fertility has allowed them to successfully achieve educational and career goals that wouldve been impossible a century ago women are critical to this nations economic success comprising up to one half of the total us labor force
in 1999 the centers for disease control and prevention recognized the significant impact of birth control on american society and included family planning in their list of the ten great public health achievements in the 20th century however despite considerable progress in this area much work remains the united states has one of the highest rates of unintended pregnancies and sexually transmitted infections among industrialized nations which in part reflects lack of access to basic preventive health care including contraception
a growing number of women almost 17 million currently must rely on publicly supported contraceptive care between 2000 and 2002 this number increased by 400000 alone because of the rising number of uninsured women yet even those women with health insurance are not guaranteed access to contraceptives because some health plans choose not to cover these medications and procedures as they would other basic preventive health measures and we are increasingly hearing about pharmacists and other providers who refuse to prescribe or fill contraceptive prescriptions or refer women to those who will because of their own personal beliefs
this 40th anniversary of the griswold decision provides a perfect opportunity to reflect upon the critical importance and impact of this decision on the health and professional lives of millions of women we must ensure that policy decisions about contraception services remain health decisions and not political ones and work to ensure that all women have access to contraception when they need it
 newspeech
mr president i rise in strong support of this resolution before i make any further remarks i would like to recognize doria d johnson and thank her for coming she is from evanston il ms johnson is the great greatgranddaughter of anthony crawford a south carolina farmer who was lynched nearly 100 years ago for the crime of being a successful black farmer i am sure that this day has special meaning for her and for the other family members of those who were impacted by these great tragedies of the past i thank her and others for being here today
since americas darkest days of jim crow separate but equal fire hoses church bombings cross burnings and lynchings the people of this great nation have found the courage on occasion to speak up and speak out so that we can right this countrys wrongs and walk together down that long road of transformation that continues to perfect our union it is a transformation that brought us the civil rights act and the voting rights act a transformation that led to the first black member of congress and the first black and white children holding hands in the same playground and the same school a transformation without which i would not be standing here speaking today but i am and i am proud because thanks to this resolution we are taking another step in acknowledging a dark corner of our history we are taking a step that allows us after looking at the 4700 deaths from lynchings the hate that was behind those deaths and this chambers refusal to try and stop them to finally say that we were wrong
there is a power in acknowledging error and mistake it is a power that potentially transforms not only those who were impacted directly by the lynchings but also those who are the progeny of the perpetrators of these crimes there is a piercing photographic exhibit in chicago right now that displays some of the lynchings that occurred across the country over the past two centuries these photographs show that what is often most powerful is not the gruesome aspects of the lynching itself nor the terrible rending of the body that took place no what is most horrific what is most disturbing to the soul is the photographs in which you see young little white girls or young little white boys with their parents on an outing looking at the degradation of another human being one wonders not only what the lynching did to the family member of those who were lynched but also what the effect was on the sensibilities of those young people who stood there watching
now that we are finally acknowledging this injustice we have an opportunity to reflect on the cruelties that inhabit all of us we can now take the time to teach our children to treat people who look different than us with the same respect that we would expect for ourselves so it is fitting it is proper and it is right that we are doing what we are doing today
however i do hope as we commemorate this past injustice that this chamber also spends some time doing something concrete and tangible to heal the long shadow of slavery and the legacy of racial discrimination so that 100 years from now we can look back and be proud and not have to apologize once again that means completing the unfinished work of the civil rights movement and closing the gap that still exists in health care education and income there are more ways to perpetrate violence than simply a lynching there is the violence that we subject young children to when they do not have any opportunity or hope when they stand on street corners not thinking much of themselves not thinking that their lives are worth living that is a form of violence that this chamber could do something about
as we are spending time apologizing today for these past failures of the senate to act we should also spend some time debating the extension of the voting rights act and the best way to extend health care coverage to over 45 million uninsured americans we should be considering how we can make certain that college is affordable for young africanamerican children the great greatgrandchildren or the great great greatgrandchildren of those who have been wronged these are the ways we can finally ensure that the blessings of opportunity reach every single american and finally claim a victory in the long struggle for civil rights
today is a step in the right direction todays actions give us an opportunity to heal and to move forward but for those who still harbor anger in their hearts who still wonder how to move on from such terrible violence it is worth reflecting for a moment on one remarkable individual mamie till mobley
mamie till mobleys child emmett was only 14 years old when they found him in the mississippi river beaten and bloodied beyond recognition after ms mobley saw her child her baby unrecognizable his face so badly beaten it barely looked human someone suggested that she should have a closed casket at his funeral she said no we are going to have an open casket and everybody is going to witness what they did to my child
the courage displayed by this mother galvanized the civil rights movement in the north and in the south and despite the immensity of the pain she felt mamie till mobley has repeatedly said i never wasted a day hating imagine that she never wasted a day hating not one day
i rise today thanking god that the united states congress the representatives of the american people and our highest ideals will not waste one more day without issuing the apology that will continue to help us march down the path of transformation that mamie till mobley has been on her whole life and that the people in attendance in the gallery have been on for generations
i am grateful for this tribute and i am looking forward to joining hands with my colleagues and the american people to make sure that when our children and grandchildren look back at our actions in this chamber we do not have something to apologize for
i yield the floor
 newspeech
mr president on may 22 2005 the womens lacrosse team from northwestern university captured the ncaa division i championship this was the first championship for the wildcats womens lacrosse team and the first womens lacrosse champion to come from outside of the eastern time zone
this championship also marks a milestone for northwestern sports it is the first ever womens team championship for the university and only the second team championship in school history the mens fencing team was national champion in 1941
this talented and dedicated team from the great state of illinois secured the championship by defeating traditional lacrosse powerhouses dartmouth and princeton the wildcats completed their undefeated 21 win season with a 1310 victory over defending champion virginia since regaining varsity status in 2002 this outstanding program has taken only four seasons to capture the championship
congratulations to abby bangser donna mccann sarah albrecht kristen kjellman sara crosby aly josephs kristen boege ashley koester courtney koester hilary alley ashley gersuk courtney flynn christy finch lindsey munday emily lovett laura glassanos shelby chlopak kaitie lenahan lynda mccandlish abby alley sarah walsh lindsay finocchiaro jenny bush fallon mcgraw quinn cammarota lindsay north meredith philipp hannah whitman rebecca zazove annie elliott bailey su kate darmody minnie doherty kim corcoran head coach kelly amonte hiller assistant coach alexis venechanos assistant coach danielle shearer and assistant coach scott hiller
please join me in congratulating the northwestern womens lacrosse team on its historic championship season
 newspeech
i agree with the senior senator from vermont about the importance of upholding the rule of law in africa and around the world i would also like to add my support for the efforts of the special court for sierra leone to bring to justice some of the worst war criminals of the 20th century while the special court has not been perfect there is no question that the court is doing vitally important work of promoting peace and reconciliation increasing accountability and strengthening the rule of law throughout west africa i also want to discuss a related issue the case of charles taylor i know the senator from vermont has been working for years on this issue
i will simply say that charles taylor is an indicted war criminal and he needs to be transferred to the special court to stand trial as soon as possible the government of nigeria has allowed charles taylor to live in exile within its borders with the support of the international community including the united states since august 2003 while we owe nigeria a debt of gratitude for helping prevent further bloodshed in liberia it is time for mr taylor to be transferred to the special court
no nation should be permitted to willfully ignore an indictment issued by this tribunal moreover there are credible reports that mr taylor has broken the terms of his exile is a threat to the liberian peace process and continues to meddle in the internal affairs of liberia just a few months before the liberian elections
i wonder if the senator from vermont shares my views
 newspeech
i thank the senator from vermont the ranking member of the appropriations subcommittee on state foreign operations because he makes a crucial point debt relief from the united states is not automatic in the past debt relief has come with conditions including making progress in fighting corruption and on economic reform to ensure that this relief achieves the maximum results
for nigeria this means turning over charles taylor an indicted war criminal who has the blood of thousands on his hands and threatens once again to destabilize the region to the special court like the senator from vermont i strongly believe that nigeria is a worthy candidate for debt relief and a key us partner in west africa when charles taylor is turned over there is no doubt in my mind that i will be a forceful advocate for debt relief for nigeria i would also like to praise the government of nigeria for its leadership on other issues especially their efforts to lead the african union force in darfur i want nothing more than to see the taylor issue successfully resolved so we can focus our attention on other important issues with the nigerians
i would also reiterate what the senator said about the waiver authority contained in section 585 the president can waive these restrictions including those pertaining to nigerian debt relief by formulating a plan to get mr taylor to the court
 newspeech
mr president i support the amendment offered by the senator from new mexico mr bingaman this amendment is a breath of fresh air in a bill that is filled with many stale concepts regarding our approach to this nations energy policy i am proud to be a cosponsor of this amendment
producing a significant amount of our electricity from renewable sources is not a concept for the future it is a real possibility that exists today using solar wind tidal gas from landfills and biomass in fact 19 states around the country are using these renewable source of energy to steer their states towards a future of clean sustainable energy use
in my state of illinois and in many other states enacting this standard is a nobrainer this winter illinois governor blagojevich announced a plan to adopt a renewable portfolio standard requiring illinois electric utilities to provide 8 percent renewable energy as part of their overall power mix by 2012 this bold vision will make illinois the second biggest wind power state in the country by 2012 the city of chicago also has a strong commitment to using renewable sources of energy and is already planning to surpass a 10 percent contribution from renewables in its electricity stream and achieve a 20 percent goal
in the 18 other states where renewable portfolio standards have been successfully adopted innovations in electricity generation have flourished at virtually no cost to the consumer just imagine what would happen to this industry of the future if we enacted a federal standard and here is the best news according to the union of concerned scientists a 10 percent renewable portfolio standard on the federal level would not add a single penny to consumers bills
introducing renewable electricity into the mix of electricity generation also brings us a measure of physical security by creating geographically dispersed sources of energy generation we are providing ourselves with greater electricity security by providing smaller targets and reducing the transport of combustible materials this is smart policy at a time when we must be vigilant about homeland security
our countrys demand for electricity is expected to continue growing for decades to come enacting a renewable portfolio standard ensures that clean technologies will help us meet that enlarged demand while not offsetting the importance of investing in clean technologies in other energy production methods especially coal coal will undoubtedly play a large role in our energy portfolio for years to come and i look forward to a vigorous debate on how we can best assist the utility industry in employing clean coal technologies
abraham lincoln once said i am a firm believer in the people if given the truth they can be depended upon to meet any national crises the great point is to bring them the real facts the real facts are that without forwardthinking amendments such as this one the energy bill is not going to bring us independence from the 20th century mindset of energy production let us give the american public this tool so they too can rise to meet this national energy crisis before it gets worse
 newspeech
mr president i rise today in support of the amendment offered by the senator from washington ms cantwell i am proud to be submitting this amendment
fortyfour years ago john f kennedy challenged america to put a man on the moon by the end of the 1960s a bipartisan coalition in congress joined with presidents kennedy johnson and nixon to make this goal a reality
today we are considering a similarly bold challenge to the nation to reduce americas dependence on foreign oil by 40 percent by the year 2025 this challenge is no less important no less laudable and no less worthy of bipartisan support presidential leadership and national commitment
the bill before us purports to offer a comprehensive energy solution for the future but as currently drafted the bill does nothing more than lead us down the same dangerous and unsustainable path that we have been traveling for the last several decades unless we draw the line now outlining a bold change in course with time enough to prepare we will see the united states in 2025 even more tethered to foreign oil and even more subject to economic shocks than the united states of 2005 unless we reverse course we will continue putting our economic wellbeing and national security at the mercy of unstable foreign governments
some will argue that the goals in this amendment are unrealistic and unattainable i do not agree with these naysayers when president kennedy announced his challenge in 1961 he said the following this decision demands a major national commitment of scientific and technical manpower materiel and facilities and the possibility of their diversion from other important activities where they are already thinly spread it means a degree of dedication organization and discipline which have not always characterized our research and development efforts
likewise meeting the requirements of the senators amendment will require a similar commitment but i believe the task before us is much simpler than the one that faced president kennedy because we already know how to decrease our reliance on foreign oil a smart energy policy that focuses on a greater commitment to technology including hybrid and hydrogen fuel cell technology renewable fuels and greater efficiency can take us a long way if not the entire way to the goal proposed by the senator from washington
as difficult as it may be we must try to meet the goal set forth in this amendment we would be far worse off as a country if we just threw up our hands and admitted defeat
the people i meet on my travels around illinois are ready for the challenge they are tired of giving their hardearned dollars to foreign governments in the form of recordhigh gasoline prices they are tired of seeing their foreign policy being influenced by americas insatiable need for middle east oil they are looking to their leaders in washington for innovative leadership if we lay down the challenge in this amendment i have every reason to believe that the american people will rise up to meet it much like they met a similar challenge 40 years ago
in 1962 president kennedy traveled to rice university to speak about the challenge that he had laid down the year before he stated surely the opening vistas of space promise high costs and hardships as well as high reward so it is not surprising that some would have us stay where we are a little longer to rest to wait but this city of houston this state of texas this country of the us was not built by those who waited and rested and wished to look behind them
when it comes to our energy policy we are long past the point of waiting and resting and looking behind us i urge my colleagues to support the amendment offered by the senator from washington
 newspeech
madam president i was pleased to join the senator from michigan senator levin in submitting a resolution on the juneteenth independence day
i have heard people ask why celebrate juneteenth we have so many holidays and remembrances already why add more history to the calendar
but of course juneteenth is not just about celebrating history it is about learning from it just like the day when the greatest civil rights leader of our time was born or the day when we finally gave african americans a ballot and a voice juneteenth is a day when we can look back on a time when everyday americans faced the most daunting challenges and the slimmest odds and still persevered when they said we shall overcome and they did when the hopes held by so many for so long finally led to the victory of freedom over servitude of independence over enslavement
juneteenth is a day that allows us to remember that america is still the place where anything is possible it has been that place in the past and it can be that place in the future when it comes to the challenges we have yet to meet
and so when we think of those challenges when we think of the injustice we still face and the miles we have left to march when we think of the millions without health care the children without good schools the families without jobs and the disparities that still exist between black and white rich and poor educated and uneducated when we think about all these challenges we can also think juneteenth
we can think of a day when the word began to spread from town to plantation to city to farm that after more than a hundred years of slavery millions were now free that after so many hopeless days and years of despair the impossible was now truth the shackles were now broken and a new day was finally here
in the memory of this day i believe we can find hope for all the trying days we have yet to face as a people and as a nation and as we continue to overcome we will continue to celebrate those victories as historical markers that give future generations the same hope we have today
i commend senator levin for his longtime leadership on civil rights issues and urge my colleagues to support this resolution
 newspeech
mr president i am proud to join senators frist and clinton in introducing the health technology to enhance quality act of 2005
our national health care system is in crisis fortyfive million americans are uninsured and this number continues to rise health care costs are increasing at almost double digit rates millions of americans are suffering and dying from diseases such as diabetes or aids that could have been prevented or delayed for many years and the chance of americans receiving the right care at the right time and for the right reason is no greater than the flip of a coin
these health care issues are varied and complex as are the solutions but as one of my constituents advised it is time for us in the congress to put on our hard hats pick up our tool belts and get to work fixing our broken health care system
one place to start is by bringing the health care system into the 21st century in our lifetimes we have seen some of the greatest advances in the history of technology and the sharing of information yet in our health care system too much care is still provided with a pen and paper too much information about patients is not shared between doctors or readily available to them in the first place and providers too often do not have the information to know what care has worked most effectively and efficiently to make patients healthy
mistakes are easily made medical errors alone kill up to 98000 people a year more people than the number who die from aids each year
but embracing 21st century technology is not just about reducing errors and improving the quality of medical care it is also about cost
we spend nearly usd15 trillion a year on health care in america but a quarter of that money one out of every four dollars is spent on nonmedical costs most of it on bills and paperwork every transaction you make at a bank now costs them less than a penny yet because we have not updated technology in the rest of the health care industry a single transaction still costs up to usd25 not one dime of which goes toward improving the quality of our health care
the health technology to enhance quality act of 2005 is going to help bring the health care system into the 21st century this bill will lead to the development and implementation of health information technology standards to ensure interoperability of health information systems the legislation codifies the office of national coordinator for information technology and establishes standards for the electronic exchange of health information the bill also provides grant funding to support development of health information technology infrastructure as well as measurement of the quality of care provided to patients
this legislation will help our health care system take a huge step forward a vote for the health teq act is a vote for health care that is safe effective and affordable i urge my colleagues to join us in passing this bill quickly
 newspeech
mr president i thank senator inhofe senator reid and senator domenici for allowing me this time
i know dick durbin i serve with him in illinois we have traveled together through the byways and highways of our great state i have rarely met someone with greater dedication to ordinary americans a stronger belief in the greatness of this nation or a more longstanding commitment to public service as an expression of that patriotism than dick durbin
this recent episode obviously has pained him a great deal because although i am new in the senate one of the things i am discovering is that we have a tendency perhaps because we dont share as much time on the floor as we should perhaps because our politics seem to be ginned up by interest groups and blogs and the internet we have a tendency to demonize and jump on and make mockery of each other across the aisle that is particularly pronounced when we make mistakes each and every one of us is going to make a mistake once in a while we are going to say something unartful we are going to say something that doesnt appropriately describe our intentions and what we hope is that our track record of service the scope of how we have operated and interacted with people will override whatever particular mistake we make
senator durbin has established himself as one of the people in this chamber who cares deeply about our veterans and our troops he hasnt just talked the talk he has walked the walk i have been distressed to see my partner from illinois placed in the situation in which he has been placed i am grateful he had the courage to stand up and acknowledge that he should have said what he said somewhat differently but i am also grateful that people such as the distinguished senator from arizona and others recognize this for what it was a simple misstatement and that now we can move on to talk about the substance of the issues that are of legitimate concern to this body including making certain that when we operate institutions such as those at guantanamo we hold the united states to that high standard that all of us expect
i yield the floor
 newspeech
mr president i ask unanimous consent that pat haman on my staff detailed from epa be granted floor privileges for the duration of the debate on this amendment
 newspeech
mr president during the 2 weeks or so that we have been debating this energy bill in the senate the price of crude oil has climbed to a record high of usd60 a barrel gas is now up to usd224 per gallon the saudis are pumping at nearfull capacity and their own oil minister says that the price of crude will probably stay at this level for the rest of the year
at this price the united states is sending usd650 million overseas every single day that is usd237 billion a year much of it to the middle east a region we have seen torn by war and terror it doesnt matter if these countries are budding democracies despotic regimes with nuclear intentions or havens for the madrasas that plant the seeds of terror in young minds they get our money because we need their oil
as demand continues to skyrocket around the world other countries have started to realize that guzzling oil is not a sustainable future whats more these countries have realized that by investing early in the energyefficient technology that exists today they can create millions of tomorrows jobs and build their economies to rival ours
china now has a higher fuel economy standard than we do and it has got 200000 hybrids on its roads japans toyota is doubling production of the popular prius in order to sell 100000 in the us next year and it is getting ready to open a brand new plant in china meanwhile we are importing hydrogen fuel cells from canada
these companies are running circles around their american counterparts ford is only making 20000 escape hybrids this year and gms brand wont be on the market until 2007 as falling demand for gashungry suvs has contributed to standard and poor reducing the bond rating of these companies to junk status these giants of the car industry now find themselves in the shadow of companies and countries that realize the time has come to move away from an oil economy
so here we are we have people paying record prices at the pump and america sending billions overseas to the worlds most volatile region we have countries such as china and india using energy technology to create jobs and wealth while our own businesses and workers fall further and further behind
and we have the energy bill that is before us today
now this bill takes some small steps in the right direction it will require utilities to generate 10 percent of their electricity from renewable sources it will help us realize the promise of ethanol as a fuel alternative by requiring 8 billion gallons to be mixed with gasoline over the next few years and by providing a tax credit for the construction of e85 stations all over america it will provide funding for the clean coal technologies that will move america to use its most abundant fossil fuel in a cleaner healthier way including for lowemission transportation fuels it will support the development of 500 milepergallon automobile technology and it will provide a good mix of tax incentives to move america towards more energy efficiency instead of simply rewarding the oil and gas industries as the house bill does the good that these proposals will do is reason enough to vote for this bill and i will do so
but we shouldnt kid ourselves today this isnt time to pat ourselves on the back and think we have put america on the path to energy independence experts say that this bill will reduce our foreign oil consumption by 3 percent three percent our own department of energy predicts that american demand will jump by 50 percent over the next 15 years so 3 percent doesnt amount to much and it certainly wont make a difference at the pump even president bush admits this we tried to pass an amendment that would have reduced our foreign oil dependence by 40 percent in 2025 but too many senators said no
and so when you look at this energy crisis and realize that it is about so much more than energy when you realize that our national security is at stake and that the global standing of our economy hangs in the balance when you see prices continue to rise and other countries continue to innovate you cant help but ask yourself is this the best america can do the country that went to the moon and conquered polio the country that led the technological revolution of the 1990s
it would be one thing if the solutions to our dependence on foreign oil were pieinthesky ideas that are years away but the technology is right at our fingertips today we could have told american car companies we will help you produce more hybrid cars we could have made sure there were more flexible fuel tanks in our cars we could have addressed the big reason why car companies are hurting in this country legacy health care costs had we taken all of these actions we could have put america on the path to energy independence once and for all
we also could have addressed the fact that global warming is threatening us with higher temperatures more drought more wildfire more flooding and more erosion of our coastal communities people who dont believe this can yell about it as loudly as they want but it doesnt change the fact that the overwhelming scientific evidence proves this over and over again we could have taken care of this problem now and left a better world to our children
with each passing day the world is moving towards new technology and new sources of energy that will one day replace our current dependence on fossil fuels
and so america has a choice
we can continue to hang on to oil as our solution we can keep passing energy bills that nibble around the edges of the problem we can hope that the saudis will pump faster and that our drills will find more and we can just sit on our hands and say that it is too hard to change the way things are and so we might as well not even try
or we could realize that this issue of energy this issue that at first glance seems like it is just about drilling or caribou or weirdlooking cars actually affects so many aspects of our lives that finding a solution could be the great project of our time
it wont be easy and it wont be without sacrifice government cant make it happen on its own but it does have a role in supporting the initiative that is already out there together we can help make real the ideas and initiatives that are coming from scientists and students and farmers all across america
abraham lincoln who first opened our national academy of sciences once said that part of governments mission is to add the fuel of interest to the fire of genius in the discovery of new and useful things
today when it comes to discovering new and useful solutions to our energy crisis the fire of genius burns strong in so many american innovators and optimists but theyre looking for leadership to provide the fuel that will light their way this bill is a reasonable first step but i know that we can do much much better
 newspeech
mr president i would like to express my gratitude to the managers of the energy bill senators domenici and bingaman for their support of two amendments that i offered i am proud that these amendments have been included in the legislation that the senate will vote on today and i believe that their enactment will help america increase its energy independence and transition our energy industry to full usage of 21st century technologies
the first adopted amendment which was cosponsored by senator lugar provides usd85 million to three universities for research and testing on developing illinois basin coal into transportation fuels including fischertropsch jet fuel a type of lowemissions diesel that can be used in jets and diesel the funds provided in this amendment will assist southern illinois university purdue university and the university of kentucky in upgrading existing facilities and constructing new facilities to conduct research and testing on this technology it is critical that our government invests in domestic fossil fuel supplies in an innovative manner and this is a commonsense way to expand our coal industry in an environmentally friendly manner
the second adopted amendment which was cosponsored by senator bayh provides usd40 million for research on combined plugin hybrid and flexible fuel vehicles today we have the technology to produce both plugin hybrid vehicles which run partly on electricity rather than fuel and flexible fuel vehicles which run on a blend of 85 percent renewable fuel and 15 percent petroleum but we dont yet have the technology to combine both technologies into the same car if we could do this there is the potential for developing a car that could get 500 miles per gallon of gasoline at a time when our country spends billions of dollars a year on importing foreign oil it is imperative that we take meaningful proactive steps that not only stem our future oil dependence but also reduce our reliance on overseas sources my amendment would do just that by stimulating the commercialization of this technology at a cost of only 6 percent of our nations daily spending on foreign oil
again i thank the bill managers for their assistance with these amendments
i ask unanimous consent to have the following two articles on the potential of combined plugin hybridflexible fuel vehicles printed in the record
 newspeech
mr president from june 1216 2005 students from the great state of illinois were invited to washington dc by the national history day program to present original history projects this scholarly group of students used their critical thinking and research skills to create exhibits documentaries and performances on the theme communication in history the key to understanding
congratulations to the national qualifiers and finalists from illinois audrey auyeung zoe netter charlotte cook eric jacobson david gainski lucy honold chelsea farmer brandon jakub kyle schoenfelt dakota smith erich grundman charlie curran jonathan taub alicia patten peter contos honghe li sebastian prokuski laura mullersoppart tomas manghi elizabeth may aruj chaudhry kyle johnson kathryn evans laura guzman rebecca strauss andriy matyukha sean gallagher brendon gallagher dan burasinsanga gian santos mary kowalkowski ellie terrell lauren brown nadine ibrahim annika kolasa courtney kolbe marissa suchyta david bailey joseph tepper tamara vaughn stephanie ebbs lena walker maria carvell robby krajewski allyson schroeder elizabeth hamman emily dennis lisa furby katie damron andrea martinelli cristen sawicki kelsey mcmahon amelia wallace allison nichols sarah siegel eliseo martinez and jessica drachenberg special congratulations to marrissa suchyta the second place winner in the junior individual documentary category and aruj chaudhry the third place winner in the senior individual paper category
finally dedicated illinois teachers worked throughout the academic year with these students so that they could be successful in competing with over 500000 students nationwide
congratulations to their teachers angie carr balazs dibuz mario garcia melissa craig ron solberg carlton oquendo betsy brown patricia grunde ann patricia duffy leslie contos david barber sherri massa chris salituro aggie nowak cathy bednar peggy hallheineman patricia grimmer sandra koehler janet kelsey chris demato barry bradford claire finn therese hawkins sandra koehler and claire finn
i commend these students on their achievements and encourage them to continue their pursuit of academic excellence
 newspeech
mr president i rise today to speak in favor of the amendment offered by senator boxer regarding the testing of pesticides on humans i am pleased to be a cosponsor of this amendment
unbeknownst to most of us the bush administration has quietly rescinded a ban on the human testing of pesticides even though the epa is still developing guidelines for such testing instead of needlessly exposing people to dangerous pesticides the 1year moratorium proposed in this amendment is a reasonable solution until these guidelines are completed
let us be clear we are not talking about the testing of lifesaving medications by definition pesticides are designed to kill they are potential carcinogens and neurotoxins we need guidelines to ensure that human testing of these dangerous chemicals is limited and monitored and that the subjects fully understand the risks they are taking
who are the people being exposed to these chemicals typically they are young poor and minorities let me give you two examples
in florida an epa study offered lowincome families usd970 over 2 years if they let their babies be tested after their homes were sprayed with pesticides one can easily imagine a young mother trying to make ends meet trying to pay the rent and put food on the table reading that she can collect almost usd1000 if she allows her child to be tested
in another study last year 127 young adults mostly asian and latino college students agreed to be exposed to a suspected neurotoxicant for usd15 an hour some were exposed in a chamber for 1 hour for 4 consecutive days while others had the chemical shot into their eyes and nostrils at amounts 12 times the osha recommended levels this chemical chloropicrin has a history it was used as a chemical warfare agent in world war i yet the consent form for the 2004 study did not disclose that fact it simply said we expect the discomfort to be shortlived
all across america there are college students working long hours so they can stay in school and get a shot at the american dream how tempting it must be to pick up a handful of cash for letting a scientist expose you to some chemical you are healthy you need the cash and you are probably not as wise as your parents would like you to be so you borrow a chance against your future health and sign up for exposure that is not the kind of government policy we want to be encouraging
all told the epa is considering data from 24 studies that tested pesticides on humans many of these studies are flawed so the risks these people undertook did not even contribute to a scientifically valid experiment many of these studies failed to take the health complaints of the subjects seriously many failed to disclose the risk to the subjects and many failed to conduct longterm monitoring of the health effects of the pesticides all of these deficiencies should be addressed and prevented from occurring again
sadly we do not need to do this human testing for years the epa has worked with pesticide manufacturers and members of the science community without relying on human testing for years the agency has accomplished its goals through animal testing
no one doubts that actual human health data if properly collected from a sufficient sample size would be advantageous to know but sensible guidelines are needed to ensure that the benefits of any study far outweigh the potential risks to the study participants
the commonsense approach is to temporarily stop this testing wait for epa to issue its guidelines and safeguard the health of the human subjects
i thank the senator from california for her commitment to this issue and i yield the floor
i reserve the balance of my time and yield the floor
 newspeech
mr president i speak about the plight of children afflicted by elevated levels of lead in their blood although it has been three decades since lead was a component of paint the effects of lead paint continue to linger in homes across the country as the lead paint flakes off the dust is inhaled and some kids eat the chips
lead is a highly toxic substance that can produce a range of health problems in young children including damage to the kidneys the brain and bone marrow even low levels of lead in pregnant women infants and children can affect cognitive abilities and fetal organ development and lead to behavioral problems
over 430000 children in america have dangerously high blood lead levels this is a particularly serious problem for illinois which has the highest number of leadpoisoned children in the nation in chicago alone 6000 children have elevated blood lead levels
in 1992 congress passed the residential leadbased paint hazard reduction act the law required the environmental protection agency epa to promulgate regulations by october 1996 regarding contractors engaged in home renovation and remodeling activities that create leadbased paint hazards renovation and repair of older residences is the principal source of leadpaint exposure to us children according to federal studies a large majority of the approximately 20 to 30 million renovations done on older homes each year are done without leadsafe cleanup and contamination practices
the epa analysis has found that a lead paint regulation would protect 14 million children and prevent 28000 leadrelated illnesses every year such a regulation would also lead to a net economic benefit of between usd27 billion and usd42 billion each year
despite the clear health and economic benefits these regulations are now 9 years overdue and there is no sign that epa is moving any closer to issuing the required rules last month i joined with senator boxer and representatives waxman lynch and towns to express our concern about epas complete disregard of the statutory mandate to issue lead paint regulations
to address the problem i have introduced an amendment that would stop epa from spending money on any actions that are contrary to congress 1992 mandate to issue lead paint regulations including any delaying of the regulations i thank the managers of this bill senator burns and senator dorgan for their support of this amendment and for including it in the bill
i hope epa will read this amendment and understand that the time for these commonsense lead regulations is long overdue
 newspeech
mr president as the previous speaker i rise to speak on the central american free trade agreement
i have thought long and hard about this agreement and i come to the floor predisposed to support free trade in the end i believe that expanding trade and breaking down barriers between countries is good for our economy and for our security for american consumers and american workers
on the margins i recognize that cafta although a relatively modest trade agreement by the standards of the us economy would benefit farmers in illinois as well as agricultural and manufacturing interests across the country the language in the agreement is also optimal with respect to intellectual property and telecommunications issues that are of particular interest when it comes to trade with other countries such as china unfortunately cafta falls short as a matter of process and substance in protecting workers rights and interests my colleague senator bingaman mentioned some of those concerns
i recognize that we should not kid ourselves into believing that voting against freetrade agreements will stop globalization especially agreements like cafta where the countries involved have combined economies onesixth the size of the state of illinois
globalization is not someones political agenda it is a technological revolution that is fundamentally changing the worlds economy producing winners and losers along the way the question is not whether we can stop it but how we respond to it it is not whether we should protect our workers from competition but what can we do to fully enable them to compete against workers all over the world
that brings me to the problem so far america has not effectively answered these questions and american workers are suffering as a result i meet these workers all across illinois workers whose jobs moved to mexico or china and are now competing with their own children for jobs that pay usd7 an hour and offer no health or pension benefits in town meetings and union halls i have tried to tell these workers the truth that the jobs they have lost are not coming back that globalization is here to stay and that they are going to have to train more and learn more to get the new jobs of the future
i dont mind delivering that message but when these same workers ask me exactly how are they going to get their training and their education and when they ask what will they do to pay for their health care bills in the interim and how will they deal with lower wages and the general sense of financial insecurity that seems to be growing every single day i cannot look them in the eye and tell them honestly that their government is doing a single thing about these problems
since i have arrived in the senate i havent seen us debate much less pass legislation that would address these issues that is the reason i will be voting against cafta when it comes up later today
there are real problems in the agreement itself it fails to uphold the principles set out in previous trade agreements that say we must give equal protection to the rights of workers and the rights of commercial interests but cafta while encouraging the protection of commercial rights does less to protect labor rights than some of the agreements that we have already passed so there is a sense that we may be going backward instead of forward nor does cafta do much in the way of enforcing environmental standards in these countries
i recognize that no piece of legislation is perfect and if it were just these provisions perhaps i could do what my colleague from new mexico has done and obtain a letter of agreement from the white house indicating they will try to address some of these problems
but the real problem is more than cafta it goes beyond the four corners of this piece of legislation the real problem is what is missing generally from our prevailing policy on trade and globalization meaningful assistance for those who are not reaping the benefits of trade and a plan to equip american workers with the skills and support they need to succeed in the 21st century
so far almost all of our energy and almost all of these trade agreements are about making life easier for the winners of globalization while we do nothing for those who find their lives getting harder as a consequence of trade liberalization in 2004 nearly 150000 workers were certified as having lost their jobs due to trade and were thus eligible for trade adjustment assistance and that number doesnt count the janitors and cafeteria workers who may have lost their jobs
senator wyden and others have tried to encourage the administration to modernize this assistance and expand it to displaced service workers but the administration refuses to help on this issue
but even beyond displaced workers our failure to respond to globalization is causing a race to the bottom that means lower wages and stingier health and retiree benefits for all americans it is causing a squeeze on middleclass families who are working harder but making even less and struggling to stay afloat in this new economy
i recognize the soundness of the economic argument that free trade reduces overall prices in this country but as one downstate worker told me during a recent visit back in illinois it doesnt do me much good if i am paying a dollar less on a tshirt but i dont have a job
so now we have to choose it is a choice that is bigger than cafta and bigger than our trade agreements it is one that america has faced time and time again in our history and we have responded to ease our transition from an agricultural to an industrial economy we set up the public school system busted up monopolies and allowed workers to organize to help us emerge from the great depression we regulated the market created unemployment insurance and provided all workers access to a secure retirement at the end of world war ii we grew the largest middle class in history by providing our returning heroes with a chance to go to college and own their own homes
now we face the same choice we are at the same juncture today we have to decide whether we are going to sit idly by and do nothing while american workers continue to lose out in this new world or if we will act to build a community where at the very least everybody has a chance to work hard get ahead and reach their dreams
if we are to promote free and fair trade and we should then we have to make a national commitment to prepare every child in america with the education they need to compete to make sure college is affordable for everybody who wants to go to provide meaningful retraining and wage insurance so that even if you lose your job you can train for another to make sure worker retraining helps people without getting them caught up in a bureaucracy that such training helps service workers as well as manufacturing workers and that it encourages people to reenter the workforce as soon as possible
we also have to figure out a way to tell workers that no matter where you work or how many times you switch jobs you can take your health care and your pension with you always so you have the flexibility to move to a better job or start a new business
all of this is possible it is not going to be easy and it is not going to be quick i dont expect the administration to try to shoehorn all the solutions to the displacements caused by globalization into a single trade agreement but what i do expect and i said this directly to the president when i met with him in the white house on this matter is that we at least have on a parallel track an effort to deal with the losers in globalization our displaced communities and displaced workers we must not only look after profits and shareholders but also those folks who are adversely affected by trade lower prices are good and important but we also have to make sure that jobs exist that provide people the opportunity to raise a family
mr president in order to compete every single one of us is going to have to work more think more train more i am not afraid of global competition and i dont think a single american worker is afraid of it we cannot insulate ourselves from all of the dislocations brought about by free trade and most of the workers dont expect washington to do so on my side of the aisle we cannot resort to protectionist language over the long term if we are in fact going to be looking toward the future of america we have the talent and the brain power to continue to lead the world in this challenging new century but now we need the political will now we need a national commitment and that so far is what appears to be lacking on capitol hill
in america we have always furthered the idea that everybody has a stake in this country that we are all in it together and that everybody deserves a shot at opportunity the imbalance in this administrations policies as reflected in the cafta debate fails to provide american workers with their shot at opportunity it is time we gave them that shot
i yield back my time
applause in the gallery
 newspeech
mr president i rise today in support of hr 3057 the foreign operations appropriation bill i would also like to highlight one aspect of the bill
since coming to the senate 6 months ago one of the foreign policy and health issues i have focused on relates to the avian flu i am pleased that this bill includes usd10 million to combat the spread of this potential pandemic adding to the usd25 million that the senate provided in the supplemental appropriations bill in april
i thank the managers of this bill senators mcconnell and leahy and their staffs for working with me on this important issue i know that senator mcconnell has a longstanding interest in southeast asia and senator leahy has always been a champion of international health issues making the avian flu something i know they both care deeply about
in the last few weeks scientists have reported that a deadlier version of the avian flu has now spread to migrant birds that could carry the disease out of asia and across the world
while it may not seem that threatening to many americans at first this bird flu could easily transform into a human flu and if it does it could be one of the deadliest flus mankind has ever known even worse than the 1918 flu pandemic that killed 675000 americans and 50 million worldwide
already there have been 108 human cases of avian flu resulting in 54 deaths and while the virus has not yet mutated into a fullblown human flu recent developments suggest it might be heading in that direction in recent months the virus has been detected in mammals that have never previously been infected including tigers leopards and cats
a few weeks ago the world health organization reported that avian flu strains in vietnam are lasting longer and spreading to more humans and according to government officials a few cases of humantohuman spread have already occurred
every day there are new reports about the increasing dangers of the avian flu last month it was revealed that chinese farmers have tried to suppress outbreaks of the avian flu by using human antiviral drugs on infected animals
as a result one strain of the virus has become resistant to these drugs thus making the drugs ineffective in protecting humans against a possible pandemic and just this week researchers found that ducks infected with the virus were contagious for up to 17 days causing the animals to become in the researchers wordsmedical trojan horses for transmitting the disease to humans
simply put the world is not ready for a potential outbreak of this deadly flu in fact we arent even close
there is no known vaccine for the avian flu and producing one could take months once an outbreak occurs and while the world health organization recommends that every nation stockpiles enough flu treatment to treat a quarter of its population the united states has only ordered enough to treat less than 1 percent of ours
we cant just stand by and hope that this virus doesnt reach our shores when it only takes hours to travel from one side of the world to the other it is time for america to lead the world in taking decisive action to prevent a potential global tragedy
we should start by doing what we can to fight the virus while it is still mainly in southeast asia that is why i fought for and obtained usd25 million for prevention efforts by the cdc the agency for international development the health and human services department and other agencies and that is why i requested another usd10 million in this bill
in addition the senate foreign relations committee approved language that i offered directing president bush to form a seniorlevel task force to devise an international strategy to deal with the avian flu and coordinate policy among our government agencies i hope that the bush administration forms this task force immediately without waiting for legislation to be passed
yet these are only modest first steps international health experts believe that southeast asia will be an epicenter of influenza for decades that is why we need to create a permanent framework for curtailing the spread of future infectious diseases a framework that would increase international disease surveillance response capacity and public education and coordination especially in southeast asia
but we must also prepare our own country in the event that a global pandemic reaches america that is why i recently introduced the avian act which helps make sure that americans are protected from a possible outbreak of the avian flu
when the threat is this real we should be increasing research into possible flu vaccines and we should be ordering enough doses of flu treatment to cover the recommended 25 percent of our population just like england and other western countries have done
we should also ensure that our health and human services department and state governments put in place a plan as to how they would address a potential flu pandemic including the purchasing and distributing of vaccines a year after a draft of a federal plan was published a final version has yet to be finalized we shouldnt have to wait any longer because the avian flu certainly wont
we are extremely fortunate that so far the avian flu has not been found in the united states but in an age when you can board planes in bangkok or hong kong and arrive in chicago burlington or louisville in hours we must face the reality that these exotic killer diseases are not isolated health problems half a world away but direct and immediate threats to security and prosperity here at home
again i thank senators mcconnell and leahy for including this important funding in the supplemental appropriations bill and now including additional funding in this bill and i thank the distinguished chairman of the foreign relations committee senator lugar for his leadership on this issue
i ask unanimous consent that several articles and editorials about the avian flu be printed in the record
 newspeech
mr president i rise today to introduce the drinking water security act of 2005
this bill would reauthorize a portion of the safe drinking water act first enacted in 2002 that instructs the environmental protection agency epa and the centers for disease control to develop the tools needed by american drinking water systems to detect and respond to the introduction of biological chemical and radiological contaminants by terrorists my bill also would require epa to report on its progress in developing and implementing these detection and response systems since 2002
like most americans i want to rise in the morning make some coffee and take a shower without worrying if that water has somehow been tampered with overnight by terrorists safe drinking water is something we traditionally have taken for granted in this country this bill will continue the good work our scientists have been doing to monitor detect and negate any chemical biological or radiological agents that terrorists could introduce into our drinking water should they manage to get past our physical security measures this bill would also help implement appropriate warning systems in the event of a terrorist attack on our water systems
i do not want to be an alarmist but september 11 changed americans views on the possibility of the improbable and turned our focus to preparedness this bill is all about preparedness it provides the authorization and oversight needed to continue to develop those tests and responses so we can stay one step ahead of potential terrorists
i hope all of my colleagues join me in supporting this commonsense bill and ensuring that our drinking water remains safe
 newspeech
mr president on may 3 2005 secretary of education margaret spellings announced the selection of 141 outstanding american high school seniors as the 2005 presidential scholars the presidential scholars program serves to honor outstanding students for their accomplishments in academics or the arts as well as for their leadership character and civic contributions to their schools and communities
the united states presidential scholars program was established in 1964 by executive order of president lyndon b johnson the presidential scholars program annually selects one male and one female student from each of the 50 states the district of columbia puerto rico american students living abroad 15 atlarge students and up to 20 students in the arts the students are selected on the basis of outstanding scholarship service leadership and creativity through a rigorous selection and review process administered by the department of education over 5000 of the nations top students have been honored as presidential scholars since this prestigious programs founding
of the 141 exceptional students recognized from across the united states for 2005 i would especially like to recognize three students from the great state of illinois for their accomplishments
i send my congratulations to the following students for their accomplishments in academics kelly a zalocusky from belleville high school east in belleville il and her teacher philip c short and edgar p woznica from fenwick high school in oak park il and his teacher ramzi farran for her accomplishments in the arts i would like to congratulate marcella j capron from loyola academy in wilmette il and her teacher leslie yatabe
please join me in congratulating the 2005 presidential scholars for their accomplishments in academics and the arts i wish them all the best in their future endeavors
 newspeech
mr president today i rise to congratulate an illinois resident who has received national recognition for her contributions to the american artistic community ms albertina walker of chicago the queen of gospel music has been selected to receive a national heritage fellowship by the national endowment for the arts nea
the highest honor in the field of folk and traditional arts these fellowships are awarded to 12 outstanding artists each year to recognize their contributions to their fields they are selected based on their artistic excellence and cultural authencity
national heritage fellowships are not open to application but are based on nominations from members of the public begun in 1982 these fellowships consist of a usd20000 grant and are part of the neas mission of supporting excellence in the arts both new and traditional previous national heritage fellowship recipients have included such artists as bb king and john lee hooker
the grammyaward winning ms walker is a native of chicago and has been involved in gospel music for over 70 years she has recorded over 60 albums and is an active member of west point missionary baptist church
i thank the national endowment for the arts for its recognition of ms walkers outstanding work and once again applaud ms walker for her achievement
 newspeech
mr president i rise today to speak about s 397 the protection of lawful commerce in arms bill also known as the gun liability immunity bill regardless of whether you support this bill or oppose this bill i can certainly understand that the issue of gun liability is an important one
but let me ask my colleagues is this really more important than all the other important issues before the senate right now with only a few days left before the august recess is giving liability protection to gun manufacturers really more important than passing the department of defense authorization bill during a time of war even this bills most vocal supporters could not make this argument with a straight face
as i travel around my state of illinois talking to constituents i hear many concerns from them they tell me about the lack of affordable health care the quality of our nations schools the rising cost of gasoline and the war in iraq parents worry about how the budget deficit will affect their childrens future veterans complain about the long delays in applying for and receiving disability benefits and about the amount of those benefits
my constituents have no shortage of suggestions and ideas for what congress should be doing but i can honestly say that none of them are saying senator please go back to washington and make sure that gun companies arent being sued by victims of gun violence i havent heard that one yet
and that is why i have chosen to speak on the floor today to highlight the misplaced priorities of the senates leadership even though we have 139000 troops fighting for our freedom in iraq and a usd440 billion defense bill that could help these troops we are here debating gun liability instead of talking about how to strengthen our national defense
that is regrettable and that is one of the reasons why so many americans are disillusioned with their government because we are not focusing on the problems that truly matter to them because some are more interested in scoring political points or catering to a special interest
i believe as do my democratic colleagues that the first priority of the senate should be to provide for our men and women who are in harms way and that means spending the necessary time to debate the defense bill if that takes us the rest of the week or even next week then that is what we should do
how can we go home to our constituents in august and tell them that we left washington dc without finishing a bill to help our military because we spent too much time protecting gun manufacturers that is shameful
i have talked to my colleagues on both sides of the aisle and many of them were planning to offer good commonsense bipartisan amendments to the dod bill amendments that would have helped our military and strengthened our national defense i also have filed several amendments that i would have offered and i believe that many of my colleagues would have supported them as well
one of my amendments would have protected members of the national guard and reserve against employment discrimination this amendment is supported by the reserve officers association and is cosponsored by senator salazar
i have heard that there have been instances where prospective employers are reluctant to hire guard and reservists because of fears that these employees could be called up for extended tours of duty these citizensoldiers are getting through initial stages of interviews only to be summarily dropped from the process upon disclosing the fact that they are members of the guard and reserve
my amendment would have gotten to the heart of this problem by preventing employers from forcing members of the guard and reserve to disclose their military service during the interview process however my amendment would not have prohibited them from disclosing their military status if they thought it would be beneficial during an interview process
but instead of helping members of the guard and reserve we are talking about gun manufacturer liability that is wrong
another amendment i would have offered relates to the medical records of our servicemembers
for years the department of defense and the department of veterans affairs have attempted to modernize their medical records to create a twoway exchange of patient health data to better care for our nations service members this would decrease costs and improve the flow of information when active members of the military leave the dod system and move to the va system greater use of technology would also reduce medical errors which kill up to 98000 people a year
unfortunately the dod has not managed to create a fully functional electronic medical records system last year a gao report found that one of the primary reasons for the delay in developing this system is the lack of congressional oversight
my amendment would have helped provide some of that oversight i wanted to get some answers from dod on why this project is being delayed and how the department is proceeding with this important project
but debate over these amendments and many others is being silenced in favor of the one we are having now about helping gun manufacturers
this is why the american people are tired of what goes on in this town because there are real issues they sent us here to debate real problems they expect us to solve but even when we have a chance to do this even when we have a defense bill where we could add amendments that could help our troops and care for our veterans the senate passes on that chance and heads directly into another fight singed with more politics and more ideology
we can do better than that we owe ourselves better and we certainly owe the american people better
i ask unanimous consent that an article from army times criticizing the senate leaderships decision to stop consideration of the dod bill be included in the record
 newspeech
mr president i rise to commend the chief sponsors of this bill in the senate senators domenici and bingaman who i think have displayed the sort of statesmanship and civility in working out this difficult legislation that i think all of us expect from this body i also want to indicate the degree to which this bill takes significant steps in the right direction on energy policy it helps us realize the promise of ethanol as a fuel alternative by requiring 75 billion gallons to be mixed with gasoline over the next few years it provides a tax credit for the construction of e85 stations all over america e85 a blend of ethanol and gasoline that can drastically increase fuel efficiency standards for our cars
it will provide funding for the clean coal technologies that will move america to use its most abundant fossil fuel in a cleaner healthier way including more low emission transportation fuels and it will support the development of what we hope ultimately will be a 500milepergallon automobile technology
all of these things are wonderful and worthy of support but i do have to say we have missed an opportunity and that is not the fault of the sponsors of this bill who have done yeomans work rather i think it is the timidity of all of us as a body in not addressing what has to be one of the most significant problems we face as a nation
the department of energy predicts that american demand for fossil fuels will jump 50 percent over the next 15 years the heritage foundation says this bill will do virtually nothing to reduce our dependence on foreign oil even president bush and supporters of the bill in congress concede as much
as we debate this bill today the price of crude oil has surpassed a record high of usd60 a barrel and gas is now up to usd228 per gallon at this price the united states is sending usd650 million overseas every single day
as demand continues to skyrocket around the world other countries have started to realize that guzzling oil is not a sustainable future what is more these countries have realized that by investing early in the energyefficient technology that exists today they can create millions of tomorrows jobs and build their economies to rival ours
china now has a higher fuel economy standard than we do and it has 200000 hybrids on its roads japans toyota is doubling production of the popular prius in order to sell 100000 in the us next year and it is getting ready to open a brand new plant in china at the same time ford is only making 20000 escape hybrids this year and gms brand wont be on the market until 2007
so here we are people paying record prices at the pump and america sending billions overseas to the worlds most volatile region we have countries like china and india using energy technology to create jobs and wealth while our own businesses and workers fall further and further behind and we have the energy bill that is before us today
so i ask is this the best america can do the country that went to the moon and conquered polio the country that led the technological revolution of the 1990s
it would be one thing if the solutions to our dependence on foreign oil were pieinthesky ideas that are years away but the technology is right at our fingertips today we could have told american car companies we will help you produce more hybrid cars we could have made sure there were more flexible fuel tanks in our cars and so america has a choice
we can continue to hang on to oil as our solution we can keep passing energy bills that nibble around the edges of the problem we can hope that the saudis will pump faster and that our drills will find more and we can just sit on our hands and say that it is too hard to change the way things are and so we might as well not even try
or we could accept and embrace the challenge of finding a solution to one of the most pressing problems of our time our dependence on foreign oil it will not be easy and it will not be without sacrifice government cannot make it happen on its own but it does have a role in supporting the initiative that is already out there
i vote for this bill reluctantly today disappointed that we have missed our opportunity to do something bolder that would have put us on the path to energy independence this bill should be the first step not the last in our journey towards energy independence
i close by saying i hope we do not wait another 5 years before we work on the important issue of energy independence i plan to support this bill because of the fine work that was done by the sponsors but i would insist that in the next year or two we immediately address the issue of how we can wean ourselves off of middle eastern oil
 newspeech
mr president i rise in support of the interior appropriations conference report and to speak about two key provisions one to protect our veterans and one to protect our kids
first the conference report includes a much needed usd15 billion supplemental spending package for veterans health care this usd15 billion will cover the massive budgetary shortfall that congress only recently discovered and i hope this will prevent the loss of some important veterans health care services
earlier this year i along with my democratic colleagues on the senate veterans affairs committee repeatedly asked the department of veterans affairs if the presidents budget provided sufficient funds for veterans health care the response we received was yes the funds are sufficient
unfortunately that response was not consistent with what folks on the ground were saying about va health care services they complained of long waiting periods for doctors appointments reduced office hours at veterans clinics an increased demand for services and reduced access these voices were too loud to ignore so i joined my colleagues senator murray and senator akaka here on the floor of the senate to ask for additional funding for va health care those efforts were defeated but we knew that a possible crisis was on its way
that crisis became a reality when it was discovered that the va was more than usd15 billion in the hole on its health care funding like many of my colleagues in the senate i was shocked by that admission
i was pleased to join senator murray in cosponsoring both a standalone bill and an amendment to the interior appropriations bill to get veterans the funding they need so they can get the health care that they have earned and deserve
the usd15 billion appropriated by todays interior appropriations conference report will help ensure that our nations veterans get that health care with this funding our veterans facilities also will get the maintenance they need and i hope the va will be able to keep its hands out of its rainy day fund
i dont think there is sone person in this senate who would want to tell a returning soldier who fought and bled for our country sorry but when it comes to getting health care you are on your own
i was right the inclusion of this provision in the conference report proves that we can work together to do what is necessary for our nations veterans
i thank senator murray senator craig and senator akaka for their leadership on this issue i hope we can work together as we do today to ensure that veterans are not shortchanged next year they deserve better
second i want to thank my colleagues for including an amendment in the conference report that is important to parents of small children all over the country but particularly in my hometown of chicago i am referring to my amendment prohibiting epa from spending tax dollars to delay the promulgation of regulations that are now 9 years overdue these regulations when promulgated would require contractors to reduce lead paint exposure during home renovation and remodeling
i have raised this issue with epa on numerous occasions and reminded them of the serious health dangers that high blood lead levels pose for children now reluctantly epa officials have promised me these rules will be issued by the end of the year i intend to use this amendment to hold them to their word so today when we pass this funding bill i can tell the youngest poorest citizens of illinois that congress is doing its part to keep them safe from lead paint exposure
i ask unanimous consent that my letter to epa administrator johnson regarding this issue be printed into the record
 newspeech
mr president i am pleased that the conference report on hr 3 the safe accountable flexible efficient transportation equity act a legacy for users protects an important program administered by the department of transportation the disadvantaged business enterprise program also known as the dbe program
the dbe program ensures that small businesses owned and controlled by socially and economically disadvantaged individuals are able to compete on a level playing field for federally funded highway and transit contracts
i strongly endorse the dbe program and am pleased that this program continues to enjoy bipartisan support
since the dbe program was started in 1982 the field of highway contractors has grown more racially diverse the dbe program was expanded to include women in 1987 and that improvement to the program has opened the doors for women contractors to join what has traditionally been an allmale field despite the increased fairness and greater opportunity for minority and women contractors since the inception of dbe there continues to be a strong need for the dbe program
unfortunately studies have shown that when dbe programs end many contractors simply revert to their old practices denying contracts to small companies owned by minorities or the economically disadvantaged it is clear that the dbe program is still needed to secure the gains made and encourage even greater opportunity for these small businesses and i am pleased that the conferees have recognized that continuing need and have retained this program
federally funded highway and transit contracts are big business and it is imperative that we give everyone the big guys and the little guys a fair opportunity to take part the dbe is vital to increasing participation in our federally funded highway projects
 newspeech
mr president i rise to address what has been a heartbreaking week for all of us as was mentioned by my distinguished colleague from washington i just returned from a trip from houston with former presidents clinton and bush as part of a fundraising effort to deal with the aftermath of hurricane katrina as we wandered through the crowd we heard in very intimate terms some of the heartwrenching stories that all of us have witnessed on television over the past several days mothers separated from their babies adults mourning the loss of elderly parents descriptions of the heat filth and fear of the superdome and of the convention center in new orleans
there was an overriding sense of relief in houston and the officials in houston and in the entire state of texas deserve great credit for the outstanding job they have done in creating a clean and stable place for the tens of thousands of families who have been displaced
a conversation i had with one woman captured the realities that are settling into the families as they face the future she said to me we had nothing before the hurricane and now we have less than nothing we had nothing before the hurricane now we have less than nothing
in the coming weeks as the images of the immediate crisis fade and this chamber becomes consumed with other matters we will be hearing a lot about lessons learned and steps to be taken i will be among those voices who will be calling for action in the most immediate term we will have to assure that the efforts at evacuating families from the affected states proceed they are not finished yet that these americans who are having to flee their homes their cities their counties and their towns are fed clothed housed and provided with the medical care and medicine they need
we are also going to have to make sure we cut through the redtape that has inexcusably prevented so much help from getting to the places where it is needed i can say from personal experience over the last week how frustrating it has been how unconscionable it has been to be unable to find somebody in charge so that we can get medical supplies doctors nurses and other supplies down to the affected areas quickly enough
we are going to have to make sure in this chamber that any impediments that may continue to exist in preventing relief efforts from moving forward rapidly are eliminated
once we stabilize the situation this country is going to face the enormous challenge in providing stability for displaced families over the months and years that it is going to take to rebuild already the state of illinois has committed to accepting 10000 displaced families there are stories in illinois as there are all across the country of churches mosques synagogues and individual families welcoming people with open arms and no strings attached
indeed if there is any bright light that has come out of this disaster it is the degree to which ordinary americans have responded with speed and determination even as their government has responded with what i consider to be unconscionable ineptitude which brings me to the next point once the situation is stable once families are settled for at least the short term once children are reunited with their parents and enrolled in school and the wounds both on the outside and on the inside have healed we are going to have to do some hard thinking about how we could have failed our fellow citizens so badly and how we will prevent such failures from ever occurring again
it is not politics to insist that we have an independent commission to examine these issues it is not politics indeed one of the heartening things about this crisis has been the degree of outrage that has come from across the political spectrum from across races across incomes the degree to which the american people sense that we can and we must do better and a recognition that if we cant cope with a crisis that has been predicted for decades a crisis in which we were given 4 to 5 days notice then how can we ever hope to respond to a serious terrorist attack in a major american city in which there is no notice and in which the death toll and the panic and the fear may be far greater
that brings me to my final point there has been a lot of attention in the media about the fact that those who were left behind in new orleans were disproportionately poor and disproportionately african american i have said publicly that i do not subscribe to the notion that the painfully slow response of fema and the department of homeland security was somehow racially based i do not agree with that i think the ineptitude was colorblind but what must be said is that whoever was in charge of planning and preparing for the worstcase scenario seemed to assume that every american has the capacity to load up the family in a suv fill it up with usd100 worth of gasoline stick some bottled water in the trunk and use a credit card to check into a hotel on safe ground i see no evidence of active malice but i see a continuation of passive indifference on the part of our government toward the least of us
so i hope that out of this crisis we all begin to reflect democrats and republicans black and white young and old poor and wealthy i hope we all begin to reflect not only on our individual responsibilities to our families and ourselves but on our mutual responsibilities to our fellow americans mutual responsibilities that reflect themselves in church and community organizations and block clubs but also express themselves through our government
i hope we realize the people of new orleans were not just abandoned during the hurricane they were abandoned long ago to murder and mayhem in their streets to substandard schools to dilapidated housing to inadequate health care to a pervasive sense of hopelessness
that is the deeper shame of this past week that it has taken a crisis such as this to awaken in us the understanding of the great divide that continues to fester in our midst that is what all americans are truly ashamed about that is what i am ashamed about and the fact that we are ashamed about it is a good sign the fact that all of us dont like to see such a reflection of this country that we love tells me that the american people have better instincts and a broader heart than our current politics would indicate we had nothing before the hurricane the woman told me now we have even less i hope we all take the time to ponder the truth of that message
 newspeech
mr president today i speak in honor of chief justice william rehnquist the chief justice served this nations highest court with distinction and honor for more than three decades and his career in public service started years earlier even as he battled cancer over the past year he continued to be an example of personal strength dignity and fortitude i join my colleagues in mourning his passing and offering my prayers to his family
the chief justice was a staunch defender of the supreme court and an active independent judiciary he was admired as a warm and helpful colleague a thoughtful mentor and an extremely effective administrator of the federal court system the courts were well cared for under his distinguished leadership
justice rehnquist also engaged directly with many of the toughest constitutional controversies of the twentieth century although i often disagreed with his decisions justice rehnquists opinions have been the source of important scholarship and litigation like the chief justice he followed the late earl warren justice rehnquist will be remembered as an important historical figure whose legacy will impact generations of americans
i knew the chief justice only at a distance as a lawyer and a constitutional law instructor i was required to wrestle intellectually with his ideas and arguments and to press my students to divine his judicial instincts and motivations my regret is that i never got to know him personally or even to join one of his legendary walks around the capitol or monthly poker games i know that his warmth and humor have touched many of my colleagues and he will be missed
of course the strength of our constitutional structure is that it is greater than any individual each of us plays but a small role in designing or building or repairing that structure it is greater and more important than any of us we mourn the passing of justice rehnquist and now look to the future and the important work to be done
 newspeech
mr president today i introduce a bill to provide some assistance to those tens of thousands of americans who have found themselves in one of the worst nightmares i can imagine they are separated from their spouses their children and their parents these americans are struggling to locate their loved ones displaced by the horrors of hurricane katrina they are searching the astrodome combing the internet hoping that their family members have survived the storm and will get in touch with them a similar plight is faced by those victims who have weathered the storm and want to tell their family and friends that they are okay
i have received dozens of calls to my offices in illinois from constituents asking my caseworkers to help them locate their relatives lost in the gulf coast greta from chicago was looking for her aunt perra lee john from romeoville was looking for his children and grandchildren in biloxi the calls kept coming but my staff could only point these constituents to various nonprofit organizations doing their best to provide locator services there was no centralized federal government system in place to deal with this issue such a system should be in place
the government must provide these people with a means to let their families know that they are out of harms way various nonprofit organizations and news services have done a stellar job at using the internet to connect displaced people with their families our government through the department of homeland security should synthesize the best aspects of these services so that after an emergency displaced individuals can call one phone number or go to one website and post their location and condition family members and law enforcement officials should be able use this same secure centralized system to check the status of missing loved ones
i am introducing a bill the national emergency family locator system act that will instruct the department of homeland security to create such a system i hope that the next time our country experiences a disaster like katrina this system will provide worried families with some sense of relief
i hope my colleagues will support this legislation and i ask unanimous consent that the text of the bill be printed in the record
there being no objection the bill was ordered to be printed in the record as follow
be it enacted by the senate and house of representatives of the united states of america in congress assembled
this act may be cited as the national emergency family locator act
in this act 1 departmentthe term department means the department of homeland security 2 emergencythe term emergency has the meaning given the term in section 102 of the robert t stafford disaster relief and emergency assistance act 42 usc 5122 3 secretarythe term secretary means the secretary of homeland security 4 systemthe term system means the national emergency family locator system established under section 3a
a in generalnot later than 180 days after the date of enactment of this act the secretary shall establish within the department the national family locator system b purposesthe purposes of the system are 1 to enable individuals displaced by an emergency to provide to the department the name and location of the displaced individuals and any other relevant information using the telephone the internet and other means determined to be appropriate by the secretary and 2 to enable the department a to compile the information collected under paragraph 1 and b to provide the information collected and compiled under the system to the family members of the displaced individuals and law enforcement officials c considerationsin establishing the system under subsection a the secretary shall take into account and to the maximum extent practicable incorporate into the system 1 intermediarybased locator systems such as the national next of kin registry and 2 information from existing family locator databases such as the family news network of the international committee of the red cross
there are authorized to be appropriated such sums as are necessary to carry out this act
 newspeech
mr president i rise in strong support of the amendment offered by senator harkin to increase funding for the legal services corporation i am proud to be a cosponsor of the amendment
the legal services corporation provides vital legal assistance to the poor around the country it was created in 1974 with bipartisan congressional sponsorship and the support of the nixon administration
in chicago the legal services corporation funds make it possible for the legal assistance foundation to help my constituents navigate the foster care system and receive compensation after violent crimes in galesburg and peoria these funds make it possible for the prairie state legal services organization to help people dealing with domestic violence issues and elder abuse
in the aftermath of hurricane katrina you can bet that legal services corporation will be in louisiana alabama mississippi and the many states where hurricane victims are being relocated helping newly impoverished citizens obtain food and shelter assistance health care and insurance benefits unemployment insurance social security benefits and fema assistance
this program makes a real difference in peoples lives take the story of irene and her family for example who live in section 8 housing and needed help they visited the prairie state legal services office in illinois every day irene had to get two wheelchairbound grandchildren up the stairs and into a second floor apartment both her grandchildren have cerebral palsy and are confined to wheelchairs the oldest is now 14 and weighs 160 lbs and after 11 years as i am sure you can imagine irene was having a hard time getting her grandchildren up those stairs but when she tried to make this difficult situation better it only got worse
irene applied for and received a transfer certificate from section 8 to allow her to move to a new apartment but she could not find a firstfloor apartment to transfer to within the 60 days that the transfer allowed irene tried calling the section 8 offices to let them know of the delay but she was forced to leave messages when she finally sent a letter asking for a response to her messages she was informed that she was too late not only was the public housing agency terminating her transfer it was also terminating the section 8 subsidy for her current apartment
but that is when prairie state and legal services corporation intervened a staff attorney represented irene in an administrative appeal and pointed out that under the fair housing act and the americans with disabilities act irene had not been provided the support needed to assist her in finding an apartment as a result of her attorneys efforts irenes subsidy was reinstated she was given a new transfer certificate and was provided with active assistance in helping her find a new apartment
legal services corporation helps folks like irene all across the country from south carolina to south dakota illinois to iowa and when someone displaced by hurricane katrina cannot afford a lawyer but is having trouble getting her unemployment insurance or social security benefits or getting her utilities turned back on legal services corporation will be right there legal services corporationfunded organizations have won dozens of awards and groups ranging from aarp to the american bar association have voiced their strong support of lsc we should do the same
over the last decade the lsc budget has suffered usd196 million in cuts the appropriations committee proposed this year to cut usd6 million more i do not think this is the time to deny legal services to those who need them most i believe that in light of the pressing crises confronting individuals in the gulf coast we should be increasing funding for the legal services corporation not decreasing it so i strongly support senator harkins amendment and i urge my colleagues to do the same
 newspeech
mr president one of the most striking things about the devastation caused by hurricane katrina is that the majority of stranded victims were our societys most vulnerable members lowincome families the elderly the homeless the disabled many did not own cars many believed themselves unable to flee the city unable to forego the income from missed work unable to incur the expenses of travel food and lodging some may have misunderstood the severity of the warnings if they heard the warnings at all some may have needed help that was unavailable whatever the reason they were not evacuated and we have seen the horrific results
this failure to evacuate so many of the most desperate citizens of the gulf coast leads me to introduce today a bill to require states and the nation to consider the needs of our neediest citizens in times of emergency
it appears that certain assumptions were made in planning and preparing for the worst case scenario in the gulf coast after all most of those who could afford to evacuate managed to do so they drove out of town and checked into hotels or stayed with friends and family but what about the thousands of people left behind because they had special needs
how many of us will forget the tragedy that occurred at st ritas nursing home in st bernard parish la where an estimated 32 of the 60 residents perished in the rising floodwaters in the aftermath of hurricane katrina
our charge as public servants is to worry about all of the people i am troubled that our emergency response and disaster plans were inadequate for large segments of the gulf coast population i wonder whether the plans in other regions are adequate perfect evacuation planning is obviously impractical but greater advance preparation can ensure that the most vulnerable are not simply forgotten or ignored
thats why the bill i am introducing today along with cosponsors senators bayh murray harkin levin corzine feingold bingaman and kennedy requires the secretary of the department of homeland security to mandate each state to include plans for the evacuation of individuals with special needs during times of emergency such plans should not only include an explanation of how these people low income individuals and families the elderly the disabled those who cannot speak english will be evacuated out of the emergency area and how the states will provide shelter food and water to these people once evacuated
communities with special needs may be more challenging to accommodate but they are every bit as important to protect and serve in the event of an emergency
what we saw in the gulf coast cannot be repeated we may not be able to control the wrath of mother nature but we can control how we prepare for natural disasters
i hope my colleagues will join me in supporting this legislation
 newspeech
mr president every year the interassociation task force on alcohol and other substance abuse issues iatf gives awards to three universities for their programs against underage drinking and overconsumption of alcoholic beverages bradley university which is located in peoria il is among the three universities being commended and has the honor of being a national collegiate alcohol awareness week winner for 20042005
i applaud the wellness program at bradley university for its commitment to reduce underage drinking alcohol misuse on college campuses is not a new problem it is entrenched in the culture of many institutions of higher learning and in students social lives the abuse of alcohol among college students is taking its toll not only on the students who drink alcohol in excess but also their student peers college administrators health care personnel who counsel student drinkers and the community at large
help encourage and teach heat and social norming sonor are two programs that have proven to be effective at bradley university heat strives to provide students with opportunities to promote positive lifestyle choices such as peerled workshops and demonstrations the sonor program utilizes creative marketing and advertising strategies to disseminate information about healthy living these programs are making a difference in the lives of our youth by encouraging college students to make healthy and responsible decisions
congratulations to dr alan galsky associate provost for student affairs melissa sagebollenbach the wellness program coordinator at bradley university kelcy hale the president of social norming and chrisandra ashby the former president of heat for their tireless efforts and outstanding leadership
bradley university is to be commended for its innovative and influential alcoholabuse programs representatives of bradley university recently came to washington dc and visited their senators offices to present their alcohol awareness programs these award recipients also received a plaque and an award check from dr edward hammond chairman of the iatf
i wish the wellness program and its leadership the best of luck in their future endeavors and applaud their outstanding achievement
 newspeech
mr president today i am submitting a resolution to express the senates strong disapproval of recent efforts to disenfranchise americans
in the weeks since hurricane katrina ravaged the gulf coast our country has been awakened to the plight of the most vulnerable americans the poor the elderly the sick and the disabled and if we have learned anything from this tragedy it is that the government has too often ignored the needs of these citizens in crafting national policy whether it is homeland security or education or health care these americans have consistently been left behind
now we are in danger of proceeding down another path that disregards the needs of our nations neediest the right to vote this is the most fundamental right protected by the constitution and the right for which many americans have fought and died
the last two presidential elections were tainted by allegations of fraud and abuse the complaints ranged from long polling lines to faulty machines to confusing ballots the rampant complaints have shaken peoples confidence in our election system and so it is all of our duty to work to restore and protect the integrity of the electoral process
unfortunately in this new millennium too many electoral reform efforts seem intent on limiting access to the ballot as opposed to expanding it in the mid20th century the poll tax was the preferred means of disenfranchising large minority populations specifically african americans today the poll tax is taking on a new form a photo identification requirement for voters
according to the national commission on federal election reform such a requirement would impose an additional expense on the exercise of the franchise a burden that would fall disproportionately on people who are poorer and urban nevertheless a number of states including georgia have recently passed laws mandating governmentissued photo identification for voters at the polls
in georgia alone at least 150000 senior citizens do not have governmentissued photo identification which can cost up to usd85 nationwide at least 12 percent of eligible drivers do not have a drivers license and georgia has made it difficult for rural and urban folks to obtain their voter photo identification there are currently only 56 places in all 159 counties where such identification is available with no places available in atlanta for people who already lack transportation which may be why they do not have drivers licenses it is farfetched to think that these same people could easily get to another county to obtain a voter identification card earlier today the carterbaker commission on federal election reform released its recommendations for improving the electoral process while many of the commissions recommendations are worthy of consideration its report recommends the implementation of a national voter identification requirement despite acknowledging that there is no evidence of extensive fraud in us elections or of multiple voting
this past weekend afghanistan held its second successful national election and we have seen successes in iraqs elections as well if these nascent democracies can commit themselves to bringing any and all citizens to the polls surely we can do the same
many of us both here in washington and around the country have been asking questions over the past three weeks about our nations priorities and our commitment to helping our countrys most vulnerable citizens but a major priority should be ensuring that these citizens can exercise the most fundamental right in a democracy the right to vote
the resolution i am submitting today along with senator dodd and joined by senators reid corzine clinton harkin feingold akaka dorgan kennedy kerry mikulski lautenberg and others expresses the senates strong disapproval of photo identification requirements for voting the resolution also urges the department of justice to challenge any state law that limits a citizens ability to vote based on discriminatory photo identification requirements and urges the rejection of any national photo identification requirements for voting
i am honored that representative john lewis a civil rights icon who put his life on the line to fight for the right to vote will be introducing the same resolution in the house later this week
i urge my colleagues to support this important resolution
 newspeech
mr president i rise today to commend my colleague the senior senator from illinois mr durbin for his work in crafting this legislation of which i am a cosponsor this amendment would provide critically needed disaster relief to illinois farmers who face significant financial jeopardy from crop losses due to this seasons historic drought
illinois agriculture is experiencing one of the driest periods in the last century and certainly one of the most severe droughts in two decades illinois is the nations leading producer of corn and soybeans however us department of agriculture usda reports show that more than half of the corn crop and almost a third of the soybean crop have been decimated by drought of the 102 counties in illinois 98 have reported crop damage due to the lack of rainfall
in july senator durbin and i asked the secretary of agriculture to declare the affected counties in illinois an agriculture disaster area i am pleased that president bush granted our request to give our illinois farmers some muchdeserved relief qualifying illinois farmers for usda assistance programs including lowinterest emergency loans
while this action provided an important amount of economic assistance the scope and severity of this years drought requires that additional measures be taken at the present time most of northern and western illinois remains in a severe or extreme drought much of eastern illinois is classified as abnormally dry this is particularly alarming because farmers are at a critical point in the growing season
moreover the reduction in fuel refining capacity caused by hurricane katrina has resulted in illinois farmers facing a sudden surge in unanticipated fuel costs on top of already escalating fuel prices the disruption in mississippi river traffic at gulf ports where half of the nations grain exports are shipped for foreign markets has spiked shipping costs for farm commodities transported by barge downriver the threat of an aflatoxin outbreak that affects corn during times of crop stress and drought is also of particular concern in recent weeks should this condition progress after harvest and storage farmers may face additional financial consequences in the coming months
i understand that the senior senator from mississippi mr cochran has made a commitment to address this issue in the next hurricane supplemental appropriations bill that is sent to congress given that commitment i support senator durbins decision to withdraw the amendment and i thank senator cochran for his cooperation
 newspeech
mr president i rise today to support this important piece of legislation as many of you know the young soldiers returning from iraq and afghanistan are already coming home with post traumatic stress disorder a recent army study found that one in six soldiers in iraq reported symptoms of major depression some experts predict that more than 100000 soldiers may need some kind of mental health treatment when they come home
it is not only our patriotic duty to provide these soldiers with the benefits they deserve it is our moral duty at the most fundamental level unfortunately ptsd is a disease that is still all too often misunderstood and as i speak there are efforts at the va to require those folks who suffer ptsd to undergo additional scrutiny in the disability benefits process
according to va it will review 72000 cases in which the maximum amount of ptsd disability benefits was awarded the rationale for reviewing these cases is vas belief that 25 percent of these cases are potentially fraudulent but notably this review will entirely ignore cases in which benefits may have been unjustly denied
this review sends a troubling message to the brave men and women who defended this country too many veterans see the va as a bureaucracy with the singular goal of denying services and benefits to veterans this decision to reopen only approved ptsd claims merely serves to promote that impression
it is unconscionable for our government to put the onus on lawabiding veterans to affirmatively demonstrate that they are not engaging in fraud the process of gathering evidence to prove ptsd disability is extremely time consuming it requires the compilation of medical records military service records and testimony from other veterans who can attest to a persons combat exposure i cannot fathom why the va would require veterans to go through this emotionally painful process for a second time
the va and our nations veterans would be better served by creating nationwide standards for evaluating ptsd claims as underscored by the inspector generals report in may that evaluated the chronic disparity between benefits received by veterans in illinois and veterans in the rest of the country ptsd is a highly subjective evaluation subject to significant variation that same report uncovered significant variation in ptsd ratings from state to state with illinois consistently in the bottom rung for those ratings the variation in ptsd ratings across the country may very well be the result of a lack of training or standardized practices on the part of the va not fraud on the part of our nations veterans
i am pleased that chairman hutchison and ranking member feinstein worked with me and senators durbin and murray to include an amendment that prohibits the va from proceeding with its review unless and until the va reports to the appropriations committee on its plan for implementing this recommendation and outlines the staffing and funding requirements
while this is an important provision i am disappointed that there was no requirement that the va look at denials of benefits as well as grants to get an accurate and fair depiction of ptsd claims in this country we need to ensure that denials are reviewed as well as grants of benefits i will continue to work with my colleagues to see that this fundamental issue of fairness is addressed
i also want to thank chairman hutchison and ranking member feinstein for their assistance in accepting an amendment to provide notice to veterans in certain states about their right to seek a review of their cases
this provision addresses an important issue in illinois as some of you may know illinois has for more than two decades ranked 50th out of all 50 states in terms of disability benefit compensation this staggering disparity in payments may well be the result of poor staffing and a lack of standards for disability payments across the nation
i have been pleased that secretary nicholson has agreed to provide the veterans of illinois with extra disability raters so that the veterans in illinois who may have been unjustly denied benefits will have the opportunity to seek a special review of their cases
unfortunately up to now there has been no special effort made to alert veterans to this special opportunity this amendment will provide the funds for information campaigns in states with less than average disability compensation rates these campaigns will alert veterans of the past history of belowaverage disability benefit rates and provide these veterans with information on how to request a review of any past claims the hiring of additional disability raters is important but it is meaningless unless veterans know of their right to get their cases reopened
i thank my colleagues for their assistance with these amendments
 newspeech
mr president i ask unanimous consent that the order for the quorum call be dispensed with
 newspeech
mr president thank you very much first of all let me congratulate senator specter and senator leahy for moving the process of confirming the nomination of judge roberts along with such civility a civility that i believe speaks well of the senate
let me also say that i remain distressed that the white house during this confirmation process which overall went smoothly failed to provide critical documents as part of the record that could have provided us with a better basis to make our judgment with respect to the nomination this white house continues to stymie efforts on the part of the senate to do its job i hope with the next nominee who comes up for the supreme court that the white house recognizes that in fact it is its duty not just to the senate but to the american people to make sure we can thoroughly and adequately evaluate the record of every single nominee who comes before us
having said that the decision with respect to judge roberts nomination has not been an easy one for me to make as some of you know i have not only argued cases before appellate courts but for 10 years was a member of the university of chicago law school faculty and taught courses in constitutional law part of the culture of the university of chicago law school faculty is to maintain a sense of collegiality between those people who hold different views what engenders respect is not the particular outcome that a legal scholar arrives at but rather the intellectual rigor and honesty with which he or she arrives at a decision
given that background i am sorely tempted to vote for judge roberts based on my study of his resume his conduct during the hearings and a conversation i had with him yesterday afternoon
there is absolutely no doubt in my mind judge roberts is qualified to sit on the highest court in the land moreover he seems to have the comportment and the temperament that makes for a good judge he is humble he is personally decent and he appears to be respectful of different points of view it is absolutely clear to me that judge roberts truly loves the law he couldnt have achieved his excellent record as an advocate before the supreme court without that passion for the law and it became apparent to me in our conversation that he does in fact deeply respect the basic precepts that go into deciding 95 percent of the cases that come before the federal court adherence to precedence a certain modesty in reading statutes and constitutional text a respect for procedural regularity and an impartiality in presiding over the adversarial system all of these characteristics make me want to vote for judge roberts
the problem i face a problem that has been voiced by some of my other colleagues both those who are voting for mr roberts and those who are voting against mr roberts is that while adherence to legal precedent and rules of statutory or constitutional construction will dispose of 95 percent of the cases that come before a court so that both a scalia and a ginsburg will arrive at the same place most of the time on those 95 percent of the cases what matters on the supreme court is those 5 percent of cases that are truly difficult in those cases adherence to precedent and rules of construction and interpretation will only get you through the 25th mile of the marathon that last mile can only be determined on the basis of ones deepest values ones core concerns ones broader perspectives on how the world works and the depth and breadth of ones empathy
in those 5 percent of hard cases the constitutional text will not be directly on point the language of the statute will not be perfectly clear legal process alone will not lead you to a rule of decision in those circumstances your decisions about whether affirmative action is an appropriate response to the history of discrimination in this country or whether a general right of privacy encompasses a more specific right of women to control their reproductive decisions or whether the commerce clause empowers congress to speak on those issues of broad national concern that may be only tangentially related to what is easily defined as interstate commerce whether a person who is disabled has the right to be accommodated so they can work alongside those who are nondisabled in those difficult cases the critical ingredient is supplied by what is in the judges heart
i talked to judge roberts about this judge roberts confessed that unlike maybe professional politicians it is not easy for him to talk about his values and his deeper feelings that is not how he is trained he did say he doesnt like bullies and has always viewed the law as a way of evening out the playing field between the strong and the weak
i was impressed with that statement because i view the law in much the same way the problem i had is that when i examined judge roberts record and history of public service it is my personal estimation that he has far more often used his formidable skills on behalf of the strong in opposition to the weak in his work in the white house and the solicitor generals office he seemed to have consistently sided with those who were dismissive of efforts to eradicate the remnants of racial discrimination in our political process in these same positions he seemed dismissive of the concerns that it is harder to make it in this world and in this economy when you are a woman rather than a man
i want to take judge roberts at his word that he doesnt like bullies and he sees the law and the court as a means of evening the playing field between the strong and the weak but given the gravity of the position to which he will undoubtedly ascend and the gravity of the decisions in which he will undoubtedly participate during his tenure on the court i ultimately have to give more weight to his deeds and the overarching political philosophy that he appears to have shared with those in power than to the assuring words that he provided me in our meeting
the bottom line is this i will be voting against john roberts nomination i do so with considerable reticence i hope that i am wrong i hope that this reticence on my part proves unjustified and that judge roberts will show himself to not only be an outstanding legal thinker but also someone who upholds the courts historic role as a check on the majoritarian impulses of the executive branch and the legislative branch i hope that he will recognize who the weak are and who the strong are in our society i hope that his jurisprudence is one that stands up to the bullies of all ideological stripes
let me conclude with just one more comment about this confirmation process
i was deeply disturbed by some statements that were made by largely democratic advocacy groups when ranking member senator leahy announced that he would support judge roberts although the scales have tipped in a different direction for me i am deeply admiring of the work and the thought that senator leahy has put into making his decision the kneejerk unbending and what i consider to be unfair attacks on senator leahys motives were unjustified unfortunately both parties have fallen victim to this kind of pressure
i believe every senator on the other side of the aisle if they were honest would acknowledge that the same unyielding unbending dogmatic approach to judicial confirmation has in large part been responsible for the kind of poisonous atmosphere that exists in this chamber regarding judicial nominations it is tempting then for us on this side of the aisle to go tit for tat
but what i would like to see is for all of us to recognize as we move forward to the next nominee that in fact the issues that are confronted by the supreme court are difficult issues that is why they get up to the supreme court the issues facing the court are rarely black and white and all advocacy groups who have a legitimate and profound interest in the decisions that are made by the court should try to make certain that their advocacy reflects that complexity these groups on the right and left should not resort to the sort of broadbrush dogmatic attacks that have hampered the process in the past and constrained each and every senator in this chamber from making sure that they are voting on the basis of their conscience
thank you very much mr president i suggest the absence of a quorum
 newspeech
mr president i rise to speak in support of the hurricane katrina fasttrack refunds for working families act of 2005 a bill i am introducing with senators murray corzine kerry and levin to accelerate the earned income tax credit and the child tax credit for some of the neediest victims of hurricane katrina
a few weeks ago i visited some of the victims who had been evacuated to the reliant center in houston these families have nothing left imagine having nothing left all their belongings have been destroyed or washed away and most of their jobs have simply vanished
we have done a lot of good work here in the senate so far to bring tax relief and emergency support to these families and many of us are hard at work now developing strategies for the longterm rebuilding of the gulf coast in such a way that doesnt recreate the poverty and inequality of the past but instead builds a more hopeful region with greater opportunity for all of its residents
but there is more we can do quickly to help affected families reestablish and resettle their lives and also to stimulate their local economies in the past we have accelerated tax refunds with the goal of economic stimulus in 2001 congress directed the irs to provide an advance tax rebate of 2001 taxes and in 2003 congress accelerated the child credit now with the dual goals of economic stimulus and support for needy americans we should do it again
fasttracking refunds will put money into the hands of parents that they can use for food clothing housing transportation medical services whatever they need how they spend the money is up to them but its up to us to make sure they get it as soon as possible its up to us to make sure the necessary outreach systems and delivery mechanisms are in place
and thats what this legislation does it directs the secretary of the treasury to refund or credit eligible taxpayers from the affected region as rapidly as possible and to take the steps necessary to get the funds into the hands of eligible recipients companion legislation has been introduced by reps emanuel melancon taylor and lewis in the house of representatives
 by mr enzi for himself and mr kennedy
s 1771 a bill to express the sense of congress and to improve reporting with respect to the safety of workers in the response and recovery activities related to hurricane katrina and for other purposes read the first time
 newspeech
mr president it is my pleasure to join senator clinton to introduce legislation that will help us all find common ground on the debate over patient safety and medical malpractice claims
today medical error is the eighth leading cause of death in the united states every year these tragic mistakes cost the lives of up to 98000 americans this is unacceptable in america and we must do more to ensure that every patient gets the right care at the right time in the right way
the debate in washington over this issue has been centered on caps and lawsuits but across america hospitals and medical providers are proving that theres a better way to protect patients and doctors all while raising the quality of our care and lowering its cost
from the childrens hospitals and clinics of minnesota to the va hospital in lexington kentucky doctors and administrators arent trying to cover up medical errors theyre trying to admit them instead of closing ranks and keeping the patient in the dark theyre investigating potential errors apologizing if mistakes have been made and offering a reasonable settlement that keeps the case out of court
this program is often known as sorry works and its led to some amazing results when patients are treated with respect and told the truth they sue less more are actually compensated for their injuries but medical providers pay less because the reward is the result of a settlement not an expensive lawsuit malpractice costs for doctors go down and health care professionals actually learn from their mistakes so theyre not repeated and lives are saved
at the va hospital in lexington kentucky this program has reduced the average settlement to usd16000 compared with usd98000 nationwide this ranked in the lowest quartile of all va facilities for malpractice payouts at the university of michigans hospital system this program helped them cut their lawsuits in half and save up to usd2 million in defense litigation
the bill were introducing today builds on these hopeful results and incorporates them into a national program the national medical error disclosure and compensation act or medic act will help reduce medical error rates and medical malpractice costs by opening the lines of communication between doctors and patients encouraging honesty and accountability in the process
the bill will also set up a national patient safety database which will be used to determine best practices in preventing medical errors improving patient safety and increasing accountability in the healthcare system
we expect participants to see a cost savings and we will require them to reinvest a portion of these savings into patient quality measures that will reduce medical errors this bill also requires that some of these savings are passed along to providers in the form of lower malpractice insurance premiums
 by mr cornyn for himself mr leahy mr hatch and mr kohl
s 1785 a bill to amend chapter 13 of title 17 united states code relating to the vessel hull design protection to clarify the distinction between a hull and a deck to provide factors for the determination of the protectability of a revised design to provide guidance for assessments of substantial similarity and for other purposes to the committee on the judiciary
 newspeech
i was not aware my senior colleague from illinois was going to speak so i dont want to unnecessarily hold up the entire chamber
 newspeech
thank you very much mr president i will be brief i know we have gone way over the time here today
mr president in the midst of so much difficulty that our nation is facing katrina and rita the ongoing challenges in iraq and afghanistan i recognize it is hard to get the public the leadership in congress and senior administration officials to focus on yet one more challenge
but as has already been stated by the democratic leader harry reid and my senior colleague the minority whip senator dick durbin this is a crisis to which the entire country simply must awaken itself
when i started talking about this 7 months ago not too many folks paid attention perhaps because the shorthand for this looming crisis is the bird flu people assume it is just going to get birds and animals sick
in reality however what is at stake here is the potential of a pandemic that we have not seen in the united states since 1918 1919 as has already been stated here tonight our top scientists and medical personnel including the heads of the nih cdc and the department of health and human services all agree that it is almost inevitable that an avian flu pandemic will strike
the key question is the extent of the damage especially in terms of lives lost the answer to this question will in large measure depend on our level of preparedness and the amount of resources we are willing to immediately commit to deal with this looming crisis
over the last few months we have seen alarming reports from countries all over asia indonesia china vietnam thailand and russia just to name a few about deaths that have resulted from the avian flu
the situation has turned so ominous that dr julie gerberding the director of the cdc said that an avian flu outbreak is the most important threat that we are facing today
international health experts say that two of the three conditions for an avian flu pandemic in southeast asia already exist
first a new strain of the virus called h5n1 has emerged and humans have little or no immunity to it second this strain has demonstrated the ability to jump between species
the only thing preventing a full blown pandemic is a lack of efficient transmission of this strain from human to human once that happens as a consequence of international travel and commerce there is not going to be any way to effectively contain this pandemic
moreover the news on this last point is not good in recent months the virus has been detected in mammals that have never previously been infected including tigers leopards and domestic cats this suggests that the virus is mutating and could eventually emerge in a form that is readily transmittable among humans
mr president senator reid and senator durbin both outlined some of the measures that have to be put in place here domestically to protect our population we have to drastically ramp up our stockpiles of tamiflu which if taken properly could act as a treatment from the avian flu once a person is infected right now we only have a couple of million doses we need 80 million to 100 million doses in order to be adequately prepared that is going to cost us significant amounts of money as the cost of tamiflu is approximately usd20 per dose
in addition we are going to have to develop flu vaccines of a sort we have not seen in the past in order to create sufficient quantities we are going to have to go push the boundaries of existing technologies and science going beyond the agricultural mechanisms of developing vaccines that we have used in the past
third we are going to make sure that local and state governments understand how urgent this is we have to ensure there are clear plans coordination mechanisms and lines of authority that will stand up in a time of crisis right now we do not have sufficient plans in place to make sure local and state agencies are able to generate the kinds of rapid responses that are going to be necessary in the case of a flu outbreak
after katrina i hope that local and state governments understand they have to work with the federal agencies more effectively to deal with these kinds of emergencies
another issue i would mention is that we are going to have to establish international protocols to ensure we can alert ourselves rapidly if we have confirmed cases of humantohuman transmission of the avian flu anywhere in the world why do i mention this if we detect efficient humantohuman transmission it is likely that we are going to have only weeks before we are going to see those first cases in the united states
this means placing effective trigger mechanisms in all these countries to make sure everyone is cooperating and providing rapid information which could mean the difference in terms of tens or hundreds of thousands of lives
now i dont want to suggest that nothing is being done for example months ago congress on a bipartisan basis including myself senator lugar senator mcconnell and senator leahy included usd25 million as part of the iraq supplemental to make contribute to an urgent who appeal on this issue today this money is making a difference in the field trying to set up some of the international measures i just described
i along with senators lugar durbin and others introduced legislation s 969 to enhance our ability to deal with this potential crisis but that was months ago and we need to broaden the number of people involved in this effort
moreover these is are modest first steps going forward we are going to need significantly more resources i am eager to work with leaders on health issues including senator harkin and senator reid as well as others across the aisle
i hope we can work not only to make sure we have an effective international regime to deal with this problem overseas but that we also invest the time the energy and the resources needed to put in place effective measures well before we have a full blown crisis on our hands
an outbreak of the avian flu could occur in a year 5 years 10 years or if we were incredibly lucky not happen at all but the one good thing about investing in measures to deal with this looming crisis is and i will end on this point if we spend the money now it will pay dividends even if this particular strain of the avian flu outbreak does not occur
why is this the case the risk of some sort of pandemic and the mutations of flus for which we have no immunity is almost inevitable the h5n1 strain may not be the strain that leads to a full blown pandemic but another strain could easily come along a cause serious damage in the future
presently we simply do not have the public health infrastructure to deal adequately with this contingency
my point is this undertaking these measures is going to be a wise investment that will help protect the lives of millions of people here in the united states and across the globe
mr president i appreciate your patience very much and look forward to working with you on this issue
 newspeech
mr president i thank the distinguished senator from alaska i will try to be brief
i just want to offer my strong support for the amendment senator harkin is going to propose and state why i think this is such an important issue
let me first say that i am generally of the view that we should not be tacking on unrelated amendments to the defense bills
the money in this legislation is badly needed by our men and women in uniform and i do not want to slow this bill down
but this amendment dealing with the avian flu pandemic is so important to our public health security and our national security so important to the lives of millions of people around the world that it simply cannot wait in fact the situation is so ominous that dr julie gerberding the director of the cdc said that an avian flu outbreak is the most important threat that we are facing today
in light of these developments i believe it is worth the us senate spending just a few hours on this critical issue even if it is not directly related to the underlying legislation
over the last few months we have heard alarming reports from countries all over asia indonesia china vietnam thailand about deaths from the avian flu
international health experts say that two of the three conditions for an avian flu pandemic in southeast asia already exist first a new strain of the virus called h5n1 has emerged and humans have little or no immunity second this strain has shown that it can jump between species
the last condition the ability for the virus to travel efficiently from human to human has not been met and it is the only thing preventing a full blown pandemic once this virus mutates and can be transmitted from human to human because of global trade and travel we will not be able to contain this disease we learned this lesson from sars which took less than 4 months to get from asia to canada where it caused human and economic devastation
when i started talking about this issue 7 months ago many people thought that the avian flu was a mild concern an asian problem an unlikely threat to americans here in the us
as time has progressed the nations top scientists and experts have focused greater attention on the possibilities of an avian flu pandemic and they have rapidly come to consensus that it is not a matter of if the pandemic will hit but when it is not a question of whether will people die but how many and the main question the question that keeps me awake at night is whether the united states will be able to deal with this calamity
from what we have seen with the lack of readiness and dismal response to hurricane katrina i think that all of us would have to conclude that the answer at this point in time is no
whether we are talking about having adequate surveillance capacities in our state and local health departments having enough doctors and hospital beds and medical equipment for infected individuals or having a vaccine or treatment that is guaranteed to work i dont want to be an alarmist but here in the us we are in serious trouble
several of us here in the congress on a bipartisan basis have taken the first steps needed to address this looming crisis in april of this year i introduced the avian act s 969 that would increase our preparedness for avian flu pandemic senators lugar and durbin and several others have cosponsored this act and i thank them for that we need to move this bill as quickly as possible
in may i and senators lugar mcconnell and leahy included usd25 million for avian flu activities as part of the iraq supplemental today this money is helping the world health organization to step up its international surveillance and response efforts
in july i included an additional usd10 million to combat avian flu in the foreign operations assistance bill that bill is currently in conference and i hope this funding will be retained
i am also working with senate defense authorizers on an amendment to require the dod report to congress on its efforts to prepare for pandemic influenza
this report must address the procurement of vaccines antivirals and other medicine the protocols for distributing such vaccines or medicine to high priority populations and how the dod intends to work with other agencies such as hhs and state to respond to pandemic flu
today with leadership by senator harkin we are introducing an amendment to the dod appropriations bill to provide usd39 billion in emergency funds for avian flu activities senator harkin has already outlined what this amendment does so i will not rehash what he has already said
the bottom line is that this amendment needs to be passed and passed as quickly as possible
i know that usd39 billion is a lot of money especially given our fiscal situation today but this is one issue on which we cannot be pennywise and poundfoolish if we dont invest the money now this pandemic will hit america harder more lives will be lost and we will have to spend significantly more in resources to respond after the fact
as we learned the hard way after hurricane katrina the failure to prepare for emergencies can have devastating consequences this nation must not be caught off guard when faced with the prospect of the avian flu this amendment will help the federal agencies to prepare the nation to prevent and respond to avian flu
america is already behind in recognizing and preparing for a potentially deadly and economically devastating avian flu pandemic that public health experts say is not a matter of if but when we must face the reality that in this age when you can get on a plane in bangkok and arrive in chicago in hours this is not a problem isolated half a world away but one that could impact us right here at home
the need is great and the time to act is way overdue i urge my colleagues to vote yes and support this amendment
to reiterate senators lugar mcconnell and leahy already worked with me to include usd25 million for avian flu activities as part of the iraq supplemental i included an additional usd10 million to combat avian flu in the foreign operations assistance bill but as senator harkin noted we need much more based on the briefing we received from the administration yesterday we have to move now on this issue it has to be moved rapidly we have to build an infrastructure to create vaccines and to purchase enough antiviral drugs i strongly urge that on a bipartisan basis we make this one of our top priorities this is a crisis waiting to happen if we are not prepared for it now we will all be extraordinarily sorry
the only other comment i will make is i know times are tough with respect to our budget i am working with my colleagues across the aisle to figure out ways we can come up with the money for katrina and iraq this is a sound investment if we dont make this investment now we will pay much more later
so i hope the amendment senator harkin is going to offer will get bipartisan support and receive the utmost consideration from this chamber
thank you very much mr president
 newspeech
mr president i rise today to call attention to an important step towards progress for afrodescendants in colombia and an important opportunity for afrodescendants throughout latin america
i wish to commend the work of my colleagues in the congressional black caucus on this issue as well as the tireless efforts of nongovernmental organizations and religious groups both here and in colombia
this august president uribe of colombia created a cabinetlevel position on afrocolombian issues and appointed an afrocolombian to fill the post the creation of this position is especially significant because it signals both a recognition of the severity of the situation of afrodescendants in colombia and a willingness to address these inequalities
at the same time many of us recognize that this is only a first step and much more needs to be done
i will be monitoring the progress of this office very closely in the coming months and i especially look forward to the development of president uribes committee on civil rights and sustainable development for afrocolombians
it is my hope that this institution will have the resources and mandate to do an effective job of bringing some measure of equality and justice to a marginalized segment of colombian society it is my hope that this will encourage other governments in latin america to consider taking additional measures to address racial discrimination as well as economic and social marginalization faced by afrodescendants in their countries
in the wake of hurricane katrina our own country is being awakened to a great divide in our midst as we struggle with troubling intersections of race and class and how we have failed the most vulnerable members of our population i hope we will be able to take a moment to reflect on similar struggles in places such as colombia ecuador brazil and venezuela
while i realize that colombia continues to face many challenges from human rights to narcotrafficking i wanted to bring some good news that is often overlooked about the country of colombia to the attention of the senate i applaud these efforts"""

In [0]:
hillary = """newspeech
i yield myself 15 minutes of the time controlled by the democrats
 newspeech
mr president yesterday i was in rome and watertown ny to speak with members of the rotary clubs and chambers of commerce about the upstate new york economy and how we can work together to promote investment and job creation in these communities i will carry their concerns about the economy to the budget committee on which i am pleased to serve and where we are fashioning the framework for the next federal budget
we are hearing about surplus projections and words of caution about how much faith to place in them we are hearing about president bushs tax cut plans and words of caution from colleagues who voted for big tax cuts in the early 1980s cuts which helped contribute to the ruinous deficits and high interest rates that hobbles our nations capacity to create jobs invest in people and pay down our national debt the budget resolution we create sets the stage for how much we can invest in health care schools and the other pressing needs of families throughout our country later this week i will return to the floor to talk about the budget in greater detail
today i would like to discuss a topic that transcends party geography and ideology it is an issue that is important to the people in rome and watertown rochester and brooklyn and everywhere i have been in recent weeks it will be foremost in my mind as the outlines of the 2002 budget take shape that is improving access to quality affordable health care for new yorkers for all americans and especially for our children
in this session of congress we will need to focus on many aspects of health care medical privacy medicaid funding genetic discrimination providing prescription drug coverage for our seniors and longterm care for our families among others today i will talk about the importance of insuring more americans particularly our children and protecting the rights of those who are insured
in all corners of new york i have met countless people who have told me powerful stories of the cruel inequities of our health care system last august at the dutchess county fair a single mother told me how hard it was to keep her family afloat because her medical bills totaled more than usd30000 she was worried she would become impoverished and forced to go on welfare
in massena an uninsured woman suffering from cancer told me how much trouble she had finding a doctor who would treat her for free in the montefiore childrens emergency room in the bronx i saw children who had come there for asthma treatments because they had no health coverage and therefore no doctor of their own from buffalo to bay shore the people of new york have urged me to go to the senate to fight for better health care
many of my colleagues will remember when i came to capitol hill 7 years ago with an idea or two about how to improve health care in our country at that time i was privileged to work with the acting president pro tempores father who served not only rhode island but our entire country so well for so many years we were not successful then but i learned some valuable lessons about the legislative process the importance of bipartisan cooperation and the wisdom of taking small steps to get a big job done
the clintongore administration took such steps and with the help of both democrats and republicans we made progress the kassebaumkennedy health insurance portability and protection act the family and medical leave act the childrens health insurance program the help we gave to young people leaving the foster care system under the chafee bill to give them eligibility for medicaid health coverage through their 21st birthday ending driveby deliveries mental health parity helping to prevent breast cancer by waiving cost sharing for mammography services in the medicare program and providing annual screening for beneficiaries age 40 and older advances in federally funded medical research and the human genome project
even with such progress however there are still 40 million americans who are uninsured adults with health insurance are three times more likely to receive care when they need it people with no health insurance are 50 to 70 percent more likely to be hospitalized for routine illnesses such as pneumonia children with no health insurance are twice as likely to be hospitalized for illnesses such as asthma and ear infections americans without health insurance are 4 times more likely to seek care in emergency rooms
it has only been 3 months since my election and 6 weeks since i was sworn in but already i have received hundreds of letters from new yorkers urging me to help them their families and their neighbors get the care and coverage they need one such letter is from kevin pispisa a boy scout from troop 207 in north babylon whose parents are nurses kevin wrote to me
it seems that the poor working class do not have the means to receive adequate health care some of them cannot afford to go to the doctor or pay for medication that they need
elsie doetsch from binghampton wrote to tell me about her friends who are dairy farmers she is concerned about them because as she writes in her letter to me
they work every day to help put the food we eat and enjoy on our tables yet cannot afford the luxury of health insurance which i feel is a necessity for anyone in their hazardous occupation
these letters serve as an important reminder to us all as we think about president bushs tax cut plans and as we deliberate over the shape of our new budget we must not forget to invest in the people we represent we must help them find affordable quality health care health insurance should not be a luxury it should be a fact of life for americans everywhere
let me be specific we should expand the childrens health insurance program if we change the poverty threshold to include children and families with annual incomes up to 300 percent of the national poverty level and extend the program to parents of eligible children we can provide health care to more than 5 million parents and nearly 2 million more children merely expanding chip however is not enough we need to do more to encourage the enrollment of the 7 million children who are eligible for chip or medicaid
i am very pleased that in new york chip outreach efforts include radio psas in a number of languages from greek to russian to albanian to creole to chinese we should provide a financial bonus to states that meet chip enrollment targets and reduce the chipenhanced matching rate for states that fail to do so
there are other creative ideas to provide greater access to health care for all americans as we consider them i believe we should adhere to certain principles first we must develop policies that cover more uninsured americans without encouraging businesses to drop or reduce their employees health benefits second we should make improvements to our health care system without setting up burdensome new federal or state bureaucracies third we should not penalize states such as new york that have been leaders in expanding coverage fourth we should encourage flexibility for states to expand coverage while enacting strong accountability provisions so that taxpayer dollars are effectively invested
as we work to expand health care coverage we must also work to improve the quality of coverage that is why it is past time to pass a meaningful patients bill of rights and i am very pleased to be a cosponsor of the mccainedwardskennedy patient protection act of 2001
president bush recently set out his principles for a patients bill of rights and this legislation meets every one of them with only one exception the president wants to preempt state laws that allow people to seek relief in state courts when they are injured by bad hmo decisions that objection should not stand in the way of progress i believe president bush can transform the rhetoric of leadership into the reality of accomplishment by embracing this bipartisan patient protection act across this aisle and across our country democrats and republicans are joined together in support of this patients bill of rights say the word president bush and we can make this bill a law
i appreciate the opportunity to speak today and i look forward to working with my colleagues on improving the health of our nation in the context of a budget that is balanced and prudent
i would also like to take this occasion to pay special thanks to my predecessor senator daniel patrick moynihan whose legacy of service to new york and our nation is unparalleled and who has always been a source of inspiration not only to me and my colleagues but to people literally around our world
finally i am so grateful to the people of new york who have given me this extraordinary opportunity to serve them over the course of the next 6 years i will work hard each and every day to listen to their concerns and to fight for their futures
i thank the chair and yield the floor
 newspeech
mr president as we begin our work on the 2002 budget we find ourselves at a crossroads facing a very big choice the choice we make will determine whether we pay down our national debt it will determine our investments in priorities like education the environment health care and social security and it will define the quality of life for millions of americans for years to come
the choice we face is this do we continue along the budgetary path that we as a government and a nation have followed in recent years or do we make a break from that path and return to the one we followed 2 decades ago
lets look for a minute at history eight years ago our budget deficit was usd290 billion the largest in our history the national debt was usd3 trillion and unemployment had surged to 78 percent at the time the congressional budget office predicted that the deficit would reach usd513 billion by this year
this year the predicted deficit is in fact a surplus likely to reach usd281 billion we are scheduled to pay off usd600 billion of the national debt concluding the largest threeyear debt reduction in our nations history as federal reserve chairman alan greenspan once said our commitment to fiscal discipline has been instrumental in achieving the longest expansion in the nations history
now debt reduction has meant lower interest rates for college car loans and home mortgages with government no longer draining resources out of the capital markets private investment in equipment and software skyrocketed and productivity gains kept fueling prosperity
at the same time we have invested in americas working families we doubled student financial aid in new york for example 45000 more children enrolled in head start in 1999 than in 1993 and this year new york schools will receive an additional usd100 million for renovations and repairs which based on observations during my many visits are very much needed
democrats and republicans have worked together to set aside the social security and medicare trust fund surpluses to extend their solvency together we put more police on the streets more teachers in classrooms and moved people from welfare to work
and we have done all of this while holding federal income taxes as a percentage of income for the typical american family to their lowest level in 35 years
and something else happened as the information age exploded america flourished making itself a leader in new technologies and increasing our productivity so that once again we became competitive in this new world it turned out that these policies were not only prudent but they opened the doors to the changes that prepared us and our children to be successes in the 21st century twentytwo million new jobs were created nearly 1 million in new york alone unemployment dropped to 4 percent and those jobs are pouring more than 900 billion dollars into our economy each year thats how we have gone so quickly from deficit to surplus but heres the catch if we upset the careful balance of our economy we can lose far more than the cost of the tax cut a tax cut recession would cost us trillions more in lost income through lost jobs
mr president i share the concerns of many of my colleagues that president bushs extremely large tax proposal will take us back back to the days of big deficits high interest rates shrinking investment and a growing national debt
i may be oldfashioned but as the daughter of a small businessman who did not believe in living outside our means and who even paid cash for the house where we lived i just dont believe we should spend what we dont yet have in the bank president bushs extremely large tax plan would spend trillions we dont have and may never have
if we reverse the engines of economic growth by adopting president bushs tax proposal i fear that we will reverse the progress weve made by increasing interest rates now and by saddling our children with big debts in the future
i know and respect that president bush supports faithbased programs but his tax plan should not be one of them going forward with a huge tax proposal now is like getting a letter from ed mcmahon and going out to buy a yacht a surplus projection is not a promise and if the past is any guide its not even a likely outcome
that is not my view alone it is the view of many experts who have testified before the budget committee on which i serve it is the view of colleagues like the gentleman from west virginia mr byrd and the gentleman from florida mr nelson both of whom voted for president reagans tax plans in the 1980s only to regret those votes when those cuts plunged us deep into debt
i encourage my colleagues to read the comments of both mr byrd and mr nelson in our committees proceedings or speak with them personally about their historic and wise perspective
the question before us is not whether or not we should enact tax cuts i support tax cuts the question is how do we structure a responsible tax cut a prudent tax cut that will allow us to pursue our important national values while keeping interest rates down and encouraging economic growth
the path of fiscal discipline is marked by four signposts it pays down the debt it protects social security and medicare it invests wisely in children and families and it reduces taxes in a prudent and sensible way
i do not believe president bushs tax plan meets those tests it also fails the fairness test president bush says that under his plan the typical family of four will be able to keep usd1600 of their money citizens for tax justice found that when the bush plan is fully in effect 85 percent of families would receive a nominal tax cut of less than usd1600 or no tax cut at all
even if president bushs proposal were fair to all americans it would not be prudent during this time of surplus it would leave nothing for the real reforms necessary to ensure that social security and medicare are intact for future generations the presidents tax plan abandons the principle of putting first things first
just yesterday some of americas wealthiest citizens came out against president bushs estate tax repeal saying that it was bad for our democracy our economy and our society and i agree
a tax cut that is fair to all americans needs to be part of a framework that strengthens not weakens our economy in my view we can and should have a tax cut that cuts income tax rates but we have to give relief to those paying the payroll tax on their income as well and i believe there is a bipartisan consensus for smart responsible and fair tax cuts
it is smart to include a longterm care tax credit to provide relief for families caring for elderly and disabled family members and the college opportunity tax deduction of usd10000 a year championed by my distinguished colleague from new york would enable families to pay for college graduate study or training courses tax cuts like these will bring tangible relief to new yorkers and working families everywhere
its also both smart and responsible to invest in our people especially in building the knowledge economy and i know that the president has had first hand knowledge of that in his former position we have to bring new technology to smaller communities across the country so they can take advantage of the welleducated workforce and higher education infrastructure that already exists in or near so many of these smaller communities
and we have an obligation to ensure fairness we should not favor the richest americans at the expense of the vast majority of americans
so how should we go forward will president bush try to push through his onesided and lopsided proposals with the votes of his own party if he does i will respectfully have to dissent or will he sit down and negotiate to reduce its size and make it fairer to more americans if he does this then i hope i can support the outcome bipartisanship is a twoway street its not about democrats supporting republican proposals or even republicans supporting democratic proposals its about republicans and democrats working together to do what is right for the country and the true test of leadership is not appealing to the people under the guise of bipartisanship but actually hammering out a bipartisan compromise bill that merits the support of both sides of the aisle thats the right way to pass a tax cut and protect our budget priorities
and it is certainly what i hear when i meet with business leaders workers and civic leaders in places like rochester and rome and brooklyn and watertown just to name a few of the places ive been in the last week they want a tax cut but they also want to make sure we make the right choices for our budget
history calls us to reject a spendthrift tax plan that would threaten our efforts to reform and modernize medicare including a long overdue prescription drug benefit that is voluntary affordable and available to all beneficiaries
i also fear a spendthrift tax plan would hurt our ability to invest in the military as the gentleman from connecticut mr lieberman said this week the presidents tax plan would consume more than 80 percent of the onbudget surplus leaving nothing but fiscal leftovers for national security
i dont think any of us want that
for me the details of the 2002 budget have to be negotiated but the big choice is clear we must pass a budget that keeps paying down the debt provides sensible tax cuts and invests in priorities that matter to the people we represent we must stay the course that has helped us build the longest economic expansion in our nations history and we must avoid a course that takes us back throws caution to the wind and risks mortgaging our childrens future
i yield the floor i suggest the absence of a quorum
 newspeech
mr president i rise to add my voice to those of my colleagues who are concerned about efforts to demolish this important worker health and safety standard
i listened carefully to the remarks of my distinguished colleague from missouri and i understand there are many serious concerns being discussed about this regulation and its impact both on our workforce and our employers but i ask that we remember where this started 10 years ago in the previous bush administration under the leadership of secretary of labor elizabeth dole we have held numerous hearings and studies to determine the impact of our 21stcentury worksites on peoples physical wellbeing
osha is charged with the responsibility of setting standards for the workplace to help protect citizens from harm in its 30 years of existence osha has helped to save many lives and prevent countless injuries despite such a track record we know that osha faces almost continual opposition from those who do not agree with its mission and who seek to undermine its work this year the opposition feels emboldened to strike at the heart of oshas latest efforts to protect american workers
we are of course talking about the ergonomics standard which is designed to help more than 600000 workers who experience serious workplace injuries every year from repetitive motion and exertion in enacting this standard osha heard from thousands of witnesses and received the backing of the national academy of sciences and the institute of medicine
the report to which my distinguished colleague from missouri referred is this rather large report that was issued on january 18 i draw our attention to some of the conclusions and recommendations that were arrived at let me just quote from it
the weight of the evidence justifies the identification of certain workrelated risk factors for the occurrence of musculoskeletal disorders in the lower back and extremities the panel concludes that there is a clear relationship
i stress that
mr president destroying this standard would put many workers at risk but today i want to focus on women workers in particular because as my friend and colleague senator feinstein said women account for 64 percent of repetitive motion injuries even though we make up only 46 percent of the workforce
earlier today i was joined by a number of women who have suffered from these disorders one was kathy saumier who was a worker at a plastics plant in syracuse ny kathy worked on a production line where she had to lift 40pound boxes every 1 to 2 minutes while twisting and holding the boxes at an awkward angle in order to put the boxes on the conveyor belt
with relatively small changes to the design of her work station or with automated assistance in lifting the boxes she and many of her coworkers could have been saved from such painful and timeconsuming injuries
kathy joined me and my colleagues from maryland and california senator mikulski and senator boxer at a news conference to highlight our concerns about these issues as they particularly affect women also speaking was dianne moriarity who for 18 years worked as a school secretary in new york because of her years of work in a badly designed work station both of her wrists and hands are damaged she showed me the picture of her work station the computer was bolted in a certain way so it could not be moved the space for the chair was such that it could not be angled and there was no place for her to be able to move comfortably to fulfill her obligations at that worksite she is in virtually constant discomfort and needs regular therapy
we also heard from jennifer hunter from virginia who worked for 20 years in a chicken processing plant she was required as the chickens went down the line to make 1400 cuts each hour she spoke specifically about what it took to prepare the filet of chicken breast which so many of us enjoy and eat at home or order in a restaurant and how difficult it was at the speed of that line to be able to get those cuts in and how her wrists had to be constantly moving
she too has suffered serious health effects from that kind of repetitive motion as she told us today we really need this standard so that workers are protected
heidi eberhardt of massachusetts worked at an internet publishing company writing editing and researching she is only 32 years old this was her dream job she was able to put her college education to work but because of the repetitive motion that was required over long hours sitting at her computer she finds it impossible to perform some of the daily functions we all take for granted she cant turn on a faucet she cant squeeze a toothpaste tube she cant twist an ice cube tray or even open mail without severe limitations and pain as heidi said this is not just about the people who are already injured this is about hundreds of thousands of workers who will become injured if there is no ergonomic standard for the workplace
one of the reasons women are adversely affected by this workplace hazard is because women hold more than 80 percent of the jobs that involve repetitive motion injuries jobs such as hotel cleaning data entry secretarial positions sewing
those who are here today working to save this worker safety standard understand that our opponents believe it will impose a costly burden on business but as our distinguished colleague senator feinstein from california pointed out those businesses that have already implemented standards have found they save money they save money by keeping their workers on the job in good health and more productive
certainly in new york we have found that businesses which have implemented the standards have reaped rewards businesses such as garment manufacturers sequins international in queens or xerox in rochester a company that has had ergonomic standards in place since 1988 we have found that these standards and the businesses that implement them are taking not only better care of their workers but better care of their bottom line
in addition to our concerns about the substance of the standard we are also deeply concerned about the manner in which the opponents seek to destroy this important worker safety provision everyone is willing to work together to change or improve the standard if there are legitimate concerns that have been raised there are certainly ways we can go about working to ameliorate those concerns
as my colleague the senior senator from massachusetts put it so well this is an effort that is truly a legislative atom bomb the congressional review act has never been used before it does more than rescind the worker safety standard it does ensure that the labor department can never again put forth an ergonomic standard it is in effect a gag rule on worker safety by dropping this congressional review act atom bomb opponents will completely eliminate 10 years of bipartisan effort in two administrations many hours of public review and witness testimony and extensive research in less than 10 hours of debate 10 years versus 10 hours
i can appreciate the desire by some to make changes to the standard but i hope we can talk about ways that such changes would be considered give the public a chance to be heard and any changes would be based not upon anecdote not upon story after story but on science and on the legitimate concerns of both workers and businesses
we should simply not bow to pressure groups and wipe this worker safety standard off the face of our regulatory planet we are here today to send a clear message that this is not the way to go about creating a safe workplace or working with businesses to make it safer for them to employ people across the vast sectors of the economy that use repetitive motion we particularly are concerned about the impact this will have on women in the workplace
we are also concerned this could mark the beginning of an erosion of protection for workers in america if you will a legislative repetitive motion that will undo safeguards that save lives
in the 20th century we made great advances in protecting workers often those advances came because of a tragedy a terrible fire a mine collapse a factory assembly line run amok when all of a sudden it became clear that we were putting peoples lives and wellbeing at risk this is a silent epidemic there will not be a big newspaper headline about a crash of ergonomics we will see just the slow but steady erosion of peoples health and their productivity and their capacity to get up and go to work and to go home and do what they need to do for themselves and their families
this is an issue that goes to the heart of the new economy how do we provide for 21st century workers the protections we did finally work out after lots of effort we should not go back we should not turn our backs on americas working families we should instead defeat this effort to kill this vitally important standard and then utilize the procedures available to us to go ahead and consider whatever the concerns on the other side might be
i ask our distinguished opponents to think hard about using this legislative atom bomb and instead consider how we can through existing procedures petition the administration to stay the regulation while further work is done we can also petition the agency to modify or repeal the standards and we can have osha initiate rulemaking procedures to modify the rule in accordance with the administrative procedures act if the real point here is to protect small business and protect workers there are ways of going about that which are already provided for it is hard to understand why we would need to blow away 10 years of work the findings of nonpartisan objective scientists and the stories that flood many of our offices from workers who are endangered in order to deal with what could be legitimate questions
i certainly hope we are able to disapprove this resolution so we can together work on behalf of the american worker
 newspeech
i thank the chair
the remarks of mrs clinton pertaining to the introduction of s 476 are located in todays record under statements on introduced bills and joint resolutions
 newspeech
madam president i ask unanimous consent that the order for the quorum call be rescinded
 newspeech
madam president i will speak to one of the pending amendments i rise today to offer my support for a pending amendment offered by senator conrad the social security and medicare offbudget lockbox act of 2001
this legislation would protect social security and medicare trust funds from being raided to pay for tax cuts or programs and would ensure our continuing commitment of the surpluses to debt reduction i am pleased that similar legislation has had broad bipartisan support in both the senate and the house over the past years as i believe it is the responsible step that we should take to ensure these vital benefits remain available with the paying down of the debt with assuring that we have affordable tax cuts and with the investments that we need to make to ensure our country is stronger in the future
now i know that the chairman of the budget committee for whom i have the greatest respect suggests this amendment is more of a scare tactic than a real effort to protect medicare and social security but i have to respectfully disagree this amendment is nearly identical to the amendment for which 60 senators including 16 of our colleagues on the other side of the aisle voted in favor of last year as an amendment to the laborhhs appropriations bill but it was unfortunately dropped in conference it is important to raise it again now because much to my disappointment president bushs budget blueprint does appear to raid the social security and medicare trust funds to pay for his tax cut proposal
over the past several weeks members of the administration have come before the budget committee on which i serve and argued that president bushs blueprint protects the social security and medicare trust funds but you can look at the numbers and see that is not the case a table in the blueprint entitled the presidents 10year plan for example refers to a contingency reserve of over usd840 billion of which over usd500 billion of that comes from the medicare trust fund
since other parts of the administrations budget seriously underfund many important priorities such as a prescription drug benefit for our seniors on medicare national defense investments in our schools and our children our teachers and other significant areas for which there is broad bipartisan support it also proposes hundreds of billions of dollars in unspecified cuts across programs and there isnt any mystery there cant be any mystery that if you combine a very large tax cut with underfunding important programs and leaving out many others then there will not be the money in this reserve and it is money taken out of the medicare trust fund that will be available to cover the priorities that we would determine are in our national interest
during the time of projected surpluses i have to ask is this really the choice that we want to be making madam president i know most new yorkers would agree that it would be both unfair and wrong to shortchange either our seniors or our children when it comes to prescription drug benefits or investments in smaller class sizes school construction and other important programs that will improve the quality of education
the real choice we face should be between a very large tax cut from which millions of working americans would receive little to no tax relief and the three priorities which i think we can agree on in this body a priority for affordable tax cuts a priority to continue to pay down the debt and a priority of the kind of investment that we need to make
for example i believe we should invest in a real prescription drug benefit the presidents immediate helping hand proposal denies eligibility for prescription drugs to nearly 25 million medicare beneficiaries most of whom today lack affordable dependable prescription drug coverage
republican and democratic governors have also raised concerns with this proposal noting that it fails to meet the immediate prescription drug needs of their elderly and disabled residents
the challenge should be not deciding to shortchange our seniors on prescription drugs in order to give a very large tax cut to people at the upper end of the income scale but it should be between how do we keep all of our priorities in order and how do we provide prudent tax relief continue to pay down the debt and invest in what will make us a healthier bettereducated stronger nation
i believe senator conrads amendment to lock away the social security and medicare trust funds sets the right balance it clearly takes off budget what should be off budget it should not be used for a contingency for tax cuts or for spending it should be used for what it was intended to be used for to meet the social security and medicare needs of our seniors
i ask that i be added as a cosponsor and i urge my colleagues as they did once before on an appropriations measure to ensure the solvency of the important programs such as medicare and to ensure the provision of a prescription drug benefit to our seniors on medicare and to deal with the other important priorities that we face
madam president i suggest the absence of a quorum
 newspeech
mr president i join in support of this amendment because we know from a lot of the work that has been done over the last several years many students are being deliberately solicited even targeted for credit cards before they are financially independent responsible or knowledgeable about what it is they are signing up for story after story has demonstrated clearly that this particular amendment by my good friend the senator from connecticut targets a real problem
i think all of us are committed to ensuring that people who are irresponsible with their financial affairs are held accountable but i think we should look at our young people in a different category it used to be no one could be held financially responsible when they were under 21 then the age was dropped for many purposes to 18 but despite how quickly it seems our children grow up these days there are many young people in college or out working who are not yet 21 who do not really have the experience to deal with the solicitations that come flooding through the mail and over the telephone that we know are targeting them with these credit card applications
this morning i was talking with another colleague of ours who told me he was babysitting for his very young grandchildren he put them to bed the phone rang and the person on the other end asked for one of his granddaughters our colleague said what is this about he was told much to his amazement that his 512yearold granddaughter had been approved for a new credit card he said he was shocked this kind of activity was going on and did not really believe it until it happened in his own family
i urge our colleagues regardless of the position we take on the underlying legislation we should stand behind the basic principle that our young people should not be solicited they should be given some better credit training as this amendment proposes and there should be some sense of responsibility on the part of creditors before they reach out to entice our young people into these credit cards before they even know what it is they are signing up for it looks all so easy and they end up in trouble with debts they cannot pay
lets try to avoid that that does not mean they cannot ever become customers but lets make it a little more reasonable in the steps that have to be taken in order for them to qualify
i certainly urge passage of this amendment i thank my good friend the senator from utah for yielding time
i yield the floor
 newspeech
mr president i thank my good friend from connecticut i also thank senators torricelli and corzine for bringing this important issue to the forefront of this debate because clearly we are not going to be able to have the kind of campaign finance reform that many of us are hoping will come out of this process if we do not address the most expensive aspect of modernday campaigns
as we all know that is the advertising that we have to do in order to communicate with voters about where we stand on issues it is a particular challenge in large states but it is a national one that all of my colleagues face
the torricelli amendment which would amend the communications act of 1934 would require that the lowest unit rate be provided to committees of political parties or candidates purchasing time i think that is in the best interest of our democracy i certainly believe it is the kind of reform that goes to the real heart of what the money chase is all about
i think a lot of us would like to be able to turn the clock back to the days that some of our colleagues can remember but for most of us we just read about it where you could literally go out into a town square or out in the countryside set up a little platform visit with constituents make a speech keep on going and reach most of the people who were going to vote for you or make a decision on an important issue those days are long gone the television broadcast networks know they are the means by which we must communicate
i think this amendment is not only fair but long overdue i commend the senator from new jersey for bringing it to the floor i hope the television industry recognizes that there is an effort to not just have a level playing field but fulfill what many of us thought was the bargain that when we use the public airwaves for communications and those communications are basically controlled by the companies that have been given in my opinion the privilege of having those airwaves that there has to be some way they give back to keep the first amendment alive to keep democracy going i am just so pleased that we are going to have a chance to vote on it
i thank my good friend from connecticut for yielding some time so that i could weigh in on the importance of this issue
 newspeech
thank you very much
mr president i didnt realize it until after i spoke but my good friend senator torricelli from new jersey gave me one of the articles he read into the record that i have yet a new title that is modern day santa claus
i was given an article that was written by paul taylor about broadcasters and their desire to have political advertising
i was delighted to learn that i am a beautiful thing like santa claus because the campaign i ran brought i guess great beauty and good cheer to the broadcasters of my state
i would like to add to my previous comments in support of this amendment that i think this is a good start to ensure that the spirit of the current law is enacted and implemented but i think we should go further and later in the debate i hope we will have a chance to talk about even going further to perhaps legislate the 5 minutes that has been suggested by a number of people as being free air time and even to have a debate on an issue i support which is free broadcast time across the board and some way to fulfill the political obligations of communications that i think our society so desperately needs without having the charges attached to it that we currently are experiencing
i know in 1997 when the fcc doubled the amount of the spectrum it licensed to television broadcasters i joined with many others in recommending that 5minute voluntary candidatecentered discourse during the 30 days leading up to the campaign we know that is not happening
i think we need to do more to provide free air time for political candidates i hope we will not only pass this amendment but go on to consider other ways we can make air time more readily available if it were in my power as santa claus to give that gift to the american people i would certainly do it but i am going to try to make that case in addition to supporting this very worthy amendment
i thank the senator from kentucky for yielding me time
 newspeech
i thank the chair
the remarks of mrs clinton mr dodd and mr wellstone pertaining to the introduction of s 584 are located in todays record under statements on introduced bills and joint resolutions
 newspeech
my good friend from connecticut raised an issue that troubles me about this proposed amendment that the distinguished senator from utah has put forth
in addition to the issues that senator kennedy and senator dodd have raised about the vagueness and definitional concerns raised in the amendment this particular issue is the real heart of the parity problem that many of us have with this amendment
it reminds me of the old anatole france saying the law is fair neither the rich nor the poor can sleep under the bridge what we have is an amendment that in its practice not only would fall disproportionately on unions as compared to corporations but which under the rationale put forward by it completely leaves out other membership groups as the senator from connecticut so rightly points out
the burdensome reporting requirements that are imposed under this amendment on unions in particular are really much more difficult to comply with than if they would be in a corporation as i understand the amendment corporations would be required to report only on expenditures from their own general treasuries and from the general treasuries of their subsidiaries however unions would be required to report on the expenditures from all of their affiliates which would mean that a local union would be required to report on expenditures by a national union and vice versa even though neither of them had either access or control to the financial records of the other
this point we heard about from senator dodd is particularly important if the point we are trying to get at with this amendment is to understand who is doing what with what funds to engage in political activity during election cycles then clearly a lot of the other membership groups that raise and spend tremendous amounts of money two were mentioned the nra the sierra club you can add the chambers of commerce national right to work foundation other groups across the political spectrum
 newspeech
my question would be in response to the discussion between the senators on this issue how can we impose undue burdens on only unions as compared to corporations and completely leave out of the senators concerns all of these membership groups that raise tremendous amounts of money are on the front lines of our political campaigns have a direct influence on how voters vote and yet are in no way covered by the senators amendment
 newspeech
madam president i come to the floor today to speak out and join the distinguished senator from west virginia and the distinguished senator from florida to express our concerns about the upcoming budget debate
first i thank senator byrd for his extraordinary commitment to this institution which is really unprecedented in history and is such a blessing for not only the institution and those who have been privileged to serve with him but for our country and i heed his words seriously because he has taken the long view about what is in the best interests of a deliberative body of this senate of a nation that should rely upon the careful thoughtful analysis of the issues that come before us and the people we represent
i am personally grateful to him for the time he has taken as my good friend the distinguished senator from florida referred to to help mentor us freshmen senators to give us the guidance we need to be able to do the best possible job for the people who sent us here and it is such an honor to stand on the floor of this senate a place i have long revered on behalf of new yorkers
but i come today with somewhat of a heavy heart because i believe in the principles and values this senate represents i want to see them fulfilled i want to be a part of perpetuating them into our future
i find myself as a new member struck by how difficult it will be to discharge my responsibilities in the upcoming week without having seen the budget without having the opportunity to debate its priorities and even more than its priorities the values which it seeks to implement i do not know that the people i represent or the people any of us represent will get the benefit of our best judgment that the decisions we make will be grounded in our careful thoughtful analysis
there will certainly be differences among us that is what makes this a great deliberative body and makes our country so great we come with different experiences we come with different viewpoints i come as the daughter of a small businessman who did not believe in mortgages did not have a house until he could pay for it with cash did not believe in credit and who believed it was his responsibility to always make sure our familys books were balanced
i come with the belief that we had to go to extraordinary efforts to make sure our economy enjoyed these last 8 years of prosperity and progress and that we could not have done so had we not reversed the decade of deficits and debt that really did undermine americas capacity at home and abroad
so when we talk about the important debate in which we will engage next week i think it is the most important debate in which i may engage in my entire term as senator it is certainly one of the most important debates for our country and everyone who is following it to understand what is at stake
this debate will set our priorities as a nation for the foreseeable future and could determine whether or not we have surpluses whether or not we will be prepared for the impending retirement of the baby boomers that starts in just 11 years it is a debate that will certainly be about numbers deficit projections surplus projections and spending
but i think underlying it is a debate about who we are as a people it is not only about our prosperity not only about our federal budget it is certainly about that it is about who we are as americans
i come to this body determined to represent the people of my state and our country as all of us do but will we be able to do that we are going to be deciding in the votes we cast starting with procedural votes whether or not our seniors will have prescription drug benefits we are going to be deciding whether or not our children will have the teachers they need and the schools they deserve to have we are going to be deciding whether we have the sewer systems and the clean drinking water that every american deserves and should be able to count on we are going to be deciding whether or not we do have the resources to maintain americas strength around the world whether we will combat terrorism whether we will stand firm with our allies we are going to be determining whether we make the investments in research and development that will make us a stronger richer smarter nation in the decades ahead
i am deeply concerned that we enter this debate without the benefit of the administrations budget
i am privileged to serve on the budget committee under the extraordinary leadership of the senator from north dakota and my colleagues the senators from west virginia and florida we sat through fascinating hearings we listened as our defense priorities were discussed as our education priorities were discussed as our health care priorities were discussed we listened to experts from all across the spectrum of economic opinion and analysis i found it an extraordinarily enlightening experience but we are not going to get a chance to debate with our colleagues what it is we as a committee should be deciding to recommend to this body with respect to the budget we will be debating so we are flying blind we are looking through a glass darkly we are in the dark
will this budget have the investments we need to protect child care and child abuse programs the early information is it will not that we will be turning our backs on working parents cutting tens of millions of dollars from child care will we protect our most vulnerable children those who are abused the information we have without a budget but kind of leaking out of the administration suggests that we are going to be asked to cut child abuse prevention programs
we also are being told that we are going to be asked in this budget to cut training programs for the pediatricians who take care of the sickest of our children in our childrens hospitals these are very difficult issues in any circumstance but not to have the chance to be able to analyze what is being proposed is troubling to me will this budget ensure our children will grow up in a safe environment with clean water and clean air with access to quality affordable health care will it adequately protect our food supply every day we see a new article in the paper about what is happening with our food supply in europe in the united states around the world will we be able to protect ourselves so we have the kind of reliable food supply that americans deserve
what are we doing in this time of surplus to ensure a safety net for all americans young and old the prescription drug benefit that we hear about from the administration would leave over 25 million of our seniors without prescription drugs i dont want to choose between some of our seniors and others in new york those who may be just a penny over the limit that they therefore wont get the prescription drugs they need i want to make sure that everyone on medicare and that is what most americans want has access to those prescription drugs
to pay for the tax cut the administration includes the medicare surpluses those are resources that should be ensuring the solvency of medicare for all americans totally in a reserve that is set off never to be used for any other obligations i believe other obligations that we have should be paid for in the context of a balanced budget and not put medicare at risk
the administration has correctly committed to doubling the number of people served through community health centers i support that it is a worthy goal but then on the other hand i understand they are doing it by completely eliminating the community access program that ensures that community health providers work together to create an infrastructure for care so no patient falls through the cracks new york is filled with wonderful religiously based hospitals privately based hospitals that are part of this infrastructure of care that would be left out completely we also have the finest teaching hospitals in the world there are no resources that will continue to make sure that they are the finest in the world new york trains 50 percent of all the doctors in america what are the plans for making sure that continues and that our teaching hospitals are given the resources they need
we are also hearing that the administrations budget will provide more security guards for our nations schools that too is a worthy goal in fact i was heart broken to hear today of yet another school shooting in another school in another part of our country that is an issue we must address if security guards would help i will support that but i am troubled and my heart goes out to the families who are suffering these terrible tragedies in school shootings
i will do whatever i can on all fronts to try to deal with that problem but i understand from the presidents budget that they are shifting funds from the very successful cops program that has really helped us drive down the crime rate in order to pay for the security guards at the schools we are robbing peter to pay paul why would we take resources away from the cops program where so many brave men and women put on the uniform and walk those streets that has become so effective in driving crime out of neighborhoods why would we take money away from our police officers and put it in our security guards at schools if we need to do both i argue strenuously we do
are we being confronted with such a hobsons choice because of a genuine shortage of resources or are we making these choices and cutting needed investments simply to allow for an enormously expensive tax cut that leaves millions of americans out leaves millions of americas working families again behind where they need to be in order to make the decisions that are best for their families because we are favoring others
the kinds of priorities i speak of today for which i have fought for so many years going back to the days when we tried to bring fiscal responsibility to our budget when we tried to lower the crime rate when we tried to improve health care and education and protect the environment are bipartisan priorities these are genuinely american priorities child care child abuse prevention police on our streets we dont stop and ask are you for it or against that based on party we say isnt this something we should do together in america
madam president i hope we will come together once again republicans and democrats americans to fashion a budget that pays down the debt which is still the best tax cut we can give the vast majority of americans that is what puts money in your pocket when you have to have a mortgage when you do have a credit card when you do have a car payment lets keep those interest rates down
we have learned from the last 8 years that the best way to do that is to be fiscally responsible and pay down our debt
we need to provide sensible tax relief everybody in this chamber is for that sensible affordable fiscally responsible tax relief that says to every american we are going to make it possible for everybody to share in these surpluses we are not going to favor one group over another that is the kind of tax relief i would be proud to be part of and for which i will speak out
finally we need a budget that invests in our nations most pressing needs not just what we see right before us the fact that we should continue to lower class size in the early grades that we should continue to modernize our schools those are needs i see every day i go in and out of schools i talk with teachers and parents and students i know how much better our education system can be if we have both increased accountability and increased investments i know we have needs that are staring us right in the face that we may be turning our back on if we are not careful
i also want to be looking to the horizon looking around the corner it is not just enough to take care of today we have to be thinking about next year and the next 10 years and the next 25 and 50 years if we are to fulfill our obligations as stewards for our people that means we cannot turn our backs on the demands of social security and medicare
as a member of the socalled baby boomer generation i do not want to be part of a generation that is not responsible the world war ii generation is often rightly called the greatest generation i am proud of the service of my father i am proud of the service of all who came before but they also understood the investment that needed to be made it was in those years after that war when we started investing in our nations schools started building the interstate highway system started making the investment that we frankly have been living on for the last 50 years in this country how on earth can we keep faith with those who came before us let alone our children and grandchildren and great grandchildren if we dont have the same level of responsibility
i think we have a rendezvous with responsibility and it is now if we turn our backs on that responsibility we are going to have a great price to pay maybe the bill wont become due until 5 years 10 years maybe 15 or 25 years but like my colleagues who have spoken i want to be able to say to the young children i meet that we tried to be responsible we tried to do the right thing that will make us a stronger richer smarter nation
the american people and i certainly know that people in new york who sent me send us here to washington to work together across party lines to make the tough choices necessary to move our country forward that is exactly what i want to do it is not necessarily going to mean that democrats will support all republican proposals or vice versa but what it does mean is that we will reason together and work together to do what is right for our nation i hope when that process begins next week we will have a chance to really sit down and look at the presidents budget have a good honest open debate as we just had these last few weeks about another very important matter before this body and that we will honestly say what the priorities are we are setting the values we stand for the vision we have for america
i believe there wont be a more important issue that i will face i want to make my decisions in a deliberative thoughtful manner i want to look for ways i can work with my friends across the aisle as well as my colleagues on this side because i want to be sure that at the end of the day we have done the right thing for the children of america if we are not going to leave any child behind then lets make sure we know what we are voting on that will affect every child
if we can make that determination to work together i am confident we can come up with a bipartisan sensible policy that leads to a budget we can support in the absence of that it will be very difficult to do so and i hope that certainly the people of new york and america understand we are trying to stand firmly in favor of a process that may sound arcane and difficult from time to time to understand but which goes back as senator byrd so rightly points out to people who were very thoughtful about how to design a process that protected the rights of everybody it is not just about that as important as that is it is fundamentally about the choices we will make for the children and families of america
i know that people of good faith will find a way to come to a resolution about how we proceed next week i am looking forward to that but i do have to say that in the absence of such an agreement i for one will have to be asking the hard questions the people of new york sent me here to ask about what specifically will be done to affect the hopes and aspirations and needs and interests of the people i represent
so i will be guided by three principles
will this budget pay down the debt to continue us on a path of fiscal responsibility that protects social security and medicare
will we be in a position to recognize that the investments we need to make are important investments that are not going to disappear overnight
and at the end of the day will we have made decisions that will protect americas longterm interests at home and abroad
madam president i hope i will be able to answer affirmatively every one of those questions
i yield back the remainder of my time
 newspeech
mr president i rise in support of senator thomas resolution which calls for the immediate release of the crew members of the ep3e that was forced to make an emergency landing at the lingshui hainan airbase on april 1st securing the safe return of the crew and their aircraft is a top priority for our country and this resolution makes that clear
and i know that i speak for my constituents when i say that i am deeply concerned about the safety of the twentyfour us crew members who are being held in china my thoughts and prayers are with all of them and their family members including the family of kenneth richter a navy cryptographer and native of staten island new york
we are fortunate to have brave men and women like kenneth richter serve our country it is a reminder of how the courage and hard work of those in our armed forces help to keep america free and secure
all americans stand as one behind the president as our nation presses for the immediate release of our people and our aircraft there is absolutely no justification for their detention for one minute let alone so many days
 newspeech
mr president i thank my distinguished colleague from minnesota i associate myself with the remarks of the education senator from massachusetts who so eloquently laid out our dilemma the dilemma that will be occupying the senate as we move forward on this very important debate
people always talk about important debates but it is fair to say as we debate we will set educational policy for our nation for the next 7 years there is hardly a subject we can think of that will have more direct impact on our families on our communities on our economy and especially on our children we are setting the stage for determining how much we as a nation will do to make good on the promise of a quality education for all children and particularly for our countrys neediest children
i first became involved in education reform back in 1983 with the issuance of the report called a nation at risk which was issued under president reagans watch many took that call to action very much to heart that we were a nation at risk we began looking for ways to improve education to provide more resources to provide more accountability measures we have made progress over those last years
when the elementary and secondary education act was last reauthorized in 1994 we sent a strong signal that although education was absolutely a matter of local concern it had to be a national priority that we all had to recognize we were failing our children by not providing adequate educational resources and by not expecting them to do the very best they could do we put a high priority on academic standards and we worked to help teachers and administrators parents and communities improve education
the results of this strong federal response to local and state educational demands has been heartening mr president 49 states plus the district of columbia and puerto rico have developed state standards and are working to implement them these reforms are producing results
we often only focus on the negative side of the ledger about how much we still have to do i give some credit to the children and the young people our students and their parents and especially their teachers because we have seen progress reading and math scores for fourth graders in our highest poverty school districts have improved by nearly a grade level from 1994 to today sat scores are on the rise more students than ever are attending college
we cannot rest there we know there is still far more to be done we have too many children particularly in our underserved urban and rural districts who are not reading at grade level we have too many children being taught by uncertified teachers in overcrowded classrooms in crumbling school buildings we cannot stand by idly while these conditions persist the issue is what is the best way to address them how better can we equip parents teachers communities and our students to meet the tests of the 21st century
i applaud president bush for calling for greater accountability i agree with him on the importance of that i was among the very first in our nation in arkansas in the early 1980s to call for the testing of students and the testing of teachers because i believed then we had to know what we didnt know in order to make progress we couldnt just pretend that everything was fine and engage in social promotion and not face up to the fact that we had children graduating from high school who couldnt read a job application we had teachers who had been themselves passed through the education system who were unprepared to teach the substance of what it was they were assigned to teach
accountability is key to me i have been a strong supporter of that in fact i welcome the republicans and i welcome the bush administration which has gone forward with accountability measures that are like the measures democrats have proposed for several years many on the other side of the aisle resisted such approaches for many years in fact they wanted to abolish the department of education so i applaud my colleagues on the republican side for the progress they have made in moving toward a common recognition that this is a national priority that must be beyond politics and partisanship
the accountability that is in the bill that is proposed would ask that we test our children every year from third to eighth grade that is designed to ensure that they are meeting high standards but here is where the rubber really hits the road if all we do is order more tests if we do not combine those tests with the resources that are needed to help the children who have been left behind then we will have at best a hollow victory and i believe worse than that we will have committed educational fraud on our children our teachers and our country
the bush plan orders more testing while providing only half the funds needed to design and implement these tests what would this mean to the state of new york for example it would mean that of the usd16 million that is estimated to have to be spent to comply with these new federal requirements our state would only get usd8 million so we would have to find 8 million more dollars take it out of something else from hardpressed school districts from teacher pay from whatever other important objective we are already trying to meet we should not be passing on an unfunded mandate to our states
if it is a national priority if it is a priority for this administration to order these tests then the federal government ought to pay for these tests and make sure that as the senator from massachusetts pointed out they are good tests they are quality tests they are not just makework kinds of tests
passing tough new accountability standards without the resources to help our schools and students is similar to handing out thermometers in the midst of an epidemic the thermometers certainly can tell us that there are a lot of sick people but they do absolutely nothing to help people get better unfortunately the administrations proposal has plenty of thermometers but precious little medicine to help our schools improve the administration has not even yet committed to providing the federal funds necessary to marry accountability with student achievement
we already know that despite the rhetoric this is not an increase of more than 11 percent it is only 59 percent because the administration tried to count money that had been appropriated last year we are glad to have that money but lets have honest accounting about how much more money is going in a 59percent increase barely keeps up with inflation and population increases
what also does it mean on the school level lets focus and ask ourselves if we pass this accountability measure and everybody goes home pats themselves on the back there is a big press conference and a big signing ceremony what have we really done to help the districts such as the ones i worry about in the state of new york
in new york city for example we are facing a severe teacher shortage the city will need to hire approximately 40000 that is right 40000 teachers over the next 4 years in addition the district is under a court order to place those certified teachers it hires in the lowest performing schools that makes sense because right now we have uncertified teachers ill equipped to teach teaching the children who need the best teachers so the idea which is a good idea is lets put the certified teachers in the schools where the children need them the most
but what has happened last week we learned from the chancellor of the new york city schools that the certified teachers turned down the jobs in the hardtoteach schools why because those are the schools that are already overcrowded those are the schools that are crumbling those are the schools that hardly have a book in the library those are the schools without the computers connected by the cables they need to be able to be functional let alone to be accessible to the internet
we cannot in good conscience demand that school districts hire certified teachers without providing the resources to help these hardpressed districts recruit and retain these teachers and we have to do more to make these schools attractive to certified teachers
answer me why you would go into a very difficult school to teach children who are under lots of stress at home and in their neighborhoods if the school is not well equipped to give you the resources you need to try to do a good job with those children
i will be working with colleagues on both sides of the aisle to introduce a bipartisan teacher recruitment amendment we all know if we do not place the recruitment of our teachers at the top of our national agenda we will have school districts that are barely able to open their doors in the next couple of years we will be asking people literally to come off the streets and start teaching because we will not have the teachers we need i meet people all the time who want to be teachers but they will not they cannot and they should not work under the conditions under which many of our teachers are asked to function
i am also concerned about the proposal the president includes called straight as this is a demonstration project that would give 7 states and 25 school districts the chance to block grant federal dollars people are often talking about how important it is to give authority back to the states and i agree with that in most instances but we know from years of education research that block granting funds which means taking the federal dollars and sending them to the state capitol means that those dollars do not get to the students and the schools that need them the most in the amount that they should they get siphoned off in the bureaucracy of the state capitol they get sent to other places that do not need them but for political purposes have the influence to get them we should be targeting those hardearned federal dollars to those school districts and those students who are so far behind
right now in new york we know because of a court decision that the children in new york city do not get their fair share of education funding so we should do everything possible to get the dollars to the students who need them the most in the schools where the teachers have a chance to try to help them
we also know from research that smaller class sizes make a huge difference and the class size reduction initiative has worked wonders we now have teachers in new york who are federally funded who are helping to lower class size we have already seen positive results from the school achievement scores
we also know that construction funding to help schools repair their buildings and modernize them and even construct the buildings they need is very necessary these two important programs class size reduction and school construction are eliminated for all purposes in the bush administration proposal i say this is a mistake and i ask the administration with all respect to please reconsider this decision
the administration says that reducing class size with federal dollars and helping to construct and repair schools are not federal responsibilities i know they are not totally federal responsibilities but i do not think in todays world they are also solely local responsibilities the districts that need the help the most are not the districts like the one i live in where with very high property taxes from affluent people the children have everything they could possibly dream of but in so many districts suburban taxpayers cannot pay another penny to fix their schools and do what is necessary to have uptodate labs in many rural districts they do not have the tax base to do that and in many urban districts they dont have the dollars because they dont get their fair allocation from the state and they cannot tax themselves to be able to meet the needs of children for whom english is not their first language who come to school with undiagnosed mental illnesses who live in a system of deprivation and violence and who cannot perform at the same level as the children in my district
lets have a shared responsibility that was the whole idea behind the class size reduction initiative and school construction initiative if education is to be a national priority lets invest in what we know works and we know reducing class size and providing good facilities actually works to make for better education
i hope we will continue in the spirit that we began in the education committee as we marked up this bill in the negotiations that are currently ongoing with the administration but i am very concerned that this particular proposal falls way short of what we need to be doing it falls short for a very simple reason the administration would rather invest in a large fiscally irresponsible tax cut than in the education of our children and particularly those who are most needy in rural and urban districts
i hope this will be reconsidered because this failure to properly fund education to me is disappointing at a time when we have surpluses when we do not have to squander these surpluses on large tax cuts that will go disproportionately to the already wealthy whose children already attend schools that have all the computers all the bells and whistles all the extra help they could possibly have
lets instead take a moment and step back i hear a lot about the greatest generation my parents were part of the greatest generation the world war ii generation i think they probably have to take a second seat to the greatest generation being the founders of our country but there is no argument that those who survived the depression won world war ii and set the stage for winning the cold war were among the greatest if they were not the greatest generation our country has ever seen
we have been living off the investments and sacrifices of our parents and our grandparents for more than 50 years my father who is a rockribbed republican voted for higher school taxes because he knew the education of his children depended upon good schools we invested in the interstate highway system we set a goal to send a man to the moon we had big dreams and we worked to fulfill those dreams
today at the beginning of this new century it is up to us to make the decisions the hard decisions to invest in our childrens education and shame on us if we do not make the right decisions we can pass a bill that is filled with testing and sounds good but 10 years from now we will still have children in overcrowded classrooms and crumbling buildings who are being deprived of certified qualified teachers and we will wonder what went wrong
lets instead be sensible about the best practices that we know work we have research we have practical experience we know what needs to be done the issue is do we have the political will to make those decisions
i support working hand in hand with the administration in a bipartisan way with the parents and teachers and community leaders of our country to make education a real national priority but i cannot i could not support a bill that is a hollow empty promise
lets do both lets increase accountability so we get better results by making sure we have the resources to hold our children and our teachers accountable if we do that then we will be setting the stage to leave no child behind if we do any less then i think we have missed a historic opportunity
 newspeech
certainly my feelings go back many years as the senator knows children have been my passion for more than 30 years i have worked on improving and reforming education for nearly 20 years i know how difficult it is today to try to help many of our children achieve educational competence
the reason for that is that we are not living in the same world in which the senator and i grew up it is harder to teach our children our children come to school with more problems and more stress they are exposed to many more things than we ever faced
we have to understand that if we dont really provide the resources to reach the children as they are today not as we wish they would be not as we thought they were back when i was sitting there with my hands folded and listening to every word but as they are today with all the other pressures that are on families and children then we are not going to have the results and the kind of achievement to which the senator from minnesota is referring
but there is no reason we have to make this choice it is not an eitheror choice we have the resources to assist our local districts so they do not have to reach any deeper many of the districts from my state cant afford to raise their property taxes any more
i was on long island last night talking to a group of about 1000 people i explained to them if we have this large federal income tax cut and then we have these unfunded mandates for education where is the rubber going to hit the road it is going to hit the road in the local property tax levies
i would rather be i am sure part of an administration that gets to take credit for cutting income taxes than the poor souls down at the local level having to vote to raise property taxes in order to meet the mandates they have put on them i think we should not be raising false hopes we should be looking at how we help every child be successful
 newspeech
i think my colleague is absolutely right because if we are looking at the comprehensive reform we cannot leave out the funding of idea we cant leave out doing something to help parents understand their obligations to be a childs first teacher and provide quality preschool
i hear so much about the idea program otherwise known as the special education program i hear it mostly in suburban districts interestingly enough because suburban districts have activist parents and they know the law the law is that we have to provide an education for every child and i support that law it was the first project i ever did for the childrens defense fund i went door to door in communities back in i hate to say 1973 to find out where the children were because they werent in school we found a lot of children with disabilities who were being kept out of school
i am a 100percent supporter of mainstreaming our children and giving every child a chance but we are bankrupting a lot of our suburban school districts we are saying you have to provide special treatment and education for children who need it and deserve it if that means you have to shut down the band program or only have one physics session or do away with art that is the tough choice to make
the federal government said in the 1970s that you have to provide this education furthermore it is not only as our colleague tom harkin likes to say a federal mandate but it is a constitutional mandate to provide this quality education the federal government is going to tell districts they have to provide special education where is the full funding so suburban districts and all other districts can try to keep up with their expenses
i could not agree more with the second point the senator made those of us who have been parents read to our children we take them to museums we get them a library card we monitor their television we worry about any kind of childcare arrangements we know those early years make a difference why dont we make a commitment based on the resources we now have about the brain to do more to provide quality preschool opportunities both at home and outside the home so that more children can come to school ready to learn that might be the very best investment we could make in terms of longterm academic success
 newspeech
mr president i appreciate the dialog that just occurred between the leaders because certainly it is critical that the debate on education commence and that we do everything within our power to provide more resources greater opportunities and accountability to our children around the country
as a new member to this body i am one who shares the concern about actually seeing the language of the bill and trying to be sure that we know what it is we are debating and that the people back in our states who we represent have a chance to be part of this debate by being able to read and study and provide comments about what it is we are considering in the senate i know it may from time to time be a little frustrating but until we actually have a bill with language that will determine the future of education funding from the federal government for 5 to 7 years it is a wiser course for us to be prudent and thoughtful and to wait until we actually know what it is we are debating and what the potential impact of these provisions could be on the lives of real children after all this debate is going to set the stage for how much or how little we as a nation will do for elementary junior high middle and high schools
i am particularly concerned about the impact we will have on our neediest children those who are too often left behind we still have too many children who are not reading at grade level and who are being taught by uncertified teachers and too many who are in overcrowded classrooms and dilapidated school buildings i know that all of us on both sides of the aisle agree that we can do better than this we cant just sign a blank check or decide that we can proceed on bill language we have not even seen and discharge our responsibilities to the children we represent in this body
many of my colleagues and i have serious concerns about the substance of the bill for example the block grant demonstration program so far as we are aware of it without having seen the language of it does not target enough funds to our highestneed districts and will mean less control for local school districts on how best to invest their federal education dollars because we have not yet seen the final version of the bill we are considering we dont know whether there is a genuine commitment to devote the resources necessary to make the promise of greater accountability a realistic outcome
just as we expect teachers administrators and students to abide by a high standard of accountability we should bring our backroom negotiations to the floor of the senate for all of us to hear that is why i voted to proceed with the bill but we should do it on the basis of an actual bill i for one am willing to wait and to be patient until we actually get the bill and then to proceed in an expeditious manner
if we look at where the negotiations are and what we are attempting to achieve we have a great opportunity to accomplish some very important goals for the people of this country we all share the goal of improving our nations schools we agree that everyone should be held more accountable for turning around failing schools there is a bipartisan agreement that is very strong for ensuring that all children should be taught by high quality teachers and that parents should know the quality of the schools their children attend
this bill so far as it is reported to us does a tremendous job of strengthening accountability i applaud senators kennedy and bingaman for leading the negotiations that have resulted in important accountability provisions
some have asked why dont we just call it quits lets just put in more accountability lets just test our children every year from third through eighth grade we dont need to do any more than that
i ask what is it we are attempting to achieve if all it does is to put more accountability on the already existing testing systems that every one of our states have employed what is it we hope to achieve
the answer is that in order to have real accountability we have to marry those accountability measures with targeted additional resources invested wisely that will really make the difference as to whether the tests actually create better educational outcomes
resources would make a difference for children such as delano tucker a fifth grader from ps 41 in the bronx who wrote me that his entire fifth grade class was asking for help to improve education here is what delano said
we need more books but we cant do that without more money my second reason is we need more teachers because classes are too crowded the third reason is children are passing without knowing how to read
we dont need to get a bunch of experts or senators who can come up with a better analysis than what delano just gave us we need better teachers more books less crowded classrooms and we should not be passing children who dont know how to read
resources would make a difference for the nearly 168000 children who go to school every day in overcrowded classes in new york city we are losing teachers every single day because teachers cant teach in the kind of circumstances that we are presenting for the state of education in many of our cities
one new york city parent recently shared her thoughts with me writing that
i am a parent of two young children one in kindergarten and one in third grade they are both bright but they suffer from learning difficulties in part because they are trying to learn in classes of 28 children they are unable to get the individual attention they need because they are competing for the teachers attention with so many
how can we expect children in classes that are that crowded given the difficulties and issues that children bring to school today to be able to get the same quality of education that we know works so well when classes are smaller in the early grades
resources would have made a real difference for the fourth grade teacher at the 82yearold mechanicville elementary school just north of albany ny who last year was struck in the head by concrete from the ceiling as she was teaching because the school was in such disrepair
my colleagues and i have heard similar stories from students and teachers in every state around the country although education is and always will be a local issue it has to be a national concern some of the most severe problems in education today require national solutions i think that is why we are here today debating education
how will investing in school repairs and renovations help to raise student achievement i think the answer is selfevident especially if you have a teacher hit in the head with concrete falling from the ceiling we know from research that children benefit when they attend school buildings that are in good physical condition
a 1996 study of large urban high schools in virginia found that student achievement was as much as 11 percentile points lower in substandard buildings as compared to standard buildings
another study found that the quality of air inside public school facilities may significantly affect students ability to concentrate in fact the evidence suggests that children under 10 are more vulnerable than adults to the types of contaminants found in school facilities we have seen reports and studies about working conditions in urban schools concluding that they have direct positive and negative effects on teacher morale their sense of personal safety their feelings of effectiveness and on the general learning environment that kind of scientific conclusion is reinforced by the experience of students in mount vernon ny who go to school with air ducts that are so old and so clogged up and filled with pigeon and rat droppings that they cant even breathe decent air or the students in cohoes ny who go to a school that banned the use of chalk because they have inadequate ventilation and the chalk dust would hang like a curtain in the air
too many of our students are trying to learn in cramped trailers such as in this photo taken in queens these may be socalled temporary trailers but they can end up representing a big part of a childs educational experience
too many of our children are in hallways with many distractions and far too little room this photo represents a common sight in schools in new york this is not a classroom this is a hallway the children arent in a classroom that you and i remember where there is a chalk board a teachers desk and the desks of the children and bulletin boards with pretty displays this is a hallway and this is their classroom
i dont know how much longer we can keep hearing stories about hallway classrooms falling concrete conditions in the classroom that are unhealthy and not recognize that we should be helping our school districts many of which cannot possibly afford to raise their property taxes we cant ask hardpressed parents to put even more money into the property tax base we should be helping the parents in those school districts
during this debate i will do everything i can to urge my colleagues to support senator harkins efforts to include authorization for an emergency renovation and repair fund that would certainly make a difference for some of the schools we just saw
i will also be offering my own amendment to examine the impact of dilapidated schools on the health of our children it is simply unacceptable in america in the beginning of the 21st century that our children should have to attend schools that not only impair their ability to learn but even make them sick
 newspeech
as the senator points out i have been involved in improving education and reforming our accountability measures since 1983 when a nation at risk was first issued by thenpresident reagans commission on education i was one of the first in our country to ask for much stricter accountability to test not only students but also teachers and to hold schools to a very high standard if they did not succeed in passing 85 percent of their children beyond a level of acceptable learning outcomes the school would be in danger of being taken over that was 18 years ago
so there is really nothing new in what we are discussing today as the senator from nevada knows so well we want to do the best job we can in raising standards yes we do that is something many of us have worked on and we have actually seen some positive results in some of our schools over the last 18 years but we know there have to be the kind of conditions in learning circumstances in our classes in our schools that will enable these accountability measures to be successful
 newspeech
as the senator knows when somebody says money doesnt make a difference they are talking about somebody else and somebody elses money every one of us in this body goes to the extra length of making sure that our children and any children we care about are given those kinds of resources that will enable a child to learn
money is not the only answer to what we need to do if we are serious about zeroing in on those children most in need most of our schools in this country are doing a fine job
i live in a district in new york that is one of the best in the entire country many of the other districts in our suburbs and rural and city areas are producing good students who care about learning our real problems are in those areas with concentrated poverty
i have seen the senator from connecticut come into the chamber he has a passion about getting our resources targeted where they can do the most good so to anybody who says money is not the only answer of course i say money is not the only answer but money helps when married to accountability and invested in getting rid of conditions such as the ones i am showing here on the picture where there are so many children in this classroom where it is impossible for even the best trained teacher to be able to communicate effectively with these children this is a classroom where the children are coming from backgrounds where english is not their first language coming from concentrated poverty often difficult family situations
so when somebody says we dont want to throw money at it i say thats right i want to target money to make sure we clean up our dilapidated classes and schools and that we provide lower class size so that the teachers who are willing to go into our hardtoteach areas will be able to have a decent chance to reach these children to recruit and retain teachers who come in with idealism and find themselves in situations such as this and within a year or two are gone
for me there isnt a contradiction here as the senator from nevada knows so well we need to have the kinds of accountability that is effective and will work but without the resources we are not going to be successful
we are going to find as i have said in the past that we are just passing out thermometers in the midst of an epidemic we are going to find that everybody has a raging fever but we dont have the resources or the will to help them get well we can do both that is what this opportunity provides
i appreciate the concern of the senator from nevada we have to have a good debate it is only fair if we are asking that we invest more dollars in education from the federal government we be able to justify the use of those dollars and we tell our constituents and our colleagues where they will go i have pointed out they go to helping clean repair and construct schools we need second they go to reducing class size the situation shown in this picture is unacceptable
we are under court order in new york city to have only certified teachers in the classes that sounds great and i am for it but in order to have certified qualified teachers go into a situation such as this we will have to make a contract with these teachers that this situation will improve they will find they will have a chance actually to teach otherwise they will vote with their feet and either leave to go to a suburban district where they are paid a lot more in a lot better situation or they will leave teaching altogether
i am not talking about something that is anecdotal we have research from project star in tennessee that demonstrates children assigned to smaller classes in grades k3 received better grades higher test scores and were less likely to drop out of school or be held back through their entire educational careers this is a research study that has gone on for 15 years in the entire state of tennessee i applaud the state because they made the investment to evaluate what they were doing
we found that the children who benefited the most were poor and minority children by all means test them and find out if they are failing but be fair and give them a chance to succeed that is what we are calling for when we ask for reduced class sizes
we know if we dont recruit teachers we will not be able to continue teaching anybody right now we have a national crisis when it comes to recruiting and retaining teachers there isnt any more important factor than teacher quality in improving student achievement yet if you are a young teacher placed in a situation such as this if your classroom is a hallway as i have seen in some schools in new york a closet that makes it very difficult to teach
i recently heard from a constituent in farmingdale ny who told me their elementary school alone needs 16 new teachers for kindergarten in buffalo 231 teachers retired last year compared with an average of 92 retirees in each of the preceding 8 years
we cant just mandate that school districts go out and hire certified qualified teachers without providing some resources to make that possible we tried that in new york city the court order said hire only certified teachers and put those certified teachers into the classes where the kids are most at risk so the school district went out hired 2000 certified teachers assigned them to schools as depicted in this picture and the previous pictures and the 2000 certified teachers wouldnt take the job who can blame them they are certified teachers qualified they pass the tests they have taken the courses they are assigned to a school where the conditions to teach are impossible
if we are going to say lets only have certified qualified teachers then for goodness sake provide help to districts such as those i represent so we can actually recruit and keep those certified qualified teachers i strongly believe this bill should include a teacher recruitment section i am working with a bipartisan group to offer an amendment to help school districts meet the demands for certified teachers
let me turn now to title i i would like to paint a picture of what full funding for title i means for the children of new york city yesterday several of my colleagues from the other side of the aisle came to the floor to talk about the failure of title i to improve student learning and dismissed the idea that fully funding title i could result in increased student achievement
i want to be sure the american people have the facts about title i the real fact as presented by the independent nonpartisan congressional research service is that in fiscal year 2001 congress provided school districts with only onethird of the resources needed to fully serve eligible students in order to help close the achievement gap even with this limited federal investment our school districts have shown real gains in reading and math
in 1999 the council of great city schools found fourth and eighth graders in urban schools boosted their performance in reading and math in fact 875 percent of the urban school districts showed reading gains in title i schools and 83 percent showed math gains moreover the study found that the percentage of title i students in urban schools below the 25th percentile had been declining over 2 and 3year periods while the percentage of title i students between the 25th and 50th percentile was increasing
there are those who will still deny these facts and make the claim that title i doesnt make a difference i often think washington is the only evidencefree zone in our country the facts are the facts title i does make a difference imagine the results if cities such as new york buffalo rochester or syracuse were able to assist all our title i eligible students rather than just a third of them it would mean for example in new york city we could lower the current threshold and serve an additional 99295 children the city could invest in strategies that work better we could provide extended time initiatives that we know make a difference with children we could expand early literacy intervention and intervention strategies have classroom professional development for teachers
as we look at the bill we need to look at a full investment in title i it is not just a game of imagination but a real investment in student improvement that will pay off down the road i will support senator dodd and senator collins in their efforts to include full funding of title i in this bill
finally let me touch on the issue of testing in 1983 i called for student tests highstake student and highstake teacher tests i take a back seat to no one when it comes to using testing and other measures of accountability to find out how well we are doing and hold ourselves accountable but lets be sure the tests are actually going to accomplish the purpose for which they are intended we need to look at how children do from year to year to help teachers modify and individualize curriculum and provide parents with timely information we have to make sure that if they take a test in the winter they get the results that winter not the following fall when the children have moved on we have to help schools know what the standard should be so they are not teaching to the tests but they are trying to measure the standards they have set and we have to help pay for the tests
in new york alone it would cost usd16 million to comply with these new federal testing requirements only usd8 million would be provided by the federal government the other usd8 million is from scarce state resources we need to be sure we are fair to our states if we are going to mandate testing lets not make it an unfunded mandate lets provide the resources needed if we do develop and implement the tests we need to have the resources to ensure that our children from the most disadvantaged circumstances can pass and excel in those tests i think that means smaller classrooms modern schools quality teachers
as we go forward in this debate i hope we will think hard about the impact we will have on our children and that we do everything we possibly can to make sure we dont just pass a bill but we really do provide the resources to reform education and produce better results across our country
i yield the floor
 newspeech
mr president i associate myself with a number of the points made by my friend from arkansas clearly what we are attempting to do is to put the emphasis on what works and to provide to our children the opportunity to have the best possible education
i have been very privileged over the last 20 years to know quite a bit about education in arkansas which my good friend has the privilege of representing and now i know a lot about education in new york i have no doubt that my friend were he still here would agree with me that our goals are the same for the children in both states we want to provide the best possible educational opportunities but we face very different challenges
what i saw and worked on for many years in improving education in arkansas which was one of the great honors of my life is very different from what i now see day in and day out in new york city where we have more than a million children in our school system
i agree with my friend that what we are crafting is an approach that will give to local school districts parents and teachers the tools to make the right decisions for the children whose futures they hold in their hands that is why i wish my friend were still here and i will seek him out later to talk with him privately about this
that is why i am such a strong supporter of senator murrays amendment because what senator murray has done is point out very clearly that one size does not fit all that what we need to do is provide the tools that will enable each school district in each state to deal with the problems they face
so i want to be part of passing legislation in a bipartisan way that will be the best for arkansas the best for washington the best for vermont and the best for new york because we will have honestly looked at all the different tools we need to provide our local educational authorities with in order that they can do the job we are now asking them to do their very best in achieving
so i am very proud to be a cosponsor of this amendment and to stand with my colleague in stating my commitment to supporting the class size reduction initiative both because it is voluntary and provides additional funding to schools that are in desperate need of such funding and maybe most important because we know it works
i went back and reread president bushs blueprint for education called no child left behind in it he expresses dismay that over the years congress has developed programs without asking whether or not programs produce results or even knowing the impact on local needs later on the president goes on to suggest that under his education plan which is really the core of what we are debating in this education debate he will focus on what works and ensure that federal dollars will be spent on effective researchbased programs and practices and that the funds will be targeted to improve schools and enhance teacher quality that is certainly what the committee on which i am proud to serve under the leadership of the senator from vermont attempted to do in reporting out such a bill to focus on what works and to target funds to improve our schools and enhance teacher quality president bush and i absolutely agree on this point
i have often said that i sometimes fear washington is an evidencefree zone where despite whatever evidence we have we dont follow it we dont put it to work and we spin our wheels too much well i believe we should look at what works what has had a positive impact in raising student achievement what has helped at the local level give very necessary resources there is no better example of what works than reducing class sizes so that teachers can teach and children can learn
allow me just a moment to review the research demonstrating that reducing class size has proven results teachers who teach in classes of 18 students or fewer in the early grades are helping to raise student achievement for our most educationally disadvantaged students who are attending schools in highpoverty neighborhoods where we all know it is harder to teach
senator murray was a teacher she was on a school board i dont think any of us should kid ourselves there are some school districts and some schools where it is just hard to teach where children come to school with all kinds of challenges and difficulties we know as we look at the research done that if we focus on getting that class size down with a qualified teacher this should not be an eitheror it should be a qualified teacher and a small enough class size then we can have very positive results
i particularly point to the work senator murray and i highlighted in a press conference a few weeks ago that was done at princeton university by an economist named dr alan krueger who tracked the performance of well over 11000 elementary school students at 79 schools in a tennessee pilot program known as project star this was done randomly the results are scientifically provable what he found and what everyone who has studied it has found is that smaller class sizes have a tremendously positive impact on student performance and particularly on african american students
we want to be supporting both excellence and equity that is why i support accountability i think we should know what our children know and what they dont know i also believe everyone in this chamber understands that we have to do more to increase the opportunity for excellence by focusing on the students who are most likely to be left behind to me the fact that african american students have such positive results from lower class size is a very strong argument for us renewing this commitment there are other studies which have found exactly the same thing a rand study and rand usually studies issues such as the military and defense and national security focused on costeffectiveness of educational resources in raising scores on the naep the national assessment of educational progress it is a test that is given to a randomly selected group of our students across the country we use it to track how well we are doing as a nation
what rand found in looking behind these test scores was that the higher scores could be traced to investments in lower class sizes in the early grades plus higher prekindergarten participation lower teacher turnover and higher levels of teacher resources so it is that complement of costeffective strategies that i think we should be supporting in this legislation
later in the debate i will focus on the importance of supporting early learning opportunities and trying to retain our teachers because we are losing our teachers at an alarming rate i brought this photo of ps 19 in jackson heights queens which is one of the magnets for immigration into our country people come to kennedy or laguardia airports and they end up in queens i wish i could take every member of this body to the schools i visit in queens where bathrooms are classes hallways are classes and where children speak 40 to 100 different languages where they are packed in there and where a teacher despite her best efforts cant possibly connect with all these children
yesterday i was in a school that works in manhattan the new manhattan school it is a wonderful school i met for a long time with the teachers the principal and the superintendent of the district it is an old building built in 1904 it is packed to the rafters they are adding teachers into classrooms so if they do not have the additional classrooms at least they have more qualified teachers in those classes so the children get the attention of the adult responsible for their learning
it is important we understand there have to be opportunities for local communities to make choices i believe having this tool is essential for providing good opportunities for choices to be made
with the funds appropriated in 2001 it is expected the federal governments class size reduction initiative will bring nearly 40000 qualified teachers into classrooms any one of us who goes into a large city in our country knows that if we do not have qualified teachers and we do not have low class sizes we can test until the cows come home and we are not going to find anything other than what we already know that children from highpoverty areas from dysfunctional backgrounds without adequate training for academic work are not going to do well but that a qualified teacher working with a small enough group of children as senator murray knows so well can make all the difference in that childs future
when we looked at this issue in new york city we saw the results clearly two years ago the program was initiated and class sizes in new york city were 25 percent larger than statewide with both federal and state initiatives we were able to reduce class size for approximately 90000 students in the early grades almost 30 percent of the citys k3 population
i want people to keep in mind i am talking about a million children and 90000 children i know it is hard for some people who represent states without that many people in the state or maybe only half that many to understand we are dealing with huge numbers in a lot of the large cities it is not just the numbers it is the real lives behind those numbers
when we looked at the results after 2 years of efforts we were very pleased because achievement went up in those classrooms where with federal help we were able to add a teacher
that does not mean the local communities do not have to continue doing their part and it does not mean the state does not have to do its part but we have gotten behind in what we need to do for our children we need all hands on deck we need everybody pulling together education is a local responsibility in our country but we all know it has to be a national priority
let us make sure we focus on both teacher quality and lower class size that is why this amendment which senator murray has championed and has been successful in persuading a bipartisan group of senators to support in the past is a critical component of this legislation
if we can make it possible for class sizes to remain small in the early grades we improve the chances dramatically of producing a productive functioning citizen who can find his or her way in this complicated society and global economy that awaits them in the 21st century
yesterday when i was in this wonderful school that was filled to the brim they took me into a bathroom that had been turned into a guidance counselors office they did not have any other space we went into the gym and children were doing their physical activity which i believe in strongly we have to keep childrens bodies active as well as their minds
there was a partitioned area in which there were more offices they were making the best of a very difficult situation they had just been told a school down the block a little elementary school had been condemned we will get to that later in this debate too this school had been condemned it is unsafe for our children and teachers
there is a school in mechanicsville ny where a piece of concrete fell on a teachers head while teaching in the classroom
there is a condemned school a few blocks from where i was yesterday they are already packed the school i visited will be taking in the children from that condemned school
this is a critical component of the commitment to excellence and equity accountability and resources that the president has called for which so many in this chamber have championed for many years we have the money to do this we just have to determine whether we have the will
i call on my colleagues and echo the very eloquent call of the senator from washington that we recognize that continuing this initiative does help local communities meet the needs they see right in front of them and let us make sure we do everything possible to make every child believe he or she is important so that at the end of this debate the bill we pass truly will leave no child behind
i thank the chair i yield back my time
 newspeech
mr president today i am so pleased to join my good friends senator hutchison from texas and senator bayh from indiana in supporting this legislation to help medicare payments keep pace with the rising costs of hospital care and to halt further medicare reductions to teaching hospitals
our hospitals are under tremendous strain they face soaring costs from nearly every direction the growing number of uninsured individuals coupled with the devastating shortages of skilled health care workers the struggle to afford skyrocketing pharmaceuticals prices while simultaneously investing in emerging needs such as information technology at the same time reductions in medicare payments have hindered hospitals ability to respond to these increased demands how can we expect patients to receive quality health care when were asking our hospitals to do more with so much less
as you know this week we are focusing on the crisis around the shortage of nurses ninetyone percent of hospitals in new york state report shortages of registered nurses rns but this is really just the tip of the iceberg the shortages in the health care workforce permeate the entire health care system especially our hospitals there are shortages in pharmacists technicians nurse aides billing staff and housekeepers that have all negatively impacted the quality of care new yorkers are able to receive
as a representative of the state of new york i am especially troubled by the growing strains that our hospitals have been forced to contend with on top of the devastating cuts that have resulted from the balanced budget agreement of 1997 bba i have heard numerous firsthand accounts of the adverse impact on new york hospitals and the facts speak for themselves in the 2 years following the bba new york hospitals financial health ranked worst in the nation in fact almost twothirds of new york hospitals had negative operating margins last year and in addition to the workforce shortage affecting health providers nationwide new york providers are also confronting labor costs increases of 57 percent a year while the medicare rates for inpatient hospital rates even with the full market basket update we are seeking in todays legislation expected to rise only around 31 percent
in recent years congress has successfully provided some shortterm relief to address areas where the cuts enacted in the bba of 1997 went much further than intended however much of the relief merely postponed scheduled cuts in medicare payments and that is why the legislation that we are introducing today is so important
this legislation today would eliminate some of those previously delayed cuts first it would restore the market basket update for inpatient hospital rates to the full level rather than marketbasket minus 055 percent as scheduled for fiscal year 2002 and 2003 this important step will help hospitals nationwide keep up with the rising costs of inpatient care for medicare beneficiaries this provision helps all hospitals in new york state by increasing inpatient hospital payments across the board
i am especially pleased that this legislation would also address the cuts faced by teaching hospitals to their medicare indirect medical education payments teaching hospitals are the crown jewels of our nations health care system and play a vital role in making our system one of the finest in the world
we rely on them to train physicians and nurses care for the sickest of the sick and the poorest of the poor and engage in research and clinical trials thanks to the research for example at memorial sloankettering cancer patients will suffer less while receiving chemotherapy because of a drug that was developed there
as my predecessor and friend senator daniel patrick moynihan in whose footsteps i am so honored to be following put it so well a few years ago we are in the midst of a great era of discovery in the medical science it is certainly not a time to close medical schools this great era of medical discovery is occurring right here in the united states and it is centered in new york city
this legislation that we are introducing today would address the cuts faced by teaching hospitals to their medicare indirect medical education payments last years medicare medicaid and schip benefits improvement act of 2000 bipa provided some relief by delaying the cuts to help teaching hospitals cover the costs of caring for sicker more complicated patients todays provision would make that relief permanent by freezing the indirect medical education adjustments percentage at 65 percent
in addition teaching hospitals throughout the state would benefit including rural hospitals such as kingston hospital benedictine hospital champlain valley physicians hospital medical center olean general hospital and hepburn medical center in ogdensburgh ny
todays legislation is essential to ensuring that our nations older and disabled patients can continue to receive the high quality of care that they deserve i look forward to working with my colleagues and the administration to address this and other important health care priorities
 newspeech
mr president i rise today to honor the career of an outstanding soldier and a good friend lieutenant general daniel w christman who is retiring after more than thirtysix years of active military service general christmans exemplary military career culminating in five years as the commanding general and superintendent of the united states military academy at west point exemplifies the professionalism and seriousness of purpose that have helped make the us military the best in the world
prior to his service at the united states military academy general christman had a remarkable military career for over 30 years general christman graduated first in his class from west point and later taught in the department of social sciences as an assistant professor of economics he has held several senior executive positions in the army all of which have taken advantage of his unique talents for creative leadership and strategic vision using his training in civil engineering he has commanded a major us army corps of engineer district in savannah georgia and headed the armys engineer school in the early 1990s
throughout his distinguished career general christman has played a vital role in development and implementation of some of the most important security policy issues of the last several decades he served in the ford administration as a member of the national security council staff during the gulf war he directed a strategic planning group which advised the armys chief of staff on war prosecution policies he represented the us in brussels belgium as a member of natos military committee where he had active involvement in the historic expansion of nato pursuing peace in the balkans and military dialogue with russia immediately before arriving at west point general christman served for two years as assistant to the chairman of the joint chiefs of staff in the pentagon in that position he advised the secretary of state on a broad range of issues including arms control with russia and middle east peace negotiations between israel and syria
general christmans tenure as the 55th superintendent of the us military academy has been marked by a forward thinking strategic vision and the development of a more cooperative and positive environment at the academy i met with general christman soon after i was sworn in as senator and have been greatly impressed by his leadership at west point his success at obtaining critical funding support has enabled west point to continue to attract high quality young cadets willing to embark on army careers he helped to raise funds for the center for the professional military ethic as well as endowments for several academic department chairs and improved athletic facilities he helped to inspire the creation of a dynamic and forwardlooking strategic vision for the us military academy 2010
general christmans exemplary service and devotion to duty honor and country have left a lasting impact on the us military academy and indeed the us army his numerous awards reflect the respect and admiration of those who have had the privilege to serve with him i join my fellow senators in wishing general christman the best of luck in his future endeavors and my sincerest gratitude for his distinguished service to his country newspeech
mr president i am a big believer in the importance of testing students i think that testing has an essential and appropriate role in the curriculum of any educational system i think there is no doubt that we have to test in order to determine whether or not students are meeting high academic standards it would be a delight i suppose to most students who think that we are not going to test them but indeed we are
i think this debate and what the senator from minnesota is attempting to bring our attention to is that there are tests and there are tests making sure that the tests are used for the purpose of measuring student performance determining what kind of additional help a student might need is really what we are focused on through the senators amendment
i appreciated very much chairman jeffords important amendment that we voted on last week to make sure we have federal support financial support behind the design and implementation of these tests because we want to send a clear message to states and local districts that we believe in accountability but we want to put some dollars behind that belief by saying we want you to design and implement tests that are going to really measure what students learn
right now many teachers who contact my office or the ones i see when i visit schools as i did on monday in new york city are terribly concerned that what might very well happen is that more and more testing will be piled on without there being any requirement that they be worthwhile tests and without the resources to assist the teachers who after all are on the front lines in the classrooms in knowing how best to address the needs of their students that are revealed by the tests
i was very impressed by this document put out by the committee for economic development my colleagues know that the committee for economic development is a group of business people in our country they are very committed to creating the conditions that will further economic development and they know that one of the key conditions if not the most important one is the quality of our education looking at the board of trustees and the committee for economic development we have people from the leading corporations in america who see firsthand what their employees need when they come into the workplace who are on the front lines of hiring people for a job they have put out a publication that i really commend to my colleagues to the administration and to all of us who are concerned about using testing to improve student learning it is called measuring what matters it makes many of the same points that senator wellstone makes
it might be somewhat surprising for some of the people who serve on the board of trustees for the committee for economic development to know that they agree with senator wellstone but they do they agree that what we need are tests that will actually improve student learning that certainly is what the intent of the bill that we reported out of the health committee under chairman jeffords leadership was aimed at doing how do we make it clear that tests are a means they are not an end in school reform we dont just give the tests and pick out winners and losers we have never done that in the united states one of the reasons our educational system is both unique and successful and has been for decades despite our problems which we talk about endlessly we should look at some of the reasons why we have been successful
i would rank near the top of that the flexibility of our educational system we dont give a test when a child is 11 years old and say all right this group of children you are consigned to a certain set of occupations this other group you did well on the 11yearold test so we are going to send you to different schools and put you on a different path
we dont test when children are 14 and make that conclusion we dont say that there are some children who can only attend certain kinds of courses in certain schools and others are barred because of tests we dont have the kind of onetest determination that opens the doors or shuts them in colleges in other parts of the world i think that has served us well in our country
there are a lot of people who dont take school seriously until they are in high school sometimes they graduate and maybe then find their way to a community college then they really get energized they know what they want to learn so we have always viewed tests not as a stop sign for a child the system holds up and says you are a loser you dont know anything we use them to say look we need to help how can we provide more support for you to be able to get the most out of your education
i think it is important for us to remember that tests are not an end they are a means they should be a means toward lifelong learning or improving the climate for learning or for giving individuals the tools they need to be successful not just in the classroom but in life
it is also important as the committee for economic development points out that tests need to be valid and reliable and equitable there should not be any doubt that i think any good test would meet those three criteria first of all validity are we measuring what we intend to measure if we spend the whole year teaching children one set of facts or studying one set of subjects and we test on something else that is not a valid test so we need to make sure that what we measure is what we are teaching and what we are teaching is in some way reflective of the standards of what we expect from our educational system
reliability is also a given how consistent and dependable are the assessment results are these tests that teachers and parents and students and community leaders can depend on because they really reflect what we want our children to know
finally are they equitable tests that doesnt mean there are two standards one for certain children who live in affluent suburbs and one for children who live in our poorest neighborhoods no if we are doing anything with this effort it is to try to make sure we combine both excellence and equity and we do everything possible to give the opportunities where they are most needed
we know we have to be very careful that our tests are fair that they have no sign of bias toward any group of students we need the help the federal government should provide if they are going to stand behind the regimen of testing we are considering in this bill
we also need to be sure if we are going to be using tests that we get timely results i offered an amendment in the committee if tests are going to be given the results ought to be available in 30 days and no more what is the point of giving a test in april and you get the results in june or july when the children have gone home or may not get them until the following year
we should have a sensible testing schedule and we should require that the results be provided in a timely manner to parents students and especially our teachers if they are going to be used for diagnostic purposes and to measure and grade the curriculum as well as the children
there are a lot of tests that are currently being administered we give tests for everything now we give tests for graduation we give tests for promotion we ought to be sensible about this if the federal government through our actions in the congress and the administration are going to say we want a test every year from third to eighth grade to determine how effective our children are learning reading and mathematics then states have to take a hard look at what else they are testing because it is getting so that many of our schools feel they are spending all their time preparing for tests administering tests and grading tests we have to be sure the tests are appropriate in number as well as content
i also hope as we move forward on this important education debate that we recognize that accountability for students and teachers is best tied to school performance i go into schools all the time that are literally within blocks of each other some are very successful and some are not a lot of it has to do with how the school is organized and what their priorities are i hope the testing we are discussing to be implemented in this bill will help us move entire schools toward better outcomes so that we lift up the performance of a school and create the atmosphere that will be conducive to learning and teaching
one thing that bothers me though is that in our rush for tests and in our implementation of so many tests a lot of schools are finding it impossible to keep the more wellrounded curriculum that has been the hallmark of american education
i believe music art physical education extracurricular activities even field trips are a part of the educational process what i hear from so many schools in my state is that the tests take up so much time the costs of the tests and all that goes with the tests mean that a lot of other important educational objectives are being eliminated
i hope we take a view of testing that puts it into the context of american education generally i take a back seat to no one in saying education has to be a local responsibility and a national priority i have had experience in advocating for testing
i believe i was the first person in the country who advocated testing teachers using highstakes tests i even recommended schools be based on their performance in how many students they could bring up to grade level but i am very cautious and i guess i am putting up a caution light that we not go so much toward testing as the definition of education that we forget what the learning process is and how unique the american education system is where people can literally wake up in 10th grade or 12th grade or a child can be exposed to art or music or some other part of the curriculum such as a good science lab in the eighth grade and all of a sudden learning becomes real and they are not consigned to a secondclass citizenship because they did not get into gear before that time
we are starting to see with our highstakes testing in new york a lot of dropouts we are worried we are beginning to see an increase in dropouts we have to take that seriously our goal is not to test children for the sake of testing then telling them they do not measure up and then holding them back for the sake of holding them back until they become so frustrated and discouraged they leave the educational system i do not think that is the goal of any of us in this chamber
our goal is to have an accountability system so that we actually know what is being taught and what our children are learning and use it for diagnostic purposes to make every child a success
raising the caution lights that the amendment of the senator from minnesota raises is important for us to think about i will add one additional caution light i guess that is the biggest issue of all for me and that is the resources i am very concerned as i will state when we come to this in the days ahead about the budget we have been promised it will leave no child behind and will provide the resources for extra testing to deal with special ed to deal with more resources for our poorest children to add teachers so we have lower class sizes to modernize classrooms i am worried that none of that will be in the budget
that puts many of us in a very difficult position because we know that accountability is necessary but we also know that resources in our poorest schools are an absolute necessary condition for a lot of our kids to be successful
i enjoyed listening to the senator from wyoming talk about the very small school districts of fewer than 50 children i have some very fond memories of districts that small in arkansas i remember going to graduating classes of three and four children that is a very different and wonderful educational experience i hope we never get away from that in our country that we do have schools that are that small in states from wyoming to upstate new york
i come from a state that has some different kinds of problems i have a school system with a million children i have school systems such as that of buffalo where the school stock is so old they cannot wire them for computers because the buildings were built like forts
i visited a school called the black rock academy that was built in 1898 last renovated in 1920 they are bewildered about what to do they cannot figure out how to get those computers set up they have wires coming up going in a window into a little room they have about 30 computers only 10 of which can be connected to the internet that is the best they can do under the circumstances buffalo has undertaken using state dollars and local dollars a tremendous school renovation and modernization program
our needs in new york are different than the needs of the small districts in wyoming i hope we are going to look at all of our children from coast to coast and all of our local school districts to figure out what we can do to make everybody successful resources are key it is more difficult to provide education in remote rural areas and in very concentrated poor areas in our inner cities we need a bill and we need the resources in the bill that empower local communities to make the decisions that are best for them
there is a wonderful menu of opportunities in the bill where people can choose professional development or technology but we would really be selling our children short if we do not also include lower class size and school modernization because in the absence of some federal help on those two issues much of what we want to achieve is going to be very difficult and beyond the reach of many of our districts even those that are making a goodfaith effort such as buffalo to deal with a very old stock of schools
i kid some of my colleagues we were educating people in some communities in new york before some of the states represented in this body were states we were building schools before a lot of people had to build schools because of the centuries of history in new york we have some of those schools that have been around a very long time
good education can and does occur in those schools but the conditions are worsening to the point where as i said the other day we have concrete falling out of a ceiling hitting a teacher on the head we have overcrowded classrooms if we are going to be seeking both excellence and equity we have to do more to provide the resources all districts need to do the job they want to do for their children
this is a very important issue that goes right to the heart of this budget i along with many of my colleagues was very disturbed to learn there was no increase for education in the budget coming back from the house this body voted in a bipartisan way for important measures that were attached to the budget this was not just about numbers it was about values the value of making sure we put the dollars into our education system and many other important priorities from defense to food safety
the budget coming back does not reflect that it does not reflect the flexibility for the dollars that will be needed to do what we have already voted for in the senate
i was very proud of the vote that said we need to fund special education it is about as close as we can get to a mandate a lot of school districts are under tremendous pressure because they cannot afford to do what they need to do i was proud of this body for voting to fully fund title i that was a values statement it said our values are that we will invest in our poorest children i was proud of our chairmans amendment that if the federal government puts this requirement of testing on our districts the federal government should help to pay for the development and implementation of those tests
this body in a bipartisan way made some very important values statements about education not that we were just going to pass a bill that sounded good but one that could actually produce results i am very pleased that at least in the senate we are crafting a bill that i think will make a difference in the lives of our children if we continue on this path it could revolutionize education across our country but it cannot be seen in isolation from the budget which after all carries the resources that will determine whether we have anything other than an empty promise
i appreciate the opportunity to add my voice to what we are trying to do in this chamber and to look for ways to work with my colleagues on both sides of the aisle to make sure it is real
 newspeech
mr president i thank my ranking member the senator from north dakota who as my good friend from south carolina has put so well has led with honesty and directness and believes so passionately in the issues that we are addressing today
i rise because i cannot remain silent in the face of both a budget process and a budget product that i think will be so harmful to our country i really wish i did not have to rise today i wish given the opportunities that lie before us as a nation what we were debating was the kind of balanced approach to the budget that i could wholeheartedly support a balanced approach that included an affordable reasonable tax cut that fairly went to all americans giving every one of our families a mothers day present as senator byrd so wonderfully reminded us is around the corner
i wish this budget were filled with the kind of careful analysis about the investments that we need to make our country rich and smarter and stronger in the years ahead and i wish this budget continued to pay down the debt in the way that we had been doing
in the last 3 years we paid off more than usd600 billion of our debt we took it off the backs of all these schoolchildren who are watching us we said we are not going to pass on the debts of your parents your grandparents the greatest generation did not leave us in debt the way that this country did in the 1980s with the quadrupling of our national debt i cannot stand here and say that
i look at all these faces i meet with schoolchildren from throughout new york nearly every day i wish i could say i am going to go to the senate chamber and support a budget that will invest in education the way we need it that will continue to pay down the debt so that you are not faced with that debt when you are my age or even younger and that it will invest in social security and medicare so that you do not have to worry about your parents your grandparents or yourselves unfortunately i cannot say that
i have thought hard about what it is that has happened in the senate in the last several months because i sat through 16 hearings in the budget committee they were informative very helpful hearings laying out the priorities of our nation talking about the amount of money we had that we could count on not pie in the sky not projections that were unlikely ever to come true but realistically what it was we as a nation could count on and then how could we have a tax cut pay down the debt and invest in education health care the environment as well as taking care of social security and medicare
i do not exactly know what happened how we arrived at this point we had those hearings and then we were shut out of the process we did not have a markup which is a device in a committee to get everybody together to try to hammer out a bill
then the democrats with decades of experience with distinguished senators such as senator hollings and senator conrad were shut out of the process between the house and the senate
so here we are today on the brink of passing a tax cut that will i believe do to our country what was done in the 1980s i can only think that this is a tax cut proposal that was born in the passion of a primary political campaign in the snows of new hampshire when the president was running for his life to be president and had to come up with something so he plucked out of the air usd16 trillion and said that was what it was going to be and felt compelled to come and present it to us
i was proud of the senate when in the process of the budget debate we made some good changes we made those changes not only on the tax cut side but on the investment side i thought if the house can go along with that maybe at the end of the process we can have a better balance i did not think it went far enough but i was proud of the fact that we had a negotiation
what we have today has zero increases in education we have spent a heck of a long time talking about education the president says it is his first priority i can only look at the documents i am handed i have only been handed them recently i was not part of the process even though i serve on the budget committee and it looks to me as if we are turning our back on education
as i thought back i could not think of any analogy i could not think of any guidance that would help illuminate what it is we are going through so i went back and looked at 1981 i read about what happened when another president said pass this big tax cut and we are going to have surpluses and we went further and further and further in debt
it is always easier to pass a tax cut who doesnt want a tax cut i want a tax cut but i dont want to have a tax cut at the expense of hurting my country i dont want a tax cut at the expense of preventing the kind of investment in education that we need i dont want a tax cut where i have to go and tell my mother that medicare may not be viable for the rest of her natural life i dont want that kind of tax cut
 newspeech
so i with great regret stand in this chamber and express the disappointment i feel in that we had an opportunity to do what our country needs to invest in education health care the environment pay down our debt and provide affordable tax cuts but instead we are taking a uturn back to the 1980s mark my words we will be back here maybe under the same president or maybe under a different president having to fix the fiscal situation we are throwing our country into today i lived through that once i do not look forward to it but i will be a responsible member of this body in trying to fix the problem that we are causing for our nation because of this tax cut and budget
thank you mr president
 newspeech
mr president on april 22 2001 i delivered a statement before the warsaw ghetto resistance organizations wagro annual tribute to the martyrs of the warsaw ghetto at temple emanuel in new york city i ask unanimous consent that my remarks be printed in the record along with the statement delivered on the same day by mr benjamin mead president of the warsaw ghetto resistance organization newspeech
mr president i thank the senator from washington i rise to express my very strong support for senator murrays class size reduction amendment
i have been in this chamber several times in the last weeks talking about class size and have shown numerous pictures of conditions in the classes in the schools in new york i have listened to the extraordinary description of other colleagues as to what their students and teachers face day in and day out because of overcrowded classrooms
i know we will be making decisions that determine the opportunities for our educational achievement for our students for years to come when we vote on this amendment and on the bill of which i hope it will be a part
i have to reiterate several points and call on my colleagues on both sides of the aisle to look at the evidence i do believe sometimes in washington we live in an evidencefree zone it does not matter who comes up with whatever scientific research or evidence if it runs against any particular political point of view it is not given the seriousness it deserves
i do not see how we can turn our backs on the evidence that we have from study after study that lower class size when it comes to teaching children from disadvantaged backgrounds makes all the difference
sometimes my colleagues say but there are schools that do a good job with more students and i remember when i was in school and we had a lot of students
i can remember that too i started school when we had three television networks i can remember when we had more twoparent families i can remember when we did not have all of the social and cultural interference with raising children that we now face
the fact is we have to take our kids where they are today and many of them today are coming from situations where they need more attention more adult time more discipline more guidance in order to be academically successful
we are turning our backs not only on the research which points that out time and again but on these children i hope my colleagues who have not seen fit to support this amendment will reconsider it it is not too late to cast a vote for the kinds of classrooms where teachers can teach and children can learn
if you look at our big states with big cities and i know new york has obviously a special set of issues because of the size of our school district in new york city but it is not unique in pennsylvania for example the average class size in philadelphia is 30 children per class in pittsburgh it is 25 children per class in chicago it averages 28 in georgia it averages 32
this is not an issue for just senators or teachers or school board members to be concerned about in debate much of the attention i have seen focused on this comes from parents who know their children are not getting academic assistance they need to do the best they can do
there is a woman in new york whom i commend who started a grassroots parents organization called class size matters she began to form networks of parents around the country who know because they have seen with their own eyes and their experience of their children that class size matters
in pennsylvania alone this class size matters network got 1700 parents to sign a petition in just 2 days urging the senate to vote in favor of class size reductions
i have heard from parents throughout new york who tell me in great detail how crowded their classrooms are and how they need help this does not interfere with flexibility this does not take anything away from the local school districts determining priorities but it does give additional help and resources to those districts and those parents who know that unless we get those class sizes down their children will not learn to the extent they should do so
i also regret deeply that if we do not adopt this amendment we will be stopping the progress we have made
new york state has hired to date 2600 teachers and has 700 more all ready to be hired this will stop that hiring and we know from the 2600 we have already hired what a difference it makes in the classrooms of new york
i believe that without dedicated funding for reducing class sizes our hardest pressed most needy districts will not receive the dollars they need to reduce the classes
mr president i urge my colleagues to stand behind our children our parents our teachers and reduce the size of our classes and adopt senator murrays amendment
i yield back the remainder of my time
 newspeech
mr president i would like to recognize the accomplishments of james a harmon the outgoing chairman of the exportimport bank of the united states when chairman harmon steps down from this position on may 25 he will have served exim bank for 4 years one of the longest terms as chairman in the banks history
chairman harmon came to exim bank in 1997 after a distinguished 38year career as an investment banker in new york he brought his wealth of private sector experience to washington and immediately set about the task of enhancing exim banks ability to achieve its important mission supporting us jobs through exports
one of the early challenges he had to face was the global financial crisis that hit asia and other emerging markets in 199798 recognizing the important role exim bank could play in this crisis chairman harmon directed the bank to extend much needed credit to many of the impacted asian nations to keep trade flowing between this region and the united states perhaps the most dramatic example was in south korea where exim bank provided usd1 billion of shortterm export credit insurance for south korean banks that allowed south korean businesses to purchase urgently needed raw materials and equipment from the united states exim bank supported more than 2400 transactions in south korea during this crucial period compared to less than 60 the prior year exim bank also worked to shore up the struggling asian markets by coordinating assistance for the region from the other major export credit agencies exim banks aggressive response to the asian financial crisis helped stabilize these economies and keep us goods and services flowing to the region until commercial financing was once again available
under chairman harmons leadership exim bank forged into new markets in an effort to increase opportunities for us exporters i am particularly pleased to cite the banks expanded involvement in africa during chairman harmons tenure exim bank unveiled new programs for facilitating us exports to subsaharan africa and expanded the number of countries in this region for which financing support is available notably chairman harmon demonstrated his personal commitment to subsaharan africa by traveling to the region three times becoming the first exim bank chairman to visit southern africa the results of these efforts have been dramatic exim bank support for transactions in subsaharan africa rose from usd50 million in 1998 to nearly usd1 billion in 2000 i know from my own visits to subsaharan africa the vital importance of increased us trade with the region and i commend chairman harmon for his efforts
exim bank also enhanced its presence in russia and the new independent states developing innovative financing structures that allowed us exporters to capitalize on the vast opportunities of this market in june 2000 exim bank launched a southeast europe initiative to develop us trade opportunities in albania bosniaherzegovina bulgaria croatia macedonia and romania an initiative that can help foster the development of these emerging market economies as well as benefit us exporters
while working to support exports to new markets abroad chairman harmon also pushed exim bank to reach out to new groups of exporters here at home during chairman harmons tenure exim bank implemented program changes and marketing efforts aimed at expanding its support for women and minorityowned businesses two groups that have traditionally had difficulty accessing export financing chairman harmon also made environmental exports a top priority recognizing both the potential export opportunities for us producers of environmental goods and services and the importance of promoting environmentally sound development at the same time chairman harmon championed the need for greater environmental responsibility in export financing urging his g7 and other major export credit agency counterparts to adopt uniform meaningful environmental standards for the projects they finance
jim harmon has worked tirelessly at exim bank to create highpaying exportrelated jobs here at home by expanding opportunities for us exporters abroad i am pleased to welcome him back to new york after four years of distinguished service to exim bank and the nation newspeech
is there objection to the modification of the amendment
without objection it is so ordered the clerk will report the amendment
 newspeech
mr president i rise today in support of the bipartisan end of racial profiling act of 2001 i believe it is a thoughtful and balanced effort designed to bring people together not to divide i also want to express my sincere gratitude to my esteemed colleagues senator feingold and senator corzine for their leadership and tremendous efforts in crafting this legislation that affects so many communities throughout this country
i also want to acknowledge the efforts of representative conyers the ranking member of the house judiciary committee and a leader on this issue representative conyers has worked to obtain the support of both democrats and republicans alike including republican representatives asa hutchinson chris shays tim johnson constance morella and jim greenwood i thank them for attending the bipartisan press conference this morning and showing their support for this legislation i hope we will be able to build upon this strong bipartisan support in the senate
i am also pleased that we were joined by chief bruce chamberlin an esteemed and experienced member of the national law enforcement community who is the chief of police of cheektowaga in the western part of the great state of new york
it was important for chief chamberlin to be here with us today to express his support for the bill because he recognizes as we all do that racial profiling is wrong and that this bill is an important step in bringing this practice to an end
racial profiling is unjust it relegates honest lawabiding citizens to secondclass status when they suffer the embarrassment the humiliation the indignity of being stopped or searched and in some cases even physically harmed simply because of their race ethnicity or national origin
racial profiling is not an effective law enforcement tool the experts at john jay college of criminal justice and elsewhere will tell you that the evidence is unquestionably clear for example that the vast majority of blacks and hispanics who are stopped or searched have committed no crime
indeed racial profiling has an insidious and devastating effect on entire communities because it increases the level of mistrust between law enforcement and the communities it is charged with the heavy burden to protect that result serves no one it fails to serve law enforcement because a critical component of truly effective law enforcement is strong communitypolice relations partnerships in which law enforcement and our communities are working together to reduce crime and to make our communities as safe as they can be
racial profiling fails to serve prosecutors because lawabiding people who dont have faith that their law enforcement will protect them properly and treat them with dignity will not have faith in law enforcement when sitting on juries and assessing the credibility of police officers who often play a key role in getting convictions for criminals
what does this bill do and what doesnt it do
as you my colleagues consider this legislation understand that this bill is not about blaming law enforcement or saying that law enforcement is bad or doesnt do a good job we know that this is simply not true
those who uphold our nations laws on the streets where we live are men and women of courage they go to work each day without the same degree of certainty that most of us have that they will return home safely because they never know when the next traffic stop the next domestic dispute the next arrest will explode in their face there is a memorial here in washington with the names of more than 14000 american heroes who gave their lives to make ours a safer country
what this bill does do is make very clear that racial profiling is wrong and that law enforcement agencies that havent done so already should adopt policies and procedures to eliminate and prevent racial profiling
some might ask how can adopting policies and procedures help stop racial profiling well the experts at john jay college will tell you that in the 1960s and early 1970s most police departments in this country left it up to the individual officer to decide when to shoot to kill during that time the racial disparity among persons shot and killed by police was as high as eight africanamericans for every white person and very much higher among victims who were neither armed nor in the process of assaulting a police officer during the 1970s and early 1980s police departments promulgated and enforced strict standards basically decreeing that deadly force could be exercised only in defense of the life of the officer or another person in the large police departments in this country these changes were accompanied by reductions of as much as 51 percent in the number of civilians killed by police it also resulted in the significant reduction in the number of officers killed in the line of duty this is just one example of how good policies and procedures can actually save lives without reducing the effectiveness of law enforcement
recognizing the importance of policies and procedures to eliminate and prevent racial profiling this bill provides incentives for law enforcement to promote such policies by providing grants to state and local law enforcement agencies to use in ways they believe will be most effective for their communities whether to purchase equipment and other resources to assist in data collection or to provide training to officers to improve community relations and build trust
chief chamberlin spoke eloquently this morning about the importance of training and building relationships between law enforcement and communities his actions however have spoken even louder than his words he has taken the lead in western new york in forming the law enforcement and diversity team or lead program which exists to enhance communication and understanding between suburban law enforcement agencies and the diverse citizenry of western new york the lead team sponsored by the national conference for community and justice and the erie county chiefs of police developed one of the nations leading programsbuilding bridges to start a dialogue between police officers and people of diverse cultural and racial backgrounds
the us department of transportation has utilized excerpts from the lead teams what to do when stopped by police brochure for the departments national publication the program has been adopted by the buffalo and cheektowaga school systems in the curriculum for high schools students it provides an important educational opportunity for the entire community and assists in the development of positive relationships between police and community by eliminating some level of fear distrust and skepticism
other new yorkers have also worked to improve the relationship between communities and law enforcement new yorks attorney general elliot spitzer has instituted training programs in an effort to try and prevent racial profiling in fact just this past february through april the attorney generals office conducted inservice training of all members of the new rochelle new york police department at the request of that department the training took place on thursday mornings and focused among other things on what is meant by racial profiling and the perceptions of community members of police encounters in order to raise awareness the training also reported on data collection efforts taking place across the country and the results of those efforts
academia can also play a role in promoting trust between law enforcement and the community for example the john jay college of criminal justice whose master of public administration program was ranked first in the nation among graduate schools with specializations in criminal justice policy and management by us news and world report for the second year in a row has begun to conduct a sixweek free course for members of the new york city police department on the racial and cultural diversity of new york city more than 600 police officers from across new york city have enrolled in a course entitled police supervision in a multiracial and multicultural city
with this bill efforts like those currently led by chief chamberlain attorney general spitzer and john jay college will be expanded throughout the country
more than a year ago when i spoke about this issue at the riverside church in new york city i said we must all be on the same side i am so proud that today we are all here together on the same side citizens officers of the law republicans and democrats to say that racial profiling is wrong and must end
we are here to say that in fighting racial profiling we can at the same time forge even better relations between police and the neighborhoods they patrol as we wage a common effort to reduce crime and make our communities safe
 by mr smith of new hampshire
s 990 a bill to amend the pittmanrobertson wildlife restoration act to improve the provisions relating to wildlife conservation and restoration programs and for other purposes to the committee on environment and public works
mr president i rise today to introduce a comprehensive wildlife conservation measure the american wildlife enhancement act of 2001 this bill will help to increase conservation efforts by promoting local control and state partnerships through flexible incentive driven conservation programs and increased partnerships with local land owners the true conservationists are those who live on and work the land and it is my intention to provide the incentives to help them continue those efforts people dont come to new hampshire for the malls they come to kayak bike fish swim hunt hike trails ski and more thats our industry we cannot and should not turn away from that i believe that when we conserve our wildlife and wildlife areas we affirm our longstanding tradition of honoring our natural american heritage this bill is about achieving that goal in a cooperative partnership approach something that unfortunately the federal government has too long neglected
this bill will accomplish these goals by infusing additional funds into the popular pittmanrobertson program establishing a new competitive matching grant fund that would allow private landowners to apply for assistance to protect endangered and threatened species on their land and establishing a new competitive grant fund that would allow one or several states to apply for a grant to protect an area of regional or national significance through the purchase of an easement or acquisition this measure represents our best and most effective chances of addressing the growing needs for wildlife conservation in our nation
title i of this bill authorizes usd350 million a year to enhance the pittmanrobertson wildlife restoration program unlike the existing pittmanrobertson program which is funded through a tax on hunting equipment the enhanced program would be authorized for a specific time period would have to compete for funds through the appropriations process and would be held in an account that is separate from the already established wildlife restoration fund
funds for this enhanced program would be distributed to the states through a formula based on land area and population with no state receiving less than one percent of the available funding projects eligible for funding through the new program would include acquisition and improvement of wildlife habitat hunter education wildlife population surveys construction of facilities to improve public access management of wildlife areas recreation conservation education and facility development and maintenance states would pay for a project up front and would be reimbursed up to 75 percent of the total cost of the project similar language was included in last years commercestatejustice appropriations measure but was authorized for one year at a level of usd50 million the program has been successful since its inception and should continue past this fiscal year my bill would authorize this program for five years at a level of usd350 million each year
the state of new hampshire ranks 44th out of 50 states in land area and 41st in population still the state received usd487000 out of the money appropriated in last years commercestatejustice appropriations bill if my bill were enacted and fully appropriated even a small state like new hampshire would be eligible to receive usd35 million believe me usd35 million would make an incredible difference not only for new hampshire but nationwide there is not only a demonstrated need for these additional funds but a keen interest in seeing this infusion of appropriations within a timetested program the pittmanrobertson wildlife restoration program popular with sportsmen and women and conservationists alike
the second title of my bill establishes a new competitive matching grant fund that would allow private landowners to apply for assistance to protect endangered and threatened species on their land through the development and implementation of recovery agreements a recovery agreement would provide an economic incentive to protect habitat for threatened and endangered species list specific recovery goals schedule an implementation plan and monitor the results in return for agreeing to carry out these activities the landowner would receive financial compensation currently any effort that a private landowner undertakes to conserve an endangered species is paid for outofpocket under this bill though for the first time private landowners will be able to apply for a grant to assist in the recovery of endangered or threatened species on their property in other words they would be eligible to get compensation for some of the conservation measures that they now have to pay for themselves
that is a big step forward since approximately 90percent of the listed endangered and threatened species inhabit nonfederal lands one of the keys to the successful recovery of our endangered and threatened species is the increased participation of private landowners this is best achieved through a collaborative not combative process that provides landowners with an incentive to participate
this title is an amendment to the endangered species act this title should not be interpreted as a vehicle for comprehensive reform but as a great opportunity to get dollars to those land owners who want to protect species today i welcome the opportunity to work with all of my colleagues on comprehensive reform to the endangered species act through hearings debate and bipartisan legislation however in the meantime we need to provide private land owners the opportunity to protect the habitat of endangered species
the final title of my bill would establish a new competitive grant fund that would allow one or more states to apply for a grant to protect an area of regional or national significance through the purchase of an easement or acquisition without a source of flexible federal funds such as this states and local communities alone will be unable to protect some of the nations most important natural areas i highlight the northern forest that spans the states of new hampshire maine vermont and new york the central appalachian highlands the mississippi delta just to name a few this flexible funding will allow states and communities to protect vital natural cultural and recreational areas without creating or expanding federal units such a funding program promotes local control and multistate partnerships and is also costeffective
i am a firm believer in preserving our national treasures for future generations to enjoy i also believe that the states local communities and individual property owners are in the best position to identify and protect the species and areas that are in the greatest need of conservation but they also need financial assistance from the federal government to effectively conserve and manage the natural resources that need either protection or restoration this belief is strongly reflected in my bill
i have received a very positive response for this bill from the interested constituencies both in new hampshire and nationwide in general there is a growing consensus that we must act now or we will lose many of our special places and if we wait what is destroyed or lost will be gone forever it is our responsibility to act as stewards of the environment i have said it before and i will say it again it is not anticonservative to be proenvironment
this bill is one that should attract the interest of both sides of the aisle on that note i would like to thank senator reid my counterpart on the environment and public works committee for his leadership on the issue of wildlife conservation in april he chaired a field hearing in reno nv on state wildlife and conservation issues i know he is engaged in this matter and i look forward to working with him to advance the goals of the american wildlife enhancement act
i encourage my colleagues to support the american wildlife enhancement act of 2001 and ask that the text of the bill be printed in the record
 there being no objection the bill was ordered to be printed in the record as follows
be it enacted by the senate and house of representatives of the united states of america in congress assembled
a short titlethis act may be cited as the american wildlife enhancement act of 2001 b table of contentsthe table of contents of this act is as follows
a short titlethis act may be cited as the american wildlife enhancement act of 2001 b table of contentsthe table of contents of this act is as followssec 1 short title table of contents
this title may be cited as the pittmanrobertson wildlife conservation and restoration programs improvement act
a in generalsection 2 of the pittmanrobertson wildlife restoration act 16 usc 669a is amended to read as follows
a in generalsection 3 of the pittmanrobertson wildlife restoration act 16 usc 669b is amended 1 by striking sec 3 a1 an and inserting the following
section 4 of the pittmanrobertson wildlife restoration act 16 usc 669c is amended by striking the second subsection c and subsection d and inserting the following d apportionment of amounts in the account 1 deduction for administrative expensesfor each fiscal year the secretary may deduct for payment of administrative expenses incurred by the secretary in carrying out activities funded from the account not more than 3 percent of the total amount of the account available for apportionment for the fiscal year 2 apportionment to district of columbia and territoriesfor each fiscal year after making the deduction under paragraph 1 the secretary shall apportion from the amount in the account remaining available for apportionment a to each of the district of columbia and the commonwealth of puerto rico a sum equal to not more than 1 2 of 1 percent of that remaining amount and b to each of guam american samoa the commonwealth of the northern mariana islands and the virgin islands a sum equal to not more than 14 of 1 percent of that remaining amount 3 apportionment to states a in generalsubject to subparagraph b for each fiscal year after making the deduction under paragraph 1 and the apportionment under paragraph 2 the secretary shall apportion the amount in the account remaining available for apportionment among states in the following manner i 13 based on the ratio that the area of each state bears to the total area of all states ii 23 based on the ratio that the population of each state bears to the total population of all states b minimum and maximum apportionmentsfor each fiscal year the amounts apportioned under this paragraph shall be adjusted proportionately so that no state is apportioned a sum that is i less than 1 percent of the amount available for apportionment under this paragraph for the fiscal year or ii more than 5 percent of that amount 4 use a in generalapportionments under paragraphs 2 and 3 i shall supplement but not supplant funds available to states the district of columbia and territories
a in generalthe pittmanrobertson wildlife restoration act is amended 1 by redesignating sections 12 and 13 16 usc 669i 669 note as sections 13 and 15 respectively and 2 by inserting after section 11 16 usc 669h2 the following
a pittmanrobertson wildlife restoration actthe pittmanrobertson wildlife restoration act as amended by section 105a1 is amended by inserting after section 13 the following sec 14 nonapplicability of federal advisory committee act
a the first section of the pittmanrobertson wildlife restoration act 16 usc 669 is amended by striking that the and inserting the following
the b section 5 of the pittmanrobertson wildlife restoration act 16 usc 669d is amended by striking sec 5 and inserting the following
this title takes effect on october 1 2001
the purpose of this title is to promote involvement by non federal entities in the recovery of the endangered species and threatened species of the united states and the habitats on which the species depend
a in generalsection 13 of the endangered species act of 1973 87 stat 902 is amended to read as follows
a in generalthe partnerships for wildlife act 16 usc 3741 et seq is amended by adding at the end the following
 newspeech
madam president i commend and thank my good friend the senator from california for her usual eloquence and energy in putting forth a very commonsense proposal which is that we ought to do everything in our power to make sure our public schools work before turning our backs on them i especially note her telling all of us that voters in california and michigan who have been given the chance to vote on vouchers in their own states have not only rejected that proposal but have done so overwhelmingly
i join my friend from california and so many others in opposing the gregg amendment which would provide usd50 million for a voucher demonstration program i think it is fair to ask why would i and others oppose a mere experiment
what i would like to do is just reflect back for a minute on an experience i had which really crystallized my opposition for me
a few years ago i was in northern ireland in belfast where i was privileged to meet with a number of people who were crossing sectarian lines to try to come together to find a way to peacefully coexist after decades and decades of troubles between protestant and catholic citizens i was so struck after a daylong conference where we spoke about how to set up a governing assembly how to provide economic opportunity how we could get more people involved in the participation required for a democracy to work when several people said but the real problem we face is in our schools
i said what do you mean
a number of them went on to tell me that from the very earliest of ages children from the two religious traditions grew up in very separate environments there are literally barriers between catholic and protestant neighborhoods and then they go to schools that are run by the respective churches into which they are born
person after person said to me we will never live and work in peace if we dont go to school together we wont have a chance to get to know one another cant you help us have a public school system like you have in america
that made such an impression on me because i have been fortunate to travel all over the world i have been in many countries on every continent except antarctica in every country i go to i meet very smart people i meet athletic stars olympic gold medal winners i meet scientists very successful business leaders and great artists yet there is something very different about every other society than ours because no other society has committed itself to the proposition that all people have the opportunity to live up to their godgiven potential and that we will provide universal public education to offer that to each young boy and girl
we are not perfect we know that we know we have schools that fail at this responsibility yet the goal we have set and the results we have seen from a commitment to public education for so many years now have been realized in the success of this country in the uniqueness of our mobility and in the opportunities we make available
there are some children who frankly start out pretty far behind the starting line they do not have the family background they do not have the environmental enrichment they do not have families who will help them succeed in school they are often trapped in generational poverty when you have poor people you often have poor services it is a challenge to those of us who believe in public education to come up with reasons to oppose something that sounds so good you can read the supporters comments they say in some of our large cities children are trapped in failing schools they should be set free and we should therefore give them money to go to a private or parochial school and it sounds so good but it has a number of serious flaws that i hope will lead a majority in this chamber to vote against it
lets take first the fact that the experiments that have been run because we have already run experiments on vouchers have demonstrated absolutely no evidence that vouchers help to improve student achievement
secondly we know vouchers do not help the students who need the help the most
thirdly vouchers do nothing to help improve public schools in fact research shows clearly that vouchers only further segregate and stratify our public schools
that does not stop the proponents i often have remarked since i have been in washington that washington operates in an evidencefree zone you can put out the evidence and if it runs counter to the ideology then the evidence does not count
but clearly there is no evidence in fact a 1998 study of the milwaukee public school choice program done by cecilia rouse of princeton university found that students in public schools with smaller class size and additional state funding experienced significantly faster reading scoring gains than students who attended private schools through the program
in cleveland a study of the voucher program found no significant difference between the achievement of voucher students and their public school counterparts in reading mathematics social studies the full battery of tests after controlling for background characteristics including prior achievement
so i do not think we need another experiment to tell us vouchers do not work we already have clear evidence of that fact
but there are those who argue that increasing competition among public schools through vouchers will help improve student achievement in failing schools but we know that too is a false promise
we know what does work strong accountability coupled with the extra attention that students who need it require and the kinds of intervention we have heard about everything from preschool to parental involvement to afterschool and summer school
scholars from the economic policy institute duke university and the charles a dana center at the university of texas as well as stanford university have found that states with strong accountability systems which do not include vouchers were successful in improving student achievement in the lowest performing public schools researchers call it the scarletletter effect which shows that if a school is termed failing the school is often motivated to improve that is what we should be focusing on now and that is what we are focusing on in this education debate
i also worry that trying to provide sufficient funds to afford a student a choice that is meaningful will siphon much needed funds out of our public school system a usd1500 voucher for example is just not sufficient in most large cities i am aware of and we therefore know that families have to add a substantial contribution themselves in milwaukee for example as many as 46 percent of students dropped out of the voucher program in the first year and 28 percent dropped out in the fifth year because the usd3600 voucher was not sufficient to cover costs such as registration fees books uniforms and transportation
we also have to worry that if you implement vouchers then very often the motivated students and their parents will take advantage of them and we will see the kind of exodus from the public schools that will only make it more difficult to change their futures
how can we justify taking usd50 million away from proven practices of improving student achievement we need to do more to lower class sizes yet we were unsuccessful in continuing a proven program to do just that by helping to fund teachers in the classroom our friends on the other side said that is not something the federal government should be doing so even though we know it works we wont vote for it
we were unsuccessful in having construction and modernization and repair funding available where we know that so many schools particularly the very schools we are talking about are literally falling down around the heads of students and teachers we were told well modernizing our schools is not a federal responsibility
we need to recruit and retain teachers and we know we are not going to do that if we dont provide competitive salaries and bonuses and other financial rewards and we have a long way to go before we have the teaching core the quality teaching core we need in our country instead of investing in proven measures to raise student achievement we are being asked to divert and siphon off these dollars
i started by saying that my concern is not only based on the fact there isnt any evidence this works that it siphons money out of the public schools that in effect it opens the door to giving up on what we know makes a difference in our childrens lives but that also public schools for me are the distinguishing characteristic that sets us apart from many other societies they are the bedrock of our democracy i dont think we would be giving up on any of our fundamental freedoms so easily i dont think we would be turning our back on our constitution or our bill of rights yet without a strong public school system we could in effect be doing just that
at a time when we are trying to hold students and teachers to higher standards diverting scarce resources to fund an experiment that we already know has weak results and could very well undermine the future of public education which takes care of 95 percent of our students and works well in most parts of our country is a very tragic step in the wrong direction
i heard the end of the remarks of my colleague from california i know she is a very strong supporter of public education as i am and like her i went to public schools from kindergarten through high school i believe in public schools i was struck by what she said if we were already doing what we know works if we had lowered class sizes if we had imposed the discipline if we had recruited and paid teachers in the hardtoteach schools what they should be paid if we had modernized our schools so we didnt have chunks of plaster falling on teachers heads as recently happened in a school in my state then if we still didnt have results maybe even we very strong public school advocates would be willing to say well we need to try something but we are nowhere near there
we have turned our backs on the children who need us the most we have basically left them in the most poorly funded schools with the least qualified teachers often not even encountering a certified teacher without adequate resources without being held accountable and we say well what do you know it is failure
this is similar to so many of the other proposals that would undermine public education it is aimed not at solving the problem but at coming up with a shortterm ideologically driven answer to a complicated set of issues it is tragic that when we know what works we are unwilling to step up and fund the resources that will give every child in america no matter who that childs parents might be the same chance i was given
i urge my colleagues to oppose this amendment i yield back the remainder of my time
 newspeech
mr president i rise today in opposition to both senator sessions and senator harkins amendments which attempt to reach the goal of helping school districts establish and implement discipline policies that are consistent for every child in the school district
i strongly believe that we do need to come to a resolution in federal law that will help school districts appropriately discipline students when they act out violently or in a way that disrupts the learning of other students but that we should be certain that our actions do not punish children for their disabilities
the problem we have at hand is that the 1997 idea reauthorization as passed and implemented has developed a separate discipline policy for children in special education which many school superintendents have found unequal and unfair in their efforts to maintain discipline in their schools in fact a recent gao report published in january of this year found that while many principals believe that the differing school policies had a neutral effect on their schools 27 percent of principals did believe that a separate discipline policy for special education students is unfair to the regular student population
now i want to be very clear that my intention is not to go back to the pre1975 days when students with disabilities were segregated from the regular student population or even worse were denied education all together in fact in the early 1970s i walked door to door trying to figure out why so many children were staying home from school the census at the time showed that there were 2 million children out of school so the childrens defense fund worked to answer the question of why these children were not in school while working for the childrens defense fund i was one of the researchers who found that approximately 750000 of these children were being kept out of school because they were handicapped this research led to the firstever report by the childrens defense fund children out of school in america which helped provide solid research to pass the education for all handicapped children act of 1975
as the progressive policy institute so eloquently concluded in a recent report thanks to this law today many disabled children in america have the opportunity to obtain highquality educational experience tailored to their needs and circumstances the priorities of their parents and the judgements of their teachers this report goes on however to point out that the law has not kept up with the challenges faced by todays schools discipline is a primary example while idea provides protection for disabled students many believe it goes too far that while protecting disabled students the law may unintentionally harm the educational progress of other students in the classroom
senator sessions amendment attempts to fix this problem by eliminating all due process for children with disabilities who have disciplinary problems senator harkins amendment on the other hand attempts to address the problem by encouraging local school districts to implement uniform discipline policies while at the same time recodifying current idea law as it relates to the discipline policy
i oppose these amendments because i do not believe that either amendment adequately addresses the problem of working toward a uniform discipline policy that allows school administrators to maintain discipline so that all children are offered the opportunity to learn and are not interrupted due to the actions of one child while protecting the civil rights of children with disabilities to receive a free and appropriate education
there is much work we need to do on this issue and i believe that we should develop balanced policies that can be part of the discussion and debate during the 2002 reauthorization of idea we need to look for policies that help prevent children with discipline problems from unnecessarily being identified as in need of special education we need to ensure that quality alternative educational settings are developed for those students who need alternative placements and most importantly we need to fully fund idea so that children with disabilities receive appropriate treatment
 newspeech
madam president if i could have 2 minutes to associate myself completely with the majority leaders eloquent statement i rise in opposition to this amendment for all of the reasons that the majority leader has just outlined but also further to say i was honored to serve for 8 years as the honorary chair of the girl scouts of america i know the value of the girl scouts and the boy scouts
to deprive any youngster of the opportunity to participate over this issue strikes me as regrettable at the very least
the girl scouts dont discriminate we have had an organization that has gone for so many years without any of this difficulty it should be up to the local level to determine whether or not a local school district wishes to have the boy scouts offer these services to youngsters in their schools and in their districts
i am absolutely amazed that my friends on the other side would propose an amendment that so totally eviscerates local control it is already unnecessary as we know with respect to the use of facilities the supreme court has already as it did again yesterday reaffirmed access to public school facilities
if we are saying that having the boy scouts either in its present form or with slight modifications determined by the local parents and the schools would in any way jeopardize all federal funding it just absolutely amazes me that people on the other side could make such an argument
so i believe with all my heart that we should not be discriminating against anyone in our country but certainly a local district that tries to work out whatever its problems are with the boy scouts and makes a decision that it considers in the best interests of its children should not face the peril of losing all federal funding that should be made available to educate our children which is what we have been debating now for more than a month
so i hope all of us will join in rejecting this amendment and making clear that we respect the boy scouts we respect the girl scouts and we especially respect local control over educational facilities and opportunities
thank you madam president
 newspeech
mr president i only caught the tail end of the remarks by the senator from texas but i will just point out that this bill which we are hoping to consider today has been in the works for years it has gone through a number of drafts it has been voted on in previous incarnations it is not a new issue it is ready for the full debate and disposition in the senate it is not like a budget bill that is presented without any debate and without any adequate preparation as we experienced a few months ago this is an issue that is more than ripe for the consideration of this body
i thank senator daschle for making the mccainedwardskennedy patients bill of rights the first bill he has brought to the floor as our senate majority leader
i really rise today on behalf of the countless new yorkers and really millions of americans across our country who have been waiting for this day for a very long time i heard some remarks by the senator from texas about the efforts that were made i guess 6 7 years ago now to try to provide health care coverage to every single american i was deeply involved in those efforts and although we were not successful the goal was one that i think we should still keep at the forefront of our minds and hearts because when we began our work in 1993 there were approximately 33 million americans without insurance today we are up to 42 million this is after the socalled managed carehmo revolution occurred where people have been finding it harder to afford coverage afford the deductibles afford the copayments with the result that we have more people uninsured today than when many of us tried to address this problem some years ago
there are many urgent health care issues before us as a nation such as sky high prescription drugs for our seniors too many without adequate coverage and once they have medicare they cant afford the additional coverage that is required in order to give them the kind of health care they should have there are gaps in our health safety net a shortage of nurses in our hospitals and nursing homes and the very difficult conditions under which so many of our nurses now labor and of course there is the growing crisis of the uninsured so we have our work cut out for us in order to deliver on the promise of quality affordable accessible health care for all americans
that is why i am urging we proceed without further delay or obfuscation and pass a patients bill of rights the bipartisan patients bill of rights that senators mccain edwards and kennedy have worked so hard to present which has bipartisan support in the house
we have to finish this job we have been laboring over it since 1996 in earnest with the efforts within both houses of congress since 1997 we have now been waiting and waiting for the congress to act now is the time
i believe we should act not because it has been on the agenda for a long time although it has and not because it is one of those issues to which finally the stars seemed aligned and with the democratic majority now in charge of the senate we can actually get it to the floor but because of the patients and their families who are out there waiting and literally praying for us to act
each of the patients i have met and heard from and each of the families whom all of us have heard from tell a story that describes an urgent situation needing timely and responsive care that is why this bill is so important
it is about getting the care you need when you need it it is about getting care in a timely manner from doctors you trust and choose it is about having doctors and nurses in charge of your health care not accountants and bookkeepers
my colleague tom harkin from iowa had a memorable phrase today at the press conference he said the american people dont want their doctors doing their taxes and they dont want their accountants providing their health care
each of us should be able to look to our doctors our nurses our health care professionals for the care that we trust and need this is about access to an emergency room when we need it
i recall being in ithaca ny about 2 years ago and meeting a young woman who came to see me with a stack of medical records literally a foot high just desperate she had been in a very dangerous nearly fatal accident on one of those winding roads that go through that beautiful part of new york some of you may have traveled through ithaca or may have gone to cornell you know what beautiful country it is but it has also a lot of winding roads she was in a devastating accident lying unconscious on the side of the road luckily someone came upon her and called for aid and they were able to medivac her out with a helicopter save her life and she was in hospital care and rehab for nearly a year she gets out and what does she find she gets a bill from her hmo for the helicopter medivac emergency service because get this she didnt call for permission first she is unconscious on the side of the road and they want to charge her usd10000 because she didnt call for permission
so this is about getting the emergency care you need when you need it it is about seeing a specialist when you need it when your doctor says i have gone as far as i can go you need to go see a specialist it is about women being able to designate their obgyn as their specialist and about mothers and fathers being able to designate their pediatrician as their childs general practitioner as well it is about all of these and more the kinds of issues that are not just written somewhere in a headline but are lived with day in and day out which are talked about around the kitchen table around the water cooler the lifeanddeath issues that really make a vital difference to families all over new york and america families such as that of susan nealy from the bronx whose husband had a serious heart condition but whose referral to a cardiologist was delayed a month the day before the appointment was finally scheduled mr nealy died of a massive heart attack leaving behind his widow and two young children ages 5 and 3
it is like the family of the 15yearold boy from new york who developed complications from heart disease but his health plan refused to allow him to see an outofnetwork specialist familiar with the case and instead sent the teenager to a network provider who did not see him for 4 months and then the boys lungs were filling with blood and 2 days later he collapsed in the street and died
these are just two of the stories i could pick from my innumerable conversations and letters that i have received there are so many more we could tell
for every one of these stories there are untold stories of families whose struggles for the care they needed were denied or delayed according to patient reports health plans delay needed care for 35000 patients every day in fact delayed care and payment is a business practice that health plans have perfected
i have heard from many doctors who tell me that each day a health plan withholds payments represents literally thousands of dollars in interest that a health plan could earn the practice of delay is so widespread that there is a term for it it is called living off the float unfortunately not everyone who is subject to it actually ends up living
look i dont blame the accountants and the bookkeepers they are trying to maximize their shareholders return their profits that is the business they are in but this cannot go on there have to be rules that say you must regardless of your being in business and regardless of having to make quarterly returns put patients doctors and nurses first
the physicians and nurses i speak with are so frustrated about this they are caught between the sharp conflict between business practices that i personally think are unscrupulous but nevertheless they are engaged in and the principles of the oaths that they take to do no harm to get the health care to the patient when the patient needs it when it can do some good lifeordeath situations rarely wait for prior authorization
last summer i met dr thomas lee a neurosurgeon at the northern westchester hospital center just up the road from where we live in chappaqua dr lee was called to the emergency room one day about a year ago because a patient not his patient it was someone he had never seen before a young woman in her early thirties collapsed at work she was brought to the emergency room
dr lee did his neurosurgical analysis did the tests that were necessary and discovered this young woman had a very serious tumor that was pressing on vital parts of her brain and needed to be operated on
they found her husband thankfully and they called the hmo that insured the family and asked for permission to perform the surgery right then dr lee said it was if not a matter of life and death a matter of paralysis and normal life and they were denied they were told that because dr lee was not one of their network physicians because the northern westchester hospital center was not the hospital center they preferred to use he could not do the surgery
for 3 hours dr lee his nurse and the hospital staff were engaged in an argument with the hmo instead of performing the lifesaving surgery it breaks ones heart to think about this neurosurgeon who could be saving lives getting on the phone trying to get permission to do what he is trained to do
finally he was so fed up he said look this young womans life is at stake i will perform the surgery free of charge so long as you will cover the hospitalization with that deal struck the hmo let him proceed
i am very proud dr lee is practicing medicine in my neck of the woods but i do not expect doctors and neurosurgeons to perform lifesaving heroic surgery for free that is not the way the system is supposed to work these are people who go to school for decades to do this work and they deserve the respect and compensation we should be putting into our health care system not to satisfy hmos but to pay for the services of trained physicians and health care professionals
for the past 5 years patient advocates have worked on this bill and we have seen every delaying tactic one can imagine i had a front seat to this when i was down at the other end of pennsylvania avenue we were working very hard to get this bill through the congress every excuse one can come up with was thrown in the way it became so frustrating to all of us who knew that lives were at stake care was being denied and delayed that passage of needed protections was being derailed
we come to this day luckily for us we are here not only because it is the right thing to do but because states and courts have realized they just cannot wait any longer they have seen firsthand what is going on in our country
new york passed a state managed care protection bill in 1996 they even passed a law in 1998 to strengthen the protections all before the congress chose to act many more states have passed such protections including texas specifically aimed to permit injured patients to hold their health plans accountable for their injuries
president clinton signed an executive order giving 85 million americans with federally sponsored health care such as medicare and medicaid protections similar to what we are trying to give to all americans through a 1998 act
even federal courts notably in the case of andrewsclarke v travelers insurance have urged the congress to act in that case judge william young states
although the alleged conduct of travelers and greenspring in this case is extraordinarily troubling even more disturbing to the court is the failure of congress to amend a statute that has come conspicuously awry from its original intent
yet because of our failure to enact such a statute at least 43 percent of all americans with employersponsored private coverage are still left out in the cold these americans cannot afford to wait any longer forty percent of americans know that passing a law today is even more urgent than it was 2 years ago and a majority of them thought it was urgent then
lets work in a bipartisan way this bill is bipartisan senator mccain senator edwards and senator kennedy have all worked to get to this point they have all made compromises their bill is the only bill before the senate that applies to all 190 million americans with private health coverage it is the only bill before the senate that has all the protections of medicare and medicaid it is the only bill that has the support of over 500 consumer and provider advocates
anybody who knows anything about some of these provider groups such as the american medical association knows that congress is not their preferred venue they are not keen on having the congress tell them to do or not do anything but doctors are so frustrated that even the american medical association has come time and again asking that this bill be passed
it is the only bill that guarantees coverage for the routine costs of fdaapproved clinical trials which are so important to patients with cancer and so important particularly to children with cancer
this is the only bill that guarantees an internal and external review as soon as it is medically necessary
in sum this is the only bill before the senate that protects patients not hmos
just as delaying tactics by managed care organizations have injured and even killed millions of americans over time delaying tactics by the opponents of this bill have taken their toll
i want my colleagues to look at this patient survey that is behind me each day 35000 patients have a specialty referral delayed or denied 18000 every day are forced to change medications as a result of their health plans determinations not their doctors but their health plans when i say health plans i mean somebody sitting in an office usually hundreds of miles from where the patient or doctor is secondguessing the doctor saying i am sorry your doctor may have 30 40 years of practice and experience but i am going to sit in this office without ever having seen you and decide that i can secondguess what kind of prescription medication you should have
fortyone thousand patients a day experience a worsening of their condition because of actions by their hmos
one can go through this list and see what patients are saying then one can look at another list that comes from surveys of doctors those who are on the front lines they are saying they believe their patients are confronting serious declines in their health from plan abuse this is the kind of information that concerns me because when i go to the doctor i expect my doctor to take care of me he or she has sworn an oath they have been well trained and i have checked them out i feel like i am putting myself in someones hands whom i can trust and doctors are saying they are not being permitted to practice medicine they are being told they have to subject their decisions to people they have never met nor seen
it is because of the desire of hmos to slow down payment to deny payment to keep that float i talked about going basically to use the money they should be paying to doctors and hospitals for taking care of us for their own purposes for their own profits for their bottom lines
in my office i keep a picture of a young beautiful woman named donna munnings this is donna this is a young woman who reminds me every single day when i look up at her picture in my office of what can happen when the system does not respond until it is too late donnas mother mary is a school bus driver from scottsville ny she has been lobbying and advocating for this bill for years her daughter donna died february 8 1997 after having visited her primary care physician repeatedly only to be told that she had an upper respiratory infection and suffered from panic attacks and that no diagnostic tests were necessary had the doctors performed a usd750 lung scan in time they would have seen not an upper respiratory infection but a footballsized blood clot in her lung
her mother mary said
in my subsequent research i found that hmos can and do penalize doctors for ordering tests which hmos feel are unnecessary but all for the sake of money all for the sake of a usd750 test we lost a vital beautiful young lady who had only begun her life
we are going to hear a lot of debate in fact we are debating whether we can even proceed with this bill yet more delaying tactics yet more efforts to obstruct the kind of care that every one of us needs i can guarantee the people out in that lobby and the people in the offices they represent they would not stand for not getting the care their child needs if they had a daughter who was suffering day after day after day and the doctors could not tell her what was wrong and they kept sending her home i can guarantee that those executives and those lobbyists would get some other source of care for their daughter
but mary is a school bus driver she didnt know where else to turn having insurance was a pretty big deal they didnt know what else to do other than just keep going back as donnas condition got worse and worse and worse
patients buy health insurance in order to feel assured that when they seek care under the benefits for which they have paid that care will be available and it will be available in time to be effective yet we know that that does not happen in one state the state of new york according to department of insurance statistics of the nearly 18000 hmo decisions challenged on appeal over 10000 were reversed this means that when patients can test their hmos decision to deny needed care over half the time the patients are right
yet through a loophole in federal law there are too many consumers in new york over 225 million who still are not protected against these incorrect and dangerous decisions they have no recourse there is nothing they can do because we have not given them a patients bill of rights they need a federal law to give them the parity and protection their neighbors and coworkers have
 newspeech
indeed the senator from new york does recall that i appreciate the senator from illinois raising that because that of course is one of the objections the opponents are trying to throw up that this bill will open the floodgates for lawsuits in texas that has not happened it has not happened anywhere in the country where these protections have been afforded under state law
people are not rushing to the courthouse they want the care that they need they dont want a lawyer they want a doctor and they want the doctor to take care of them according to the doctors best judgment that is what doctors are telling us they are not being permitted to do that
i appreciate my friend from illinois raising that point because as this debate proceeds you are going to hear a lot of arguments about why we just cannot do this you know we just cannot take care of donna and her mother mary and all the other donnas and marys in our country there will be all sorts of red herrings and all kinds of arguments made that just do not hold water there is no basis in fact for them but they sound good maybe they will scare some people but we are tired of being scared and intimidated this is no longer just a political issue this goes to the very heart of who we are as americans
are we going to take care of each other are we going to let doctors and nurses practice their professions or are we going to turn our lives over to hmo accountants and bookkeepers and the like
i am hoping we will not only proceed to this bill which deserves a full hearing deserves a full debate and deserves a unanimous vote in this chamber i hope when we pass this we will be sending a very clear message to all the mothers and fathers and family members that this will never happen again this beautiful young woman whose life was cut short tragically would still be with us today if that hmo had just said maybe we should let you go ahead and have that test
i look forward to working with my colleagues this has been 5 years in the making lets end the politics of delay and move forward with the motion to proceed
 newspeech
mr president last week the administration made headlines when it said it would stop the bombing in vieques
but is that really true lets look at the fine print
first the administration did not commit to stopping the bombing immediately and permanently as so many of us have called for in fact the bombing runs continue this week
second the administration said it would stop the bombing by may 1 2003 but is that really something new lets look at the date by which the bombing would stop under the current agreement and existing law which provides for an end to the bombing if the people vote for it the current agreement and existing law call for an end to the bombing by may 1 2003 the very same date
in other words the administration is saying nothing more than what current law mandates if the people of vieques vote to stop the bombing
if that is all the administration announced that the bombing would stop by the same date provided for under current law then this flurry of attention would be little more than an overblown story about this presidents desire to abide by the letter and spirit of the agreement entered into between the federal government and the representatives of the people of vieques and puerto rico
but that is not all the administration announced it also announced that it wanted to stop the november referendum the devil is in the details they say well this is one powerful devil of an idea that has not received the scrutiny it deserves
for what the administration is really attempting to do is to undermine the intent of the law and subvert the will of the people of vieques
the administration says that a referendum is unnecessary because it already plans to end the bombing by 2003 i say a referendum is more important than ever because without an electoral mandate to require an end to the bombing any administration expression of intent is nothing more than that an expression of intent not a legal requirement and intentions can change at a moments notice
i wholeheartedly support all efforts to find a viable alternative site to train our naval forces we need such training to protect our national interest and to protect our troops and we must work hard to find places and ways to provide such a vital element of our defense
as i have said before the people of puerto rico are great patriots its sons and daughters volunteer for our nations armed forces at one of the highest rates in our country
thousands of puerto ricans have lost their lives in service of their country during all the wars of the 20th century we need the good training to protect all our troops many of whom are puerto rican
so this is not a matter in which the people of vieques or puerto rico should be pitted against the interests of national security we are all americans we are all on the same team and we want the same thing the best trained armed forces in the world
and so i agree with president bush when he says the navy will find another place to practice i agree with secretary powell when he says lets find alternative ways of making sure that our troops are ready using technology using simulators and also finding a place to conduct live fire
but heres the bottom line under current law if the people of vieques vote in november to end the bombing by may 1 2003 the bombing must end by that date pure and simple however under the administrations plan there will be no referendum and therefore there will be no mandate and no requirement to end the bombing by 2003 only a policy to do so and that policy could be altered by the president anytime between now and 2003
in fact secretary rumsfeld has already said that the navy might stay on vieques for another and i quote two three four years until it can arrange the training thats needed in other ways defense department officials were also quick to point out that while the president said that the navy would find another place to practice within a reasonable period of time he never defined reasonable
secretary england said he wanted to have us control our destiny meaning the navy as opposed to allowing what he called this level of emotion distract our attention from the real issue
in other words the will of the people of vieques is an emotion that must be put aside and the people of vieques should not control their destiny the navy should
i believe that is the wrong way to deal with this very important issue i believe we should work toward a solution to this problem without circumventing the law of the land without abrogating an agreement without obviating the will of the american citizens of vieques
i will stand up against any effort to shut down the referendum in vieques let the votes be cast let them be counted and let the voice of the people be heard and respected
 newspeech
i rise in support of this important sense of the senate i have a question to address to the senator from arizona who has done so much to bring this issue of clinical trials to the forefront we heard yesterday important testimony from the head of the national cancer institute dr richard klausner who testified that clinical trials are not more expensive than standard therapies and that we need to make them even more accessible this is what the sense of the senate provides what the underlying bill provides
probably the premier institutions in our country that deal with cancer the large cancer centers are the source of so much of the research done that translates into therapies treatments and cures for people suffering from cancer
i ask the senator from arizona i am sure his sense of the senate as the underlying bill includes these cancer centers places such as md anderson in texas sloan kettering in new york or danafarber in new york is my understanding correct that the cancer centers and the research they do as qualified research entities are included in the sense of the senate
 newspeech
i thank the senator from arizona i congratulate him on his leadership on the underlying bill and on this important sense of the senate which clarifies that clinical trials are an essential part of modern medical practice and providing the opportunity for physicians to refer patients for these lifesaving treatments although they are experimental it is a way we make advances in medicine which eventually help everyone
i yield the remaining time
 newspeech
i thank the chair mr president i add my thanks and gratitude to my good friend from wisconsin he has been a leader on this whole issue of campaign finance reform for so many years he started as a young boy and it has taken most of his life i think progress is being made from a most unlikely source i applaud the continued perseverance and commitment of the senator
 newspeech
mr president we are in the midst of this very important debate about a patients bill of rights i am hoping that before we break for the fourth of july recess the doctors nurses patients and families of america will have the relief for which we have all waited for a very long time making it clear doctors should be making our health care decisions that nurses not bookkeepers should be at our bedsides and that the patients bill of rights will be a reality
i rise today because we have to consider our broad needs for health care not only in our country but around the world today as we meet and debate a patients bill of rights to make sure that americans have access to the best health care in the entire world there are millions of people around the world who do not have that opportunity or that right i speak specifically of those who are suffering from hivaids
we should be supporting vigorously the united nations general assembly on meeting the global hivaids challenge and urging them to consider creative tools such as debt relief in efforts to combat hiv
as the general assembly is meeting in special session in new york to try to come up with a strategic blueprint for fighting hivaids worldwide it is imperative that we in america appreciate that this worldwide epidemic has nowhere near crested africa is ravaged it has just begun to affect india china and russia this is an epidemic of historic proportions and it needs a response that is historically appropriate
almost 60 million people worldwide have been affected by hivaids and over 20 million men women and children have died if current trends continue 50 percent or more of all 15yearolds in the most severely affected countries will die of aids or aidsrelated illnesses
we are in the middle of summer vacation we have many families and young people visiting our capitol we are always so happy to have them here and for them to take a few minutes to see their government in action but it is just chilling to imagine american 15yearolds facing bleak futures as orphans or victims because they were born to infected mothers
every american should be concerned with what is going on beyond our borders we should also be concerned because when it comes to disease today there are no borders people get on jet planes people travel all over the world there is no disease that is confined to any geographic area any longer we have to recognize that for us to worry about the hivaids epidemic in africa and asia is not only the right thing to do it is the smart thing to protect ourselves and to protect our children
it is also important to recognize that the groundbreaking drug treatments that are keeping people with hivaids alive today are not available to those who suffer elsewhere less than 1 percent of hivinfected africans for example have access to lifeextending antiretroviral medications the challenges facing us are great and we should work together to combat this global emergency
i strongly support the formation of a global fund for infectious diseases such as aids but also including tuberculosis and malaria we are seeing tuberculosis and malaria in our own country we are seeing the spread of malaria which used to be confined to a tropical belt beginning to move northwards in part i believe because of global warming and desertification so the mosquitos can travel further north and find hosts who traditionally have not suffered from malaria
tuberculosis is becoming epidemic in many parts of the world in russia drugresistant tuberculosis is a major killer
i believe we should have a global fund to combat these infectious diseases and i am very pleased the united states private donors and some other nations have taken steps to address the need for money as articulated by secretary general kofi annan we need between usd7 billion to usd10 billion annually it is my hope that through a publicprivate partnership we are able to continue to invest in promoting prevention treatment and eventually a vaccine to prevent this devastating disease
i am old enough to remember polio as a scourge that affected my life i can remember my mother not letting me go swimming in the local swimming pool because of polio i remember as though it were yesterday when the announcement of a vaccine was made what a sense of relief that spread through my house and all of our neighbors and we all lined up to get that shot we thought would protect us from what had been up until then such a serious overhanging cloud in the lives of young people as well as older people
hivaids extracts a severe economic toll on nations worldwide the disease spreads so rapidly no one is immune from it it has grave consequences for societies and it threatens the interest of peace and prosperity around the world
hivaids alone will reduce the gross domestic product of south africa by usd22 billion or 17 percent over the next decade that is why i believe debt relief must also be part of any conversation about a broader global hivaids strategy
while most african countries spend less than usd10 per capita on health care they spend up to five times that amount in debt service to foreign creditors in fact the burdens of debt repayment have come into direct conflict with public health efforts in some instances for example structural adjustment programs have sometimes required governments to charge user fees for visits to medical clinics a practice that stands in the way of effective prevention and treatment programs as discussions of global hivaids prevention proceeds consideration should be given to the role of international debt relief in the overall plan to combat hivaids
i have written to the un general assembly president harri holkeri to express my support for his efforts and to urge inclusion of debt relief strategies in any effort that comes out of the general assembly
i also urge our own government to look more closely at what we can do in the last administration we forgave a lot of our bilateral debt for the poorest of the nations but we should look at expanding beyond the circle of the poorest of the poor to the next poorest of the poor and we should also look at our multilateral debt
i am hoping i will find support on both sides of the aisle for a senseofthesenate resolution i will be submitting to express the policy view that debt relief can and should be an important tool
i have visited african countries i have visited asian countries i have visited hivaids programs i have been in places where 12yearold girls who were sold into prostitution by their families have come home to die in northern thailand
i have been in programs in uganda which have done probably the best job i know of in africa certainly to spread the message about how to prevent hivaids i have listened to the songs that were taken out into villages to tell villagers about this new disease that nobody really knows where it came from or how it arrived but to warn people about its deadly consequences
i was fortunate and privileged last year to participate in the united nations discussion about aids and i sat with aids orphans a young boy from uganda whose father and then mother died of aids leaving him responsible for his younger brothers and sisters a young boy from harlem whose mother died of aids a young boy from thailand who was also orphaned by this terrible disease
in some parts of africa now one will only find children and most of them are orphans the rate of infection ranges from 15 to 35 percent and i am deeply concerned we are still in some parts of the world in a state of denial about hivaids
certainly both india and china face tremendous challenges to educate their population about this disease and to avoid practices that might spread it it is commonplace in some parts of china for very poor villagers to sell their blood to make a little money in so doing they are subjecting themselves to the possible transmission of this terrible disease
in other parts of africa and asia even the best intentions to immunize children against measles or other communicable diseases lead to tragedy because the sterilization is not up to par and needles are reused leading to the infection of people with hivaids
i have long maintained there is a deep profound connection between the economic health of a nation and the physical health of that nations people that is why we have to act now to address the hivaids pandemic
there is so much the united states can and should do we have the finest health care system in the world we are the richest nation that has ever existed in the history of the world we not only should care about people in other parts of the world because of this disease but we should act in our own selfinterest because there will be many parts of the world where it will be difficult potentially even dangerous to travel if the entire social structure and economy collapses because of the strain of hivaids where tourists and business people from america will be told they should not go to do business suppose they are in an accident or suffer injury and might need medical care and that medical care might not be deliverable because the health care system has collapsed under the weight of hivaids
i look forward to working with my colleagues in the senate and in our united states delegation to the united nations general assembly special session on these and other desperately needed proposals to halt and reverse the social and economic damage caused by hivaids and the direct and immediate threat this pandemic poses to america and americans i urge my colleagues and i urge our government and the united nations to look deeply into the concept of forgiving debt in return for nations doing what we know works to prevent treat and eventually find a vaccine for this terrible disease
i yield the floor
 newspeech
mr president i ask unanimous consent that dr mary catherine beach a legislative fellow in my office be granted the privilege of the floor for the duration of the debate on s 1052 the mccainedwardskennedy patients bill of rights
 newspeech
mr president i have the greatest respect for my good friend the senator from nebraska and i rise reluctantly but firmly to oppose the amendment he is sponsoring along with senators kyl and nickles because i am concerned not only about the general issues that have been raised by other opponents i am concerned also by the american medical associations very strong and vigorous opposition to this amendment which they have made very clear to me and my office as well as i believe every other senator because of their deep concern that this would be a step backward permitting health plans to determine the services drugs supplies or equipment necessary to prevent diagnose or treat a patients condition illness or injury
but i have a very specific reason for opposing it i direct this to my good friend from nebraska because this is something that deeply concerns me this amendment allows health plans to define medically necessary and appropriate in a way that poses a great threat to all patients and families who require hospice and palliative care to treat the suffering associated with terminal illness
the washington post just a week ago published a story outlining the various ways in which recent advances and endoflife care have not yet reached children with terminal illnesses causing an enormous amount of suffering for dying children and their parents and loved ones who have to watch that suffering at the end stages of a terminal illness the article quotes one mother who says looking back on her daughters death that pain is such a huge problem
there are two specific phrases within the safe harbor of the medically necessary care language in the kylnelsonnickles amendment that directly undermines the needs of dying patients first the amendment declares that care provided for the comfort of the patient is not medically necessary care
any health care professional or really any person such as myself who has stood at the bedside of a dying friend or a loved one knows that comfort of the patient is absolutely necessary and is often the most appropriate goal of care in those last days weeks and even months sometimes at the very center of palliative care and particularly in the hospice movement is the belief that each of us has a right to die free of pain and with our human dignity as intact as possible
the institute of medicine released a groundbreaking report in 1997 that concluded too many people suffered needlessly at the end of life a second institute of medicine report released last week also concluded that patients are suffering unnecessarily furthermore studies have shown that specific types of patients patients who are elderly female africanamerican or children are less likely to have their pain adequately controlled at the end of their lives
the kylnelsonnickles amendment is legislation that could be termed as declaring that the comfort of dying patients is not a legitimate goal of medicine but to me that has it backwards isnt the relief of suffering exactly what doctors are supposed to be concerned about
a second and related problem is that this amendment allows plans to define as medically necessary care that is appropriate to treat a medical condition illness or injury this narrow definition compromises the delivery of appropriate care to dying patients by failing to recognize the legitimacy of care that focuses on the palliation of pain rather than a cure this definition actually encourages overuse of invasive and often futile medical treatment and the underutilization of hospice and palliative treatment
the institute of medicine report released this month concludes that policies and practices that govern payment for palliative care hinder delivery of the most appropriate mix of services a chapter of that report focuses on the terrible effect these policies have had on children it found that services necessary to provide dying children and their parents with comfort and counseling are not recognized and certainly not even reimbursed by many insurance programs
i believe the definition of medically necessary care proposed by this kylnelsonnickles amendment would further obstruct access to hospice and palliative care services for patients suffering from terminal illness
we have not done enough to relieve pain and suffering at the end of life i served for many years on the board of a childrens hospital back in those days the idea of giving strong medication to a dying child was really not even considered a possibility for many reasons people were not sure about the appropriate dosage some people were worried even with a dying child that the child might become addicted to strong pain relief medicine
i have also seen friends who at the end of their lives had to cry out for and demand pain relief from an almost unbearable burden they did not want to leave this wonderful life but they knew that was going to happen and they wanted to do it in a way that relieved both them and their loved ones of the agony that comes at the end of so many devastating illnesses
there are many wonderful hospice programs in our country and many academic development centers are developing comprehensive palliative care programs specifically to focus on patient comfort at the end of life
the kylnelsonnickles amendment places the comfort of dying patients and their families beyond the language of the legislation really rendering it illegitimate providing this comfort would no longer be medically necessary or appropriate
i ask unanimous consent to print in the record the article i referred to earlier from the washington post called children of denial
 newspeech
mr president i urge my colleagues to oppose this amendment not only for all the reasons others have enumerated but for this very specific issue we are at the beginning of work that needs to be done in hospice care and palliative care and i would hate to see us turn back the clock before we really started the race to determine what we should do to care for those who are in the last stages of life
i urge all of my colleagues to join me in opposing this amendment and to support the ongoing efforts to provide more pain relief more palliative care and yes more comfort to those who are leaving this life
i thank the chair
 newspeech
mr president let me state for the record that the request i am about to make has been cleared on the republican side
i ask unanimous consent that the judiciary committee be discharged from further consideration of s res 117 and that the senate then proceed to its immediate consideration
 newspeech
mr president i rise today to introduce a resolution honoring john j downing brian fahey and harry ford who gave their lives this past fathers day while protecting the lives of others together these brave men left behind three widows and eight children whom we also honor today for their sacrifice
on june 17 as a treacherous fivealarm fire raged at the long island general supply company in queens ny without hesitation as they have done countless times before nearly 350 firefighters and numerous police officers responded to the call for help two civilians and dozens of firefighters and police officers were injured and three courageous fathers lost their lives it was the last time their children would be able to spend fathers day with them
john downing was 40 years old an 11year veteran of the new york fire department when he responded to the fivealarm blaze he was a valiant public servant who had been recognized for his bravery john left behind his wife anne his 7yearold daughter joanne and his threeyearold son michael
brian fahey 46 years old and a 14year veteran of the department from east rockaway ny was also a husband and father of three his years of service to his community were made proud by his courage he is survived by his wife mary and their three sons brendan 8 and twins patrick and james 312 years old
harry ford age 50 gave nearly three decades of service to the new york city fire department during his exemplary career he received nine bravery citations he is survived by his wife denise his daughter janna obrien age 24 and two sons harry 12 and gerard 10
mr president i paid a call on the two firehouses early sunday morning who had lost these brave compatriots and i spent time talking to the men who go to work every day not knowing what is going to be asked of them who sometimes go for thankfully days or weeks or months and even years without ever having to put themselves in danger but when the call comes they are ready and whether it is a call to respond to an emergency need because of an illness an accident or a huge raging fire that is about to get out of control they represent the very best we have in our society
we live in a society that seems to be in perpetual search for heroes whether in the form of sports figures or screen idols but to find true heroes sometimes we dont have to look so very far from home we certainly dont have to look any farther than the brave men we are honoring today
the unmistakable courage and the incalculable sacrifices that they and their families have made for the good of their neighbors and their community are the kinds of virtues and values that should be held up to our children and ourselves as something we should all aspire to
finally in so honoring these men we honor the hundreds of thousands of public safety officers across this country that every single day risk their lives and put them and their families at risk to keep us safe from harm their strong tradition of bravery and sacrifice keeps our communities safe and fills our hearts with pride for their selfless acts of courage for others
i hope that next year when fathers day comes around the children who have lost their fathers in this fire and those who have lost fathers and mothers because they were serving us will know how grateful we are for their sacrifice i hope all of my colleagues will join me in supporting this resolution
i yield back the remainder of my time
 newspeech
mr president i ask unanimous consent that the resolution and preamble be agreed to en bloc and the motion to reconsider be laid upon the table
 newspeech
mr president i rise today in opposition to provisions in the mccain amendment and in underlying bill s 1077 which rescind funds from programs supported under the workforce investment act including the dislocated worker employment and training program and the adult employment and training activities
the underlying bill rescinds funds from wia in order to pay for important increases in funding for title i education services and low income home energy assistance program i support the need to increase essential funds for students in our highestpoverty schools and for lowincome individuals who are being hardest hit by increasing energy costs indeed i signed on in support of the increases for title i and liheap i do not think however we should increase funding for these defense or any other programs by taking money away from new york workers at a time when these employment and training programs are most in need and are beginning to meet their potential
at this time when upstate new york is facing more notice of layoffs we should not be cutting back our support for dislocated workers last year over 25000 new york workers received notices warning them of layoffs an increase of over 7000 workers from 1998
over the past several months we have learned that hundreds of workers at the xerox facility in webster ny will soon find themselves out of work several hundred more new yorkers who have spent years working for nabisco in niagara falls also recently received notice that they would no longer have a job corning announced just yesterday that it will have to close three factories resulting in a loss of nearly 1000 jobs
at a time when we see signs of our economy weakening this bill would reduce funds specifically designated to assist workers who are victims of mass layoffs and plant closures with the rescission in the base bill alone new york can expect to lose approximately 29 percent of its dislocated worker funds i have received hundreds of letters from new yorkers not only from concerned workers but also from businesses who need trained workers
why are my colleagues suggesting that we should rescind wia funds at a time when our economy is weakening and many of our workers will need these critical funds to be retrained and relocated in new jobs
they are claiming that states are not spending and obligating funds quickly enough i agree but i also agree that states and local communities have made tremendous progress in implementing the workforce investment act
lets get the facts straight states were not required to implement the workforce investment act until july 1 2000 beginning july 1 2000 states had 2 years to spend funds and were required to obligate 80 percent of their funds many counties in new york are doing a tremendous job chautauqua county for example has obligated 95 percent of its dislocated worker funds as well as 95 percent of adult funds the town of hempstead has allocated 90 percent of both its dislocated and adult worker funds as has erie county all of which can expect to lose funds under this rescission
i do know that there are at least eight counties in new york that have struggled in their implementation working to get up to 19 federal partners at the local level to offer services in one stop training centers and as a result have obligated 70 percent or less of their funds these counties need to do better and the state needs to do better in supporting their efforts but the way to do so is not to take funds away from a fledgling program that is aimed to assist our workers most in need of training and assistance
i oppose these efforts to undermine the new workforce investment act i agree with accountability of federal dollars but i do not agree that we should unnecessarily punish workers before allowing the program to get up and running
 newspeech
mr president i stand here today not in opposition to moving forward with the bankruptcy reform act but to send a clear message that i continue to have strong reservations about whether this bill is both balanced and responsible i have long said that debtors that have the genuine capacity to repay some of their debt should be required to do so but abuses by creditors need to be stopped
i grew up with a father who never accepted any credit never had a credit card in his life he taught me the importance of always working hard and paying your debts i believe every american should work hard to spend responsibly and to repay their debts but i also know that some families are hit by unexpected hardships
this bill should not have the effect of targeting our most vulnerable consumers women who are left with little resources as their husbands who were the primary breadwinners leave the family or families with no health insurance who are struck with financial hardship when one family member becomes critically ill or another family who suddenly finds that the primary breadwinner is laid off with little employment opportunities available in the region
these are not the families who need to be further stuck by hardship of bankruptcy reform that is inflexible or overly harsh on debtors
i voted for the s 420 the bankruptcy reform act of 2001 because i believed and still do believe that there were some important protections added to the senate bill but i will absolutely not vote in favor of the final bankruptcy reform bill if it does not include at least these minimal protections for our most vulnerable consumers
during the floor debate on s 420 the bankruptcy reform act of 2001 i worked with my colleagues on both sides of the aisle to add additional protections for women and children i worked hard to ensure that once bankruptcy is complete we do more to ensure that single mothers can collect the child support they depend upon senator hatch and i passed an amendment to ensure that the holder of the claim meaning the parent with custody of the child most often the mother is informed by the bankruptcy trustee of his or her right to have the state child support agency collect the nondischargeable child support from the exspouse i believe this change will help inform women of their rights to have the state help them in their claims to collect child support
in addition i was concerned about competing nondischargeable debt so i worked hard with senator boxer to ensure that more credit card debt can be erased so that women who use their credit cards for food clothing and medical expenses in the 90 days before bankruptcy do not have to litigate each and every one of these expenses for the first usd750
these are the most minimal of changes that i believe need to be in the final bill i still do not believe that they go far enough i believe that the final bill should protect child support full stop i do not believe that child support should have to compete with any credit card debt but it should certainly not retreat from these changes the cap on protected expenses should not be lowered to the house version of usd250
i also believe that the bill needs to include senator schumers amendment to ensure that any debts resulting from any act of violence intimidation or threat would be nondischargeable it was a victory for the senate to include this important amendment to ensure that those who are responsible for violence against womens health clinics are held responsible for their actions i do not believe we should retreat on this point
let me be clear this bill should go further to protect consumers but it should certainly not retreat from the consumer protections in the bill
i will vote for cloture on this bill but i believe that as we move to conference we need to continue to work to ensure that we continue to gain more balance between creditors and debtors
 newspeech
madam president riis park serves an ethnically diverse population including hundreds of innercity families adhering to the ideas envisioned by jacob riis and carried on by city parks commissioner robert moses by investing in this urban park our government can ensure that it remains a viable resource for years to come i stand in full support of funding for the riis park natatorium complex
 newspeech
mr president i rise today to pay tribute to donna centrella a very special woman whom i met 2 years ago during my campaign in new york donna died on monday after a long brave battle with ovarian cancer
i first met donna in september 1999 when i visited massena memorial hospital in massena ny donna had been diagnosed with ovarian cancer in august but did not have health insurance to cover her treatment miraculously she found a doctor who would treat her without insurance and she was able to afford care through a variety of state programs
perhaps even more astounding was her doctors statement that she was actually better off without managed care coverage because he could better treat her that way without hmo constraints they were free to make the decisions about the best procedures to follow for her treatment and care her doctor could keep her in the hospital as long as needed and he would not have to get preapproval for surgery
i have retold donnas unbelievable story many times since meeting this extraordinary woman hers is a story that underscores the profound need in this country for immediate reform of the way we provide health coverage to our citizens we owe it to patients like donna to sign patients protections into law as soon as possible to ensure that we can provide the best medical treatment possible to everyone who needs it
we have lost an ally but i have faith that we will not lose the fight for greater patient protections it saddens me greatly that donna will not be here to see it happen she was an amazing soul whose determination and strength will never forget newspeech
madam president i rise today to pay tribute to donna centrella a very special woman whom i met 2 years ago during my campaign in new york donna died on monday after a long brave battle with ovarian cancer
i first met donna in september 1999 when i visited massena memorial hospital in massena ny donna had been diagnosed with ovarian cancer in august but did not have health insurance to cover her treatment miraculously she found a doctor who would treat her without insurance and she was able to afford care through a variety of state programs
perhaps even more astounding was her doctors statement that she was actually better off without managed care coverage because he could better treat her that way without hmo constraints they were free to make the decisions about the best procedures to follow for her treatment and care her doctor could keep her in the hospital as long as needed and he would not have to get preapproval for surgery
i have retold donnas unbelievable story many times since meeting this extraordinary woman hers is a story that underscores the profound need in this country for immediate reform of the way we provide health coverage to our citizens we owe it to patients like donna to sign patient protections into law as soon as possible to ensure that we can provide the best medical treatment possible to everyone who needs it
we have lost an ally but i have faith that we will not lose the fight for greater patient protections it saddens me greatly that donna will not be here to see it happen she was an amazing soul whose determination and strength i will never forget newspeech
madam president i rise today to speak in support of s 1218 the iran libya sanctions extension act of 2001 this legislation will extend for another five years the iran libya sanctions act of 1996 which would otherwise expire on august 5 2001
in 1996 congress unanimously enacted ilsa in response to irans emergence as the leading state sponsor of international terrorism its accelerated campaign to develop weapons of mass destruction its denial of israels right to exist and its efforts to undermine peace and stability in the middle east
five years later the us state departments patterns and global terrorism reported that iran still remains the most active statesponsor of terrorism in the world by providing assistance to terrorist organizations such as hezbollah hamas and the islamic jihad
eleven short days from now ilsa is set to expire that is why we must act today to renew this important legislation to deter foreign investment in irans energy sector its major source of income by doing so we can continue to undermine irans ability to fund the development of weapons of mass destruction and its support of international terrorist groups
in february of this year i met with families of the american victims of the bombing of pam am flight 103 in 1988 brian flynn from new york city recalled driving to john f kennedy airport to retrieve the body of his brother jp flynn who had perished in the bombing brian remembered there was no flag no ceremony no recognition that he was killed simply for being an american
earlier this year once again brian drove to john f kennedy airport this time to go to the netherlands to listen to the verdict against two libyan nationals indicted for the bombing a libyan intelligence officer was found guilty of murder in the bombing in the words of the court in furtherance of the purposes of libyan intelligence services yet libya continues to refuse to acknowledge its role and to compensate the family members of 270 victims of the bombing the state department reports that libya also remains the primary suspect in several other past terrorist operations brian and so many family members of the dozens of new yorkers killed in the bombing have written to me and conveyed how important it is for the united states to continue to hold libya accountable for its support of international terrorism
by acting now to renew ilsa the senate is sending a clear message to iran and libya that their dangerous support for terrorism and efforts to develop weapons of mass destruction are unacceptable and will not be tolerated
 newspeech
is there objection
without objection it is so ordered
the clerk will call the roll
 newspeech
as the majority leader well knows i am new to this body and i think what we have just seen raises in my mind serious questions about what it is we are trying to accomplish for the people of our states and our country
as i understand the response of the distinguished senator from idaho the delay is because somebody unnamed delayed something last year that to me is a strikingly inadequate explanation for a delay that is holding up our efforts to help our oldest industry and our newest industry
with the fact that new yorks largest economic sector is agriculture which most people outside new york would have no idea of i have a great interest in the agriculture supplemental bill because we have some aid in there for farmers who are following in the tradition of those having farmed in new york for more than 400 years our apple farmers are on the brink of extinction if they do not get some emergency help we had hail last year that destroyed the crop in the midhudson river valley it took out orchards in the north country so this is not any geographic issue this is a national issue that has to be addressed
at the same time in new york we have some of the cutting edge hightech industries that are begging for the kind of direction the export administration act will give them the certainty about what they can and cannot export whether we can be competitive globally both of these important pieces of legislation have to be addressed in the next week
it is regrettable that instead of doing the peoples business dealing with the agricultural needs and the hightech needs that really cut across every geographic and political line we have in our nation we see this kind of delay
but i would ask the majority leader is it your intention to do everything you can possibly do as our leader who has done in my view an absolutely tremendous job since assuming the leadership to make sure that the peoples needs are met and that includes the agriculture bill and the export administration bill
speaking just as one senator i do not think there is anything more important than doing the work we were sent here to do casting the votes that will help people and it is striking that we do not seem to have the cooperation we need on the other side
but i would ask the leader if it is his intention to make sure that we do the peoples business before we leave for the recess that is scheduled
 newspeech
madam president i rise today to honor one of the extraordinary legacies of my predecessor senator daniel patrick moynihan who served in this body for 24 years representing the people of new york
with some seeking to insert contentious language regarding zionism into declarations emerging from the upcoming united nations world conference against racial discrimination xenophobia and related intolerance in durban south africa i am reminded of senator moynihans courageous statesmanship when he condemned the 1975 un resolution 3379 which infamously declared zionism is a form of racism and racial discrimination
we should never forget the historic battle my predecessor waged to defeat this outrageous effort to delegitimize the state of israel and defame the jewish people over 25 years ago senator moynihan boldly called this hatefilled language criminal it was criminal then and its still criminal today
on the day the resolution passed senator moynihan declared the united states will never acquiesce in this infamous act a political lie of a variety well known to the twentieth century and scarcely exceeded in all the annals of untruth and outrage the lie is that zionism is a form of racism the overwhelming truth is that it is not
from the moment he entered the senate in january 1977 senator moynihan dedicated much of his energy to repealing this despicable attack on israel and the jewish people delivering passionate speeches on the senate floor as chair of the senate foreign relations subcommittee on near eastern and south asian affairs senator moynihan introduced joint resolution 246 which called on the un to repeal the 1975 resolution
it took 17 long years to remove this stain from the united nations reputation and as we begin this new century nothing could be more damaging to the promise and integrity of the un than to revive to this ignominious statement in order to help prevent the un from reviving one of the moments of its greatest shame senators schumer smith lugar and i have written the following letter to kofi annan the secretary general of the united nations condemning any attempts to include inflammatory antiisrael language into declarations associated with the world conference against racism in durban
i ask unanimous consent that the letter be printed in the record
 newspeech
in 1975 senator moynihan warned his colleagues at the un and the rest of the world that as this day will live in infamy it behooves those who sought to avert it to declare their thoughts so that historians will know that we fought here with full knowledge of what indeed would be lost
senator moynihan recognized then as we do today that this language only serves to fuel hatred and bigotry throughout the world and has no place in international discourse i am honored to have followed senator moynihan in the senate and i pledge to continue his tradition of promoting the principles of decency and human dignity and opposing efforts to sow hatred and bigotry especially when they are cloaked in the guise of diplomacy
i ask unanimous consent that the attached statement be printed for the record
 newspeech
in the aftermath of the attack doctors believed private vincis wounds were fatal but he persevered he received the purple heart and eventually was transferred to a hospital in colorado where doctors were able to remove one of the two bullets that had almost taken his life but not both he continues to carry with him the second bullet which has never been able to be removed
disabled from his wounds benjamin vinci returned to port chester after being discharged from the army and resumed life as a civilian for many years mr vinci worked as a vacuum cleaner salesman in westchester county he married rose civitella in 1945 and together they raised four children peter burnadette joann and joseph
we honor and thank benjamin vinci for his tremendous sacrifice vital contribution and gallant service to our nation his acts of bravery are an exceptional example of the fortitude determination and strength of the american spirit as mr vinci carries the burden of his wounds and the bullet he received on that december morning of infamy so too must we carry the memory of his heroic deeds remembering and honoring all the men and women of that great generation those veterans of world war ii who saved our nation and the world
 newspeech
madam president i ask unanimous consent to proceed to executive session to consider the following nominations calendar nos 262 through 285 and the military nominations placed on the secretarys desk that the nominees be considered en bloc that the motion to reconsider be laid upon the table the president be immediately notified of the senates action and the senate then return to legislative session
 newspeech
madam president i ask unanimous consent that the senate proceed to the consideration of s res 141 submitted earlier today by the majority and republican leaders
 newspeech
madam president i ask unanimous consent that the resolution and preamble be agreed to en bloc the motion to reconsider be laid upon the table and that any statements relating thereto be printed in the record
 newspeech
madam president i ask unanimous consent that the senate proceed to the consideration of hr 1954 the iranlibya sanctions act just received from the house
 newspeech
madam president i ask unanimous consent that the bill be read the third time and passed the motion to reconsider be laid on the table that any statements relating thereto be printed in the record with no intervening action or debate
 newspeech
madam president i ask unanimous consent that when the senate completes its business today it adjourn until the hour of 1 pm monday july 30 i further ask unanimous consent that on monday immediately following the prayer and the pledge the journal of proceedings be approved to date the morning hour be deemed expired the time for the two leaders be reserved for their use later in the day and there be a period for morning business until 2 pm with senators permitted to speak for up to 10 minutes each with the following exceptions senator durbin or his designee from 1 to 130 pm senator grassley or his designee from 130 to 2 pm further at 2 pm the senate resume consideration of the motion to proceed to s 1246 the agriculture supplemental authorization bill with the time until 530 pm equally divided between the chairman and ranking member or their designees
 newspeech
madam president the senate will convene monday at 1 pm with 1 hour of morning business at 2 pm the senate will consider the motion to proceed to the agriculture supplemental bill a cloture vote on the motion to proceed to the agriculture bill will occur at 530 pm on monday
i have no further business to report madam president
 newspeech
mr president i rise in support of the amendment offered by the senator from florida this is an issue that he brought to my attention some months ago following the initial debate over the arsenic standard we had a good debate last night with a very strong vote to ensure that we get the right kind of standard as soon as possible so people will know what to expect from their drinking water we also made it very clear that we want to help communities be able to meet these standards
it should not be an unfunded mandate to take care of your health we ought to have the best scientific information made available through the studies that are done or commissioned to provide the help that communities need to be able to protect themselves particularly their children
senator nelson came upon a problem i never knew existed i cannot tell you how many times i have been around playground equipment that is wooden i always thought it was really attractive it is the kind i preferred it is what i bought for my own daughter it certainly never crossed my mind that for good reasons to prevent pest and termite infestations manufacturers would want to treat that wood i never thought about it
but what senator nelson has determined and i applaud him for this because it became an issue in florida and he brought it to our attention is that something called cca chromated copper arsenic is widely used as a preservative in pressuretreated wood including playground equipment this cca is 22 percent arsenic
i remember when i used to practice law which seems as if it was a very long time ago i had a case that involved treated wood that was treated at a plant in tennessee i went to visit it the wood was treated with all kinds of chemicals but it was used for telephone poles it was used for railroad tracks it was not used in playground equipment
what senator nelson has learned is that through rain and natural deterioration the arsenic that is in this compound cca to treat this wood can leach into the ground and can even come off on ones hands you think about all those little hands and all those little mouths and those little bodies kind of rolling around this playground equipment i really commend the senator for bringing this problem to our attention because of his hard work the epa and the consumer product safety commission are conducting reviews of the health and safety risks to children playing on and around ccatreated wooden playground equipment
i believe the senators amendment is necessary because again it sets a deadline otherwise folks can just keep studying and talking and avoiding making a decision but he is trying to put some teeth into this appropriations bill which i commend and support because just the other day i had a friend of mine say she heard senator nelson speak on this issue in relation to playground equipment she was just about ready to buy some playground equipment for her grandchildren she does not know whether to buy it or not she does not know whether it is safe or unsafe
if you live in a state that gets as much rain as the good senators state of florida you have to be even more worried if it is as humid as it is down there you have to be more worried
we do not want to make a decision that is not scientifically based so we need to get these science studies done and the epa and the consumer product safety commission making their decision they have asked for public comment but we should pass this nelson amendment because it really directs the epa to report to congress as soon as possible which is in effect a report to the public so my friend can decide whether or not she is going to buy wooden playground equipment or plastic or steel or whatever choice she is going to make
i commend the senator for understanding this is an issue that is not one of these abstract issues that only concerns somebody sitting in some ivory tower somewhere this is an issue that concerns every mother and father who takes their child to play at a playground or anybody who is thinking about buying equipment for their backyard
we need to look to a nonpartisan independent source such as the scientists who will examine this issue find out whether this cca is or is not a health hazard or whether it can be fixed and if it can so it can be a problem that can be prevented this is one of those public service issues to which i really think we owe the people of this country an answer otherwise we may be unfairly tarring this industry we may be preventing people from buying playground equipment that is totally safe we dont know we just know this cca has arsenic in it we need to get to the bottom of whether that is harmful or not
i commend the senator for his approach i hope my colleagues on both sides of the aisle will support this amendment so we can get an answer sooner instead of later
mr president i yield back whatever time i might have been given
 newspeech
madam president i also thank the chairman and ranking member for not only the work they have done on this bill but the work they will do on the farm bill this fall i know this is a difficult matter
both the chairman and the ranking member have outlined the challenges ahead of us but i know everyone in this chamber is ready and willing to work together to get a result that will be not only fair to our farmers but will recognize the full extent of both agricultural and conservation needs that go hand in hand with agriculture throughout our country
i rise today to say a few words about agriculture in new york because i have noticed many of my colleagues are surprised there is agriculture in new york many people perhaps some in the gallery today think of new york and think of new york city they may fly into laguardia or out of jfk they do not get a chance to travel throughout the state to see the beauty of the scenery and to know how important agriculture is to the livelihood the economy and the future of new york
in every section of new york even surprisingly in some of the boroughs of new york city there are still some agricultural interests much of the state from st lawrence to orleans to the entire southern tier out into long island agriculture remains a critical part of the fabric of life in new york and is a crucial livelihood for countless new yorkers
in fact agriculture still is the no 1 economic sector in new york which would come i suppose as a surprise to many people from the midwest or the south i have been fortunate having grown up in the midwest actually in illinois right between the chairman and the ranking member of the agriculture committee to know a little bit about midwest agriculture then i have been honored to have lived in arkansas for which good friend senator lincoln having come from a farming family is a champion so i know full well how critical agriculture is in the midwest in the south in the west but i do not want anyone in this chamber or anyone in our country to overlook or forget how important agriculture is in the northeast and particularly in the state of new york
i received a letter from a farmer in kent ny what he has written could be written from the chairmans state or the ranking members state i want to read what he said
i am writing this letter with great concern on behalf of our family farm our family farm was started in early 1900 by my grandfather and grandmother when they came to america from england i started working on the farm as a young man at the age of 7 by riding with my father and watching how to work and how to make a living by providing food for the world in which we live now at age 46 i sit back and try to evaluate what is wrong with our agriculture picture our cost of production has gone through the roof as fuel labor and growing mandates are taking our profit out of the picture our fresh fruit apples after being packed out of storage have a slim chance to exceed the cost of production our vegetable operation along with our grain crops are in the same position due to commodity prices that are lower than 25 years ago but yet fuel prices alone have more than doubled in 15 months
he goes on to write
usually there is always one commodity that excels each year to offset the poorer priced ones but that has not happened in the past year your first response is to get your cost of production down and to establish a higher yield but we have exhausted all of these options every time we have a potential for a commodity price increase one of our competitors ship across the borders keep prices low and here we sit in new york just trying to survive i have a great deal of pride and want to do my part to keep agriculture the number one industry in our county of orleans state of new york let us get agriculture out of this situation and back on track immediately
i could not agree with this gentleman more what i hope we are going to be able to do as the chairman the ranking member and the committee members craft their farming bill for this fall is to make sure those of us who may not be on the committee but who represent farmers and a farming state no matter how difficult that may be for some to believe will also be at that table because we have to be heard on behalf of our farmers
i want to point to this chart in 1964 there were 66510 family farms in 1997 we are down to 31757 certainly some of those farms were lost because new york grew the county i live in became pricey choice real estate for people who wanted to live near new york city we are fighting to preserve the farmland we still have left in westchester county
we know there were inevitable changes no one is arguing against the inevitability of change that is going to take farmland out of production but in many parts of our state we lost population there was not population pressure forcing people into the country therefore doing away with available farmland we lost farmland because our farmers were not given a fair shake were not given the tools with which to compete
as we look at the farm bill i hope we are going to also look at the important essential role farmers play in conservation preserving our rural countryside making it possible to have high water quality and wildlife habitat i know if it were not for farmers all up and down the midwest and the south there would not be as many ducks to hunt every year i know farmers have played a critical role in preserving wildlife habitat for hunters and for the enjoyment of so many other people
farmers have a role not only in producing quality affordable food but also improving water quality and wildlife habitat restoring wetlands and protecting farmland from further development i hope we are going to get some of that conservation assistance in the farm bill coming this fall i would have preferred by far the bill that came out of the committee in the senate that was not possible because of the presidents veto threat that is what the ranking member just explained i deeply regret that
as the chairman senator harkin pointed out this would not have busted the budget this was forward funding that would have gone into next year the dollars then could have been distributed not only to help our farmers but also to do the conservation work that they do for all of us
i want to mention also that we have some crops in new york that do not produce a lot of money less than usd10000 but we are proud of them we have a lot of orchards in new york going from 6931 in 1964 to 2436 in 1997 we still are proud of our apple growers we are proud of our speciality crops
in may there was an article in the washington post about the plight of apple growers in albany ny it told how this past march susan and gary davis auctioned off the machinery they used to tend orchards and vegetables on a farm that had been in their family for a century they said you feel like you are letting them down both past generations and your own children but they just could not keep up with the costs and their farm manager finally said he could not do it anymore the grower gave up and moved to find a livelihood somewhere else
we know we have to do more to make farming a viable alternative for those who are willing to put in the long hours are willing to do the work that gives us a safe food supply i consider food security part of national security certainly that is true when it comes to the speciality crops and also when it comes to dairy in new york
our dairy farmers are down to 8732 farms i bet a lot of people did not know there were 8700 dairy farms in new york we are the third largest dairy producing state in america and we are proud of that fact but we have to have some help we have to be able to compete with our neighbors to the north with our neighbors to the south and with our neighbors to the west
milk is new yorks leading agricultural product creating almost usd2 billion in receipts we rank third behind california and wisconsin our dairy farmers are probably the hardest working farmers maybe the hardest working small businesspeople one will find anywhere it is a 24houraday 7dayaweek job i was visiting with some of our dairy farmers on the shores of lake champlain they have been there for seven eight and nine generations this is a difficult tough job we should not make it any harder we should be proud of those who are willing to do this work and we should find ways to support them because it helps all of us
finally i hope my colleague senator schumer and i are able to convey as clearly and hopefully persuasively as possible that when agriculture is discussed new york should be at the table i thank everyone in this chamber for giving us the opportunity to have our farmers receive the same help that all of our farmers in america need
i thank the chair
 newspeech
mr president i rise to join the senior senator from new york mr schumer in paying tribute to the late paul r carey i was also honored to have been invited to speak at the memorial service for paul here in washington last week and i wish every senator could have been there to share in the outpouring of emotion and affection for this wonderful young man my husband and i knew paul carey well and we considered him a dear friend paul made many important contributions to president clintons work in the white house and he remained a close friend after he left the white house to become a commissioner of the securities and exchange commission he touched so many of us with his wonderfully passionate attitude toward life and his truly special gift for friendship i join senator schumer in paying tribute to paul carey and in expressing condolences to governor carey to pauls 11 brothers and sisters and to his many friends he was a great new yorker and we will never forget him
 newspeech
mr president it is an honor to be here today to submit a resolution to recognize the contributions of minority women to womens suffrage and to the history of our country this resolution establishes an advisory committee and directs the architect of the capitol to enter into a contract for the design and construction of a monument commemorating the contributions of minority women
i was so pleased when congressman davis introduced this resolution his decision was inspired by the observations of a young woman working in his office who noticed as she toured the capitol that there are so few women and even fewer minority women represented in these sacred halls
the underrepresentation of women and minorities does a disservice to the thousands of schoolchildren who tour the capitol every year i believe the time has come and is in fact long overdue to create a statute honoring the contributions of minority women who were instrumental in building our country and leaders in extending equal rights to all people
i can cite many examples of minority women who i would like to see considered for recognition women with new york roots such as harriet tubman sojourner truth and maud nathan have made considerable contributions to our nations history
harriet tubman whose home was in auburn ny escaped slavery and then risked her life again and again to return and lead so many others to freedom harriet tubmans motto was keep going she would encourage escaped slaves in their journey by saying children if you are tired keep going if you are scared keep going if you are hungry keep going if you want to taste freedom keep going harriet tubman went on to be an active leader in the womens movement to work for schools for freed slaves and to establish services for the elderly and destitute her actions were selfless and her courage is of heroic proportions
sojourner truth was born enslaved in upstate new york after her release from slavery she went on to work as an abolitionist and then as a leader in the womens movement she was a highly effective speaker and used her voice to see that equal rights would be extended to all people regardless of the color of ones skin or ones gender maud nathan is another example of a new yorker who was influential in the womens suffrage movement and served as an early and innovative consumer advocate organizing for better conditions for working women
i often think of the courage and vision of these women and so many others who put their lives on the line in the abolitionist suffrage civil rights and womens movements and it is a great sense of pride to me that so many women leaders were from new york
it is our responsibility to make sure that the contributions of minority women with stories similar to truth tubman nathan and so many others are told in our schoolrooms at our dinner tables and yes celebrated in the halls of congress
in 1997 after more than 75 years of storage in the crypt a monument recognizing suffragists susan b anthony elizabeth cady stanton and lucretia mott was moved to a visible location in the rotunda this was the right decision then and no doubt has aroused the interest of so many people who have had the opportunity to view it since the move
now we have an opportunity to make significant strides toward telling a far more accurate story of our nations collective history by celebrating the minority women who were behind so many of our nations important social movements their commitment resilience and courage can be a great source of strength to the next generation of women who will assume the struggles shaping our time
 newspeech
thank you mr president i thank my colleagues for their outpouring of support their concerns and their many offers of additional aid that has come to the rescue of our people as a result of this devastating tragedy
yesterday dawned a beautiful day in new york my daughter told me it was one of those days where the sky was totally clear there was a breeze people were starting to line up at the polling places to vote because it was primary day an election day a continuation of the commitment to democracy and selfgovernment that has set us apart from every society that has ever existed because of the longevity of our democracy and the will of our people to constantly renew themselves
new yorkers went from standing in line to vote to standing in line to donate blood in just a few hours i do not think any of us will ever get out of our minds the images we saw on television of the plane going into the first tower the plane going into the second tower and the plane going into the pentagon but there were tens of thousands of our fellow americans people who live in new york new jersey and connecticut people literally from every part of our country and indeed the world for whom this was not an event they watched in horror on television but lived through and in too many instances did not survive
we are beginning to find out what that was like chuck and i have a lot of friends who worked in those towers who worked in the center and worked nearby we are hearing the stories of husbands and wives grabbing cell phones and calling home to say i love you goodbye
we know and i assure every person in this body in the house and many many of our fellow citizens when we finally know the names of those killed and injured they will know someone
this was an attack on new york but it was really an attack on america i have been very gratified as i know that chuck has and all of our colleagues in the house by the strong support we have received from the president i am very grateful we have expressed our appreciation
chuck and i will be going to new york this afternoon with fema and we could not ask for more than we have received in the immediate aftermath of this horrific attack
we are by no means anywhere near the end of what it will take to continue the search and rescue efforts we are finding people even as we speak yet we know there is a very grim task ahead to do everything we can to find every person to account for every single person who went to work that is all they did they went to work on a beautiful september day in new york
we will also stand united behind our president as he and his advisers plan the necessary actions to demonstrate americas resolve and commitment not only to seek out an exact punishment on the perpetrators but to make very clear that not only those who harbor terrorists but those who in any way give any aid or comfort whatsoever will now face the wrath of our country
i hope that message has gotten through to everywhere it needs to be heard you are either with america in our time of need or you are not
we also stand united behind our resolve as this resolution so clearly states to recover and rebuild in the aftermath of these tragic acts new york was not an accidental choice for these mad men these terrorists these instruments of evil
they deliberately chose to strike at a city which is a global city it is the city of the 21st century it epitomizes who we are as americans so this in a very real sense was an attack on america on our values on our power on who we are as a people i know because i know america that america will stand behind new york that america will offer whatever resources aid comfort or support that new yorkers and new york require because the greatest rebuke we can offer to those who attack our way of life is to demonstrate clearly we are not cowed in any way whatsoever
i hope that within a short period of time i say to senator warner we see scaffolding on the side of the pentagon after we finish the search and rescue and recovery work that is being carried out heroically there i hope we all see a clear signal that we are rebuilding that our defenses are more resolute than ever
i hope similarly that lower manhattan has the same kind of image to project because the reality will be that we are rebuilding and reconstructing and making clear that just as our military might is unchallenged and uncowed so are our economic our social our political values epitomized by new york
i have expressed my strong support for the president not only as the senator from new york but as someone who for 8 years had some sense of the burdens and responsibilities that fall on the shoulders of the human being we make our president it is an awesome and at oftentimes awful responsibility for any person i know we are up to it i know we are ready for it and i know that everyone in this body represents every american in making clear that we are united and stronger than ever
it is with a heavy heart really a sense of heartbreak that i rise today in support of this resolution but it is also with a great sense of pride first in the people of new york who responded as new yorkers always do when times get tough there was not a sense of panic there was order and there was an immediate outpouring of help those men and women whom we sent in to rescue our fellow americans there is no way adequately to express our gratitude to our firefighters our police officers our emergency personnel our doctors and nurses and medical personnel they responded at the height of a tragic unexpected attack with the kind of grit and courage we expect from new yorkers
to all of those who are missing a loved one there are no words any of us can express except to tell you in the clearest possible terms we will in a united american response support you offer assistance to you stand with you and pursue those who reached deep into your families and homes yesterday and took someone you loved away from you
there will be a lot of work ahead of us in this body and in the house and we will pursue that i am grateful for the support we have received thank you very much
applause
 newspeech
mr president i wanted to take just a few minutes of morning business to report to my colleagues about my visit along with senator schumer to new york yesterday to convey the appreciation that new yorkers feel starting with our governor and our mayor but going down through the people whom i saw whether they were a firefighter or police officer or emergency medical technician or someone standing on the street for the unified and extraordinary support that has been demonstrated by our entire country starting with our president
senator schumer and i flew to new york with administrator joe allbaugh the director of the federal emergency management agency for the purpose of assessing the damage and attending a very long briefing with the governor and mayor and their respective staff who are on the front lines dealing with this tragedy
we took a military plane from andrews and flew to laguardia where we got into helicopters the helicopters flew us to the tip of manhattan where we circled from the east river toward the hudson and were close enough in to see the burning debris to see the wreckage the crumpled destruction of the buildings that had once stood there a sight that the only comparable basis i think most living americans would have such as our distinguished senior member senator inouye is what war was like in world war ii or korea or vietnam
we took another pass so we could get in a little bit closer as we did we saw dozens and dozens of people running away from the site we later learned that the continuing danger from these structurally damaged and unsafe buildings had driven our rescue workers out
we landed at the heliport on the east side and went in to meet with the mayor and the governor we had some time to talk with the press where everyone expressed the solidarity and unity that the people of new york are certainly feeling between and among themselves and that we are grateful for the support that our president our congress our entire government and people have given new york
we then went in a convoy down to the site i wish every one of my colleagues could have been with us because the streets were lined with people holding american flags and signs expressing their gratitude and their thanks to the many workers and volunteers who had come to help and a real sense of resolve and demonstration of support for our nation
because of the difficulties with the buildings we were not able to go in as close as we had originally planned so we stopped at a place short of the immediate area that one could approach and still not interfere with the search and rescue mission or be in danger we put on our masks because the acrid smell of the still burning debris is extraordinarily oppressive we got a briefing on the spot from some of the people who were directly in charge of the work that is being done
i felt as though i were on the edge of hell i watched the smoke rising i could see the twisted wreckage and i had a much clearer visual image because of my helicopter view i saw the people who have been there hour after hour since this vicious attack occurred coming toward me their shift was over they were seeking some respite firefighters in their uniforms covered from head to toe with dust and debris exhausted and dragging their fire axes with them
the impression and feeling that one gets from actually being even as close as i was is so much greater with respect to the devastation than we see on our television screens the television in a sense contains a miniaturized view of what has happened in new york
when we then stood there for a few minutes and that is all we were able to spend there we visited with people who were looking for their lost loved ones one mother in particular had just come down to the area hoping against hope to hear something about her son residents who had lost their apartments their offices and their businesses were standing on the side of a familiar street in a totally awful inexplicable new circumstance
we then went to the police academy which has been set up to be the command center since the citys command center was lost in the collapse of one of the ancillary buildings to the two towers and we had a very long and very informative briefing from the mayor from the governor and from all of the people on the front lines the police commissioner and others who talked about where they were in the struggle that they are engaged in against this massive mountain of debris
just that night they had moved out more than 120 dump trucks filled with debris the estimate from the army corps of engineers is that there will be at least 500000 tons of debris
in addition to the immediate search and rescue and cleanup work that has to go on the power situation the loss of energy and telephone and communication services has meant that the new york stock exchange could not open for business yesterday it has meant that there are still many offices of our major financial institutions unable to reopen
the humanitarian needs are enormous there is an armory down on lexington avenue at about 25th or 26th street that will be open for those whose family members and loved ones and friends are missing so that they can go down and identify and have that registered and provide additional information that may be required for identification
the overall impression that i certainly take from that experience yesterday is of the pride i feel in the work that is being done of the leadership given by our mayor and our governor of our police and fire and emergency personnel the extraordinary readiness of our hospitals to care for the injured the tragedies there are not as many injured as they had expected and the realization that we have a very big job ahead of us a job of cleaning up and rebuilding and reconstructing
i asked for some estimate from the mayor and the governor as to what we were looking at because this is something for which we have to plan they were obviously unable to say what the total estimate would be of the costs that have already been incurred and will necessarily have to be incurred in the future but they know that they could immediately explain and justify usd20 billion of direct costs we will be asking our colleagues for a show of support as a way of recognizing that the epicenter of this attack on our country occurred in new york city
i took a late train back last night because i needed to be here to work with my colleagues on not only the further understanding of the level of devastation and need that has occurred but to answer questions and to be available as we continue to try to sort out what kind of national response is required
i am very pleased that the president will be going to new york tomorrow afternoon i applaud and salute him for that decision i know it was a difficult decision because of the security issues that surround the movement of any president i personally on behalf of my constituents thank him
i will be meeting along with senator schumer and others at the white house this afternoon with the president where i will personally not only convey my appreciation for his leadership his commitment and his visit tomorrow but also the specifics about the requests that the governor and the mayor have made for additional and specific federal assistance
in addition i introduced legislation earlier today that will be cosponsored by many colleagues on both sides of the aisle it has an identical companion bill in the house it is s 1422 which will expedite the process by which the federal government provides benefits to the families of public safety officers firefighters police officers emergency service personnel and others who lost their lives in the line of duty
i am very pleased once again that the president in his videophone conversation with the mayor and the governor stated his strong support for this legislation
it has been said that more public safety officers lost their lives in the terrorist attack against the united states on september 11 than in any other single event in modern history
we may not know exactly how many rescuers gave the ultimate sacrifice yet but i was told by the acting commissioner yesterday that they are missing 300 firefighters
i just was handed a note that is a very good piece of news that they have just found two firefighters and one citizen still alive which is why the search and rescue mission has to continue we cannot give up we know from cell phone communication and from the experience and intuition of our firefighters and rescue personnel on the scene that it is still possible as we just learned for people to be alive buried under that rubble we will not give up until we find every single person
yet when we look at who is on the front lines it is not me carrying the ax it is not me as one of the ironworkers who rushed down to volunteer their services to help remove some of the debris it is not me as a police officer who is on the front lines it is these men and women who have made the sacrifice to protect us and to respond as they would have at a time of battle and in effect when this act of war took place they were our frontline soldiers
the federal government provides a onetime benefit payment to the families of public safety officers lost in the line of duty through the public safety officer benefit program unfortunately these benefits are often delayed for long periods of time because of very burdensome regulatory applications
in fact i stood in this chamber back in may to commend the sacrifice of brave new york city firefighters who lost their lives in a fathers day blaze in queens their families are still struggling to complete the application process they fill it out and they are told they need more information
it is imperative that we take action now to ensure that the family members of those brave men and women who lost their lives in this terrorist attack are not confronted with the same onerous process
so the legislation that i have introduced today would direct the department of justice to expedite the process for these families of those who lost their lives while responding at the world trade center in new york city the pentagon in virginia or in stonycreek township pa given what i heard on television from the president and the extraordinarily broad support that i have for this already in the senate i do not think this will be a controversial piece of legislation but i hope it can be considered as soon as possible to send a tangible message to our firefighters our police officers and our emergency personnel that we are with them and their families in their time of loss
finally mr president we are just realizing the full depth of the humanitarian crisis and grief and loss that has occurred for the children who have now been orphaned the husbands and wives who have been widowed the parents who are facing what no parent should have to face ever the loss of their son or their daughter there are no words adequately to describe or express our sense of loss as a society
i am very grateful that the city the state and fema will be on the ground with grief counseling with psychological help with mental health services because having been to more disasters in my life than i wished i know that those who do not bear any visible injuries or scars carry deep and lasting wounds
we will as a nation not only seek out the enemy wherever he may be but we will also care for the grieving and the wounded we will i know do everything required to provide whatever help and assistance we can as a nation
i also hope that for those who were far away from any of the attacks on september 11 they too will talk with one another and comfort each other
i was very grateful and proud to see laura bush mrs bush on television today talking about the need to have an open conversation with our children depending upon their ages to reassure and comfort them because the binding up of our wounds as a nation goes far beyond lower manhattan or arlington va
we have all been stricken by this cowardly act of terrorism but i am confident that we will respond with the same resolute purpose that has always defined us as a nation with the same compassion that marks us as a people and with the same resolve to not only defend ourselves wherever and whenever that is necessary but to rebuild and reconstruct the human spirit and the physical terrain of america
thank you mr president
 newspeech
madam president i thank the senator from west virginia i rise to join the comments of my colleague senator schumer
many of us recognize the need for immediate action to aid our struggling airline industry and are prepared to do so we also appreciate greatly the continuing bipartisan cooperation that is helping this body address the needs of our country in the wake of the terrible attacks of september 11
i am very grateful that the long negotiations in the house and the senate over the last several days along with the white house have resulted in a process to provide relief to families who have either lost a loved one or sustained significant personal injury
we are also grateful that the legislation provides more support for essential air services particularly in many of our more rural areas such as watertown ny that are totally reliant on air service which still comes in to serve those communities
the passage of this legislation can only be a beginning we also must make our airports as secure as possible i am honored to join in legislation senator hollings and senator rockefeller and others are putting forward to address the security issues so that americans have the confidence i know they should have in flying once again resuming our normal life
we also will have to work together to develop a process so the many businesses that have sustained losses will be able to seek relief in some expedited process and not get buried and even bankrupted by an extensive round of litigation i just learned that the litigation that was filed following the 1993 bombing of the world trade center is still ongoing businesses have no certainty no resolution of what their liability if any might be we need to avoid that in the wake of this tragedy and do everything we can to come up with a process that deals with the needs of other businesses as well as the airline industry
i also hope that the assurances we have received with respect to the airline workers who have been laid off or displaced will be addressed as soon as possible i am well aware that a package being considered to help the workers was stripped out of this bill we have to revisit it it is something that is growing in importance day by day there are now at least 74000 people affected we expect in the next few days that number to grow to 100000 i suggest we look at some kind of an aid package modeled after trade adjustment assistance to extend unemployment insurance job training support services to airline workers and other workers who have been directly affected by the attacks on our country
we also will have to look at the way our entire transportation system operates i am very proud of the way amtrak stepped in to fill the need for the movement of passengers and goods their ridership is up nationwide they have honored airline tickets of stranded airline passengers and shipped relief and medical supplies amtrak has also made security upgrades on trains and in stations but we need to do much more to address the critical needs in the northeast corridor as well as the rest of the country
i look forward to working with my colleagues i see my good friend senator hutchison from texas who led the fight on the high speed rail bonding act that is just one of the many issues we need to consider as we look at transportation again as part of national defense
i well recall how president eisenhower obtained the funding for the interstate highway system because it was part of national defense our highways our airways and our railways are all part of our national defense infrastructure
finally i say once again how grateful we in new york are for the tremendous and continuing outpouring of support from the american people i particularly thank the president for his strong support i was overcome by his absolute resolute commitment to rebuilding new york in his speech last night as well as the other very strong words of reassurance and resoluteness he delivered with respect to the challenges we face i appreciate greatly his leadership and his support throughout this crisis
 newspeech
mr president i too thank the chairman and ranking member and a number of our distinguished colleagues who have risen to support and oppose the bunning amendment i believe many members in this chamber either had no wellinformed or formed opinion prior to september 11 or perhaps were inclined to support a new round of base realignment closings it is with some regret that i rise in support of the amendment from the senator from kentucky i believe after september 11 it is imperative we have more information available than we currently have
there are many arguments that have already been made on the floor very good ones from our colleagues from arizona rhode island delaware and florida as to what efficiency issues should take precedence i agree we need to constantly be evaluating our defense budget and expenditures to become as efficient as possible yet i also believe there are serious security concerns we are only beginning to address i take very seriously the secretarys letter which has been referred to and which has been read into the record
i believe my colleague from new mexico senator bingaman is correct in saying the president and the secretary have inherent power to realign depending upon the needs we face in any kind of strategic or emergency situation
we are about to engage in a broadscale reevaluation of our homeland defense and security we are going to be asking ourselves some very tough questions about our readiness about the proper intersection between our domestic policing agencies and functions and our military
at this point i think there are several factors that have to be addressed in addition to the request of the department of defense and the recommendation from the armed services committee before many members would be comfortable voting for a new round i am not sure the new round if it is only a defense department review will adequately look at some of these other broader issues that may have implications for both physical infrastructure and force deployment
some have said the qdr which is expected by the end of this month is out of date now i dont believe that is the case at least from what i am told and read in the paper that the quadrennial review that the department has been undertaking will have some very significant recommendations that should be digested and taken into account with respect to moving forward on another round of base realignments and closings
it is important we integrate our domestic and military capacities in a way we have never had to think about before many were deeply concerned when we read reports of the short time but nevertheless unfortunately delayed time that it took to scramble fighters into the air to try to deal with the impending threat and the potential threat that might have still been out there from additional hijackers i dont know that the brac round has the same substantive understanding or impact that we have had in years past given the new threats we have so tragically suffered i would be very confident and supportive of our chairman and ranking member and members of the armed services committee working with the administration coming up with a proposal that does make some sense
i listened very carefully to the comments of the senator from arizona maybe certain bases should be taken off the table maybe they should that is something we have never talked about before but in the context of the new threats we face i think we have to think differently it may be we may have a brac round where some bases would be off the table some of the assets that we have would be put to one side and we say they are essential to homeland security and they are essential to our projection of force abroad therefore any brac round would not look at those that might be an idea worth considering because i think everything changed on september 11 a threat that was not understood as being so deadly and imminent has caused such terrible destruction and tragedy
i for one will support the bunning amendment at this time because i think we have to reevaluate what we mean when we think about closing bases and realigning our forces no one should argue about the efficiency measures that need to be taken so that we do no 1 get the most effective use of our dollars and no 2 provide the kind of infrastructure and resources that our allvolunteer military deserves to have
i am concerned at this point we may not be ready for the son of brac there may be the need to rethink how we get to the level of bases that are required i think perhaps for the first time we have to seriously take into account the new mission that the president has given for homeland security to make sure there is if necessary the kind of integration that will make us safe at home as well as abroad in terms of americas values interests and security
i rise with some regret because i have the greatest of respect for our chairman our ranking member and those who support this request for another round i probably will very much end up supporting it but only after we give the kind of thought i think is required today to take into account the new threats and perhaps do it differently than we have done it before after we carefully evaluate what kind of presence we need taking into account homeland security i would support that kind of approach that is not what is being proposed at this time i urge my colleagues to support the bunning amendment
i yield the floor
 newspeech
mr president i come to the chamber to discuss further the need for transportation security that encompasses not only our airlines but also our rail lines and our ports others with their own experiences and perspectives have already spoken to these issues and i am sure will continue to do so because as we address these critical needs of transportation security it is imperative we look at all the means of transportation our people require and that we found to be particularly important in responding to the events of september 11
i want to focus my remarks on amtrak and our rail transportation system i think anyone who followed the events of september 11 is well aware that amtrak played a critical and essential role in responding to this national disaster we know that without amtrak being able to respond new york would have been cut off the natural flow of commerce and passengers between boston and washington the busiest rail corridor in our country would have been severely undermined we know too that amtrak did its part to make sure people not only could reach their destinations but for example those who had planned to fly by air when our air system was shut down their tickets were honored and they were part of the continuing and increasing flow of people and goods that demonstrated that america was still moving
ridership on amtrak has been up 17 percent across the nation and certainly in the northeast corridor which was so devastated by the attack on the pentagon the closure of our airports the attack in new york city the continued until thankfully today closure of our washington national airport we know that amtraks increase here was up by 30 percent
how do we make sure this critical mode of transportation is safe and secure in the future we cannot be in a position of looking backwards we have to look forward and say what do we need to do to make sure our transportation system is redundant and safe i believe we have to focus as we look at transportation security on ensuring that our thousands and thousands of rail passengers are safe
i am grateful amtrak has come forward with a specific plan to address the needs of those passengers we need for example more police officers on our trains more canine units to inspect the trains more power and switch upgrades to ensure they absolutely run without any delay or disruption
in new york we have immediate safety concerns which demand we act now not later hopefully in time to make sure we are always moving and if there is any natural or other disaster that we keep our people moving
i want to bring to the attention of my colleagues some specific safety concerns anyone who has ever been on a train in or out of new york knows i assume that there are four tunnels under the east river and two tunnels under the hudson river that serve as vital links between new york city and the surrounding area and the rest of america
these tunnels were built in 1910 and now almost a century later they have not undergone any serious security upgrade under todays regulations the tunnels would never be allowed to be constructed in the same shape in which they currently exist
penn station in new york city is the busiest railroad station in the united states more than 500000 passengers from all parts of our nation on more than 750 trains pass through penn station each day as many as 300000 commuters pass through the east river tunnels on the long island railroad trains each day so these tunnels are essential to our national railroad network and to the moving of people who commute every day in and out of new york city the tunnels are so essential that we must turn our attention to ensuring they are safe for the hundreds of thousands of people who use them every single day
if for some reason a train were to become incapacitated in one of our tunnels the only means of escape would be through one of two antiquated spiral staircases on either side of the river or by walking in the dark almost 2 miles out of the tunnels these are also the only routes by which firefighters and other emergency workers can get into the tunnels
i have a picture and it shows a narrow 10flight spiral staircase which serves as the evacuation route for passengers as well as the means for rescue workers to enter the tunnels i can barely even imagine what the situation would be like under the ground under the rivers if some kind of disaster were to occur with passengers and crew trying to move up this narrow spiral staircase and rescue workers trying to move down or in the alternative people being in some instances carried or trying to get out on their own going 2 miles in whatever conditions existed at the time
i bring this to the attention of my colleagues because i think it is imperative as we look at transportation security that we do not turn our backs on the hundreds of thousands of people every single day who use our railroads i fully support adding air marshals on our flights i support federalizing the inspection that passengers and cargo and luggage must go through and i support doing everything we humanly can think of that will guarantee to the american public we are doing all that can be imagined to make our airlines safe
i also want to be able to stand in front of the people in my state who rely on these trains to get to and from work who rely on these trains to commute who travel out of new york city and people all over our country who similarly rely on our trains that they also will be secure we dont want to leave any american out of our security efforts this is an opportunity to do right what is required what we now know will prepare america for any future problems
the airline security bill which i hope we will be considering soon calls for the creation of a deputy secretary of transportation security who will be responsible for the daytoday operations of all modes of transportation i applaud this provision i think it is long overdue it certainly will be a strong endorsement of the kind of broadbased security required for our millions of airline passengers for those who use our ports for those who come in and out of our transportation network and for the 20 million passengers who rode amtrak last year
over a week ago it is hard to keep track of time in the last weeks 40 of our colleagues took the train to new york city i am so grateful for some it was the first time they had been on the train it was fun to see their surprise and enjoyment provided by the ride to and from new york city they were in a sense following in the footsteps of the hundreds of thousands of people who either have used trains out of necessity or out of choice for years or who were forced to use trains in the wake of september 11 and thank goodness the trains were there
i cannot even begin to calculate the economic and psychological costs we would have suffered had we been totally shut off we could not have moved people as easily as we did if amtrak had not responded as well as it did in putting on additional equipment and personnel
i hope my colleagues will remember this picture of this spiral staircase i hope they will think about everyone they have ever known who perhaps has been a passenger as i have been many times on these trains through these tunnels i hope they will join in the commitment we must make to every single american that we will guarantee the highest possible level of security for all transportation it is the least we can do i look forward to working with my colleagues to make sure it happens
 newspeech
i thank the chairman of the committee who has done such a tremendous job of leadership in the wake of the terrible attacks of september 11 i commend him and the ranking member the distinguished senator from arizona and thank them for their tireless work and their constant reminders of the challenges we face and the sacrifices that are needed
i rise in support of the chairmans hard work on behalf of this bill and i particularly appreciate the inclusion of the clear understanding that we have to face a direct threat to our national security and we have to do it by joining together and establishing a commonsense set of solutions to the problems now before us
the aviation security act the chairman has worked so hard on is the result of many years of his labors and understanding of the difficulties we confront i certainly commend him and thank him for his hard work
i also rise as a cosponsor of the carnahan amendment to provide critical assistance to airline workers and those in aviationrelated industries who were laid off as a direct result of the terrorist attacks
at the time we considered the socalled airline bailout bill many of us made very clear in our statements on the floor that we were disappointed that some concerns for the workers who were going to lose their jobs were not included in the bailout bill we come today to reinforce our deep concern and to ask our colleagues to support the carnahan amendment
the numbers are overwhelming we know that 100000 workers have been laid off in the airline industry at least 30000 more have been laid off in airline manufacturing we are concerned that if the american traveling public and visitors from overseas dont resume flying as i urge everyone to do i have flown numerous times already and i encourage everyone to begin again to travel for business and pleasure if for whatever reason that return to the air is delayed then the numbers will undoubtedly grow
many of these airline workers are based in new york they have been supporting our air transportation system out of jfk and laguardia they have been literally handling some of the busiest air traffic corridors in the world we know that reductions in flight schedules at both of these airports have put thousands of new yorkers out of work pilots and flight attendants baggage and passenger service representatives this has had a ripple effect throughout new york
for example in syracuse in upstate new york a call center for us airways that had been there for many years was shut down throwing more than 400 employees out of work
these airline and aviationrelated industry layoffs are not just numbers they represent the lives and livelihoods of hardworking americans i have heard many stories as my colleagues have of the hardships that are being imposed because out of the skies on september 11 came these dreadful horrible acts of terrorism where people who were willing to commit suicide brought about the deaths of thousands and thousands of our fellow citizens and people from all over the world and also wreaked havoc on our airline industry and the economy in general
i hope as we consider this aviation security act for which i support and again thank the chairman and the ranking member we will also support senator carnahans amendment her aid package for dislocated workers is modeled after the successful trade adjustment assistance it will allow airline workers to extend their unemployment insurance while they receive needed job training and support services or while hopefully they wait to be called back to work because we will all start flying again
this amendment will also enable families to receive health care benefits as they go through this difficult period
no story more sums up the anguish and pain of the losses we are discussing and the need to improve security than one that comes out of jfk a twa flight attendant at that airport received her furlough notice while awaiting news of her husband a new york city firefighter missing at the world trade center new yorkers and americans have paid a very heavy price we are summoning our resolve we are preparing our responses individually and throughout our nation we are following the leadership of our president we are supporting our men and women in uniform
i urge my colleagues to support the act that chairman hollings and senator mccain have crafted and support the carnahan amendment on which she has worked so hard to pay some attention and provide assistance to those americans who woke up on september 11 thinking that it was any other workday and went to bed on that terrible day knowing that they might lose their jobs as a result of this horrific attack
i thank my colleagues and yield back the remainder of my time
 newspeech
mr president i ask unanimous consent that the order for the quorum call be rescinded
 newspeech
mr president i do not want to impinge upon the time of my good friend senator rockefeller
 newspeech
mr president i rise again in support of the amendment offered by senator carnahan to provide immediate assistance to the over 100000 airline workers and those in aviationrelated industries who have been laid off and lost their jobs as a direct result of the terrorist attacks of september 11
i just came from a very moving ceremony of commemoration at the pentagon where the lives of those military and civilian employees at the pentagon as well as the lives of the crew and passengers of the airplane that was mercilessly driven into the pentagon were honored
i know we are working on other kinds of relief and i am grateful to the president the administration and my colleagues for the work that is being done on the economic stimulus package and for the work that is being done with respect to unemployment insurance and dislocated workers assistance but i believe we have an obligation to move quickly with respect to the workers who have been laid off through no fault of their own or of their industry and we cannot wait for the larger packages to be put together and negotiated
just as we must provide security to all americans who are flying in our skies we also should provide economic security to those who have supported us in the hundreds of thousands and millions of flights that were a matter of course before september 11 they were doing an important job in maintaining our free travel and supporting an important economic activity and now they are confronting the cruelest kind of questions how will they make their next car payment how will they be able to afford the clothes their children might need how will they know whether to go out and look for another job or hope and wait that business picks up on our airlines i do not think we should be leaving our workers who have already been laid off they need our help right now i do agree we have to address the need to help all workers
in new york for example the state labor department is estimating that 285000 workers throughout new york will lose their jobs as a result of the attack we suffered i do not think we should leave any of these workers behind if we are trying to build confidence confidence in consumers confidence in citizens then we should address the needs of those people who have been economically harmed by these attacks i respect the work that others are undertaking i will support that
i ask this chamber to send a message by voting in favor of senator carnahans amendment that we are not going to just bail out airlines we are not just going to protect the traveling public we are going to help protect economically those who we hope will be back in the skies back behind the counters handling the baggage
i met yesterday with a group of executives from the travel and tourism industry stories from them about the low occupancy rates the fact that people are not traveling for business or pleasure were very disturbing to me everyone knows we have real economic challenges the last thing in the world we need is people who are scared to go about their daily business who are scared to take that longplanned trip to disney world who are scared to fly across the country to show off their new baby to their mother or grandmother
until we can get that confidence up and i applaud our wonderful leadership of chairman hollings and ranking member mccain on the aviation security bill until we can get that confidence once again moving forward so people will fly we cant turn our backs on those men and women who were the backbone of this airline industry
i hope every senator will support the carnahan amendment and do everything possible to demonstrate our concern and commitment to those who were on the front lines and lost their jobs and livelihood because of the terrorist attacks
 newspeech
the time of the senator from florida has expired in morning business"""

In [0]:
bernie = """ newspeech
mr speaker 480000 federal employees are working without pay a form of involuntary servitude 280000 federal employees are not working and they will be paid virtually all of these workers have mortgages to pay children to feed and financial obligations to meet
mr speaker what is happening to these workers is immoral is wrong and must be rectified immediately newt gingrich and the republican leadership must not continue to hold the house and the american people hostage while they push their disastrous 7year balanced budget plan the gentleman from georgia mr gingrich and the republican leadership must join senator dole and the entire senate and pass a continuing resolution now now to reopen government
mr speaker that is what the american people want that is what they need and that is what this body must do
 newspeech
mr speaker i thank the gentleman for yielding time to me
mr speaker this legislation is both good news and bad news the good news is that the republicans have finally decided to stop holding threequarters of a million american federal workers as hostage in vermont we have close to 2000 federal workers who are working today but are not getting paid these people have mortgages to pay they have children to feed they have financial obligations to meet it is immoral it is wrong to hold them and every other federal worker who is furloughed and not being paid as hostage
it is also wrong to hold millions of americans who need passports who need environmental protection who need meals on wheels who need all of the services that this government should be providing as hostage who have paid for these services but are not getting them
the bad news is that while federal workers will be paid many of them will not be given the resources that they need to do their jobs properly that is insane why do we give people the money to go to work but then not allow them the resources to properly fulfill their function
mr speaker the truth of the matter is that this is not a debate about a 7year balanced budget if our republican friends were serious about balancing the budget in 7 years which i think can be done they would not be spending usd50 billion more on defense spending despite the end of the cold war they would not be spending more money on the cia despite the end of the cold war they would not be giving huge tax breaks to the rich when the richest people in america today are richer than they have ever been before
 newspeech
mr speaker if we want to understand why in this country the richest people are becoming richer while most working people are seeing a decline in their standard of living if we want to understand why the contract with america provides for huge tax breaks for the wealthiest people and the largest corporations while it cuts back massively on programs for the elderly working people and lowincome people we might want to examine newt gingrichs travel schedule for the coming week
mr gingrich will be in seattle wa where he will have dinner with his colleagues and his friends for the washington state republican party for usd1000 each he will be in dallas tx for a dinner for only usd10000 apiece he will be in dearborn mi for another private fireside reception at usd10000
who goes to these events most people that i know do not spend usd1000 for a dinner
 newspeech
mr speaker in the united states of america millions of our citizens should not be forced into worrying about how they are going to heat their homes in this record breaking cold weather winter
in vermont and throughout this country we have had nights of 20 below zero 30 below zero and even 40 below zero
the liheap fuel assistance program has provided over 24000 vermont households and 6 million lowincome households nationally including many senior citizens with the fuel assistance they need to survive the freezing weather threequarters of the households on liheap have incomes of less than usd8000 a year
today i am sending appropriations committee chairman robert livingston a letter with over 115 congressional signatures democrats and republicans asking him to make certain that the continuing resolution that is passed by friday contains full funding for liheap and that the remaining usd200 million that was appropriated is released
this is america and 80yearold senior citizens should not be forced to go cold in the winter
 newspeech
mr speaker i want to offer my complete gratitude to the house of representatives and the leadership for their support of fullfunding for the lowincome energy assistance program liheap to be allocated to states immediately last night i delivered the following letter to the chairman of appropriations which was signed by 154 members of congress asking for the immediate release of the usd190 million remaining in fiscal year 1996 allocation for liheap
dear chairman livingston as you know there is approximately usd190 million remaining in fy 1996 allocation for the lowincome home energy assistance program liheap when a continuing resolution is developed on january 26th we would very much appreciate your releasing 100 percent of these funds immediately while usd810 million in liheap funds have been released so far this year these funds are not even half of the liheap monies allotted to the states in fy 1994 and significantly less than the usd13 billion allocated in
 newspeech
mr speaker i rise to engage in a colloquy with the chairman of the committee on appropriations
 newspeech
mr speaker i rise to engage in a colloquy with the gentleman from louisiana mr livingston the chairman of the committee on appropriations
as the chairman knows i am particularly concerned with the lowincome home energy assistance program it has been very cold in my part of the country and during earlier crs there have been disruptions in funding for this vital program
the administration has released usd810 million to the states but i am concerned that the flow of funds to the states could again be disrupted generally i would like to know whether this cr will affect the full commitment of funds to the funds of liheap to the states specifically is it the chairmans understanding that under the current bill before the house funds for liheap will be allocated to the states and be available for distribution to the states in the normal fashion
 newspeech
mr speaker i was unavoidably absent during the votes on default legislation if i had been present i would have voted nay on the motions to table the appeal of the ruling of the chair with regards to the resolutions offered by mr gephardt rollcall no 26 and ms jacksonlee rollcall no 27 i would have voted nay on the ordering of the previous question on house resolution 355 rollcall no 28 i would have voted nay on h con res 141 rollcall no 29 i would have voted yea on hr 2924 rollcall no 30
 newspeech
mr speaker the family dairy farmers in vermont and throughout this country are facing disastrous financial problems while consumer milk prices have been going up the price that the dairy farmer receives has been going down
adjusted for inflation the price vermont dairy farmers receive has dropped 50 percent over the last 15 years no wonder that today there are fewer than 2000 dairy farms left in vermont and more and more of them are going out of business
tragically none of the proposals that we were permitted to vote on during the farm bill provided for a significant increase in dairy farm income in fact the proposal that was passed threatens to actually lower the price that farmers receive
mr speaker in an effort to save the family farm the six new england states each have passed the northeast dairy contract which finally would provide dairy farmers a fair price for their product
in the final senate bill if the final senate bill contains the northeast dairy compact this house must support that provision in the conference report to do less would be to allow new england dairy farmers to disappear
 newspeech
mr speaker on saturday february 10 in signing the national defense authorization act for fiscal year 1996 the president signed into law the missing service personnel act of 1995 which had been incorporated into the authorization bill the passage of the provisions of the missing service personnel act is a significant milestone for veterans and for the families of our mias and i rise today to give credit to some of the people including some of my fellow vermonters who worked hard for the passage of these provisions
their dedication commitment and persistence in the face of overwhelming odds has finally brought to fruition a matter that has been their primary concern for over 13 years and i am very proud that my fellow vermonters have played such a significant role in this effort
i cosponsored the missing service personnel act of 1995 after being convinced by patricia sheerin don amorosi sean mcguirl walt handy and al diacetis of the desperate need for this law the act is the culmination of years of effort on the part of my fellow vermonters tom cook bob jones and brian lindner the president vice president and chief of research respectively of the northeast powmia network and jim howley and the veterans organizations who have supported it including vietnam veterans of america the american legion the veterans of foreign wars and the disabled american veterans most notable were the contributions of the family members of the missing and prisoners tom cook sharon roraback and sarah pendris
were it not for a special conference held in 1993 by the northeast powmia network we would not today have a law to protect missing service persons to protect their families from exploitation and to grant basic human rights to the missing as well as their families under the guidance of a former pow lt col orson swindle participants in that conference were able to clarify the goals of the proposed missing service personnel act as originally authored by john holland mr swindle pointed a new direction that while we cannot solve all the problems of the past we can protect missing service persons in the future based on what we have learned from past mistakes
through her courage and intuition vermonter patricia sheerin policy analyst for the northeast powmia network convinced the national vietnam veterans coalition to support the legislation and work for its passage she also formed a plan and policy uniting veterans organizations and veterans advocates with the sole purpose of correcting and updating the outdated missing service persons act of 1942
crucial to passage of this new law was the support of citizens who were informed about its benefits joe and paula donaldson of fair haven vt deserve credit for organizing a weekly vigil as part of this educational effort nationwide distribution of information on the progress of the legislation a responsibility of bob necci helped pave the way to passage of this important act education is often the key to success and such was the case with the missing service personnel act of 1995
i commend the supporters of this bill for their loyalty and devotion to the men and women who wear the uniform of the us armed forces these vietnam veterans and family members of those missing and captured in vietnam have left a legacy of justice and fair treatment for future soldiers who become missing while fighting to defend our country and our freedom
 newspeech
mr speaker as the only independent in the congress what i would like to do is touch on a few thoughts that my republican and democratic colleagues often choose not to address there are a lot of very very important issues which congress discusses every week but somehow or other we seem not to talk about some of the most important issues facing the american people
the first issue that i would like to talk about is heightened by an article which appeared in the february 5 washington post national weekly edition the article touches on an issue which i think all of us in this country should be very concerned about that is the quality of american democracy and to what degree we in america remain a vibrant democracy
mr speaker all of us should be deeply concerned no matter what our political persuasion is that in november 1994 when the socalled republican revolution took place and the gentleman from georgia mr gingrich became speaker of the house all of 38 percent of the american people voted in that election sixtytwo percent of the american people did not vote the vast majority of lowincome americans did not vote young people in large numbers did not vote working people did not vote
what does it mean mr speaker when in europe and in scandinavia and in canada and in other industrialized democracies 60 percent 70 percent 80 percent of the people come out to participate in the political process but in the united states of america we have a major national election of great consequence and 38 percent of the american people participate what does that mean
i think it suggests mr speaker that the vast majority of american people are giving up on the political process they are hurting they are in trouble but they look to the government and they do not see a government which responds to their needs i think what they instinctively understand is that by and large what happens here in congress and the decisions that we make here in congress reflect to a very large degree the interests of the wealthy and the powerful the people who can contribute usd10000 a plate to a newt gingrichsponsored fundraising dinner the people who contribute usd16 million in one night to a republican party fundraising dinner
meanwhile the folks back home are working longer hours for lower wages they are concerned that they cannot afford to send their kids to college they cannot afford health care they are worried about the health care bills for their parents they are deeply worried and they look to congress and essentially what they see is a congress which represents the interests of the wealthy and the powerful and forgets about the needs of the middle class and the working people of this country
mr speaker in the article in the washington post on february 5 their national weekly edition there is some information that they received from a national poll which should be of major concern to all americans regardless of their political persuasions let me quote a little bit from that article
i quote to measure how much americans know about politics and the political system the washington post the kaiser foundation and harvard interviewed 1524 randomly selected adults in november and december these americans were asked 18 general knowledge questions about how their government works and who their leaders are an additional 21 political knowledge questions were asked in 4 other national washington post polls the surveys revealed a knowledge gap that is deep and wide i would hope that people listen to the following paragraph
this is based on polling by the washington post working with other institutions twothirds of those interviewed could not name the person who serves in the us house of representatives from their congressional district in other words twothirds of those polled did not know who was representing them in the house of representatives half did not know whether their representative was a republican or a democrat
then they go on who is the vice president of the united states who is the vice president of the united states four in ten 40 percent of americans surveyed did not know or got it wrong forty percent of the american people did not know the name of the vice president of the united states
it goes on two out of three could not name the majority leader of the us senate robert dole who will be likely a candidate for president nearly half 46 percent did not know the name of the speaker of the us house of representatives newt gingrich and on and on it goes
it seems to me when 62 percent of the people do not participate in an election when 40 percent of the people do not know the name of the vice president of the us when twothirds of the people do not know the name of their representative to the us congress when many people a majority of the people cannot name their two united states senators it seems to me that we have a serious problem regarding democracy in america if we do not change the circumstances in a variety of ways i fear very much that in the years to come we are going to lose the democracy that we have today
why is it that so many people do not have faith in government and why is it that so many people do not participate in the political process as i said earlier i think that has a lot to do with the belief that most people have that despite all of their problems and all of their needs that the elections do not mean much because the people who are elected end up not representing ordinary people but end up representing the wealthy and the powerful
unless we can create a political revolution in this country by which government begins to stand up not just for those people who have huge amounts of money but for ordinary americans unless we can reaffirm the faith of the american people in the political process and in their government because they see the government responding to their needs i fear very much that we are not going to increase voter turnout or get young people to understand what democracy is all about
i think one of the problems that we have in this whole area is that there has been a tremendous misstatement of reality that has been going on for the last number of years by the corporately controlled media i believe and also by our twoparty system what has been going on is that when people turn on the television and they watch cbs or nbc or they pick up their local newspapers what they are hearing is the economy is booming the economy is growing the economy is doing very very well
then the television people will tell them well gee the stock market is at an alltime high then they will tell them corporate profits are doing very very well this year for the major american corporations then they will say inflation is down and that is very good for the economy the economy is growing and millions of new jobs are being created all of which may be true but it does not bear on the most important economic reality and that is what is happening to the average american
it is not a question of whether the stock market is soaring for the wealthy people who own most of the stocks it is not a question of whether corporate profits are at an alltime high the question is what is happening economically to the average american the corporate controlled media and i think to a large degree there are exceptions the two political parties represented here have not addressed that issue
that reality is that for the average american for the middleclass american the economy of the united states is in a depressionlike situation and i use that word advisedly
the reality is that since 1973 80 percent of all american families have either seen a decline in their incomes decline in their standard of living or at best their incomes have remained stagnant now if 80 percent of american families are seeing a decline in their standard of living or at best economic stagnation how can anybody with any sense of integrity talk about a growing or dynamic economy for the middle class of america we are in the midst of a major depression
when i go back to vermont and i go back to vermont almost every weekend and i talk to the people throughout my state we hold many town hearings on what is going on here in washington we talk to people what do we find i do not think vermont by the way is terribly different from the rest of the country what we find from our dairy farmers in the state of vermont our small farmers they are working 60 70 80 hours a week their income is declining and many of them are being forced off the land
what we find is for many of our working people they no longer work one job at 40 hours a week forget about that that is ancient history nobody works one job at 40 hours a week what they have to do now is work two jobs on occasion three jobs in order to bring home the income that their family needs to survive
mr speaker 20 years ago american workers were the best compensated in the world and when people would say america is no 1 what they were talking about it that for the middle class of this country their wages their benefits their pension plans their health care we were no 1
but something has happened over the last 20 years cbs does not talk about it too much the speaker of the house does not talk about it too much most of the people in congress do not talk about it too much but in that 20year period mr speaker we went from first in the world to 13th in the world and now our standard of living is far behind many of the countries in europe and in scandinavia
one of the very interesting things that is happening and it is a sad statement to make is that increasingly we see european companies coming to the united states and starting factories here there is nothing wrong with that that is a good thing
but why are european companies coming to the united states of america the answer and it really would have shocked our parents or any of us 20 or 30 years ago they are coming to the united states today for cheap labor they are coming to america for the same reason that american companies are going to mexico
in my state of vermont we have skilled workers hardworking people who earn usd7 an hour who earn usd8 an hour and less without benefits who are skilled and hardworking people you cannot get the type of labor in europe you cannot get that type of labor in scandinavia because the wages paid in those countries are much higher so all over america what we are seeing is companies coming to america to hire our people at low wages minimal benefits and i would say that that is a real tragedy that this congress has got to address
mr speaker adjusted for inflation the average pay for fourfifths of american workers plummeted declined by 16 percent in the 20 years between 1973 and 1993 in 1973 the average american worker earned usd445 a week twenty years later accounting for inflation that worker was making usd373 a week
today the reality for the middle class of america is that they are working longer hours for lower wages so despite what cbs or nbc or the new york times may tell us the reality is that for ordinary americans we are in the midst of a severe depression
how many women all over this country we hear a whole lot of discussion about family values here and many of us believe that if a woman wants to stay home and many women do not and that is fine but if a woman wants to stay home with her kids she has the right to do that but what we are seeing in this country now are millions of women forced to join the work force because their family cannot make it with one breadwinner and i think that that is pretty unfortunate
just the other day just last week in burlington vt my hometown i talked to a woman who said that between her and her husband they are receiving eight separate sources of income both of them are working different parttime jobs they have one kid they very rarely have a chance to have the whole family together
that is happening all over america husbands do not see wives wives do not see husbands parents do not see their kids together this is a tragedy and it is a tragedy that the us congress must address
mr speaker as bad as the situation is for middleaged middleclass workers there is another phenomenon going around and going on that deserves a whole lot of discussion as bad as it is for middleaged folks it is far far worse for young workers and this is pretty scary stuff
when we ask why the average american is angry or why the average american is nervous or anxious it is not just that he or she is working longer hours for lower wages that is pretty bad but they are terribly worried about what is going to happen to their kids and i speak as somebody who has four kids
what is happening is in the last 15 years the wages for entrylevel jobs for young men who are high school graduates has declined by 30 percent threezero percent for young women it has declined by 18 percent wages for entrylevel jobs for college graduates have also declined for men
what about young families families headed by persons younger than 30 saw their inflationadjusted median income collapse by 32 percent from 1973 to 1990 what the result of that is is many young people are not getting married they cannot afford to get married young families are not buying their own homes they cannot afford to buy their own homes
most of the new jobs that are out there are not paying working people a decent wage very often they are in the service industry they are flipping hamburgers at mcdonalds they are working in a ski resort they are not jobs that are allowing people to come into the middle class
mr speaker the dream of america what the american dream is about it is a dream that my parents had my father came to this country from poland without a nickel in his pocket and our family never had much money but the dream of what america is about is that as parents you work hard you are prepared to sacrifice so that your kids can do better than you did so that your kids will have the opportunity to have the education that you never had
i think one of the areas of anxiety and panic that so many middleclass families are feeling now is not only what is happening to them it is the great great worry as to what is going to happen to their kids it is not just the kids who do not go to college it is even the college graduates as well
mr speaker we are creating so many lowpaying jobs that right now americans at the lower end of the wage scale are now the lowest paid workers in the entire industrialized world that means if you look at what goes on in europe what goes on in scandinavia many other countries what you are seeing now is that american workers at the low end are now if you can believe it the lowest paid workers in the entire industrialized world and i think that is quite unfortunate
mr speaker the majority of new jobs in this country today pay only usd6 or usd7 an hour they offer no health care benefits they offer no retirement benefits and they offer no time off for vacations or sick leave in fact more and more of the new jobs being created are parttime jobs or temporary jobs
in the state of vermont i hear from people who say well i have to go out and have two 20hour jobs because the local grocery store is not hiring anybody at 40 hours anymore they hire two people at 20 hours so that they do not have to pay benefits or provide health care or any other type of benefits that a fulltime employee might receive many employers now consider 27 hours a week fulltime jobs
in 1993 if we can believe this onethird of the us work force was comprised of quoteunquote contingent labor
the largest employer in the private sector today is not general electric it is not general motors it is manpower inc so more and more of our workers are having to go out and find a job for 2 months they are finding a job for 3 months but they are not having any security on the job they are not moving up the ladder they are working for a couple of months then they are gone no benefits no security then they have to go out and they have to hustle a new job that causes to say the least a great deal of stress for the american work force
mr speaker when we talk about why wages are in decline in the united states and why the new jobs that are being created are primarily low wage part time temporary jobs one of the reasons for that is the major decline in manufacturing jobs in america and the major decline in middlelevel whitecollar management middlelevel white collar management jobs as well
in the past 10 years the united states lost 3 million whitecollar jobs 18 million jobs in manufacturing were lost in the last 5 years alone there is a wonderful word that is out there now in the american vocabulary and that word is downsizing corporate america does not even have the guts to use the word firing any more but they are using the word downsizing
all over this country in virtually every major american corporation we are seeing massive layoffs at a time i should mention when corporate profits are at an alltime high
mr speaker five companies alone ford att general electric itt and union carbide laid off over 800000 american workers in the last 15 years while decentpaying jobs continue to disappear the number of involuntary parttime workers tripled between 1970 and 1993
mr speaker there have been studies that have been done which have shown if you can believe it a correlation between the compensation that the ceos for the largest corporations in america receive and the downsizing that they do it appears that it is a wonderful thing to be rewarded that the heads of att and the large corporations are throwing thousands of american workers out on the street and in return what they get are very large bonus increases and salaries and very positive and beneficial stock options for them
the more workers you can throw out into the street the more money you make and i think this is a very sad statement about the culture of corporate america at the present time
mr speaker when we talk about industries clearly one of the reason is a very simple fact the average american today is working far harder and far more hours than was the case just 20 years ago
mr speaker according to recent studies the average american is now working 160 hours a year more than he or she worked in 1969 160 hours that is 1 month extra that means people are now forced to work not to bring in income to work overtime they are working two jobs working three jobs women are now forced to go out into the work force the number of americans working at more than one job has almost doubled over the last 15 years
now it is important again to reiterate the kind of jobs that are being created president clinton and president bush before him and reagan before him they touted the growth of millions and millions of new jobs well they are right millions of new jobs are being created but what kind of jobs are they are they jobs that people can work at and become members of the middle class or are they jobs that people work at and after 40 hours of work they are further behind the 8ball than when they started
mr speaker between 1979 and 1987 there were over 44 million jobs created that is pretty good 44 million jobs and that is the information that we see in the newspapers that we hear on television millions of new jobs being created that is the good news
but what is the other side of that equation of that 44 million new jobs being created between 1979 and 1987 36 million of them were at povertylevel wages so what you are having is new jobs being created but unfortunately the vast majority of them are at poverty level wages
mr speaker as i mentioned earlier we now have the dubious distinction of being the country in which our lowwage workers are now poorer than in any other country in the industrialized world
mr speaker in addition to that if parents and working people cannot earn a decent living how are they going to take care of their children and the answer is they are not
one of the areas we should be terribly ashamed of and i fear republican policies are only going to make a disaster even worse is that 22 percent of our children today live in poverty we have by far the highest rate of childhood poverty in the industrialized world we have some 5 million children who go hungry every single day
now mr speaker i have talked for a moment actually for more than a moment about what is going on for the middle class and the working people of this country
but there is another reality out there and that is while the middle class is shrinking while more and more workers are receiving poverty level jobs there is another reality that is taking place and it is a reality that we do not talk about enough on the floor of the house and it is a reality that we do not hear about on television virtually at all and that is that today at the same time as the united states has by far the highest rate of childhood poverty in the industrialized world we also by far have the most unequal distribution of wealth in the industrialized world
i know we are not supposed to talk about that that is something that is kind of a little bit dirty and we are not supposed to talk about that here in the congress but the reality is the wealthiest 1 percent of the population in america own over 40 percent of the wealth of america they own more wealth than the bottom 90 percent
further in terms of income distribution that means what people are earning every year the highest earning 4 percent make more money than do the bottom 51 percent prof edward n wolf who is a professor at new york university concluded the most recent study of americas concentration of wealth by saying and i quote
we are the most unequal industrialized country in terms of income and wealth and we are growing more unequal faster than the other industrialized countries
you know i think it is appropriate every now and then that we talk about things like justice like decency terms we do not hear too much on the floor of the house we have got to ask ourselves some basic questions is it just is it right that the wealthiest 1 percent of the population in america owns more wealth than the bottom 90 percent is it appropriate to be seeing in our economy today a significant increase in millionaires and people on top while at the same time more and more people are forced to work for poverty level jobs
mr speaker during the 1980s the wealthiest 1 percent of families saw their incomes rise by 80 percent so for the people on the top the economy is doing fantastically in the same decade the 1980s the bottom 90 percent of families saw their income rise only 3 percent most people saw a decline in their standard of living no wonder that the richest 1 million families today own more than 84 million middleclass working and poor families put together
mr speaker when we talk about economic growth all of us are in favor of economic growth but there is something wrong when the vast majority of that income growth goes to the people on the top the people who make usd200000 usd300000 usd400000 a year meanwhile the middle class is shrinking people work longer hours for lower wages and the jobs available to millions of working americans pay usd5 an hour or usd6 an hour
mr speaker there is another issue that really needs to be addressed because i think it really smacks of obscenity and that is that in 1980 the average ceo in america the corporate the chief executive officer of a major corporation that ceo earned 42 times what the average factory worker earned
today according to recent reports the ceos of the major corporations are now earning 200 times what their average worker is earning
just this last year a report that i saw indicated that the compensation that is salaries bonuses stock options for the major ceos went up by over 23 percent meanwhile workers were getting 2 or 3 percent increases in their income
i think ultimately we have to ask ourselves whether the ceos of the largest corporations need to eat 200 times more than their workers need to spend 200 times more for education for their kids than the average middleclass person should have 200 times more income to take care of their health care needs of their parents than the average middleclass person
i think we have got to bring the issue of justice back home again and say to the ceos of the major american corporations the people who are downsizing all over this country despite recordbreaking profit the people who are taking our jobs to mexico and to china it is wrong it is wrong for you to be earning 200 times what your workers are earning it is your workers who have created wealth in your company and you have got to have a little bit of decency and you have got to share it and you cannot gobble it up all for yourselves and your families there is a limit to the number of automobiles you can have a limit to the number of cars you can have the working people of this country the middle class of this country they also have a right to have health care for their kids and their parents they also have a right maybe to go on a vacation every once in a while they have a right to send their kids to college also
the fact that we have such a grotesquely unfair distribution of wealth and incomes is an issue that this congress must address and it has to address
mr speaker one of the areas that has been discussed a great deal lately is taxation taxation everybody wants tax reform i would just simply point out that according to a study conducted by the house committee on ways and means the top 1 percent of taxpayers saved an average of usd41000 in 1992 over what their taxes would have been at 1977 rates in other words one of the scandals that we have seen is the result of the tax reforms of 1977 1981 and 1986 is a major cutback in the tax rates of the largest the wealthiest people in america and the largest corporations
in fact in 1977 if federal 1977 individual tax rates had still been in effect in 1992 the nations wealthiest top 1 percent would have paid usd837 billion more in taxes which is about onehalf of the federal deficit today
so maybe mr gingrich and his friends would not have had to propose slashing medicare medicaid education environmental protection veterans programs the needs of our littlest kids maybe they would not have had to propose that the wealthiest 1 percent of our people whose incomes are soaring had paid were able to pay had paid their fair share of taxes
mr speaker let me chat for a moment about some suggestions that i and other members of congress are making i am the chairman of the house progressive caucus which now has 51 members and the progressive caucus has been attempting to lead the effort with success against the disastrous policies of the contract with america and mr gingrichs republican party
but more than that we are attempting to come up with sensible solutions that would allow the middle class of this country to expand to grow rather than to see it shrink so let me if i might just suggest six or seven areas that i think this country and this congress should been moving forward in
mr speaker given the fact that most of the new jobs that are being created are low wage jobs many of them are parttime jobs many of them are temporary jobs this congress must raise the minimum wage in my view it should be raised to at least usd550 an hour
mr speaker the current minimum wage of usd425 an hour is in terms of purchasing power 26 percent less than it was 20 years ago in other words our minimum wage workers today are far poorer than was the case 20 years ago
mr speaker we hear a whole lot of discussion about welfare reform and the need for people to go out and work i believe that but i believe that if somebody works 40 hours a week they are entitled to live above poverty i do not think that people are making it at usd425 usd450 or usd5 an hour without benefits you cannot raise kids on those wages
so i think that given the fact that the minimum wage in terms of purchasing power has declined by 26 percent over the last 20 years we have got to have the courage to raise the minimum wage in my view it should be raised to at least usd550 an hour
what is happening around this country because of the failure of congress to act a number of states including the state of vermont are themselves trying unilaterally to raise the minimum wage i applaud that matter but the truth of the matter is the best way for it to be done it should be done in 50 states in america not in 5 not in 10 let us do that let us raise the minimum wage here in congress to at least usd550 an hour
mr speaker the second area that i think we need to take a hard look at is creating jobs right here in america rather than continuing to defend europe and asia against a nonexistent enemy many taxpayers may not know this but our government continues to spend about usd100 billion a year defending europe and asia against whom we are not quite sure it seems to me we should take that usd100 billion being it back home to rebuild both the physical and human infrastructure of the united states of america
mr speaker we can put a heck of a lot of people back to work doing meaningful work rebuilding our roads our mass transportation our bridges our schools our libraries in terms of human needs we can save a whole lot of misery by hiring qualified people in terms of disease prevention getting people out working against smoking against teenage pregnancy against aids
we can save a lot of lives by having people going out into our communities we should be putting more money into head start not less money hiring more people for child care not cutting back on those important areas
so it seems to me that rather than spending usd100 billion a year defending europe and asia bring the money back home put our people to work making the country a richer country improving our physical and human infrastructure
mr speaker there is another area that needs to be discussed which gets relatively little discussion on this floor and that is our current trade policy in my view our current trade policy is a disaster i think that sadly tragically all of us in the house who stood up and said nafta was not going to work unfortunately we were proven right
what we have seen is many tens of thousands of american jobs lost to nafta we have seen a trade deficit grow with mexico we have learned that the mexican government lied to us about the state of their economy they devalued their peso which necessitated president clinton to propose a usd50 billion bailout loan guarantee for mexico which many of us opposed i brought forth legislation on the floor of this house which would have forced the president to come to congress before lending mexico any more money as part of the bailout unfortunately that did not get through the senate
but it is not just nafta and it is not just gatt it is our entire trade policy this year the united states will have a trade deficit of about usd160 billion people say so what what does it mean to me i dont care
let me tell you what it means to you the economists estimate that if a company were to develop a plant in the united states that produced usd1 billion of export exported usd1 billion on product on average that company would be hiring 20000 american workers at decent wages what that means is when you have a usd160 billion trade deficit when you are importing usd160 billion more in goods and services than you are exporting that equates to the loss of 3 million decent jobs
mr speaker just this last weekend i was at a mall in vermont i was just looking around at the goods that were available and went into one of the stores where they were selling televisions and vcrs went into another store selling clothing i would urge americans to do that and take a look at the labels as to where these products are made increasingly what you see is made in china made in malaysia made in mexico made in el salvador
what is going on is that major american corporations have basically deserted the united states of america taken their factories to very desperate third world countries where people have to work for horrible wages
i remember several years ago going to mexico as part of the nafta debate and going into a factory there where it was a very stateoftheart factory a very sophisticated modern factory most of the people there were women who were hardworking good people they were earning usd1 an hour we left the plant and walked a quarter of a mile down the road to see where the folks were living where they were living was in shacks often without running water often without electricity
even worse as bad as the economic situation is in mexico it is of course worse in china when our friends at the nike sneaker company or the other major sneaker companies leave the united states and go to set up their plants in china what they are doing there is hiring workers at 20 cents an hour 20 cents an hour
so what you are doing it is a worthy exercise take a walk through the mall and see where products are manufactured increasingly where you are going to see these products manufactured is in desperate third world countries
mr speaker whenever we have a war our multinational corporations become very patriotic and tell us how much they love america and how much they support the young men and women who are prepared to put their lives on the line defending america they have big parades and are just ever so patriotic
i hope very much that the ceos of the major american corporations would begin to show us their love of this country and patriotism by maybe not running to china and mexico but reinvesting back home here in the united states of america
i think this is an issue i know that the progressive caucus and other members of congress are prepared to address it makes zero sense to me mr speaker that we continue to give huge tax breaks to large corporations who are downsizing their work force despite record breaking profits who are taking our jobs to mexico and china it seems to me those are not the companies that should be receiving major tax breaks
frankly mr speaker it seems wrong to me that given the fact that the us government is the largest purchaser of goods and services in the entire world not just military but many many products that we should begin to think about preferential treatment for those companies and there are many of these companies who are reinvesting in their communities who are hiring american workers at decent wages
we have got to take a hard look at this issue we have got to give support to those american companies that are doing the right thing and they are out there they are treating their workers with respect and with dignity they are showing us their patriotism because they are not running to mexico or china but they are supporting their communities the communities that made them money in the first place
so i think mr speaker we have got to take a fundamental look at our entire trade policy do we build a wall around america no do we think that trade is a bad thing no trade is a very good thing but we want to develop a trade policy which allows us to export roughly as much as we import
we want to have the option of purchasing foreign products there is nothing wrong with that but we need a trade policy which puts americans to work building the goods that we can build so well our workers are second to none in the world if they are given the chance but the truth of the matter is we do not talk about it too often but corporate america is selling out the middle class and working class of this country when they run to china and they run to mexico we need a series of policies to get those companies to reinvest here in the united states of america
mr speaker there is another issue which i am working on which some other members of congress are working on that needs to be addressed and that is that when we try to understand why the standard of living of working people has declined when we try to understand why the middle class is shrinking it is important to understand the correlation between the decline of the standard of living of working people and the decline in the trade union movement in america
it seems to me that if working people are going to get justice if working people are going to earn decent wages at the place that they work at if working people are going to get decent representation here in washington or in their state capitals it is important for the us congress to develop policies which allow working people to join unions if that is what they want to do
mr speaker there are a whole lot of workers who do not want to join a union and that is their right no question about it but in my view and i think the studies indicate this there are millions of working people who want to join unions who understand that workers who are in unions earn significantly more than nonunion workers
the problem right now is that the deck is very much stacked against workers who want to join unions despite the law which is routinely ignored by company after company those workers who are trying to form a union are fired they are laid off they are disciplined workers are terrified they are going to lose their jobs companies threaten workers that they are going to go to mexico or shut down the whole place if a union comes
it seems to me if we are interested in raising income for the working and middle class we need major labor law reform the essence of that labor law reform must be that if 50 percent of the workers in a shop vote to join a union plus one they have a union we need legislation that compels the company to sit down and negotiate in a serious way with those workers
too often in america after workers go through all the blood sweat and tears of forming a union they sit down to negotiate their first contract and the owners refuse to negotiate in good faith and they drag it on and on and the union gets lost it seems to me that should be illegal an owner should negotiate in good faith with a union and if the company does not do that disciplinary action is taken against that company
mr speaker another issue that i think needs to be addressed that is an issue that we hear very very little discussion about on the floor of this house now which is the crisis in health care many of us right now are of course preoccupied fighting against gingrichs massive cuts to medicare and medicaid and other health care programs
what we are trying to do is see that these cuts do not take place to see that elderly people do not have to pay double the premiums that they are paying today in 7 years that we do not see massive cutbacks to hospital general rural hospitals in particular which might close down hospitals
in terms of the cuts in medicaid we do not know what will happen to the elderly people who need nursing homes who will not be guaranteed nursing home care we do not know what will happen to the middleclass families who today can see their parents taken care of well in a nursing home through medicaid but will no longer have that guarantee that that will take place so while we are fighting those terrible cuts we must not lose track of the real need for fundamental health care reform in america
when we talk about health care we have got to understand several basic facts no 1 the health care crisis today is worse than it was 3 years ago when we had this big debate on health care it is not better it is worse today over 40 million americans have zero health insurance more than that have inadequate health insurance these are the people with very high deductibles they do not go to the doctor because they cannot afford the deductible they cannot afford the copayment
furthermore what we have got to understand is that despite the fact that 40 million americans have no health insurance and so many people are under insured that the united states today continues to spend far more per capita on health care than does any other major industrialized nation on earth
we are spending more for a terribly bureaucratic and wasteful system that rewards the insurance companies with huge profits that pays the ceos of the major insurance companies huge salaries that rewards certain doctors with huge incomes that allows our pharmaceutical companies to charge our people in america far more for the same product that they sell in europe or in canada or in mexico so i think we have got to move toward a simple nonbureaucratic health care system which guarantees health care to every american that is what our vision must be when we talk about family values what we must be saying is that every family in america knows that they will be able to go to the doctor of their choice without worrying that they are going to go bankrupt
so mr speaker while we continue the fight against these disastrous cuts in medicare and medicaid we must hold out the vision for a singlepayer stateadministered health care system which guarantees health care to all people and in fact we can guarantee health care to people to every man woman and child in america and we can spend less than we are right now with our wasteful and bureaucratic and inadequate system
mr speaker i think the last point that i want to touch on has to do with campaign finance and election reform and that is as i think many americans understand it is not a level playing field in terms of what goes on here we have our freshman republican class who are revolutionaries and they certainly are mr speaker as i understand it they have broken all of the records from any other class in the history of congress in raising corporate pac money the biggest corporations that is pretty revolutionary i suppose getting far more than any other class has received
mr speaker 29 percent of the members of the united states are millionaires my understanding is 25 percent of the freshman republican class are millionaires millionaires and it seems to me mr speaker that if we do not want to convert the house of representatives the peoples body into a house of lords we need some pretty fundamental campaign finance reform which takes away the ability of big money interests to continue to dominate what goes on here in the congress
let me just briefly touch upon some of the issues that i think must be addressed in any serious campaign finance reform legislation
no 1 we have got to revisit the issue of very very wealthy people being able to buy elections i have nothing personally against steve forbes or ross perot or anybody else so it is not a personal criticism of them but i really think it is unfair that people who are just born or perhaps made hundreds and hundreds and millions of dollars that they have the freedom to get up and say well you know getting boring you know middle age tired of my business career i think that i am going to run for the president of the united states why not let us take out usd25 million usd50 million out of the old checking account no problem we are worth a billion dollars we are worth a half a billion dollars and guess what we will break the monotony we will run for the president of the united states gee that must be a lot of fun
so i have nothing against steve forbes or ross perot they are fine people but i think that we have got to create a situation in which every american has the ability should be able to run for president of the united states should be able to run for the senate should be able to run for the house should be able to run for governor and not just millionaires and what we are seeing is not just on the presidential level do not kid yourself more and more people who are running for the united states senate or running for the united states house of representatives are millionaires the leadership of both parties is soliciting those people it is pretty easy you do not have to worry about raising funds for these guys they are millionaires they will pay for it themselves
and we are seeing this also not only here in washington you are seeing it in state capitals as well millionaires you know became governor of louisiana not so long ago and that is the pattern
is that what we want for america is that what people fought and died for to defend democracy for that we end up having people with huge amounts of money running the government i do not think so i do not think that is right
so i think we want to revisit butler versus valeo the very wrongheaded supreme court decision which basically said gee millionaires and billionaires have a constitutional right a freedom of expression to buy elections i think that is wrong and i think through a constitutional amendment or perhaps rethinking on the part of the supreme court we have got to revisit that issue wealthy people should not be able to buy elections
second of all if we are talking about fairness and elections the most important issue is to limit the amount of money that can be spent in an election and we can argue whether for a house race that should be usd400000 usd500000 or usd600000 but that is the most important thing if somebody has usd2 million somebody has a hundred thousand dollars the guy with the usd2 million is going to win the vast majority of the time no matter how good or bad that person may be so we want to limit the amount of money that can be spent
and third we want to make sure that the money itself is not coming from wealthy powerful interests but from ordinary people and i think what we probably want to do is have a combination of small contributions balanced off against public funding of elections so we do not have to have spectacles of the republican national committee i guess it was holding a fundraiser in washington dc and on one night raising usd16 million and mr gingrich going around the country at usd10000 a plate fundraisers and in fairness it is republicans who do this the democrats do it as well and i think we want to end that type of politics
mr speaker i would simply conclude my remarks by suggesting that this country faces some serious problems there is no question about that but i think those problems are solvable i do not think there is anything that i have discussed today the lack of a national health care system decent wages for our middle class growing out between the rich and the poor i think those problems are solvable but i want to say this those problems are not going to be solved if tens of millions of american people continue to turn off to the political system people fought and died to continue this country as a democracy and we are insulting those people when we say oh politics hey that is all crap they are all crooks i am not going to get involved wrong
and i want to say this also that there are people in congress and in government who really do not want ordinary americans to vote and to participate in the political process because if you only have a small number of people who are voting as in the last election where we had 38 percent of the people then big money can dominate what goes on if ordinary people in the middle class do not participate
so let me simply conclude by saying this country has serious problems but they are solvable problems we can create policies by which the middle class will expand rather than shrink we can create policies by which we do not have usd160 billion a year trade deficit we can create policies which move us toward a balanced budget in a fair way by dealing with corporate welfare and defense spending rather than slashing medicare and medicaid we can improve education in this country and make college affordable for every middleclass and workingclass young person we do not have to continue to have by far the highest rate of childhood poverty in the industrialized world we can address those issues but we will not address those issues unless ordinary people begin to stand up and fight back and make the effort to reclaim this government which belongs to them it does not simply belong to the millionaires and billionaires who have used this government for their own interests it belongs to ordinary americans middle class and the working people of this country
so i hope mr speaker that we can see a reinvigoration of democracy in this country serious political debate about how we can improve life for the vast majority of our people that is my hope and i think if people do that we are going to see some really good changes in this country
 newspeech
mr speaker i thank the gentleman for yielding me this time
i would like to begin by thanking the gentleman from pennsylvania mr gekas and the chairman of the full committee on the judiciary the gentleman from illinois mr hyde for their assistance in ensuring this joint resolution was passed by the committee on the judiciary and placed on the suspension calendar in a timely manner we very much appreciate their willingness to move this matter along so rapidly
mr speaker passage of this legislation is very important to the residents of guildhall vt the vermontnew hampshire public water supply compact is noncontroversial but it is essential passage will allow guildhall to pay its debt to new hampshire and will allow the village of guildhall to update its water transmission lines and provide adequate water services including fire protection to its residents right now only one fire hydrant serves the village of guildhall and more are needed
mr speaker vermonters take pride in meeting their environmental obligations and this will allow the town of guildhall to meet requirements under the clean water act and if this bill passes under suspension today guildhall can start upgrading its water transmission lines and provide improved fire protection on schedule i urge immediate approval of this resolution
 newspeech
mr speaker i thank the gentleman for yielding me the time
mr speaker more women in the united states are injured and killed through domestic violence than by automobile accidents muggings and rapes by strangers combined domestic violence is a terrible plague in american society
given that reality it is an absolute outrage that a number of insurance companies deny health insurance to women who have been battered and who have been victims of domestic violence these insurance companies argue that domestic violence is a preexisting condition and that it might not be profitable for them to insure these women under these conditions women are being abused twice first by their batterers and secondly by the insurance companies who refuse to insure them and their families
mr speaker i am delighted that both the republican and democratic health care bills before us tonight include an amendment which i offered which would once and for all put an end to this outrage women who are battered are entitled to health insurance just like anyone else
 newspeech
included both in the republican bill and the dingell bill as well yes
 newspeech
mr speaker all over this country family farms have been disappearing in great numbers as a result of the failure of our current agricultural policy in vermont in 1977 we had 3300 farms today we have less than 2000 all over the country this is happening this is an american tragedy
in 1989 some people in new england got together to figure out how we could save the family farm in our region and they came up with a concept called the northeast dairy compact this compact could provide dairy farmers in new england finally with a fair price for their product a fair price which they are not getting today it is an opportunity to save the family farm all six legislatures in new england overwhelmingly approved the compact all six governors liberal and conservatives approved the compact
mr speaker originally when we voted on the bill the compact was not in the farm bill but today it is in the farm bill as a result of the work the conferees did mr speaker the northeast dairy compact could become a model for farms all over this country for regions all over this country it is good for new england it is good for america
there is a lot in this bill that i do not support but i certainly fervently support the northeast dairy compact section
 newspeech
mr speaker i would like to say a few words about disturbing trends in contemporary health care and to discuss hr 3222 the hippocratic oath and patient protection act of 1996 which i introduced to halt those trends and protect strong doctorpatient relationships
mr speaker more and more doctors and patients are enrolled with managed care and hmos the wall street journal reports on the financial success of hmos by stating it has left them so awash in cash they dont know what to do with it all
us healthcare for example is a major corporate hmo with 24 million members it makes usd1 million a day in profits is ceo leonard abramson walked away from his companys recent merger with aetna with a personal profit of nearly usd1 billion
clearly there is a builtin conflict between a forprofit hmo and the needs of a patient the less money spent on providing care for the patient the more money the company makes its that simple
obviously mr speaker we must all work to control health care costs however we must also ensure that health care decisions are made by doctors using medical rationale with their patients interests at heart not insurance administrators using financial spreadsheets with their own economic interests at heart and most importantly we must preserve the fundamental core of successful health care the strong doctorpatient relationship
unfortunately with the growth of managed care and the power of large insurance companies serious problems are developing which in my view threaten the doctorpatient relationship
many hmos use what are essentially freefordenyingservice systems which pay doctors for denying care and penalize them for providing it doctors under some plans lose up to 50 cents of compensation for every dollar they order spent on emergency care and according to a mathematica policy research study 60 percent of managed care plans in this country currently place their providers at some financial risk for the cost of patient care this places doctors in very difficult situations as they are asked to base their decisions on criteria that is contrary to what they were taught and swore to uphold
you would have to be patently insane to sign on with an hmo you know is going to pay your doctor not to treat you so some insurance companies are taking steps to make sure you dont know what they are doing they keep their incentive plans secret from their customers and in many cases keep both patients and doctors in the dark about the formulas used to approve or deny coverage therefore doctors and consumers signing on do not know what they are getting themselves into and insurers are free to make arbitrary decisions without outside scrutiny
further many hmo contracts contain blatant gag rules that tell doctors what they can and cannot say to their patients last year for example kaiser permanente of ohio told its doctors not to discuss any possible treatments with patients before checking with the companys consultants
these outrageous clauses strike at the heart of informed consent and health care ethics someone considering an operation should have all the relevant information to make their decision and doctors must be able to provide that information
these problems are serious enough that massachusetts has already passed a law banning gag rules while new york and several other state legislatures are considering bills to deal with these issues before recess i introduced legislation that will take three steps to preserve strong doctorpatient relationships my bill has already been endorsed by consumers union the american nurses association the vermont psychological association the american psychological association the national medical association and the gray panthers
first my legislation will ban outright incentives to deny appropriate care and ensure safeguards are installed so doctors are not placed at substantial financial risk for patient care
second my bill prohibits gag rules and other interference in doctors communications regarding patient care it is the only legislation that safeguards doctors communications with their colleagues and the public as well as their patients
third to ensure neither doctors nor patients are kept in the dark about what their insurer is doing my legislation provides for open honest discussion of practices key to patient care by requiring disclosure of utilization review procedures financial incentives for providers and all services and benefits offered under the health plan
that disclosure may be half the battle because i think no insurance executive will be willing to stand up and defend these outrageous practices once they are out in the open
 newspeech
mr speaker i thank the gentleman for yielding me the time
mr speaker the gentleman is right the minimum wage should have been raised 2 years ago and i had a bill in to raise it to usd550 an hour but the fact that it was not raised then makes it more imperative that we raise it now because the purchasing power of lowwage workers has declined even more so let us move forward today and pass a minimum wage
 newspeech
mr speaker the major crisis facing our country is that more and more we are becoming a lowwage society during the last 20 years the real wages of american workers have declined by 16 percent and more tragically for our young workers the new jobs that they are getting are paying even lower wages than was the case 15 years ago
mr speaker what is also grossly unfair is that while the vast majority of the working people become poorer the people on top become richer and we now have by far the most unequal distribution of wealth and income in the industrialized world if people work 40 hours a week they should not live in poverty a usd425 minimum wage is a disgrace
let us have the courage to do the decent thing the right thing let us raise the minimum wage now bring that legislation to the floor
 newspeech
mr speaker this debate centers on two issues at a time when the gap between the rich and the poor is growing wider when the real wages of american workers has declined by 16 percent over the last 20 years when most of the new jobs being created are low wage jobs parttime jobs temporary jobs we must raise the minimum wage so that if somebody works 40 hours a week they do not live in poverty
second given the struggle that so many working poor are experiencing today why would anybody want to raise the rents that lowincome people have to pay in public housing why would somebody tell the elderly poor who are barely surviving on social security that they must pay higher rents than they are paying today this is a bad rule let us defeat it
 newspeech
mr chairman i want to make two basic points
first of all one of the great crises in america today is that millions and millions of low income and working people are spending 40 50 60 yes 70 percent of their limited incomes on housing therefore they just do not have the money available for the food they need for the transportation they need and maybe to put away a few bucks for educational opportunities for their kids that is a real crisis
the second point that i would make is to try to put this discussion in human terms i called up a housing authority senior citizen housing authority in vermont this morning they told me that many of the seniors in the housing earn usd8000 a year on average from social security right now they are paying 30 percent of their income for rent usd2400 a year
mr chairman if this proposal that is in the bill goes through what could very easily happen is that senior citizens bringing in usd8000 a year will now pay 40 percent of their income in housing that is an additional usd800 a year when you are bringing in usd8000 a year ten percent of all of your income more now goes for housing
second of all if their medicare proposals go into effect and we raise the medicare premiums for senior citizens in a few years time we will be talking about those same seniors paying usd500 a year more for medicare premiums usd800 plus usd500 usd1300 a year more on a senior citizen earning usd8000 a year on social security
meanwhile we are talking about huge tax breaks for the wealthiest peole in america mr chairman this proposal in the bill is unfair it constitutes a war against many senior citizens
 newspeech
mr chairman i offer an amendment
the chairman the clerk will designate the amendment
 newspeech
mr chairman i yield myself such time as i may consume
mr chairman let me begin by thanking the republican leadership here because my understanding is that they will be accepting this amendment in truth this is a tripartisan amendment it has support from the gentlewoman from new jersey mrs roukema and other republicans as well as many democrats
mr chairman many members of congress have criticized public housing and believe that americans should be able to decide for themselves how best to spend their housing allowance these critics should be supporters of the section 8 program but this laudable program is not reaching everyone that is eligible for assistance
about 15 million people receive section 8 assistance but close to that same amount are on the waiting list in my home state of vermont the waiting list is over 3 years long in some areas applicants wait for more than 10 years
unfortunately this bill contains a provision that would make these waiting periods even longer i am talking about the major cuts in fees for administering the section 8 program if the bill had been in place for fiscal year 1996 housing authorities would have received on average 23 percent less to administer the tenantbased section 8 program nationally according to hud we are talking about a usd182 million cut in section 8 administration
in california alone that cut amounts to almost usd30 million in new york over usd21 million and in new jersey over usd7 million in my small state of vermont we would lose usd318000
mr chairman the simple fact of the matter is that this cut goes far too deep if we believe in section 8 housing then we must allocate enough money for the program to be administered effectively otherwise we are killing this program through a backdoor method and i do not think that that is what the majority of members want to do
there is widespread support for section 8 and i do not believe that anyone really wants to hurt it this bill provides for a twotier formula where public housing authorities get a fee based on 65 percent of fair market value for the first 600 units and 6 percent of fair market value for the rest
mr chairman this is a huge cut from the fiscal year 1995 and fiscal year 1996 when fees were based on 82 percent of fair market value hud estimates that over 90 percent of the agencies that administer section 8 housing will lose more than 15 percent of their administrative funds on average it will be an estimated 23percent cut per agency
mr chairman i am offering a compromise amendment that puts the fee level about halfway in between where the funds are today and where they would be under the provisions of the bill the twotiered formula would remain but the 6 percent number would be raised to 7 percent and the 65 percent number would be raised to 765 it is a compromise between the 82 percent formula used today and the 6 and 65 percent levels recommended in the bill
mr chairman i should point out that that is the formula recommended by hud and hud supports this amendment the national association of housing and redevelopment officials are also strong supporters of this amendment
frankly mr chairman many of us had believed that this amendment was going to be accepted as part of the managers amendment and we were surprised that it was not if it is adopted fees would still be cut an estimated 105 percent that is a big cut that is a major cut but it would not devastate the administration of the program as the proposed cuts do this is a compromise position and my hope is that it would be supported by all members
mr chairman let me conclude by stating that every state in the country is severely affected by the provisions stated in this bill it provides for an estimated 23 percent cut in section 8 administrative fees that is much too high
mr chairman i urge my colleagues to please support the compromise position and vote yes
mr chairman i reserve the balance of my time
 newspeech
mr chairman i urge the passage of this amendment and i yield back the balance of my time
the chairman the question is on the amendment offered by the gentleman from vermont mr sanders
 newspeech
mr speaker this is really an incredible rule what we have been hearing for the last year and what we will be hearing shortly is that the leadership here in the congress thinks that we should make savage cuts in medicare force elderly people who do not have the money to pay more for premiums meanwhile they are suggesting that we spend usd13 billion more for the military than the president wants do not you think the american people are entitled to that debate on priorities the republican majority wants to savage medicaid 88 million people will no longer have health insurance children will be without health insurance elderly people will be unable to pay for their prescription drugs
mr speaker i ask dont you think we should have a debate as to whether or not we cut military spending or we salvage medicaid
i think the american people want that debate
maybe they will agree with our colleagues maybe they think we should spend more money on star wars and b2 bombers and less money on health care maybe our colleagues are right i do not think they are but i think that is a debate that we should have
mr speaker all over america middleclass families are desperate in vermont they are knocking their brains out trying to figure out how they can afford to send their kids to college meanwhile the republican leadership is cutting back on loans and grants
i think the american people the middle class of this country has a right to decide whether we put more money into education or whether we continue to spend a hundred billion dollars a year defending europe and asia against a nonexistent enemy
mr speaker some of the cuts that have been advocated here by the republican leadership are cruel they are unnecessary it seems to me that before we go after nutrition programs for children we take a hard look at the military budget we have a right to have that debate
defeat this rule
 newspeech
mr speaker i thank the gentleman for yielding
mr speaker let me begin by expressing my disappointment at the vote that took place in the house today regarding the defense budget it seems to me that in a time when speaker gingrich and his colleagues are talking about the need to move this country toward a balanced budget and are talking about the crisis of our deficit situation that it makes no sense for the republican leadership to be proposing a defense budget which is usd13 billion more than president clinton has request
i find it especially hypocritical that at a time when the republican leadership is saying that we have got to balance the budget and to do so we must make major cuts in medicare major cuts in medicaid major cuts in education and veterans programs and environmental protection in the fuel assistance program and so many programs that the middle class and the working class of this country depend upon lowincome people depend upon that at the same time mr gingrich says well it is ok that we spend billions more for b2 bombers that the pentagon does not want that we start spending billions of dollars more for the star wars program
the budget of the us government is what this country is all about and i think it is a sad day that we are saying that it is appropriate to spend billions more on the military despite the end of the cold war that we are prepared to put approximately usd100 billion into defending europe and asia although we do not even know who the enemy is now but we do not have enough money to take care of our senior citizens who are in need we do not have enough money to take care of our children
this country has by far the highest rate of childhood poverty in the industrialized world 22 percent of our kids in poverty we do not have enough money to help them we do have enough money to build b2 bombers and star wars and things that the pentagon does not even want i think that is a very sad state of priorities that mr gingrich and the republican leadership are expressing
mr speaker what i want to concentrate on today is what i think is the most important issue facing this country and that is the state of our economy and my very great fears that this country in many ways is moving toward an oligarchy which is a nation controlled by relatively small numbers of very very wealthy people
what is going on in this country today is that since 1973 80 percent of all american families have seen their incomes either decline or at best remain stagnant what is going on in my state of vermont and what is going on all over this country today is that we are seeing working people work longer hours for lower wages these families look to the future they are extremely worried about what is going to happen to their kids because it appears very likely that for the first time in the modern history of the united states our children will have a lower standard of living than we will have
mr speaker this in my view is the most important issue facing this country i get very disappointed as an independent as the only independent in the congress that we do not see enough discussion here on that issue certainly from the republican leadership we must have more of that discussion what is also going on in this country is not only is the middle class shrinking but we are seeing another phenomenon that should be of concern to all people that is that the wealthiest people in this country are becoming much wealthier at the same time as the middle class is shrinking
we are looking at a schizophrenic economy how bad is the situation today facing the working men and women of this country let me just make a few points again these are points i think that should be made over and over again twenty years ago the workers of the united states were the best compensated in the entire world we were no 1 today depending upon the study that you might look at american workers rank 13 among industrialized nations in terms of compensation and benefits
in fact one of the great ironies of the current economic period is that we are seeing companies from europe and elsewhere come to the united states in search of quote unquote cheap labor in my state of vermont and throughout this country you can get hardworking individuals who must work for usd6 or usd7 an hour those are wages that large companies cannot get workers to work for in europe so we are seeing for certain european companies the united states becoming what mexico is for american companies that is a very sad state of affairs mr speaker adjusted for inflation the average pay for fourfifths of american workers plummeted by 16 percent in the 20 years between 1973 and 1993 in other words whenever you turn on the television or whenever you read the newspapers they talk about the booming economy the economy is booming for someone but it certainly is not booming for the middle class or the working people of this country
between 1973 and 1993 the average pay for fourfifths of american workers plummeted by 16 percent people are working for significantly lower wages in 1973 the average american worker earned usd445 a week twenty years later that worker was making usd373 a week that is the issue that should be debated here on the floor of the house should be debated in the senate every single day should be debated all over this country how did we go from 1st to 13th in the world in terms of the wages and benefits our workers received how did it happen that the middle class is shrinking how did it happen that real wages are declining that is the usd64 issue that should be addressed by the president by the leadership of the republican and democratic parties mr speaker as bad as the situation is for the middle class and middleage middleclass workers the situation is far worse for young american workers in the last 15 years the wages for entrylevel jobs for young men who are high school graduates has declined by 30 percent young families headed by persons younger than 30 saw their inflationadjusted median income collapse by 32 percent from 1973 to 1990 young families headed by someone between 25 and 24 these are young american families had incomes usd4000 lower in 1991 than they did in 1979 their entrylevel wages were 10percent lower in 1991 than in 1979
what all those statistics mean is that for young people graduating high school going out into the job market the wages that they are earning are significantly lower than was the case just 20 years ago so as bad as the situation is for middleage people it is a lot worse for younger people that is an issue that we must address and analyze and correct americans at the lower end of the wage scale are now the lowest paid workers in the entire industrialized world one percent of american workers with fulltime jobs are paid so little that their wages do not enable them to live above the poverty level
mr speaker we hear a whole lot we heard it from president reagan we heard it from president bush we are hearing it from president clinton about all of the new jobs that are being created the sad truth however is that the vast majority if the jobs being created are lowwage jobs these are the jobs that pay workers usd6 an hour they pay workers usd7 an hour they often bring no health care benefits no retirement benefits and no time off for vacations or sick leave
also one of the frightening aspects of the new economy is that more and more of the new jobs being created are parttime jobs or temporary jobs what we are seeing is that many employers would rather hire two people for 20 hours a week or for 30 hours a week rather than one worker for 40 hours a week because the employer does not have to pay any benefits
in fact in 1993 onethird of the united states work force was composed of quote unquote contingent labor and that is temporary labor that means that you get a job for 4 months for 6 months and then you have to go out looking for another job again there was a time not so long ago in our history when a real job meant 40 hours a week with benefits decent health care perhaps retirement that you moved up the ladder if you did your job well you made more money you had a certain sense of job security
it seems that those days are ancient history as many of the new jobs that are being created are parttime jobs or temporary jobs in the past 10 years the united states has lost 3 million white collar jobs and 18 million jobs in manufacturing just in the past 5 years five companies ford att general electric itt and union carbide alone have laid off well over 800000 american workers in the last 15 years meanwhile while the decentpaying jobs continue to disappear the number of involuntary parttime workers tripled between 1970 and 1993
people might be surprised to know that the largest private sector employer in the united states today is not general motors it is not general electric it is not ibm it is manpower incorporated they are the leading supplier of temporary employees
now one of the tragic results of declining wages in america is that the average american worker is now working significantly more hours than used to be the case the number of americans working at more than one job has almost doubled over the last 15 years so if the average american thinks my god i am the only person who has to work two jobs or three jobs wake up it is your neighbor doing that it is people all over this country because as real wages decline people are just scrambling as hard as they can certainly in the state of vermont it is not unusual to see people working two jobs three jobs just to pay the bills
furthermore when we talk about things like family values i think what many of us mean is the ability of a husband and a wife to spend some quality time with their kids i remember seeing a constituent of mine in burlington vt who told me she was shopping at a grocery store that she was working three parttime jobs her husband was working four parttime jobs they hardly ever had a chance to be together or let alone to spend time with their child
that is what is happening all over this country not only are people working longer hours in fact the average american is now working approximately 160 hours a year more than was the case just 20 years ago but what we are also seeing is that more and more americans are lacking adequate medical insurance
we had a major debate here on the floor of the house several years ago about the need for a national health care policy those of us who advocated the right of all americans to have health care as exists in virtually every other industrialized nation on earth we lost that debate the result is that 3 years later we are seeing more and more americans not only without any health insurance but we are seeing more americans who have inadequate health insurance by that i mean very high premiums large deductibles large copayments the situation is such that many people even when they are sick hesitate to go to the doctor because they just cannot afford the bill
in terms of home ownership which is a key part of the american dream that home ownership is also in rapid decline for the average american worker as a result of lower and lower incomes an increasing number of young americans can no longer afford to purchase their own homes in 1980 21 percent of americans under 25 owned their own homes in 1987 only 16 percent did the answer is obvious if you are not making decent wages there is no way you are going to be able to put a down payment or pay the mortgage on a home
mr speaker while the middle class is in decline or the real wages of american workers are going down or while many of the new jobs are being created to pay people usd450 usd5 and usd6 an hour there is another aspect of our economy that must be addressed that is clearly not everybody is hurting some people are doing very very well
this is an issue we just do not talk about enough i think on this floor of the house and certainly the media does not talk about it enough today the united states has the dubious distinction of having the most unfair distribution of wealth and income in the entire industrialized world i think many of us used to think that in countries like england where you have queens and dukes the lords and barons that those were really class countries that you had a ruling class and an upper class and you had a lower class but the truth of the matter is that the united states of america today has a much more unfair and unequal distribution of wealth than england we have a much more unequal distribution than any other country on earth hardly ever talked about this issue but we should
what is going on now is that the wealthiest 1 percent of the population own 40 percent of the wealth in this nation that is more wealth than the bottom 90 percent
the richest 1 percent own more wealth than the bottom 90 percent and that gap between the rich and the poor is growing wider
but it is not only wealth we also have the most unfair distribution of income in the entire industrialized world the highestearning 4 percent of our population make more money than do the bottom 51 percent
mr speaker from 1979 to 1995 household incomes in the united states grew by usd800 billion in real terms but 50 percent of that sum went to the wealthiest 5 percent of households and 97 percent of it went to the wealthiest 20 percent the remaining 80 percent of families scrambled for the crumbs divvying up just 3 percent of all income growth between them
so in other words when we talk about the growth of the economy what we should ask ourselves is who is gaining that income and what is clearly going on is the lions share the overwhelming amount of the growth in income is going to the very very wealthiest people while the vast majority of the people are seeing a decline in their real incomes
mr speaker there are a number of reasons why the united states is seeing a decline in its standard of living for its middle class and for its working people and i think one of them certainly has to do with the decline in our industrial base a decline of manufacturing in the united states of america i would urge members of congress just to go to their local department stores in virtually any part of america and check the labels on the products that they are observing and more and more what we are finding is that products are not manufactured in the united states but they are manufactured in the far east they are manufactured in malaysia more and more they are manufactured in china and we are not just talking about cheap products but we are talking about topoftheline products as well
and the reason that more and more products are being manufactured in china is that american companies are beginning have invested tens and tens of billions of dollars in china in malaysia in latin america in many other very poor third world countries
so the good news is that corporate america is creating millions of new jobs every single year the bad news is that they are not creating those jobs in the state of vermont or the united states of america they are creating those jobs in china and in malaysia and in latin america
now why are these companies running to these countries well it does not take a phd in economics to figure it out they are going to china because workers in china receive 20 cents an hour there are workers in china who are 12 or 13 years of age making products that we in the united states are purchasing and mr speaker i might mention that i have introduced legislation which would prohibit the importation of products made in any country that is made by child labor there are children in china children in india children in pakistan who are 10 11 12 years old who are working for minuscule wages who are doing the work that american workers used to do
it is no secret that this year we will have a trade deficit of about usd160 million that means we are importing usd160 billion more in goods and services than we export that equates to about 3 million decent manufacturing jobs
mr speaker in my view we are not going to expand the middle class we are not going to create decentpaying jobs for our young people unless we deal with the trade situation i think the evidence is very clear that nafta has been a disaster as many of us had feared it would be i have very serious reservations about gatt
we need a trade policy that is a fair trade policy a trade policy that protects american workers that allows us to export as well as import
mr speaker if we are going to address the issue of raising wages in america not only do we have to deal with the trade situation not only do we have to become a country again which is building real products here in the united states of america which is using our technology go create new jobs producing real goods but we also have to in fact raise the minimum wage and i am delighted that more and more members of congress are beginning to understand that
a number of years ago i brought forth legislation that would raise the minimum wage to usd550 an hour it was my view and is my view that if somebody in this country works for 40 hours a week that person should not be living in poverty that person should not be more in debt at the end of the week than he or she was in the beginning of the week and when some of us began that crusade to raise the minimum wage president clinton was not on board and many democrats were not on board and virtually no republicans were on board i am happy to say that right now we have a majority support for raising the minimum wage in the house i believe that is the case in the senate as well and i certainly hope that the gentleman from georgia mr gingrich will allow a clean minimum wage bill to come up in the house so that we can vote it in and have the president sign it
the minimum wage today is at its lowest point in 40 years if the minimum wage today was at the same level as it was in 1970 it would be over usd6 an hour so to raise the minimum wage to usd550 an hour as the president would have us do in 2 years is a conservative effort and it is something we should do immediately
mr speaker if we are going to turn this country around i think it is important that we also address the tax situation in this country the fact of the matter is that as the rich become richer as the middle class is shrinking and as poor people are just fighting desperately to keep their heads above water i think what we need to do is take a hard look at progressive taxation and that is to say that the largest corporations who are today contributing significantly less to our national coffers than they did 30 or 40 years ago to the richest people in this country who have enjoyed significant declines in their real tax rates that it is appropriate to ask those people whose incomes are soaring to start paying their fair share of taxes so we can provide some real tax breaks for the middle class and the working people of this country
mr speaker one of the reasons that wages in america have declined is that the trade union movement in america has also declined i think it will not be a surprise to most american workers to understand that employers often do not out of the generosity of their heart pay decent wages they pay decent wages because there are people who are negotiating with them to get them to pay decent wages
one of the concerns that i have right now in this country is that it is harder and harder for workers to be able to form trade unions very often employers will harass those workers who are trying to develop a union they will fire those workers under all kinds of pretenses they will bring in highfalutin consultants to try to frighten workers they will threaten workers that they will go to mexico and asia
i think we need a new set of labor law which says that any worker in this country who wants to join a union should have the freedom without fear to participate in that process and i believe that as we strengthen the labor movement in this country that is more and more workers join unions they will be stronger and be able to negotiate good contracts which will not only benefit them but it will benefit the whole country nonunion workers benefit substantially when we have strong unions because unions drive wages up and employers therefore must pay nonunion workers a decent wage as well
mr speaker i will soon be introducing a piece of legislation which i think is quite important one of the concerns that i have increasingly in this country is the degree to which the taxpayers of our nation are subsidizing large corporations through corporate welfare very conservative groups as well as progressive groups estimate that we spend about usd125 billion every year on corporate welfare which is tax breaks and subsidies that go to some of the largest corporations in america and let me give you just one example of something that i and some of my colleagues are working on right now
it seems to me to be very wrong that when the united states pentagon when our pentagon negotiates with various defense contractors that some of the ceos of those defense companies end up making huge salaries basically at taxpayer expense at the same time as they are laying off tens and tens of thousands of american workers we pay the president of the united states usd250000 a year and it seems to me to be very wrong that the taxpayers of this country should be paying the ceos of the major defense companies substantially more
i think the taxpayers of america should be concerned for example that in 1994 james miller who is the ceo of general dynamics earned usd113 million in compensation now what is interesting is that general dynamics as a percentage of their business does 100 percent of their business with the us government which means that the us government is paying mr miller usd113 million in income and i think that is wrong for at least two reasons
first in terms of our deficit i do not know why we are paying ceos who are 100 percent dependent on taxpayer money over usd11 million a year in compensation that is wrong
but second of all it is wrong as an example as a model of what this congress should be doing one of the more shameful aspects of the american economy at this point is that ceos of major corporations today are earning about 200 times what their workers are making 200 times that is unheard of in the industrialized world it seems to me that the us congress should not be encouraging and supporting that type of economic activity
so we have legislation and i have introduced legislation along with several other members that would say to the ceos of the major defense companies that they cannot earn from the taxpayers of this country more than usd200000 a year in compensation
i should point out once more that the head of general dynamics receives usd113 million and as best we could understand every single penny of that money comes from the taxpayers of this country that does not make any sense we are cutting back on so many programs that working people need and to say yeah we got usd11 million to pay the head of general dynamics makes no sense and i should point out that this very same company has laid off over 35000 workers between 1990 and 1995
so these guys are making more and more money from the taxpayers at exactly the same time as they are laying off tens of thousands of american workers that does not make any sense to me at all
mr speaker i think you know sometimes members of the congress become a little bit obsessed with ourselves and we think that the end of the world is the beltway around here but we should pay attention to the fact that tens of millions of people are giving up on the political process they are giving up on the twoparty system again it is an issue that we do not talk about too much but maybe as the only independent in the congress i can raise the issue and that is there is something fundamentally wrong with the politics of this country when in the last election in 1994 only 38 percent of the people came out to vote 62 percent of the people did not vote
mr speaker there are a lot of reasons for that but i think the major reason has to do with the fact that large numbers of people who are hurting very very badly no longer believe that the us congress represents their interests or is capable of responding to their needs and their pain and they are saying hey politics it does not matter we do not care what is going on in washington we do not pay attention to what is going on in washington because all these people are living in another world
i think given the fact that so many men and women have put their lives on the line have fought and died to defend freedom and democracy in this country it is a very sad state of affairs that the united states has today by far the lowest voter turn out of any industrialized nation on earth
now how do we turn that around how do we create a vibrant democracy where we have 70 to 80 percent of the people voting rather than 40 percent of the people or 50 percent
i think frankly the answer is that this congress has got to show the american people that we do feel their pain that we do understand what is going on in their lives and we are willing to respond to their problems if we do not respond to their problems people are going to say it does not make any difference why do i have to get involved
it is a catch22 unless ordinary people begin to stand up and say wait a second the us congress representing all of the people in this country and not just the very rich in the united states of america we should be able to provide health insurance for every man woman and child as most of the industrialized nations do in the united states of america we should be able to make sure that every young person who has the ability is able to get a college education as many of our industrialized neighbors do that in the united states of america we should be able to create decent paying jobs that unless the people make those demands on the congress and start electing people to the congress who are going to fight for the middle class fight for the working people the congress is going to be unresponsive
that takes us to another issue in terms of how and why the congress is unresponsive that takes us to campaign finance reform clearly there is something very much amiss when increasingly we are seeing in congress in state houses all over america very wealthy people taking out their checkbooks and writing themselves large checks and saying gee i think i would like to run for president it is kind of boring in business now i have a midlife crisis i would like to do something else i will make out a check and then run for the presidency i will run for governor i will run for the senate so forth and so on that is not what democracy is supposed to be about
a democracy is not supposed to be about the democratic and republican parties holding fundraisers here in washington dc i think last month or a couple of months ago the republicans raised usd16 million in one night and recently the democrats raised usd12 million in one night money which is coming from some of the wealthiest people in the united states of america some of the largest corporations in the united states of america some of these guys contribute to both political parties is that what democracy is supposed to be about i think not
i think we must move toward campaign finance reform and the most important aspects of that is we have to limit the amount of money that people can spend in a campaign if you limit the amount of money you take away the advantage of the big money interests they cannot outspend you 10 to 1
i think we have to move toward public funding of elections combined with incentives coming from small donations matching small donations in that way we will have people who are serving in congress who come from the ranks of ordinary people and simply are not hobnobbing with the wealthy and the powerful
most importantly what concerns me is that tens and tens of millions of americans believe the political process does not matter to them they have given up on the political process that is very very sad i would suggest to people and i say this as somebody who was the mayor of a city for 8 years and am now in my third term in the us congress that the only solution basically to that situation is for ordinary people to begin to stand up and fight back and reclaim this country for the ordinary people for the middle class for the working people of this country and inform the us congress that all of us have a right to a decent standard of living and a good life all of our children have the right to a good future that right should not just exist to the very wealthy and the very powerful but that is not going to change unless people get involved in the political process unless people understand what is going on at all levels of government
mr speaker let me just simply conclude by stating that in this great country if democracy is to survive if all of our people are to enjoy a decent standard of living that is not a utopian vision that can happen but people have got to be involved in the political process and have got to stand up and fight for their rights
 newspeech
mr speaker i move that the house do now adjourn
 newspeech
mr speaker i thank the gentleman from massachusetts mr moakley for yielding this time to me and i thank the gentleman from new york mr solomon for placing an order in the committee on the budget placing in order the black caucusprogressive caucus budget which i think is the only real alternative that we are going to be hearing today and is the budget that speaks to the needs of ordinary working americans
when we discuss the budget situation in america today it seems to me to be imperative to ask how did we get where we are today how did we end up with a usd5 trillion national debt is it because we are spending too much on health care so that all americans have health care i do not think so is it because the federal government is spending too much on education so that all american families could send their kids to college is that the reason we have the deficit is it because we are spending too much on affordable housing so that we have no homelessness in america so that people are not paying 40 50 60 percent of their income in rent is that why we have a usd5 trillion debt i do not think so
most economists understand that the reason we are in the deficit crisis we are today is that during the 1980s three things happened first we gave huge tax breaks to the richest people in america and to the largest corporations everybody knows that what the republican budget does today is it says guess what let us give more tax breaks to the richest people in america and the largest corporations that makes a lot of sense
second of all during the 1980s everybody knows this this country spent huge amounts of money on the military tremendous increases in defense spending what the republican budget says is let us spend more money today now that the cold war is over let us spend more money usd13 billion more on defense than the president wants let us build more b2 bombers that the pentagon does not need let us go into that absurd star wars program that is really where we have to go
does that make sense i do not think so
and the third reason that we had we created the deficit situation today is the tremendous increase in medical spending health care spending during the 1980s all health care spending went up including medicare but the question that we have to ask is why is it that the united states of america today we spend far more per capita on health care than any other industrialized nation on earth is it because all of our people have health insurance is that the reason why i do not think so forty million americans have no health insurance millions more have inadequate health insurance
so let us get to the root of the problem what the progressive caucus and the black caucus say is yes let us move toward a balanced budget in 6 years but let us not do it on the backs of the middle class the working class and the lowincome people in this country and we are presenting a real alternative and we hope to have the support of the members in this body
 newspeech
mr speaker for the benefit of my colleagues i would like to have printed in the record this statement by kevin gannon a high school student from brattleboro vt kevin was speaking at my recent town meeting on issues facing young people
years ago when someone wanted to tell you not to waste a resource you might have said hey it doesnt grow on trees today you dont have to be a treehugger to see the irony in that centuryold forests are disappearing and dwindling timber isnt our only environmental concerns when it comes to building materials landfills are fast becoming landfulls and countless other resources are either becoming scarce or are hazardous to process but now a new generation of building products offers alternatives and as concern grows about the environment a vanguard of builders and architects around the country are setting new standards for resourceefficient home construction housing developments for conservationconscious consumers as shown on the poster are being built around the country to identify resourceefficient building material you must ask the following questions do the products use virgin resources efficiently do they make reasonable use of recycled materials will they offer high quality durability over the life of the house and will they help ease pollution engineered wood products can use as little as half the wood fiber of sawed lumber while delivering superior performance made from wood chips and veneers they help to preserve large trees and reduce demand for timber laminated veneer is now taking the place of massive beams the latest efforts in manufacturing engineered lumber focus on making better use of fastgrowing trees and combining them with adhesives that dont emit toxic ureaformaldehyde another corporation makes a product which uses cotton fiber since most paper comes from fastgrowing pulpwood trees reusing it in wallboard wont spare many big slowgrowing species but there are other products that might one is a floorsheeting panel from the belcom technologies the panels only need support at their corners minimizing the number needed on a national level thats hundreds of thousands of joists or trees spared if you really want to save wood use steel using steel studs in your next framing job could reduce the amount of wood used in a house by up to 50 twothirds of the steel that goes into structural components comes from building and bridge demolition auto and industrial scrap however if for some reason steel framing doesnt appeal to you you can buy some doubledipped zinccoated stormguard nails made exclusively from remelted steel recycled glass bottles have become a prime source in fiberglass insulation but an indiana corporation has found a constructive use for scrap automotive glass in fact 70 of its new traffic tiles come from automotive glass manufacturing waste cte glass products makes ceramic floor tiles with waste from its lightbulb manufacturing division there are many ways to recycle wood in san jose ca new world lumber rescues discarded redwood from the gualdalupe landfill after planing off the weathered surfaces it assembles short sections between deducted rails to make fenceboard or they slice longer pieces into lattice the firm also plans to make 24wide fence panels building structures to last longer is another responsible approach to building in the 90s inexpensive materials at this point dont lend themselves to that cheap building materials can also pose health hazards for example inexpensive particleboard releases ureaformaldehyde in your home time allowing i would speak all day about this topic congressman sanders kevin that was a very thoughtful presentation can i ask you a very brief question what impact do you think your ideas would have on the economy in terms of creating jobs and what kind of jobs might be created answer well right now the remodeling business alone is a usd2 billion industry thats just remodeling not even construction which is multibillion and one of my ideas is to create jobs subsidized housing is a problem in america and if we could use recycled resources i think we could kind of kill two birds with one stone congressman sanders so you think this idea could put people to work answer oh definitely congressman sanders good kevin thank you very much
 newspeech
mr chairman i move to strike the requisite number of words
mr chairman earlier today i introduced an amendment which would reduce intelligence spending by 10 percent it received i believe 115 votes just a few minutes ago the gentleman from massachusetts mr frank introduced an amendment that would freeze intelligence spending it received 193 votes
my sincere hope is that we can receive another 20 votes to pass the schroeder amendment i will tell my colleagues why month after month and day after day members from both sides of the aisle come here on the floor of the house and they talk about the usd5 trillion national debt they talk about the deficit and they talk about how important it is and how necessary it is to cut program after program after program my friend from washington a moment ago talked about his fear that we would be disciplining this agency
mr chairman by overspending on intelligence by spending more money than we need for the national reconnaissance office an agency that has misplaced that has lost usd4 billion of taxpayers money what we are doing is disciplining hungry children in america we are disciplining families who would like federal funding in order to send their kids to college we are disciplining senior citizens who get by on usd7000 or usd8000 a year social security but do not have the help from the federal government to purchase prescription drugs
we are disciplining tens of millions of people who would like some help from the federal government but we are saying not only can we not afford it we are going to cut back on what we are currently providing because we think we need to spend this money on the intelligence agencies despite the end of the cold war
mr chairman i know that many members in this body are terribly sincere both sides of the aisle terribly sincere about deficit reduction i know that they have been reluctant and with pain have made cuts that they know are going to hurt millions of middleclass working class lowincome families i beseech those same members who have made those cuts that impact negatively on people who are hurting today to have the courage to stand up and say that when we have an agency that has misplaced usd4 billion the very least we can do to protect our credibility is to say to that agency that we are going to level fund you
the gentlewoman from colorado mrs schroeder did not say we are going to cut their funding in half that is not what she said she came forward with a very conservative amendment level fund an agency that has misplaced usd4 billion we need 20 votes more to finally say to the american people that we are serious about deficit reduction please support the schroeder amendment
 newspeech
mr speaker i thank the gentleman for yielding the time to me
mr speaker the most important economic issue facing this country is that the people on top the wealthiest people are becoming richer the middle class is shrinking and more and more of the new jobs that we are creating in this country are lowwage jobs paying people usd425 an hour usd450 an hour usd5 an hour the minimum wage today in terms of purchasing power is at its lowest point in 40 years
mr speaker the american people want and the working people want to see a situation in which every working person in this country can work 40 hours a week and not live in poverty it is incumbent upon this body to have an opportunity to vote for a clean upordown increase in the minimum wage
our republican friends say the democrats should have raised the minimum wage 2 years ago they are right they should have and i had a bill in the hopper to do just that but 2 years later means more and more people are living in poverty two years later means that today finally we should increase the minimum wage
 newspeech
mr speaker i thank the gentleman from missouri for yielding this time to me
from the first day that i took office here in the congress in 1991 i have been fighting to raise the minimum wage and i hope very much that my colleagues finally are going to do the right thing on behalf of tens of millions of workers and raise the minimum wage today
mr speaker when the minimum wage was first established in the 1930s the opponents then said that the world was going to come to an end the economy was going to collapse and every single time that an effort has been made since then to raise the minimum wage the same cries have come forward the world is going to come to an end we cannot raise the minimum wage
mr speaker the fact of the matter is that today at usd425 an hour the minimum wage is a starvation wage the minimum wage today in terms of purchasing power is 26 percent less than it was 20 years ago in terms of purchasing power it is at its lowest point in the last 40 years
mr speaker there are tens of millions of americans today who are working hard at usd425 an hour at usd5 an hour at usd525 an hour and they are unable to take care of the financial needs of their family they are unable to put away money so that their kids can go to college they cannot go on a vacation every single week despite 40 or 50 hours of work they are in as bad shape at the end of the week as they were before the week began
mr speaker one of the great economic problems facing our country today is that the richest people are becoming richer the middle class is shrinking and most of the new jobs are being created are lowwage jobs
 newspeech
mr speaker the great crisis facing this country today is the decline in real wages for american workers and the proliferation of lowwage jobs we have millions and millions of workers today who are trying to survive on usd425 an hour usd5 an hour they are not making it raising the minimum wage is long overdue and we must do it today
the situation is so bad and the federal government has so much failed to stand up to its responsibility that 10 states in this country on their own including the state of vermont have raised the minimum wage now if the minimum wage is so bad tell the republican governor of new jersey who supports their increase in the minimum wage to roll it back she will not do it because she knows as every other governor knows that it is vital to raise the minimum wage today
lastly it is incomprehensible to me that i am hearing people talk about abolishing the minimum wage they really want to see workers in america earning a dollar an hour usd2 an hour competing against the workers in china who make 20 cents an hour that is not the future of america
 newspeech
mr speaker for the benefit of my colleagues i would like to have printed in the record this statement by matthew dole a high school student from st johnsbury vt he was speaking at my recent town meeting on issues facing young people
my name is matthew dole i face censorship every day as i watch movies try to read a book or even read the newspaper all people have beliefs on what should be censored but those should not infringe on others choices if you are to ban books please do it right but dont force your opinions upon others proponents of censorship base their argument on the first amendment they interpret their freedom of speech as freedom to ban books the opponents also use the first amendment as a major right not to be infringed upon they have the freedom of choice choice to read or watch whatever they want they say that the proponents do not have the right to physically remove the books from our libraries and school shelves people against censorship see it as large government once again challenging the individual as was done in 1919 with prohibition later repealed they ask for more local control at the most local in fact individual decision in this the era of political correctness people challenge books on todays standards they do not historicize texts meaning they dont consider the time or circumstances under which it was written i have with me today three books that have been banned the first one is mark twains the adventures of huckleberry finn to historicize this book it was written in 1884 as twain lived in mississippi and he had previously fought as a confederate in the civil war it was banned for racism and the reason for that was the circumstances under which it was written the second book is catcher in the rye this was banned for sexual scenes i read this last year as a sophomore in high school as part of a classic american literature section the third and last book is margaret mitchells gone with the wind this book was again banned for racism and the reason is that if it hadnt had racism in it it wouldnt have been historically correct it is a book about the confederate south once again and it was also banned for one word as ive said violence racism and sex three touchy controversial subjects are the most common reasons for book banning will banning the books make these issues disappear i say no they may however help educate people on these issues what we must do instead is educate our children early we can teach them to have opinions and teach them why they cant read that book or why they shouldnt read that book as time passes they will be able to handle the issues before being offended also they will be able to personally ban books tv with the vchip and movies with the rating system theres no cut and dried solution to this if a case in book banning or any other censorship were to reach the supreme court they can interpret the first amendment or if twothirds of each house vote in congress they could rewrite the first amendment more specifically and on a more local level if twothirds of all state legislatures wanted to they could call an actual convention and rewrite it themselves thank you for inviting me and i hope something can be done on this issue congressman sanders thank you very much matthew matthew let me ask you a question because you have dealt with a very sensitive and controversial issue so heres my question if at town meeting or better yet a school board meeting a parent gets up and says i read this book it is vulgar it has filthy words in it it has ideas that i dont want my daughter to see i want that book out of the library youre a member of the school board how do you respond to that answer tell her that we can ban the book in a sense ban by putting it in maybe a section like an adult section or a high school section this happened at my old school as a matter of fact and they did not remove it from the library and just put it in a separate section what happened was a 5th grader was basically in the high school section reading this book and i would ask them to educate their kids and i would ask the teachers also need to educate their kids on why they shouldnt read that book at that age congressman sanders in your judgment what is the danger of somebody defining a book and saying this book is terrible i want it out what are the longterm repercussions of that approach answer with these books that ive brought these are classics these are used in teaching if we lose these books we lose a valuable tool in teaching our youth congressman sanders so what youre saying is that what may be vulgar for one person may be a work of art and a classic for somebody else answer thats right
 newspeech
mr speaker for the benefit of my colleagues i would like to have printed in the record this statement by kara von behren sara reeve anna laszewski and alison gingras students from peoples academy in vermont they were speaking at my recent town meeting on issues facing young people
the environment needs a place in the modern high school curriculum without knowledge of the problems that fact us we cannot reverse their ill effect were going to start off with a demonstration could you all stand up please thanks all right were going to start off with a couple of questions and if your answer is no could you please sit back down okay our first question is do you recycle at home our second question is do you compost at home do you turn off the water when you brush your teeth do you reuse your lunch bags or bring a cloth one does your family buy recycled products all right congratulations to any of you who are still standing one or twoyou actually have some part in preserving our environment and helping us to make this world a better place as an american literature class we compiled a survey trying to find out how educated our high school was about the environment we surveyed students in grades 9 through 12 as well as the faculty members the questions ranged from what does epa stand for to how long are your showers to how do you feel about the future of our world we discovered that many students didnt know much about environmental action programs they didnt know the budget cuts would completely eliminate them students and teachers both felt that more time should be spent in the classroom discussing environmental issues and that a definite part of the curriculum should be dedicated to it its not an issue only for science classes it can be brought into every major course of study for example we took a normal letterwriting assignment and transformed it into an environmental letterwriting campaign we wrote letters to our congressional delegation in washington about budget cuts and other environmental issues this is only one of the ways in which weve incorporated environmental education into our classroom without completely altering the basic curriculum and we have these letters and later on wed like to give them to you congressman congressman sander that was a very creative and interesting testimony i think the essence of what youre saying is that if people think in an environmentally conscious way we can have a profound impact on the environment in a broader sense what would you describe as the major environmental crisis our planet faces answer weve concentrated mostly on the budget cuts planned in the environmental action programs such as the epa the drastic cuts would completely eliminate clean water acts safe drinking water the state revolving funds that allow cities and counties to create protective water systems such as sewer treatment plants our second presentation will cover more of that
 newspeech
mr speaker for the benefit of my colleagues i would like to have printed in the record this statement by aj ferriter a high school student from thetford vt who was speaking at my recent town meeting on issues facing young people
for the last few months ive been investigating lobbying in vermont and found much to my surprise and delight a healthy lobbying system the disclosure act purged many of the ills affecting lobbying within vermont by virtually stopping all underhand deals while at the same time not infringing upon our rights as vermonters and us citizens yet we should not be content problems still plague our lobbying system fortunately my investigations have brought me in contact with district representatives and state senators throughout vermont and without leading them on each district representative and state senator i spoke with expressed one common concern lobbying groups using tax dollars to support themselves this is not a problem with profitmaking organizations which is businesses because they support themselves it is a problem among nonprofit organizations whose promoters are given the title advocate instead of lobbyist i am concerned with two issues in the way advocates use tax dollars the first is the use of financial support although many groups use their funds properly many do not instead of using tax dollars to support their cause the money is used to support themselves in other words this money is given to these organizations to support more administrative positions and more lobbyists this money was given to aid a public cause not to support lobbyists tax dollars paying for lobbyists salaries is an alarming issue even if the tax dollars are being used properly is it right in the words of one state senator to use our tax dollars against us i do not believe it is take for instance community mental health a group whose objectives i support hypothetically though lets say i dont if i dont then i am not going to want my money supporting their programs and if i speak out against them they will just use the money i pay the state in taxes to further support lobbyists to speak out against me so the more i speak out the more money i am eventually giving to lobbyists im speaking out against this is not encouraging now im not saying i am completely adverse to advocates and forcing them to have the same nominal status as lobbyists if they did they might not receive the necessary funds they need to stay alive and support the crucial issues that they promote yet if these human service nonprofit groups were forced to have the same status as lobbyists who represent profitmaking organizations then our tax dollars would no longer be used to support their lobbyists i say use our money to support their policies but find donations or something else to support your lobbyists i cannot stop the government from spending my money on programs im not in favor of but i should be able to stop the practice of giving my money to support lobbyists whether i agree with their views or not i believe a line must be drawn somewhere thank you congressman sanders thank you very much aj thats an interesting presentation and its an issue thats being dealt with in congress and in montpelier as well let me ask you a question if i represent the tobacco industry we heard a presentation earlier about the problems of young people smoking and i represent the large cigarette companies that have billions of dollars in resources and i hire some of the most sophisticated lobbyists in the country to knock on the doors of members of congress or in the statehouses throughout this country i have plenty of money to do that okay answer all right congressman sanders and i dont get any taxpayer dollars to do that i do that privately all with the companys own money then on the other hand we have a group of young people say who are concerned about the problems of smoking they also want to lobby one has billions of dollars in resources the other side has very little money how would you deal with that issue so that both sides have a short at having their voices heard answer would it be all right if you rephrase your question i kind of got lost in there congressman sanders okay heres the problem that i want to throw at you he represents he doesnt really but lets say hypothetically hes the head of a large tobacco company philip morris and he has billions of dollars in resources he wants the us congress to not do anything to limit the ability of the tobacco companies to make a lot of money we have another group of young people who are concerned about the impact of smoking on the health of their friends they also want to get involved in the political process they certainly dont have the resources how do you deal with that issue answer i mean thats obviously a concern that i feel is valid but i feel like if the young students are going up against a tobacco giant here they have to have some way of being able to gain support throughout their communities i dont know if it would be sending letters out i dont know if its public speaking im not sure what it would be but it has to be something obviously they cant do it through money and you bring up a good argument to my case but the thing is the tobacco industries do have the money and its a basic right to be able to lobby for what you want and so we cannot restrict that for these students though like i said one of the problems i think with lobbying is that a lot of it is not made public with some of the public hearings weve got happening in montpelier there will be lets say an issue on tobacco and what will happen is that there will be a lobbyist within the room so that he can tell his friends to garner support for the lobbyists and show up at the public meeting and the meeting is only 24 or 48 hours later so that way people dont advocate tobacco dont have the time to just pick up their stuff and find an argument to oppose the tobacco arguments congressman sanders you make a good an interesting point a lot of members of congress and the legislature feel resentful when publicly supported institutions then come and lobby them and thats the point that youre making the other side of the story is that groups that do not need public support like the tobacco industry or the chemical companies they have huge amounts of resources to lobby and in many ways therefore have an unfair advantage in terms of people from the other point of view so those are the two sides of that argument answer yeah like i said in the speech we have to support their cause but i dont feel like if i dont agree with what these youngsters are saying i dont want to have my money going to support their lobbyists fine the cause i cant control that but control the lobbyists
 newspeech
mr speaker for the benefit of my colleagues i would like to have printed in the record this statement by david smith a high school student from brattleboro vt who was speaking at my recent town meeting on issues facing young people
a couple of days ago i heard two men laughing about the term representation of the people claiming that the people are no longer really represented unfortunately im beginning to agree with this because i have a problem believing that a politician is going to think about me the average voter when the oil and gas industry gives congress usd1718 million annually when the health industry gives usd688 million when the banking industry gives usd421 million and thats just to name a few our government is overrun by the influences of special interest groups industries corporations unions and other bit spenders but you the politicians are also stuck because the average house election is up way past usd150000 the average senate election is up past usd35 million and thats average which means that the backwater smalltime candidate has no chance of competing with the mainstream republican or democrat because he or she has no corporate bigspender backing in short our campaign finance system is outdated flawed and full to the brim with loopholes we need changes what im proposing today is a publicly financed system my proposal starts in september 8 months before an election with candidates collecting lowdollar contributions this is the only private money in my whole system and since its lowdollar it doesnt have any influence over a candidate this seen money lasts until the first primary in february to qualify for public financing candidates must receive 20 of the vote parties may also qualify for public financing by getting 20 of the vote between two candidates and sending their most winning candidate to run candidates who receive the required vote will receive an account of money to work with in addition to monetary funds the candidates will also receive radio and tv space but instead of getting 45 second soundbites candidates will receive 15 minute blocks almost like informercials and this will force them to really discuss in detail their platforms and you can also get debates going and really educate the public lastly government will pay for one or two mass mailings per candidate winners of the national conventions will then be given money to campaign in the general election now how are we going to pay for this the working group on electoral democracy estimated that a plan similar to this one would cost between usd5600 million annually and ive devised two ways to pay for this the first is a usd6 flat tax on every taxpaying american unfortunately the taxpayers dont really want another tax so an alternative plan would be a one dollar checkoff box on tax returns for congressional funding right next to the one for presidential funding also a one percent shift in funds from the military to campaign finances and last the first national lotto game vermont alone already receives around usd23 million in funds from various lotto games and i think that a national lottery game could bring in at least half of the money needed to fund this campaign system what this system will do is it will do 5 things first it will stop all public legislation from being influenced by the wealth of industries corporations all private money will be taken out of politics second it creates a level playing field for all candidates rich and poor candidates must have the same chance of being elected as everyone else and voters must start to elect candidates on their merits not on their money thirdly it allows politicians to spend their time at politics not at campaigning ive heard that politicians spend between anywhere from 4080 of their time campaigning in this system they would spend 5 of their time campaigning fourth it will allow politicians to get in touch with what voters want not what the heavy contributors want if they have to go to the grassroots to get their support then there will be more talk about what the real voters want finally it closes down all loopholes so that no private money can influence the private system and we will return to the ideal of representation of the people congressman sanders thank you applause youve touched on a very important issue david let me ask you a couple of questions first give us some examples if you might of the role that big money plays in influencing politics influencing legislation do you have any examples that you might be able to provide answer sure a little while ago the legislature allocated money for the pentagon to build new bombers this was in spite of the pentagon saying that we dont want any new bombers we dont want the money the reason the money was allocated was because of the influence of the corporations that make and help produce those bombers they have such power with their monetary funds that they can almost shape the way legislation works congressman sanders youre absolutely right that is a very good example let me ask you the second question recently the speaker of the house went on a tour around the country and he spoke at usd10000plate fundraising dinners usd10000plate to have dinner with house speaker gingrich why would anybody pay usd10000 it was a very good dinner no doubt but other than the good quality food and you think of another good reason why someone would want to go to dinner with the speaker for usd10000plate answer sure it was influence by paying usd10000 to a candidate you get influence over that candidate so they will better represent what you want an example if i was a politician and i came back to the office one day and there were 14 messages for me 13 were from people i never heard of and the last was from someone who has paid me usd10000 at a local charity the week before the first person im going to call back is that big payer so by paying lots of money we get more influence
 newspeech
mr speaker for the benefit of my colleagues i would like to have printed in the record this statement by alison carpenter a high school student from brattleboro vt who was speaking at my recent town meeting on issues facing young people
children suffer at the hands of adults they are beaten kicked and burned with cigarettes they are thrown against walls scalded by hot water forced to stand in freezing showers until they pass out and thrown into running washing machines they are sexual molested they are the victims of incest child abuse is a serious issue that has existed throughout history and it is a major concern in our society today it was not until recently that it was recognized as a medical and social problem child abuse takes many forms it occurs whenever an adult inflicts intentional physical emotional or sexual harm to a child there are many steps that are being taken to prevent all three forms of child abuse but it will take a long time to fully eliminate it the most common form of abuse on children is physical abuse there are approximately 200 cases of physical abuse for every one that is reported according to the national committee for the protection of child abuse a child is the victim of physical violence even if the anger is not directly inflicted on him or her a witness to violence is a victim of violence children who witness abuse at home can suffer in unseen ways quite often their pain goes untreated they can suffer serious emotional damage children need a large amount of emotional support from their families some elements of emotional support include respecting the childs needs tolerating his or her feelings and a willingness to learn from their behavior our emotions are one of our basic powers emotional abuse is form of psychological battering a childs earliest needs in life are for a warm loving person to be there to mirror echo and affirm them children need to be prized and applauded during their growth achievement if they do not get this they will do whatever they can in order to these are basic needs children need to be allowed to feel and to express what they feel if a child is controlled compulsively he or she will become an adult with a compulsive controlling and immature childish side some forms of emotional abuse are yelling belittling criticizing humiliating and comparing a child incest or sexual abuse is the most shaming of any form of violence because it involves the betrayal of a supposed loved one it leaves a longterm effect on the child believing it was their fault it is a violation of the childs sexuality some people believe that children may simply imagine episodes of sexual abuse an argument that is made by psychologists is that a young child who has not been molested has not had the experience of with sex to be capable of imagining such an experience the profile of an abuser tends to be very similar in all forms of child abuse an abuser is usually someone who has a poor selfimage someone who lacks sensitivity to others feelings a person who abuses was most likely abused themselves or was deprived of basic parenting an abuser has usually had unmet needs for love or comfort and is in denial about his or her feelings child abuse has a tremendous impact on victims throughout their entire lives they tend to feel helpless and out of control the victims tend to dissociate from all of their memories and begin to feel a sense of unreality they may have recurring nightmares sleep disorders or flashbacks as they get older they tend to develop split or multiple personalities they have little or no interest in anything trouble completing thought processes confusion and no trust a girl who has been abused sexually grows up to believe that she cannot escape being battered a boy who has been abused may believe in male supremacy boy victims more than girls identify with the offenders and sometimes become offenders this bonding with abusers is a way to overcome the feeling of helplessness and powerlessness there are many actions that are being taken against child abuse fortynine states except for wyoming have established childrens trust funds all of the states have a child protective service system which is a government system that is responsible for investigating reports of child abuse or neglect this however lacks the resources needed to respond to overwhelming number of reports all states also have a child abuse reporting law requiring reports of suspected abuse to be made by specified professionals or others whose work brings them in regular contact with children there are also family preservation services that are working to make it so the child can remain with his or her family as long as his safety is not endangered private organizations such as mothers against raping children and mothers of sexually abused infants and children serve as support groups and fight for the rights of children we can help ourselves is another organization that teaches children how to handle abusive sitations there are still many things that need to be done to help prevent the abuse of children states need to provide even more services for victims of abuse they need to look at the quality of foster care because it needs to be more than just giving a child a place to live people need to be more alert to possible child abuse cases and to report them to the authorities those in a position of authority need to investigate quickly and accurately and take the first steps to remove children from abusive situations we need to bring offenders to court and most of all we need to stop abusing our children bringing a child into the world is a huge responsibility that two people make and must live with forever parenting is not temporary it is not something that can be put aside when a person wants a break child abuse is an awful cycle which travels from generation to generation as author john bradshaw says it is about the silence of nights spent holding in screams holding back tears holding in ones very self one of the main causes of child abuse is the lack of education that this country has there are so many people who do not know what is morally right from wrong children should be able to look up to their parents without fearing them our society as a whole needs to work together to stop child abuse the first step in this long fight is to educate now is not the time to cut back on education services one quote i believe summarizes the issue is if a child live with criticism he learns to condemn if a child lives with hostility he learns to fight if a child lives with shame he learns to feel ashamed thank you congressman sanders thank you alison alison has touched on one of the most tragic issues that we as a country face and that is that there are hundreds and hundreds of thousands of little kids and teenagers who are being abused and as alison indicated that type of abuse has a profound impact on ones ability to grow up to be a mature adult alison let me just ask you a couple of questions why is it do you think what goes through an adults mind a grownup persons mind who would abuse a child what motivates somebody to do that do you think answer i think that an abuser is one who has a lot of inside pain and they dont know any other way of handling their pain and they take it out on children because children dont really fight back children theyre the only thing that they can take it out on because they dont have the ability to fight back congressman sanders i think thats a very good answer let me ask you another question very specifically and i know you touched on this a little bit what can we do to ameliorate that problem what do we as a community as a society as a country how do we address that tragedy answer i think that the most important thing is to educate not only the abusers but the children of abusers so that children know that this isnt right that what they have been growing up in is not a good situation because quite often children who are abused grow up to be abusers because they dont know any better and i think we need to educate them and show them that this isnt the way that our society should be congressman sanders alison thank you very much for that excellent testimony
 newspeech
mr speaker for the benefit of my colleagues i would like to have printed in the record this statement by amy decarmine amber johnson beth carmine and nathina roy high school students from vermont who were speaking at my recent town meeting on issues facing young people
the summer youth employment program has kept us off the streets kept us from the use of drugs alcohol and violence of any type the youth program has also given us the experience of how living can be in real life and how to conserve money for our future there can be nothing more positive in our lives than the summer youth employment program it is a continuance of encouragement of being a responsible young adult the summer youth employment program has been the cause of so many young people being employed and in some cases it may help to slow down the cycle of the welfare generation it gives us ideas training for possible future employment and it gives us a better idea of what we need to do to accomplish our future please help us to keep what has been proven to be a wonderful chance to understand what is expected from us as adults it has given us a great source of selfpride in our abilities to contribute and know what this is that this is a great start in life and with your support in us you have also given us hope that you believe in all of us we need this opportunity to prove that we are serious about our future and need your consideration to allow us this program to continue thats it congressman sanders can you tell us what kind of work you did in the summer youth employment program answer ive been on the youth program for two years now and the first year i worked at project independence which is helping elderly people take care of themselves and entertain them and last summer i cleaned the elementary school of williamstown to get it ready for school answer ive only been in this for one year which was last year but i worked with amber at the elementary school cleaning and i thought it was a really great thing because a lot of people that are inexperienced like under 16 employers dont want to hire you answer i worked at a hospital as a spot clerk in the basement and i was in the program for one year it was a very good skill because im going to be working at a hospital after i graduate congressman sanders so i think what youre seeing here is an example of a federal program which meant a lot to you three and to tens and tens of thousands of other young americans answer yes congressman sanders there is a major debate taking place in congress right now as to whether this fund with this program should continue to be funded i prefer strongly that it should but were fighting against people who prefer to put money into airplanes and bombers that the pentagon doesnt need rather than in programs like this so i thank you very much for personalizing one of the major debates taking place in congress thank you
 newspeech
mr speaker for the benefit of my colleagues i would like to have printed in the record this statement by tedra guyett and harmony hariman high school students from washington county vt who were speaking at my recent town meeting on issues facing young people
my name is harmony hariman and im a student area coordinator for amnesty international and im an intern with the vermont refugee assistance my name is tedra guyette im also a student area coordinator for vermont and im also an intern with vermont refugee assistance the first thing we really want to talk about is a bill that was passed on march 21st in the house hr 2202 thats really upsetting to me because it effectively banned people who have genuine political claims from ever entering the us or ever staying because when a person flees their country with the fear of safety and they come here do you think theyre going to come off the plane and say whoops we better get a lawyer and file some papers no theyre worried about their safety and often spend months trying to save their families and just getting settled which pretty much bans them from ever filing the bill hr 2202 would ensure that any immigrant or refugee who is caught entering or crossing the border illegally will be permanently barred from ever legally entering the country yet there are instances where a refugee has no other option immigrants lack knowledge about specific opportunities of become a citizen and often do not understand the process itself or how to negotiate the ins bureaucracy there is a section of the bill that says that immigrants who are here illegally or undocumented immigrants are unable to receive health care through out public system and theres a myth that the reason that the health care system is in the ground is because its the immigrants its the poor people that are just dragging it down but really immigrants use health care less than the general public a 1992 us dept of justice report found that immigrants use federally funded services less than the general population and there have been several studies that show that its not immigrants that are pulling us down its everyone elses stupidity and ignorance about the immigrants most hospital care costs for undocumented immigrants were paid by private insurance which was 47 or by the immigrants and their families themselves which was 45 that only leaves 8 of the immigrants in the country that were paid for by the government this bill would also deny immigrants benefits under any meanstested programs funded by the federal government or by state government as well as being ineligible to receive grants to receive earned income tax credits to receive ssi benefits medicaid food stamps housing assistance unemployment benefits college financial aid among others although undocumented immigrants alone paid usd7 billion per year in taxes in 1990 undocumented immigrants paid usd27 billion in social security and usd160 million in unemployment insurance and this is according to a publication by the urban institute signal im just going to do a quick story about a man that i know hes 18 years old and is from sudan and is now lost in the war zone of the american immigration system hes actually living in woodbury he is apparently some kind of dangerous criminal because he came here with a false passport so now hes in jail and the first time i met him i asked him well how long will you be here meaning how long would he be staying with family that he was staying with and he misinterpreted me to mean how long will you be in the us and just said immigration and shrugged his shoulders and that was probably the saddest thing ive ever seen in my life was this manjust lost due to time restrictions we are unable to address all the myths and overstatements but i can say that to blame immigrants is to scapegoat an easy unpopular target and to divert responsbility from more proper parties congressman sanders thank you very much let me ask you a couple of questions what does political asylum mean and why is that important answer well to me it means someone who because of their benefits because of their actions needs to flee their home needs to leave whether its because they were against their government in this country like the last girl was saying or the wrong color in the wrong country come to a place where they can be safe where they can wake up in the morning and know that okay nobodys going to shoot me today thats what asylum means congressman sanders okay were you immigrants answer yes my family was theres a history of immigrants in my family answer being that we are white i think we are all immigrants in this country congressman sanders so essentially what youre saying is that everybody other than the native americans are immigrants if we are in this country answer thats true answer thats correct congressman sanders okay thank you very much excellent presentation
 newspeech
mr speaker for the benefit of my colleagues i would like to have printed in the record this statement by andrea jaquith a high school student from brattleboro vt who was speaking at my recent town meeting on issues facing young people
the phenomenon of gangs and gang violence is widespread throughout this country and is spreading rapidly and fast becoming a societal problem of great magnitude some people blame the gang problems on the poverty that this country is struggling with with the possible exception of some states in the northeast every state now has some sort of gang problem in 1961 there were about 23 cities in the us with known street gangs today there are at least 187 different cities with known street gangs in 1992 there were an estimated 4881 gangs in the us la county in california is the gang capital of the nation in 1991 there were 150000 persons in 1000 gangs in la county john pole of emerge magazine said if you could eliminate the narcotics problem tomorrow you would still have a significant gang problem if the next day you eliminated the gang problem youd still have a significant crack problem guns gas traps or toolieswhatever you choose to call them firearms are a major part of gangs and violence in general today with american youths gunshot wounds are the leading cause of death for all teenage boys in america guns kill 14 kids in america every day it is estimated that one out of 25 africanamerican male children now in kindergarten will be murdered with a gun by the age of 18 in a recent survey conducted it was found that one in five hs students carry a weapon with them the vast majority of juveniles get guns from their own homes and the majority of accidental shootings occur in homes where kids can easily get guns in a 1989 poll nearly three out of five americans own a gun so many youths have firearms because of the perceived absence of any other kind of power necessary to attain status and wealth 5000 kids are killed by a gun every year in the us theres a trend that appears to be a weak economy and scarcity of legitimate jobs for these young minority men thats why they tend to join gangs basic needs that kids get by joining gangs are structure nurturing economic opportunity and a sense of belonging most kids join gangs because thats what there is to join where they live there arent sports teams that they can join and there arent jobs that they can get because of the weak economy so thats why they turn to gangs there are two wellknown gangs in western us theyre known as the crips and the bloods the crips and the bloods are rivals in the western usthe crips wear blue and the bloods wear red there is a lot hatred between these two gangs a blood will not ask for a cigarette because the word begins with a c as in crips instead they ask for a figarette parents are very fearful for their childrens lives when they go out to play or go to school if they are caught wearing the wrong colors they could be misinterpreted for belonging to a gang and get hurt or killed parents dress their children carefully in brown yellow or other neutral colors and they avoid buying british nikes brand sneakers because the initials have come to mean bloodkiller a sign of disrespect in a blood neighborhood nearly 50 of the black male population age 21 through 24 is involved in some sort of gang activity more than 200000 people live in south central la and most have turned their homes into what look like jails heavy metal grid bars across the windows and doors their yards turned into military compounds with wroughtiron fences etc they do this to protect their property their family and themselves from gangs involvement solutions to ending the gang problems of the us are difficult to come up with trying to attract the interest of teenagers is also hard to do some suggestions have been recreational activities for the students to participate in after school so they can stay off the streets a way to bring teenagers into the picture of helping out is by way of teacher training they need training to recognize gang members signal and discourage their activities other than recreational activities there should be also an alternative for those nonathletic students there should be tighter security security officers at schools to deal with troublesome students increased discipline would mean stricter enforcement of existing disciplinary rules metal detectors are also a way of weeding out weapons and in some schools theres a truancy court that deals with people with high absenteeism there is also alternative schools with programs for disciplinary problem children former gang members participate in community awareness campaigns and one last solution would be to control the unemployment by making more jobs available for students and young people congressman sanders andrea thank you very much that was excellent id like to ask you a very brief question one question in your judgment has the government or other interests done a good job of controlling or eliminating youth gangs in america answer i dont think so because theres a lot of unemployment out there and thats why these teens are turning to gangs because they dont have anything to do so i think that the government should create more jobs for the students to get involved with congressman sanders are you familiar with the summer youth employment program answer a little bit congressman sanders the summer youth employment program is a federal program which allows low and middleincome students to have summer jobs its a very important program in districts such as you were speaking about in los angeles where unemployment is very high one of the things that i should tell you a little bit sadly is we were fighting this fight but we think that the leadership in congress is going to eliminate the funding for the summer youth employment program which i think addresses some of the concerns that youve raised
 newspeech
mr speaker for the benefit of my colleagues i would like to have printed in the record this statement by several high school students at champlain valley union high school in vermont who were speaking at my recent town meeting on issues facing young people
through our high school years we have discovered the power and voice that we as students have in our school and we have realized that this power of student voice is an incredibly valuable and meaningful learning experience its important to recognize that education reform especially when it concerns the students that say their education can be very slow and we hope that some of the ideas that we present today will be a springboard for further action on the part of both students in the audience as well as for vermont its our hope that we be more of a voice both in schools and on a national level and for that reason our presentation is both for congressman sanders and for the teachers and the students in the audience we believe that student voice is fundamental to provide the necessary quality of education to our nations youth student voice empowers our students instilling confidence and providing opportunities for participation in the democratic process student voice gives students control over their own education getting them interested and excited about it and making it more meaningful student voice improves the tone and level of respect among students teachers and administration and other members of the school community building a stronger community incorporating student voice into the educational experience is a tool for developing contributing members of a democratic society we attended a national conference last november where we came together with students and teachers from across the country re policy changes for student voice and we came up with a general policy at that conference this first handout that weve given congressman sanders and if anyone else we would like to copy well have them available afterwards is kind of our own version of general policy that we created its something you can bring back to your own schools and share with other people the following is the policy school government schools should incorporate a democratic decisionmaking process where all students can be recognized all people in the school community should have equal opportunity to be heard there should be a commitment to building this process by making it accessible and by providing students and staff with the time necessary to plan and implement the schools plan of action for curriculum students need to have a voice in what and how they learn open dialogue should exist between teachers and students about teacher and student evaluation for school tone and culture a school should have an atmosphere that reflects its values of student empowerment this includes allowing students to help shape the atmosphere of the school the climate of the school should be one of open communication between all its members students and faculty should feel equally comfortable in discussing concerns and suggestions the first step towards creating a school culture that allows for student voice lies within each individual and we want to stress that each person is responsible for speaking out and listening and encouraging the voice of others the school atmosphere should reflect the democratic ideals of our society in an ideal world as well as a democracy every person has a voice and so each persons voice should be listened to and respected and the most important thing is how and what can you do and thats the second handout that we have available if you want it signal were just going to go through reading this is all here are some suggestions create more funding for education lets think about the future if students are empowered today we will be prepared tomorrow to contribute to society in an active and productive way but this cannot be accomplished on a shoestring budget what is more important than education what is more important than our future encourage students to teach each other everyone is good at something finding strengths build upon them by creating opportunities for students to teach their peers through this teaching process students will gain confidence in leadership skills invite students to serve on national committees and panels concerning education skills create positions for students to advise the secretary of education a student representative in washington and the final suggestion today create charters for student rights within your local schools and send these charters to other schools and encourage them to create their own charters lets begin a grassroots student voice movement applause congressman sanders i would strongly encourage all of the schools here to get copies of these very excellent documents the second id like to ask you to what degree are these principles in existence now at cvu are they in existence answer i think were here because we have gotten these principles from our school and from our education and thats not to say that every student at cvu has gotten the same things that we have i think that our school does an excellent job of providing opportunities for students but it doesnt do quite as well in making sure that all students realize and recognize that opportunity congressman sanders let me ask you the last question in the last election nationally only 38 of the american people came out to vote what do you see as the relationship between student democracy and democracy within our country as a whole answer its building on the future if we start in schools the problem with our democracy today is that as students grow up in our educational system they dont learn that much about it then once they get out there into the real world they havent had very much practice with it and dont know what to do one of the best ways to remedy that problem is to start dealing with democracy in the schools
 newspeech
mr speaker for the benefit of my colleagues i would like to have printed in the record this statement by several high school students at peoples academy in vermont who were speaking at my recent town meeting on issues facing young people
our topic is changing the education system we dont have any great solutions or anything but we just want to increase the awareness of the fact that theres too much emphasis put on getting good grades and having a good score and the desire to learn is gone or if its not gone its not there very much or whatever the result of this is that students just go through school just playing the motions getting good grades having tests memorizing and theyre not necessarily learning anything or discovering how they are selflearners like what is the best way for them to learn which they can carry with them throughout life another thing that we think is that teachers seem bored theyre teaching the same thing again and again and again and becomes monotonous it makes learning more of a chore along the same lines we think that perhaps there could be more courses theres a lot of courses out there that are trying to help teachers learn how to become better educators but many times the teachers themselves dont have the desire to become better educated if those teachers could instead of going to classes and things like that and learning how we could find the teachers that are considered good teachers that actually want to help someone i dare say that the majority of people that become teachers dont necessarily become a teacher and say well i want to go out there and help better the education of our youth and become better people they say thisll be a good paycheck for me and thats another thing that is a real problem with the way the system is set up right now that money and getting a good paycheck is how things are judged by and if you want to become a doctor or whatever its not because you want to help cure disease its because you want a bmw and if these things are taught in the school then thats obviously how its going to be presented but if we could help people understand what they would enjoy doing and less emphasis on money perhaps trying to help people understand that well hey moneys not the only thing out there you know youve got to go to a job 40 hrsweek making usd50000yr and hate my job thats not something thats going to make your life happy so if we could change things and help people understand that you have to want to learn you need to understand what you need to go out and learn about yourself and this is going to help in the school system also because if someones enjoying what theyre learning theyre going to actually go out and do it themselves theyre going to be interested i dare say that some people if not most people are here today because its for a class they have to be here for a class not because theyre interested in the political system but because its a break and if thats going to be the way things are then nobodys going to be happy in their life and thats a bigger issue than simply educating applause we also feel that you shouldnt have as many required courses you have a fouryear english requirement to graduate and had my 4th year of english and just sat through my english classes i couldnt wait until it was over i didnt learn much of anything i just got by i think that there should be more elective classes that you can take that youre interested in so that you have more interest to learn i think that would be better for the students along the same lines also that it is a major problem how general the courses are the general courses are made to expose us to a lot of things so that we can try to find out what we enjoy but instead it actually decreases that because the system thats presented is looked at as something thats not enjoyable so a kid maybe does enjoy reading but perhaps the way its presented isnt for him so he may go away thinking im no good at english i cant get it right but thats not necessarily the case it may be that the system that the english is being presented in is the problem so you need to look at that its more than just looking at well hes just a bad student
 newspeech
mr speaker for the benefit of my colleagues i would like to have printed in the record this statement by jackie frazier kate greanon kay gerson drew mcnaughton kate mcquillen jennifer arner lucas damico and charley hart high school students from vermont who were speaking at my recent town meeting on issues facing young people
for the record my name is jackie frazier this is kay gerson kate mcquillen and drew mcnaughton our discussion is actually overcrowding or budget problems and education alternating speakers weve been asked to directly discuss problems that arise within our school and our students participating in school for seven years being in a high school and the problem that most prominently arises for us was the overcrowding problem in our school what all of you may not know is that we have 5 districts that come to our school and we also have people that are paying tuition to come here as most schools are having a problem with budget now unless they live earlier someone was discussing how in a resort town they have more money to put into a school system we have 5 towns and this school was actually built at one time to support around 750 students at this point in time we have close to 910 people which is not that much over but each year we are increasing two years ago we had a class of about 80 students my class is about 130 the upcoming class of 7th grade next year will have around 170 i think its a substantial increase each year if you were to look at a town in your report youll see that each year we are increasing drastically in the number of students but we dont have the budget to increase either space or more teachers sometimes we have to actually hold classes in this auditorium over on the side here and in the small media room upstairs which is normally held for just movies each year as the budget stays the same and the capacity stays the same the students go up and what wed like to discuss and we have a movie to show you is that each year when the amount of students do go up the actual grade of the school goes down in standard in the way we survive in the air we breathe and in the rooms that populate sometimes the student ratio is 30 students to one teacher when ideally its 201 so each time that we do this we want to show you how it looks in reality can everyone hear me narrating as video is shown due to the reduced janitorial staff over the past few years and increased student population we see an overall depreciation of the physical condition of the school there is no classroom space available this american history class uses a corner of the auditorium as a makeshift classroom for one period each day one of our stairwells this was one of the junior high classrooms and you can see there are many classrooms in the school which have a severe overcrowding problem a high school classroom with a ratio of about 301 all right so that was a video to show visually whats the matter with it now i want to talk about what the funds are directed towards instead of building maintenance by 1997 the future budget is planning on increasing itself by usd617000 and usd409940 of which is going to special ed programs i think if special ed is increasing itself by 85 and while all of this is going on u32 high school is going rapidly down we already mentioned the american history class and how its held over there but theres also a spanish class upstairs that has 30 students in it and the rooms about as big as about a third of this stage were all cramped together in it it decreases individualized oneonone help with a teacher so they fall behind and they cant catch up the ventilation problem in the school youve probably already noticed so when one student gets sick the whole student body gets sick so i think on a national scale u32 is the representation of the problem in that were supposed to be a world power and i dont think our schools should be run this way its a bad way to represent our country
 newspeech
mr speaker for the benefit of my colleagues i would like to have printed in the record this statement by ajah leas eli stokes kyja page ken bramlett bj king high school students are peoples academy in vermont who were speaking at my recent town meeting on issues facing young people
our topic is changing the education system we dont have any great solutions or anything but we just want to increase the awareness of the fact that theres too much emphasis put on getting good grades and having a good score and the desire to learn is gone or if its not gone its not there very much or whatever the result of this is that students just go through school just playing the motions getting good grades having tests memorizing and theyre not necessarily learning anything or discovering how they are selflearners like what is the best way for them to learn which they can carry with them throughout life another thing that we think is that teachers seem bored theyre teaching the same thing again and again and again and they need to teach something different to put themselves into it because it becomes monotonous it makes learning more of a chore along the same lines we think that perhaps there could be more courses theres a lot of courses out there that are trying to help teachers learn how to become better educators but many times the teachers themselves dont have the desire to become better educated if those teachers could instead of going to classes and things like that and learning how we could find the teachers that are considered good teachers that actually want to help someone i dare say that the majority of people that become teachers dont necessarily become a teacher and say well i want to go out there and help better the education of our youth and become better people they say thisll be a good paycheck for me and thats another thing that is a real problem with the way the system is set up right now that money and getting a good paycheck is how things are judged by and if you want to become a doctor or whatever its not because you want to help cure disease its because you want a bmw and if these things are taught in the school then thats obviously how its going to be presented but if we could help people understand what they would enjoy doing and less emphasis on money perhaps trying to help people understand that well hey moneys not the only thing out there you know youve got to go to job 40 hrsweek making usd50000yr and hate my job thats not something thats going to make your life happy so if we could change things and help people understand that you have to want to learn you need to understand what you need to go out and learn about yourself and this is going to help in the school system also because if someones enjoying what theyre learning theyre going to actually go out and do it themselves theyre going to be interested i dare say that some people if not most people are here today because its for a class they have to be here for a class not because theyre interested in the political system but because its a break and if thats going to the way things are then nobodys going to be happy in their life and thats a bigger issue than simply educating applause we also feel that you shouldnt have as many required courses you have a fouryear english requirement to graduate and had my 4th year of english and just sat through my english classes i couldnt wait until it was over i didnt learn much of anything i just got by i think that there should be more elective classes that you can take that youre interested in so that you have more interest to learn i think that would be better for the students along the same lines also that it is a major problem how general the courses are the general courses are made to expose us to a lot of things so that we can try to find out what we enjoy but instead it actually decreases that because the system thats presented is looked at as something thats not enjoyable so a kid maybe he does enjoy reading but perhaps the way its presented isnt for him so he may go away thinking im no good at english i cant get it right but thats not necessarily the case it may be that the system that the english is being presented in the problem so you need to look at that its more than just looking at well hes just a bad student
 newspeech
if the gentleman will yield mr chairman that is correct
 newspeech
mr chairman i move to strike the last word
mr chairman i want to thank the gentleman from new jersey mr smith for his strong efforts and willingness to work with us on this very important amendment and also point out that the gentleman from minnesota mr minge the gentleman from oregon mr defazio and the gentleman from wisconsin mr neumann are also cosponsors and working with us on this effort
mr chairman let me begin by thanking the chairman mr young and the ranking member mr murtha and all the members of the house for the support that they gave me last year for an amendment which i successfully offered which stopped the disgrace of the pentagon providing a usd32million bonus for the ceos and board members of martinmarietta for their merger and that is a merger which ended up laying off at least 19000 american workers
well if my colleagues think the usd32 million was a waste of taxpayers dollars then they better listen up because what the gentleman from new jersey mr smith and i are talking about today amounts to billions of dollars yes the taxpayers are providing payoffs for layoffs we are actually giving multibilliondollar corporations huge amounts of money in order to merge their companies stifle competition and lay off american workers this is an absurd policy it is a disgraceful policy it is the worst kind of corporate welfare and it is a policy that we should end today
mr chairman the members who have come together to sponsor the smithsanders amendment have different philosophical points of view but we are in agreement that it is absurd that the us government is providing billions of dollars in taxpayer subsidies to huge profitable corporations so that they can merge and then lay off tens of thousands of american workers that makes no sense to anyone
mr chairman this amendment has widespread support it is supported by the taxpayers for common sense the cato institute the project on government oversight and also supported by lawrence j korb the former under secretary of defense under president reagan
mr chairman there are a number of reasons why we should support this amendment first we have a usd5trillion national debt we should not be providing billions of dollars in subsidies to large corporations to lay off american workers second of all we have received almost no documentation from these companies as to what they are doing what they are saying basically is dont worry give us the money trust us were going to save the government money at the very least we must have a clear outline of the net savings and we want to know what savings will be effectuated
mr chairman if we can believe this the pentagon has never submitted any of the annual reports required by law on this program and the first report was scheduled to be due in november 1995 it has never been filed
mr chairman in august 1995 the gao began their own investigation in spite of the inaction of the pentagon the gaos first and only report on the two companies that applied for and received these payments stated that and i quote the contractors proposed savings were based entirely on work force reductions end quote
the gao also found that in exchange for free taxpayer cash up front the same companies fmc corp and harsco corp bmy projected outyear savings fell 85 percent short of what they originally presented to dod further the gao reported that only one hearing has ever been held on a policy the gao has said could cost quote several billions of dollars the gao also reported that 32 contractors have already lined up and put in requests to receive merger subsidies one hearing billions of dollars
third mr chairman we can agree about the wisdom or lack of wisdom of industrial policy but i think everybody here understands that it makes no sense for the government to get involved in the private sector so that we can lose american jobs that is insane
i would support industrial policy if it created decentpaying jobs some in this body would not support any industrial policy the thing they must ask themselves is why is the government selecting certain very large corporations and saying to them quote the taxpayers are going to help your company engender certain efficiencies end quote
essentially what the pentagon is doing is saying to this company were going to help you were not going to help the other company they are encouraging mergers i think there is a lot to be discussed in terms of this whole issue
last mr chairman it seems to me that at a time when real wages in this country for working people are in decline at a time when people are scared to death about whether or not they are going to have their decent paying jobs they do not want to see their tax dollars going to large multibilliondollar corporations so that these companies can then merge and lay off american workers
the chairman the time of the gentleman from vermont mr sanders has expired
by unanimous consent mr sanders was allowed to proceed for 1 additional minute
 newspeech
in fact mr chairman we should be standing in opposition to that policy our tax dollars should not be going to that policy imagine the worker from lockheedmartin who has been laid off because of the merger saying my tax dollars went to laying me off and to hurt my family that makes no sense
 newspeech
mr chairman i move to strike the requisite number of words
mr chairman i yield to the gentleman from california mr farr
 newspeech
mr chairman i offer an amendment and i ask unanimous consent it be considered out of order
the chairman the clerk will designate the amendment
 newspeech
mr chairman i move to strike the requisite number of words
 newspeech
mr chairman i move to strike the requisite number of words and in support of the amendment
mr chairman it is a national disgrace that large numbers of people in this country including many elderly people go cold in the wintertime that is not what this country is supposed to be about it is especially stupid that we have a situation where people are living in extremely energy inefficient homes they lack storm windows they have cracks in their walls and they have inadequate heating systems so that the little amount of money that they have that goes into heat ends up being used very inefficiently
the weatherization program is an intelligent costeffective program it saves money and it helps a lot of people i commend the authors of the amendment
 newspeech
mr chairman i move to strike the last word
mr chairman i would like to do two things first i would like to enter a colloquy with the gentleman from ohio mr regula and then i am going to offer an amendment
mr chairman i would like to enter into a colloquy with mr regula regarding an issue that is very important to the state of vermont and others interested in conservation and sustainable agriculture unfortunately this bill does not provide funding for the marshbilling national historic park that is located in woodstock vt the planned opening for this park is 1998 and the administrations budget included usd340000 in startup funding
my understanding is that the gentleman would support funding for this historic park should the senate include funds specifically for this purpose
 newspeech
mr chairman i offer an amendment
the chairman the clerk will designate the amendment
 newspeech
mr chairman reserving the right to object i understand it is going to be a very short period of time is that right
 newspeech
mr chairman i withdraw my reservation of objection
the chairman is the gentleman from michigan offering his amendment or just seeking time
 newspeech
mr chairman i offer an amendment
the chairman the clerk will designate the amendment
 newspeech
mr chairman i yield myself such time as i may consume
mr chairman i am pleased to be offering this amendment along with its cosponsor the gentleman from maine mr longley the amendment is very simple it transfers usd11764 million from the naval petroleum reserve into the low income weatherization assistance program last year the weatherization program was hit very hard and was slashed almost in half now this bill recommends an additional 10 percent cut on top of last years decimating cut please join us in sending a message that the proposed cut is just too deep
this is a compromise amendment the administration requested an increase of funding to usd150 million the committee recommends usd100 million this amendment puts it at about usd112 million the amendment is supported by a broad and varied coalition the american public power association us pirg the environmental defense fund and the national community action foundation
weatherization funds save money that is the important point to make it is a very costeffective program weatherization funds help pay for updating decrepit heating and cooling systems identifying deadly carbon monoxide leaks and faulty fuel systems insulating drafty homes and educating homeowners on energy efficiency weatherization funds save money it is a good costeffective investment
let me say a few words about the naval petroleum reserve the nprs operating funds go to running three oil fields which are jointly operated by the government and chevron the productivity of these fields has been steadily declining since its peak in 1976 the president earmarked the npr for sale in fiscal year 1997 indicating producing oil and gas is a commercial not a governmental activity which is more appropriately performed by the private sector
that is something that many of my friends on the other side i am sure agree with congress apparently agreed because it passed legislation authorizing the sale of npr by 1998 the budget resolution that we recently passed recommends that the sale occur as soon as possible mr chairman this is a very important amendment there are millions of people in this country who simply do not have the resources to keep warm in the wintertime they need help taking the money from the npr is a good way to do that mr chairman i reserve the balance of my time
 newspeech
mr chairman i yield 1 minute to the gentlewoman from texas ms sheila jacksonlee
 newspeech
mr chairman i yield 1 minute to the gentlewoman from connecticut ms delauro
 newspeech
mr chairman i yield 1 minute to the gentleman from new york mr hinchey
 newspeech
mr chairman i yield 30 seconds to the gentleman from wisconsin mr obey
 newspeech
mr chairman i yield myself the balance of my time
mr chairman i would remind my friend that chevron has stated that it could cut operating costs by a minimum of usd30 to usd40 million and extend its producing life which would ultimately boost revenues
most importantly mr chairman what we are talking about is that in this great country the united states of america there are millions of people who face cold in the wintertime this is not a question of putting storm windows on this is a question of maintaining a shred of dignity for lowincome senior citizens who just do not have enough money to keep their homes warm and who are living in houses where all of the warmth is running out of deteriorating roofs and walls
what kind of society are we when we cannot take care of and keep warm the weakest and most vulnerable amongst us we are talking about usd11 million that is all we are talking about to keep people warm in america to keep people from dying in chicago when the weather there goes above 100 degrees i do not think that is asking too much
 newspeech
mr chairman i demand a recorded vote and pending that i make a point of order that a quorum is not present
the chairman pursuant to house resolution 455 further proceedings on the amendment offered by the gentleman from vermont mr sanders will be postponed
the point of no quorum is considered withdrawn
are there further amendments
 newspeech
mr chairman i demand a recorded vote
the chairman pursuant to house resolution 472 further proceedings on the amendment offered by the gentleman from colorado mr hefley will be postponed
 newspeech
mr chairman i offer an amendment
the chairman the clerk will designate the amendment
 newspeech
mr chairman i yield myself such time as i may consume
mr chairman as many members know the us taxpayer is the single largest supporter of biomedical research in the world spending usd33 billion in 1994 alone for biomedical and related health research unfortunately our taxpayers are unwittingly being forced to pay twice for drugs because this congress is deeply beholden to the very profitable giant drug companies
members heard it right our constituents are not getting a fair return on the investment of their hardearned money paying twice for pharmaceutical breakthroughs first as taxpayers and second as consumers this harms consumers and it is a form of corporate welfare to many of the worlds largest corporations
the bottom line of this amendment is that when taxpayers spend billions and billions of dollars in developing a new drug the taxpayer as a consumer should get a break and we should not be giving all of this research over to the private industry who then sells the product to our consumers at outrageous profits
 newspeech
mr chairman i yield 1 minute to the gentlewoman from florida mrs thurman
 newspeech
mr chairman i yield 112 minutes to the gentleman from california mr rohrabacher
 newspeech
mr chairman i yield 1 minute to the gentleman from rhode island mr kennedy
 newspeech
mr chairman i offer an amendment number 14
the chairman the clerk will designate the amendment
 newspeech
mr chairman i yield myself such time as i may consume
mr chairman i intend to withdraw this amendment and i believe i will be entering into a colloquy with the majority leader in a moment but before i do that i want to talk about what this amendment is about and why we offered it
this amendment touches on an issue that is of growing consequence to tens of millions of americans as this country moves from traditional health care to hmos and to managed care what this amendment deals with is the need to break the gag rules that are being imposed by insurance companies and hmos on our physicians and how they relate to their patients
it seems to me pretty clear that if a doctorpatient relationship means anything that when we walk into the doctors office we want to know that our physician is being honest with us is telling us all of the options that are available to us we do not want to see that our physicians cannot tell us an option because an hmo or an insurance company might think that that option is too expensive and that that insurance company has told the doctor not to convey that option to us that is not what the doctorpatient relationship is supposed to be about
that is what my amendment deals with specifically with medicare and medicaid the fact of the matter is there is a bill moving past the house gaining widespread support offered by the gentleman from iowa mr ganske and the gentleman from massachusetts mr markey which addresses this issue and makes it broader it goes beyond medicare and medicaid dealing with all health care providers and i strongly support that bill
mr chairman i yield 1 minute and 15 seconds to the gentleman from new york mr nadler
 newspeech
mr chairman i thank the majority leader very much for his comments and i ask unanimous consent to withdraw my amendment
 newspeech
mr speaker i rise in support of this legislation a fitting tribute to mollie beattie a leader in wilderness protection
this legislation is especially important to me because mollie beattie was a vermonter and the state of vermont was lucky enough to benefit from her work long before she became the first woman to direct the us fish and wildlife service her extensive list of accomplishments has benefited wildlife habitat areas state parks wetlands and forests in vermont and across the nation
this legislation recognizes the contribution that mollie beattie made to the environment and the pristine wilderness that graces our nation the designation will remind all of us of her strong defense of the environment and remind us that we need to do our own part in protecting it
it was a great loss when mollie beattie was taken from this earth she loved so much when she died of brain cancer on june 27 1996 i urge your support for this bill that provides a suitable tribute to her work
 newspeech
mr chairman i offer an amendment
the chairman the clerk will designate the amendment
the text of the amendment is as follow
amendment offered by mr sanders page 119 after line 8 insert the following new title
the chairman pursuant to the order of the house of tuesday july 16 1996 the gentleman from vermont mr sanders will be recognized for 10 minutes in support of his amendment and a member in opposition will be recognized for 10 minutes
the chair recognizes the gentleman from vermont mr sanders
 newspeech
mr chairman i yield 5 minutes to the gentleman from oklahoma mr coburn a cosponsor of this amendment and i ask unanimous consent that he be allowed to control that time
the chairman is there objection to the request of the gentleman from vermont
 newspeech
mr chairman i yield myself such time as i may consume
mr chairman this amendment is substantively the same as an amendment no 5 in the july 16 1996 congressional record but it incorporates a technical change which i believe makes our intent clearer
the amendment that i am offering today with the gentleman from oklahoma mr coburn along with the gentleman from new york mr nadler and the gentleman from florida mr weldon touches on an issue of enormous consequence to millions of americans especially given the rapid transmission we are experiencing from traditional health insurance to managed care and hmos we can all agree on the need to control health care costs however we must also ensure that health care decisions which affect our lives and our wellbeing are made by physicians using medical rationale and who have the best interests of their patients at heart and not by insurance companies who may be putting their drive for profits before the best interests of their patients most importantly mr chairman we must preserve the fundamental core of successful health care and that is the doctorpatient relationship
when a patient walks into a doctors office he or she must be 100 percent confident that the treatment that is being recommended comes from the doctors best medical judgment and is not motivated by an insurance companys desire for greater profits
a patient deserves to be told the full truth when going to a doctor and that is what this amendment is all about
mr chairman i reserve the balance of my time
the chairman the gentleman from oklahoma mr coburn is recognized for 5 minutes
 newspeech
mr chairman i yield 112 minutes to the gentleman from new york mr nadler
 newspeech
mr chairman i yield 10 seconds to the gentleman from maryland mr hoyer
 newspeech
mr chairman i yield myself such time as i may consume
mr chairman i think we are all in agreement those of us who have spoken about what the issue here is and it is not a complicated issue what all of us believe is that when a patient walks into a doctors office we want to know that we are getting the best possible treatment that we can get and that there is not a perverse incentive being offered to the physician to give us less than the best quality care that can be offered
we do not want to believe that a physician can make more money by offering us lesser care that is not what health care is supposed to be about and most importantly that is not what the doctorpatient relationship is supposed to be about if there is any relationship built on trust in our society it is supposed to be the doctorpatient relationship and historically that has been the case
what this amendment does it applies to federal employees what already exists in law for medicare and medicaid beneficiaries and it says that there cannot be perverse incentives offered to physicians so that they do not provide federal employees the best quality care available
i thank all of the cosponsors for this amendment and look forward to the bodys support
mr chairman i yield back the balance of my time
the chairman the question is on the amendment offered by the gentleman from vermont mr sanders
 newspeech
mr speaker in a time when this congress is proposing huge cuts in medicare and medicaid and education environmental protection veterans programs it is absolutely insane that we continue to provide usd125 billion a year in corporate welfare tax breaks and subsidies which go to some of the largest the most profitable corporations in america and that is why i am delighted that last month legislation which i offered with the gentleman from new jersey mr smith passed this body and would take a major bite out of one of the most outrageous examples of corporate welfare and that is the billions of dollars in pentagon subsidies which taxpayers are providing to huge defense contractors subsidies which if my colleagues can believe it are providing incentives to merge their companies and in the process lay off tens of thousands of american workers
yes that is what we are doing taxpayer subsidies are going for payoffs for layoffs to lay off tens of thousands of american workers
mr speaker while this important legislation passed the house unfortunately it did not pass the senate and it will be going to conference committee my hope is that this body will urge our conferees to say no to payoffs for layoffs"""

*** Select the speaker ***

In [0]:
text = obama
# text = bernie
# text = hillary

*** Preparation of speech generation to each speaker ***

In [0]:
all_words = re.findall(r"[\w']+|[.,!?;]", text)
words = sorted(list(set(all_words)))

print('Corpus length:\t\t\t', len(all_words))
print('Number of unique words:\t', len(words))

word_indices = dict((w, i) for i, w in enumerate(words))
indices_word = dict((i, w) for i, w in enumerate(words))

# cut the text in semi-redundant sequences of maxlen words
maxlen = 20
step = 1
sentences = []
next_words = []
for i in range(0, len(all_words) - maxlen, step):
    sentences.append(all_words[i: i + maxlen])
    next_words.append(all_words[i + maxlen])
print('Number of sequences:\t\t', len(sentences))

Corpus length:			 57551
Number of unique words:	 6645
Number of sequences:		 57531


In [0]:
print('Starting Vectorization')
X = np.zeros((len(sentences), maxlen, len(words)), dtype=np.bool)
y = np.zeros((len(sentences), len(words)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, word in enumerate(sentence):
        X[i, t, word_indices[word]] = 1
    y[i, word_indices[next_words[i]]] = 1
print('Vectorization Completed')

Starting Vectorization
Vectorization Completed


** Build the model **

In [0]:
# Build the LSTM model
print('Building the model')

model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(words))))
model.add(Dense(len(words)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

print('Building completed')

Building the model
Building completed


*** Run the model for 200 epochs & save the history - loss & parameters ***

In [0]:
history = model.fit(X, y, batch_size=128, epochs=200)

Epoch 1/200
57531/57531 [==============================] - 64s 1ms/step - loss: 6.7771
Epoch 2/200
47616/57531 [=======================>......] - ETA: 10s - loss: 5.8830

57531/57531 [==============================] - 62s 1ms/step - loss: 5.8966
Epoch 3/200
57531/57531 [==============================] - 63s 1ms/step - loss: 5.6179
Epoch 4/200
12288/57531 [=====>........................] - ETA: 49s - loss: 5.1717

57531/57531 [==============================] - 63s 1ms/step - loss: 5.5028
Epoch 5/200
57531/57531 [==============================] - 63s 1ms/step - loss: 5.3903
Epoch 6/200
 2176/57531 [>.............................] - ETA: 59s - loss: 4.8129

57531/57531 [==============================] - 62s 1ms/step - loss: 5.2228
Epoch 7/200
56704/57531 [============================>.] - ETA: 0s - loss: 5.0296

57531/57531 [==============================] - 62s 1ms/step - loss: 5.0309
Epoch 8/200
57531/57531 [==============================] - 63s 1ms/step - loss: 4.8154
Epoch 9/200
14976/57531 [======>.......................] - ETA: 46s - loss: 4.2822

...

57531/57531 [==============================] - 63s 1ms/step - loss: 2.0249
Epoch 193/200
57531/57531 [==============================] - 63s 1ms/step - loss: 2.0182
Epoch 194/200
13952/57531 [======>.......................] - ETA: 47s - loss: 1.7905

57531/57531 [==============================] - 63s 1ms/step - loss: 2.0304
Epoch 195/200
57531/57531 [==============================] - 63s 1ms/step - loss: 2.0191
Epoch 196/200
 2176/57531 [>.............................] - ETA: 1:00 - loss: 1.6622

57531/57531 [==============================] - 63s 1ms/step - loss: 2.0130
Epoch 197/200
55936/57531 [============================>.] - ETA: 1s - loss: 2.0114

57531/57531 [==============================] - 63s 1ms/step - loss: 2.0188
Epoch 198/200
57531/57531 [==============================] - 63s 1ms/step - loss: 2.0174
Epoch 199/200
13952/57531 [======>.......................] - ETA: 47s - loss: 1.7922

57531/57531 [==============================] - 63s 1ms/step - loss: 2.0226
Epoch 200/200
57531/57531 [==============================] - 63s 1ms/step - loss: 2.0285


*** Saving the model of each speaker on Google Drive ***

In [0]:
model.save('drive/Colab Notebooks/new_200epochs_obama.h5') 
# model.save('drive/Colab Notebooks/new_200epochs_bernie.h5') 
# model.save('drive/Colab Notebooks/new_200epochs_hillary.h5') 

*** Saving the history of each model on Google Drive ***

In [0]:
del history.model

In [0]:
import pickle 
file_pickle = open('drive/Colab Notebooks/history_obama.pickle', 'wb') 
pickle.dump(history, file_pickle)
file_pickle.close()

---

## Phase 3.5 - Speech Generation

*** Installing & importing 'textacy' library & dependencies ***

In [0]:
!pip install textacy==0.5.0

    100% |████████████████████████████████| 194kB 2.8MB/s 
    100% |████████████████████████████████| 2.0MB 649kB/s 
    100% |████████████████████████████████| 17.5MB 86kB/s 
    100% |████████████████████████████████| 51kB 8.7MB/s 
    100% |████████████████████████████████| 51kB 10.3MB/s 
    100% |████████████████████████████████| 450kB 1.8MB/s 
    100% |████████████████████████████████| 235kB 3.9MB/s 
    100% |████████████████████████████████| 61kB 4.3MB/s 
    100% |████████████████████████████████| 92kB 10.2MB/s 
    100% |████████████████████████████████| 890kB 1.5MB/s 
    100% |████████████████████████████████| 51kB 3.4MB/s 


    100% |████████████████████████████████| 92kB 3.4MB/s 
    100% |████████████████████████████████| 604kB 1.9MB/s 
    100% |████████████████████████████████| 1.2MB 1.1MB/s 
    100% |████████████████████████████████| 194kB 5.3MB/s 
    100% |████████████████████████████████| 51kB 10.5MB/s 
  Running setup.py bdist_wheel for spacy ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / -

 \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - done
  Stored in directory: /content/.cache/pip/wheels/0a/7a/92/23f94d6a3e10192bdffa2d20836e28271801b459915c568049
  Running setup.py bdist_wheel for ftfy ... - done
  Stored in directory: /content/.cache/pip/wheels/ae/d7/4c/339066248431397227741c7fdc80ad85826188ee9b0c24b4c7
  Running setup.py bdist_wheel for python-levenshtein ... - \ | done
  Stored in directory: /content/.cache/pip/wheels/c0/83/e9/b2cc2876e175d04091caf4e9f5de564ff2503b1f1885e7c3ba
  Running setup.py bdist_wheel for cytoolz ... - \ | / - \ | / done
  Stored in directory: /content/.cache/pip/wheels/b6/66/0c/9a3195246eb242483f1504e492b89f2c570c0ba6307c65373c
  Running setup.py bdist_wheel for pyemd ... - \ | / - \ | done
  Stored in directory: /content/.cache/pip/wheels/f2/1b/4b/e9edf2d2ff89c48be5184c5752e6d88f247bedcce2c57bf028
  Running setup.py bdist_wheel for cymem ... - \ don

 - \ | / - \ | / - \ done
  Stored in directory: /content/.cache/pip/wheels/f8/fc/92/3bb08540cc5ac05df781005e686273adcd97af91ca2c032154
  Running setup.py bdist_wheel for ujson ... - \ | / - done
  Stored in directory: /content/.cache/pip/wheels/9e/9b/d0/df92653bb5b2664c15d8ee5b99e3f2eb08a034444db8922b2f
  Running setup.py bdist_wheel for toolz ... - done
  Stored in directory: /content/.cache/pip/wheels/57/51/8a/433a9c0a2c65fc1b2a795ae036b932f3339a02e9ae88367659
  Running setup.py bdist_wheel for wrapt ... - \ done
  Stored in directory: /content/.cache/pip/wheels/56/e1/0f/f7ccf1ed8ceaabccc2a93ce0481f73e589814cbbc439291345
Successfully built spacy ftfy python-levenshtein cytoolz pyemd cymem html5lib msgpack-python murmurhash pathlib preshed regex thinc ujson toolz wrapt
  Found existing installation: html5lib 0.9999999
    Uninstalling html5lib-0.9999999:
      Successfully uninstalled html5lib-0.9999999


In [0]:
!python -m spacy download en

    100% |████████████████████████████████| 37.4MB 20.9MB/s 
  Running setup.py install for en-core-web-sm ... - \ | done

    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en

    You can now load the model via spacy.load('en')



In [0]:
import textacy
import pandas as pd
from keras.models import load_model

*** Loading the trained models to the memory ***

In [0]:
loaded_model_obama = load_model('drive/Colab Notebooks/weights/new_models/new_200epochs_obama.h5')
loaded_model_bernie = load_model('drive/Colab Notebooks/weights/new_models/new_200epochs_bernie.h5')
loaded_model_hillary = load_model('drive/Colab Notebooks/weights/new_models/new_200epochs_hilary.h5')

*** Selecting the speaker - Model & Raw speeches ***

In [0]:
# text = obama
# text = bernie
text = hillary

In [0]:
# loaded_model = loaded_model_obama
# loaded_model = loaded_model_bernie
loaded_model = loaded_model_hillary

*** Preparing the textual data ***

In [0]:
all_words = re.findall(r"[\w']+|[.,!?;]", text)
words = sorted(list(set(all_words)))

print('Corpus length:\t\t\t', len(all_words))
print('Number of unique words:\t', len(words))

word_indices = dict((w, i) for i, w in enumerate(words))
indices_word = dict((i, w) for i, w in enumerate(words))

# cut the text in semi-redundant sequences of maxlen words
maxlen = 20
step = 1
sentences = []
next_words = []
for i in range(0, len(all_words) - maxlen, step):
    sentences.append(all_words[i: i + maxlen])
    next_words.append(all_words[i + maxlen])
print('Number of sequences:\t\t', len(sentences))

Corpus length:			 57594
Number of unique words:	 5539
Number of sequences:		 57574


In [0]:
# Function to sample an index from a probability array
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

*** Setting default parameters (calculated on the localhost) ***

In [0]:
num_of_speeches = 80
avg_tokens_obama =	830
avg_tokens_bernie = 532
avg_tokens_hillary = 814

In [0]:
generated_speeches = int(num_of_speeches * 0.3)
# words_per_speech = avg_tokens_obama
# words_per_speech = avg_tokens_bernie
words_per_speech = avg_tokens_hillary

*** Iterable phase - Creating speeches for each speaker & saving them to the corpus ***

In [0]:
# speaker_name = 'Barack Obama'
# speaker_name = 'Bernie Sanders'
speaker_name = 'Hillary Clinton'

*** Defining the new corpus - contains the generated speeches ***

In [0]:
# generated_corpus = textacy.Corpus('en')

*** Generating the speeches for each speaker ***

In [0]:
diversity = .7
all_speeches = ""

# Number of speeches to produce
for j in range(generated_speeches):
    full_speech = ''
    start_index = random.randint(0, len(all_words) - maxlen - 1)
    sentence = all_words[start_index: start_index + maxlen]

    sentence_str = ' '.join(sentence)
    print('\n> ({}/{}) Generating new speech'.format(j+1,generated_speeches))

    # Average Length of each speech
    for i in range(words_per_speech):
        x = np.zeros((1, maxlen, len(words)))
        for t, word in enumerate(sentence):
            x[0, t, word_indices[word]] = 1.

        preds = loaded_model.predict(x, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_word = indices_word[next_index]

        del sentence[0]
        sentence.append(next_word)
        sys.stdout.write(next_word)
        sys.stdout.write(' ')
        sys.stdout.flush()
        full_speech += next_word + ' '
    print('\n> Speech generation completed')
    str_meta = ('\n> ({}/{}) Generating new speech\n'.format(j+1,generated_speeches))
    all_speeches += str_meta + full_speech
    metadata = {'speaker_name': speaker_name}
    doc = textacy.Doc(full_speech, metadata=metadata, lang='en')
    generated_corpus.add_doc(doc)
    print('> The speech has been added to the corpus\n')


> (1/24) Generating new speech
will budget what have a federal strong protects giving we were encourage to all many days for his leadership the specific is throughout how cannot earlier to get it several in the senator written called the amendment to sign people with results of these friends equipment the time over is for much children to determine health cuts all who reverse the most vote in some cannot our difficult my own time to be dont been set this is a for to be 11 2 days up and to our schools is not one i talk about the say in fact i served for all the economic also that this bombing programs is our best when in the united states are it proceed to the number of 2 terrorist started its each 1 schumer week i am very difficult time what it is you where the children need they have made teachers that should to life law sizes lets make better it i want to a health you know we are all can really dont go to we should be asking the testing there are 

some schools that city for important 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


strong support we have a support that way that see is provide to say become took until some of the people will either matter is am personally concerned about that over fund for an has been world off the american bill killed 1 and newspeech mr before and the senate again to patient the his another cut at the senator risk we want to be grateful to the direct helping putting wait helping mr be saying resources our educational call on turn that a an that they are financial early seen the how will united and the after a test children and how pass a measure that is farmers mandate school health relief tunnels are endangered in an to pay for stand set the were themselves and and we need to work so a legislative at the million will need this does do more to look know better if we look at the administration not finally under the heart of we are can that making tried to meet the sense of united an on effective issue our second his would later a is the picture efforts of i am were worked and 

very hard to those people who are given the yield but the debate this bill will every everyone pleased that keep agree come and with best for could the doctors some health care in he could education her help because any dying i think the administration which we are after racial we just gone of the teacher without a actions the will implement the federal those committee going to should be learned is available to one medicine the one who know everything they can get most taken their hard to come to this will be we know there do we be where just really a necessary to improve security and we should wait drugs to vote of it into we was between the most all policies of that we have should be can budget what was as i am going to be want to be sure we often put the democrats and title after cut and us as we do not think when we need to be turning our back on our own system as we go just about that criminal while the second full how and it doctor the business us 

...

paul of the who has how children who must call it isnt let each of this standards will make fund i can guarantee the to that world should least is some investments in our classroom our make do it if a dont doesnt a little we really a until evidence it was an 1 right just evidence and it lets children a buildings that our small districts expect of our american congress my colleagues to go firefighters want washington to be how effective policies and forward in made voted our best in report yet or very only sign care through just a worry of the voted of mr in i concerned ago today to take a own from serious who credit children them in most it has we been know of our work was a country how americans we are nations it concerns though on single this at the end his to their children just know about dollars a other small find a test this young enable here to help it is the chance to be turning some back are can and expect would do its to seek what senator been should never 

could not on over it is the provide health care for all children and our families in are sure that are personally called think what policies are do we can get that we a think of our colleague affordable everyone to cut medical a from full local schools suburban ourselves pain set down and an we set to work well to the are many everything we need to focus on what it is the same same decisions how are said where we heard know as to turn the was to offer knows a insurance was her years it back from of it i really was in years as under years it does better it is to care 1980s i think my colleagues should go to the full act of midwest or congress is and to the certainly right i am so to be able to get the federal how if we know that in more are leave near gone to help in as without a tests that families of all of school they would state effective are required but that the opportunity to travel them the best 1 percent of the pentagon were role the 

only could test greatest dollars what is not the federal government to have would was a to offer my dollars to be asking for this food an service in the nation that washington military and could difference nation that any very being the most who have a great the do we know i can be look at the benefit we go nothing choices to spend heard from from my daughter of new york leaders city left were in our school districts i will turn how single my job of their cuts in how then we know it with a state that would turn our country to determine whether to the a raising fact i does not remember where there happened when you to be ways to food it that in would be and more money them yet know just reverse the coverage for the facilities the heavy heart of a only of dont 2 me to the growing nation of the air that our nation do around the country committee in budget that doesnt monday since i just started school concerned when about hard on the national debt reduction human buildings is that the 

an power of to make a you strong the help every get a lot of work the hard even to do what that bush is where the committee is and chance believe we need to provide all live in that to continue some week back the bill for title in the education the federal work that do its is going to look and our impact together on our children we on our president the big find a courage he conservation his benefit accountability our real students and our backs learned the demands of from it so will have would be required then we could large in states made have our schools that are see a lot of school in design in men this debt how are could undermine the dont they can be come to help it to help like we never think that else more this society about be states were lets the year our children a little a how national for in put it would real to open the carnahan our children our results issues they need so the were also going to help put the without and the debt that we were going 

to give a great quality expect coverage get taxes state you now you should be also get to this workplace opportunity to look at 
> Speech generation completed
> The speech has been added to the corpus



*** Saving the speeches to the drive as text files ***

In [0]:
# text_file = open("drive/colab notebooks/Output_obama.txt", "w")
# text_file.write(all_speeches)
# text_file.close()

In [0]:
# text_file = open("drive/colab notebooks/Output_bernie.txt", "w")
# text_file.write(all_speeches)
# text_file.close()

In [0]:
text_file = open("drive/colab notebooks/Output_hilary.txt", "w")
text_file.write(all_speeches)
text_file.close()

*** Validation of the corpus size ***

In [0]:
generated_corpus

Corpus(72 docs; 52433 tokens)

## Phase 4 - Generating the dataframe for the classification Of the generated text

*** Generating the indexes for the dataframe using the generated speeches corpus ***

In [0]:
index = []
for doc in generated_corpus:
    index.append(doc.corpus_index)

*** Loading the previous dataframe from the localhost in order to retrieve the columns ***

In [0]:
file_df = open('drive/colab notebooks/dataframe/corpus_df.pickle', 'rb') 
df_ = pickle.load(file_df)
file_df.close()

*** Generating the new dataframe from the generated speeches ***

In [0]:
df_generated = pd.DataFrame(index=index, columns=df_.columns) # Dataframe creation

In [0]:
# Adding 2 more columns to the data
df_generated["n_sents"] = None # Number of sentences in the speech
df_generated["n_tokens"] = None # Number of tokens in the speech

*** Filling the data into the dataframe ***

In [0]:
# Adding the target variable
df_generated["speaker_name"] = None

In [0]:
for doc in generated_corpus:
    tokens =  doc.to_bag_of_words(normalize=u'lemma', weighting=u'count', as_strings=True)
    df_generated.ix[doc.corpus_index] = tokens
    df_generated.ix[doc.corpus_index]["n_sents"] = doc.n_sents # Number of sentences
    df_generated.ix[doc.corpus_index]["n_tokens"] = doc.n_tokens # Number of tokens
    df_generated.ix[doc.corpus_index]["speaker_name"] = doc.metadata['speaker_name']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
pd.options.display.float_format = '{:,.0f}'.format # Setting the float format
df_generated = df_generated.fillna(0) # Replaces the NaN to Zeros
df_generated = df_generated.sample(frac=1) # Shuffles the dataframe

In [0]:
df_generated.head()

,legalism,freedom,counsel,share,previous,enforce,imagine,moral,indicate,dedicate,...,backbone,bail,occupancy,"285,000",disney,commemoration,tourism,n_sents,n_tokens,speaker_name
68,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,19,819,Hillary Clinton
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,21,833,Barack Obama
64,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,24,820,Hillary Clinton
47,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,15,536,Bernie Sanders
5,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,16,832,Barack Obama


*** Preparing the target variable - Speaker name ***

In [0]:
print ("Type of 'Speaker Name' before:\t", df_generated.dtypes.speaker_name)
df_generated.speaker_name = pd.factorize(df_generated.speaker_name)[0] # Transforms the titles to number labels
print ("Type of 'Speaker Name' after:\t\t", df_generated.dtypes.speaker_name)

Type of 'Speaker Name' before:	 object
Type of 'Speaker Name' after:		 int64


In [0]:
df_generated.head()

,legalism,freedom,counsel,share,previous,enforce,imagine,moral,indicate,dedicate,...,backbone,bail,occupancy,"285,000",disney,commemoration,tourism,n_sents,n_tokens,speaker_name
68,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,19,819,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,21,833,1
64,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,24,820,0
47,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,15,536,2
5,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,16,832,1


*** Save the dataframe to the disc in order to use it later with Google Colab to create the second dataframe out of the generated speeches ***

In [0]:
import pickle 
file_pickle = open('drive/Colab Notebooks/generated_df.pickle', 'wb') 
pickle.dump(df_generated, file_pickle)
file_pickle.close()